# Install necessary packages and libraries

In [1]:
!pip install gensim
!pip install GPUtil
!pip install rouge_score
!pip install nvidia-ml-py3
!pip install pynvml
!pip install rouge-score
!pip install rouge

In [1]:
import os
import pickle
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, losses, metrics, callbacks
import psutil
import time
import nltk
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
import gc
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from collections import defaultdict
import uuid
from tqdm import tqdm
import pandas as pd

try:
    import nvidia_smi
except ImportError:
    nvidia_smi = None

nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

2025-07-04 13:44:56.254661: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 13:44:56.254764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 13:44:56.354588: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 13:44:56.547633: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-04 13:44:58.096520: W tensorflow/compiler/tf2

True

# DeepSeekMoE baseline experiment code

In [1]:
def reset_kernel():
    tf.keras.backend.clear_session()
    import gc
    gc.collect()

results_table = {
    'Training Speed (epochs per second)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Training Time (second/epoch)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Total Training Time (second/iteration)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Computational Resource Usage': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average CPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average GPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average GPU Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average FLOPS Estimate (GFLOPS)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Expert NLU Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Expert NLG Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Entity Accuracy': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Intent Accuracy': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Perplexity': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Response Cosine Similarity': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0}
}

best_val_loss = float('inf')
best_model_path = "best_Deepseekbaseline_moe_model"

for iteration in range(5):
    print(f"\nStarting Training Iteration {iteration + 1}")
    reset_kernel()
    
    def load_tf_datasets_from_disk(load_path):
        print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
        try:
            with open(os.path.join(load_path, "moe_params.json"), "r") as f:
                loaded_moe_params = json.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"moe_params.json not found in {load_path}")

        element_spec = (
            {
                'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
                'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
                'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            },
            {
                'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
                'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
                'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
            }
        )

        try:
            train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
            test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
        except Exception as e:
            raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

        raw_data = {}
        for dataset_name in ['train', 'test']:
            path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
            if os.path.exists(path):
                with open(path, 'rb') as f:
                    raw_data[dataset_name] = pickle.load(f)
                print(f"Loaded raw data for {dataset_name} from {path}")
            else:
                print(f"Warning: Raw data file {path} not found.")
                raw_data[dataset_name] = []

        return {
            "train_dataset": train_tf_dataset,
            "test_dataset": test_tf_dataset,
            "moe_params": loaded_moe_params,
            "raw_data": raw_data
        }

    tf_dataset_save_path = "tf_datasets"
    loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
    train_tf_dataset = loaded_data["train_dataset"]
    moe_params = loaded_data["moe_params"]
    raw_data = loaded_data["raw_data"]
    moe_params["num_experts"] = 4

    class MoELayer(layers.Layer):
        def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.01, top_k=2, name=None):
            super(MoELayer, self).__init__(name=name)
            self.m = m
            self.k_s = k_s
            self.total_experts = num_experts * self.m
            self.routed_experts = self.total_experts - self.k_s
            self.top_k = top_k
            self.top_mk = self.m * self.top_k
            self.top_mk = min(self.top_mk, self.routed_experts)
            self.alpha = alpha
            self.input_dim = input_dim
            self.seq_length = None
            self.adjusted_expert_dim = expert_dim // self.m
            self.shared_experts = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                    layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
                ]) for i in range(self.k_s)
            ]
            self.routed_experts_list = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                    layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
                ]) for i in range(self.k_s, self.total_experts)
            ]
            self.experts = self.shared_experts + self.routed_experts_list
            self.gate = layers.Dense(self.routed_experts, activation='softmax')
            self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

        def call(self, inputs, training=False):
            input_rank = inputs.shape.rank
            if input_rank == 3:
                batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
                flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
                is_3d = True
            else:
                batch_size = tf.shape(inputs)[0]
                seq_length = 1
                flat_inputs = inputs
                is_3d = False
            flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
            shared_output = tf.zeros([tf.shape(flat_inputs)[0], self.input_dim], dtype=tf.float32)
            for i in range(self.k_s):
                shared_output += self.shared_experts[i](flat_inputs)

            gate_weights = self.gate(flat_inputs)
            top_mk_values, top_mk_indices = tf.nn.top_k(gate_weights, k=self.top_mk, sorted=True)
            top_mk_indices = top_mk_indices + self.k_s
            top_mk_weights = top_mk_values / (tf.reduce_sum(top_mk_values, axis=-1, keepdims=True) + tf.keras.backend.epsilon())
            routed_output = tf.zeros([tf.shape(flat_inputs)[0], self.input_dim], dtype=tf.float32)
            
            for k in range(self.top_mk):
                kth_weights = top_mk_weights[:, k]
                kth_indices = top_mk_indices[:, k]
                mask = tf.one_hot(kth_indices, depth=self.total_experts, dtype=tf.float32)
                expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
                kth_expert_output = tf.reduce_sum(expert_outputs * tf.expand_dims(mask, -1), axis=1)
                weighted_kth_output = kth_expert_output * tf.expand_dims(kth_weights, -1)
                routed_output += weighted_kth_output

            output_flat = shared_output + routed_output + flat_inputs

            if is_3d:
                output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
                if self.seq_length is not None:
                    output.set_shape([None, self.seq_length, self.input_dim])
                gate_weights_reshaped = tf.reshape(gate_weights, [batch_size, seq_length, self.routed_experts])
                if self.seq_length is not None:
                    gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
            else:
                output = output_flat
                gate_weights_reshaped = gate_weights

            f_i = tf.reduce_mean(tf.reduce_sum(tf.one_hot(tf.argmax(gate_weights, axis=-1), depth=self.routed_experts), axis=0), axis=0)
            P_i = tf.reduce_mean(gate_weights, axis=0)
            load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
            self.add_loss(load_balancing_loss)
            self.load_balancing_loss_metric.update_state(load_balancing_loss)

            return output, gate_weights_reshaped

        def get_metrics(self):
            return {f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric}

        def get_config(self):
            config = super().get_config()
            config.update({
                'num_experts': self.total_experts // self.m,
                'expert_dim': self.adjusted_expert_dim * self.m,
                'input_dim': self.input_dim,
                'm': self.m,
                'k_s': self.k_s,
                'alpha': self.alpha,
                'top_k': self.top_k,
                'name': self.name
            })
            return config

    class TransformerDecoderLayer(layers.Layer):
        def __init__(self, d_model, num_heads, dff, rate=0.1):
            super(TransformerDecoderLayer, self).__init__()
            self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.ffn = keras.Sequential([
                layers.Dense(dff, activation='relu'),
                layers.Dense(d_model)
            ])
            self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
            self.dropout1 = layers.Dropout(rate)
            self.dropout2 = layers.Dropout(rate)
            self.dropout3 = layers.Dropout(rate)

        def call(self, x, enc_output, training, look_ahead_mask=None):
            if look_ahead_mask is not None:
                look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
                look_ahead_mask = 1.0 - look_ahead_mask

            attn1_output = self.mha1(query=x, value=x, key=x, attention_mask=look_ahead_mask, training=training)
            attn1 = self.dropout1(attn1_output, training=training)
            out1 = self.layernorm1(attn1 + x)

            attn2_output = self.mha2(query=out1, value=enc_output, key=enc_output, attention_mask=None, training=training)
            attn2 = self.dropout2(attn2_output, training=training)
            out2 = self.layernorm2(attn2 + out1)

            ffn_output = self.ffn(out2)
            ffn_output = self.dropout3(ffn_output, training=training)
            out3 = self.layernorm3(ffn_output + out2)

            return out3

        def get_config(self):
            config = super().get_config()
            mha1_config = self.mha1.get_config()
            config.update({
                'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
                'num_heads': mha1_config['num_heads'],
                'dff': self.ffn.layers[0].units,
                'rate': self.dropout1.rate
            })
            return config

    class MoEModel(models.Model):
        def __init__(self, moe_params):
            super(MoEModel, self).__init__()
            self.embedding_dim = moe_params["embedding_dim"]
            self.max_seq_length = moe_params["max_seq_length"]
            self.num_domains = moe_params["num_domains"]
            self.num_intents = moe_params["num_intents"]
            self.vocab_size = moe_params["vocab_size"]
            self.num_experts = moe_params["num_experts"]
            self.expert_dim = moe_params["expert_dim"]
            self.turn_id_dim = moe_params["turn_id_dim"]
            self.num_heads = 4
            self.dff = self.embedding_dim * 4
            self.dropout_rate = 0.1
            self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                                 self.num_domains + self.turn_id_dim + self.num_intents)
            self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains
            self.embedding = layers.Embedding(
                self.vocab_size,
                self.embedding_dim,
                mask_zero=True,
                embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05)
            )
            self.embedding.build((None,))
            with tf.init_scope():
                embedding_weights = self.embedding.get_weights()
                if embedding_weights and len(embedding_weights) > 0:
                    embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                    self.embedding.set_weights(embedding_weights)

            self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
            self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
            self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
            self.decoder_dropout = layers.Dropout(self.dropout_rate)
            self.moe_nlu = MoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlu_input_dim, m=2, k_s=1, alpha=0.01, top_k=2, name='moe_nlu')
            self.moe_nlg = MoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlg_input_dim, m=2, k_s=1, alpha=0.01, top_k=2, name='moe_nlg')
            self.intent_output = layers.Dense(self.num_intents, activation='sigmoid')
            self.domain_output = layers.Dense(self.num_domains, activation='softmax')
            self.response_output = layers.TimeDistributed(layers.Dense(self.vocab_size, activation='softmax'))
            self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)

        def create_look_ahead_mask(self, size):
            mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
            return mask

        def call(self, inputs, training=False):
            user_utterance_tokens = inputs.get('user_utterance_tokens')
            prev_system_response_tokens = inputs.get('prev_system_response_tokens')
            decoder_input_tokens = inputs.get('decoder_input_tokens')
            domain_onehot_input = inputs.get('domain_onehot_input')
            turn_id_embedding = inputs.get('turn_id_embedding')
            ontology_multihot_input = inputs.get('ontology_multihot_input')
            if any(x is None for x in [user_utterance_tokens, prev_system_response_tokens, decoder_input_tokens,
                                      domain_onehot_input, turn_id_embedding, ontology_multihot_input]):
                raise ValueError("One or more required inputs are missing or None")

            pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
            user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
            prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
            decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)
            user_enc_out = user_embedded
            prev_system_enc_out = prev_system_embedded

            look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
            dec_output = decoder_embedded
            for i, layer in enumerate(self.decoder_layers):
                dec_output = layer(dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask)

            decoder_output = self.decoder_dropout(dec_output, training=training)

            user_attn_out = self.attn_layer(query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), 
                                          value=user_enc_out, key=user_enc_out, training=training)
            prev_system_attn_out = self.attn_layer(query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), 
                                                 value=prev_system_enc_out, key=prev_system_enc_out, training=training)
            decoder_attn_out = self.attn_layer(query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), 
                                             value=decoder_output, key=decoder_output, training=training)

            combined_features = layers.Concatenate()([
                tf.squeeze(user_attn_out, 1),
                tf.squeeze(prev_system_attn_out, 1),
                tf.squeeze(decoder_attn_out, 1),
                domain_onehot_input,
                turn_id_embedding,
                ontology_multihot_input
            ])
            combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
            nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
            nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
            intent_out = self.intent_output(nlu_out)
            domain_out = self.domain_output(nlu_out)
            intent_out_expanded = tf.expand_dims(intent_out, axis=1) 
            domain_out_expanded = tf.expand_dims(domain_out, axis=1) 
            intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1]) 
            domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])
            nlg_input = layers.Concatenate(axis=-1)([
                decoder_output, 
                intent_out_tiled,  
                domain_out_tiled  
            ])
            nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])
            nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
            nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
            response_out = self.response_output(nlg_out)

            return {
                'intent_output': intent_out,
                'domain_output': domain_out,
                'response_output': response_out,
                'response_embeddings': decoder_output,
                'nlu_gate_weights': nlu_gate_weights,
                'nlg_gate_weights': nlg_gate_weights
            }

        def build_graph(self):
            inputs = {
                'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
                'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
                'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
                'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32),
                'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32),
                'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32),
            }
            return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

        def get_config(self):
            config = super().get_config()
            config.update({
                'moe_params': {
                    'embedding_dim': self.embedding_dim,
                    'max_seq_length': self.max_seq_length,
                    'num_domains': self.num_domains,
                    'num_intents': self.num_intents,
                    'vocab_size': self.vocab_size,
                    'num_experts': self.num_experts,
                    'expert_dim': self.expert_dim,
                    'turn_id_dim': self.turn_id_dim
                }
            })
            return config
    
    class IntentAccuracy(metrics.Metric):
        def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros')
            self.total_preds = self.add_weight(name='total_preds', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
            self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
            self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

        def reset_state(self):
            self.correct_preds.assign(0.)
            self.total_preds.assign(0.)

    class IntentPrecision(metrics.Metric):
        def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros')
            self.predicted_positives = self.add_weight(name='pp', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            predicted_positives = tf.reduce_sum(y_pred)
            self.true_positives.assign_add(true_positives)
            self.predicted_positives.assign_add(predicted_positives)

        def result(self):
            return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.predicted_positives.assign(0.)

    class IntentRecall(metrics.Metric):
        def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros')
            self.actual_positives = self.add_weight(name='ap', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            actual_positives = tf.reduce_sum(y_true)
            self.true_positives.assign_add(true_positives)
            self.actual_positives.assign_add(actual_positives)

        def result(self):
            return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.actual_positives.assign(0.)

    class IntentF1(metrics.Metric):
        def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.precision = IntentPrecision(threshold=threshold)
            self.recall = IntentRecall(threshold=threshold)

        def update_state(self, y_true, y_pred, sample_weight=None):
            self.precision.update_state(y_true, y_pred, sample_weight)
            self.recall.update_state(y_true, y_pred, sample_weight)

        def result(self):
            p = self.precision.result()
            r = self.recall.result()
            return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

        def reset_state(self):
            self.precision.reset_state()
            self.recall.reset_state()

    class DomainAccuracy(metrics.Metric):
        def __init__(self, name='domain_accuracy', **kwargs):
            super().__init__(name=name, **kwargs)
            self.accuracy = self.add_weight(name='acc', initializer='zeros')
            self.count = self.add_weight(name='count', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
            pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
            matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
            self.accuracy.assign_add(tf.reduce_sum(matches))
            self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.accuracy / (self.count + tf.keras.backend.epsilon())

        def reset_state(self):
            self.accuracy.assign(0.)
            self.count.assign(0.)

    class Perplexity(metrics.Metric):
        def __init__(self, name='perplexity', **kwargs):
            super().__init__(name=name, **kwargs)
            self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
            self.total_loss = self.add_weight(name='total_loss', initializer='zeros')
            self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            mask = tf.cast(y_true != 0, dtype=tf.float32)
            loss = self.cross_entropy(y_true, y_pred)
            masked_loss = loss * mask
            sum_loss = tf.reduce_sum(masked_loss)
            num_non_padding_tokens = tf.reduce_sum(mask)
            self.total_loss.assign_add(sum_loss)
            self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

        def result(self):
            avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
            return tf.exp(avg_loss)

        def reset_state(self):
            self.total_loss.assign(0.)
            self.total_non_padding_tokens.assign(0.)

    class ResponseEmbeddingCosineSimilarity(metrics.Metric):
        def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
            super().__init__(name=name)
            self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
            self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            epsilon = tf.keras.backend.epsilon()
            y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
            y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
            y_true_norm = y_true / (y_true_norm_val + epsilon)
            y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
            cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
            non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
            non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
            masked_cosine_sim = cosine_sim_per_token * non_padding_mask
            num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
            avg_cosine_sim_per_sample = tf.where(
                num_non_zero_tokens > 0,
                tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
                tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
            )
            self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
            self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

        def reset_state(self):
            self.total_cosine_sim.assign(0.)
            self.num_samples.assign(0.)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    def calculate_flops(model, batch_size=moe_params["batch_size"]):
        flops = 0
        functional_model = model.build_graph()

        def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
            layer_flops_count = 0

            if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
                return 0

            if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
                for sub_layer in layer_instance.layers:
                    layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
                return layer_flops_count

            if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
                return 0

            if isinstance(layer_instance, layers.Dense):
                input_dim = layer_instance.input_shape[-1]
                output_dim = layer_instance.output_shape[-1]
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                layer_flops_count += 2 * input_dim * output_dim * effective_batch_size

            elif isinstance(layer_instance, layers.LSTM):
                input_dim = layer_instance.input_shape[-1]
                hidden_dim = layer_instance.units
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
                layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size

            elif isinstance(layer_instance, layers.Embedding):
                pass

            elif isinstance(layer_instance, MoELayer):
                moe_input_dim = layer_instance.input_dim
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
                layer_flops_count += gate_flops
                for expert in layer_instance.experts:
                    layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)

            elif isinstance(layer_instance, layers.MultiHeadAttention):
                config = layer_instance.get_config()
                num_heads = config['num_heads']
                key_dim = config['key_dim']
                d_model = num_heads * key_dim
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
                projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
                attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
                context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
                output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
                layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops

            elif isinstance(layer_instance, layers.TimeDistributed):
                inner_layer = layer_instance.layer
                td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
                layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)

            elif isinstance(layer_instance, TransformerDecoderLayer):
                layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
                layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
                ffn_effective_batch_size = current_batch_size * model.max_seq_length
                layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)

            return layer_flops_count

        for layer in functional_model.layers:
            flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)

        return flops / 1e9

    def get_gpu_stats():
        if nvidia_smi is None:
            return 0, 0
        try:
            nvidia_smi.nvmlInit()
            handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
            gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
            mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
            gpu_memory = mem_info.used / (1024 ** 3)
            nvidia_smi.nvmlShutdown()
            return gpu_load, gpu_memory
        except Exception:
            return 0, 0

    class ResourceMonitor(keras.callbacks.Callback):
        def __init__(self, validation_data):
            super().__init__()
            self.resource_stats = []
            self.expert_load_history = []
            self.start_time = None
            self.epoch_start_time = None
            self.flops = None
            self.validation_data = validation_data
            self.nlu_expert_usage_counts = None
            self.nlg_expert_usage_counts = None
            self.nlu_gate_weights_history = []
            self.nlg_gate_weights_history = []

        def on_train_begin(self, logs=None):
            self.start_time = time.time()
            try:
                self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
                self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
                self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
            except Exception:
                self.flops = 0

        def on_epoch_begin(self, epoch, logs=None):
            self.epoch_start_time = time.time()

        def on_epoch_end(self, epoch, logs=None):
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            epoch_time = time.time() - self.epoch_start_time
            self.resource_stats.append({
                'epoch': epoch + 1,
                'epoch_time': epoch_time,
                'cpu_usage': cpu_usage,
                'memory_used': memory.used / (1024 ** 3),
                'memory_total': memory.total / (1024 ** 3),
                'gpu_load': gpu_load,
                'gpu_memory': gpu_memory,
                'loss': logs.get('loss', 0),
                'val_loss': logs.get('val_loss', 0),
                'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
                'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
                'intent_precision': logs.get('intent_output_intent_precision', 0),
                'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
                'intent_recall': logs.get('intent_output_intent_recall', 0),
                'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
                'intent_f1': logs.get('intent_output_intent_f1', 0),
                'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
                'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
                'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
                'perplexity': logs.get('response_output_perplexity', 0),
                'val_perplexity': logs.get('val_response_output_perplexity', 0),
                'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
                'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
                'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
                'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
            })
            if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
                self.expert_load_history.append({
                    'epoch': epoch + 1,
                    'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                    'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
                })

            sample_size = 100
            for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
                inputs, _ = batch
                outputs = self.model(inputs, training=False)
                nlu_gate_weights = outputs['nlu_gate_weights']
                reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
                self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
                nlg_gate_weights = outputs['nlg_gate_weights']
                reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
                self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

        def on_train_end(self, logs=None):
            self.total_time = time.time() - self.start_time

            if len(self.nlu_gate_weights_history) > 0:
                all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
                nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
                for expert_id in nlu_top_indices:
                    if expert_id < self.model.moe_nlu.total_experts:
                        self.nlu_expert_usage_counts[expert_id] += 1

            if len(self.nlg_gate_weights_history) > 0:
                all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
                nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
                for expert_id in nlg_top_indices:
                    if expert_id < self.model.moe_nlg.total_experts:
                        self.nlg_expert_usage_counts[expert_id] += 1

        def visualize_expert_load_distribution(self):
            total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
            total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
            nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
            nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
            nlu_stability = np.var(nlu_percentages)
            nlg_stability = np.var(nlg_percentages)
            return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

    class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
        def __init__(self, d_model, warmup_steps=4000):
            super().__init__()
            self.d_model = tf.cast(d_model, tf.float32)
            self.warmup_steps = warmup_steps

        def __call__(self, step):
            step = tf.cast(step, tf.float32)
            arg1 = tf.math.rsqrt(step)
            arg2 = step * (self.warmup_steps ** -1.5)
            return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

        def get_config(self):
            return {"d_model": self.d_model.numpy(), "warmup_steps": self.warmup_steps}

    model = MoEModel(moe_params)
    embedding_layer = model.embedding

    def create_validation_split(dataset, embedding_layer, validation_split=0.2, batch_size=moe_params["batch_size"]):
        element_spec = dataset.element_spec
        data_list = [(features, targets) for features, targets in dataset.unbatch().as_numpy_iterator()]
        if not data_list:
            raise ValueError("Dataset is empty.")
        if len(data_list) < 2:
            raise ValueError("Dataset too small to split.")
        train_data, val_data = train_test_split(data_list, test_size=validation_split, shuffle=True)

        def create_dataset_from_list(data):
            if not data:
                raise ValueError("Empty data list provided.")
            features = {
                'user_utterance_tokens': np.array([d[0]['user_utterance_tokens'] for d in data], dtype=np.int32),
                'prev_system_response_tokens': np.array([d[0]['prev_system_response_tokens'] for d in data], dtype=np.int32),
                'decoder_input_tokens': np.array([d[0]['decoder_input_tokens'] for d in data], dtype=np.int32),
                'domain_onehot_input': np.array([d[0]['domain_onehot_input'] for d in data], dtype=np.float32),
                'turn_id_embedding': np.array([d[0]['turn_id_embedding'] for d in data], dtype=np.float32),
                'ontology_multihot_input': np.array([d[0]['ontology_multihot_input'] for d in data], dtype=np.float32),
            }
            response_output = np.array([d[1]['response_output'] for d in data], dtype=np.int32)
            response_embeddings = embedding_layer(response_output).numpy()
            targets = {
                'domain_output': np.array([d[1]['domain_output'] for d in data], dtype=np.int32),
                'intent_output': np.array([d[1]['intent_output'] for d in data], dtype=np.float32),
                'response_output': response_output,
                'response_embeddings': response_embeddings
            }
            return tf.data.Dataset.from_tensor_slices((features, targets))

        train_dataset = create_dataset_from_list(train_data).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        val_dataset = create_dataset_from_list(val_data).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return train_dataset, val_dataset

    train_tf_dataset, val_tf_dataset = create_validation_split(train_tf_dataset, embedding_layer)

    losses_dict = {
        'intent_output': losses.BinaryCrossentropy(),
        'domain_output': losses.SparseCategoricalCrossentropy(),
        'response_output': losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss
    }

    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }

    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={'intent_output': 0.5, 'domain_output': 0.5, 'response_output': 1.0, 'response_embeddings': 1.0}
    )

    sample_input = next(iter(train_tf_dataset.take(1)))
    model(sample_input[0])

    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=8, mode='min', restore_best_weights=True)
    resource_monitor = ResourceMonitor(val_tf_dataset)
    class TQDMProgressBar(callbacks.Callback):
        def on_epoch_begin(self, epoch, logs=None):
            self.progress_bar = tqdm(total=len(train_tf_dataset), desc=f'Epoch {epoch + 1}')

        def on_batch_end(self, batch, logs=None):
            self.progress_bar.update(1)

        def on_epoch_end(self, epoch, logs=None):
            self.progress_bar.close()

    tqdm_callback = TQDMProgressBar()

    history = model.fit(
        train_tf_dataset,
        epochs=100,
        validation_data=val_tf_dataset,
        callbacks=[early_stopping, resource_monitor, tqdm_callback],
        verbose=0
    )

    expert_stats = resource_monitor.visualize_expert_load_distribution()
    stats = resource_monitor.resource_stats
    avg_epoch_duration = np.mean([s['epoch_time'] for s in stats]) if stats else 0
    total_training_time = resource_monitor.total_time
    avg_cpu_usage = np.mean([s['cpu_usage'] for s in stats]) if stats else 0
    avg_memory = np.mean([s['memory_used'] for s in stats]) if stats else 0
    avg_gpu_load = np.mean([s['gpu_load'] for s in stats]) if stats else 0
    avg_gpu_memory = np.mean([s['gpu_memory'] for s in stats]) if stats else 0
    avg_flops = resource_monitor.flops if resource_monitor.flops else 0
    nlu_counts = resource_monitor.nlu_expert_usage_counts
    nlg_counts = resource_monitor.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) or 1
    total_nlg = np.sum(nlg_counts) or 1
    expert_nlu_percentages = [(nlu_counts[i] / total_nlu) * 100 for i in range(8)]
    expert_nlg_percentages = [(nlg_counts[i] / total_nlg) * 100 for i in range(8)]
    val_intent_accuracy = np.mean([s['val_intent_accuracy'] for s in stats if s['val_intent_accuracy'] > 0]) if stats else 0
    val_entity_accuracy = np.mean([s['val_domain_accuracy'] for s in stats if s['val_domain_accuracy'] > 0]) if stats else 0
    val_perplexity = np.mean([s['val_perplexity'] for s in stats if s['val_perplexity'] > 0]) if stats else 0
    val_cosine_similarity = np.mean([s['val_cosine_similarity'] for s in stats if s['val_cosine_similarity'] > 0]) if stats and any(s['val_cosine_similarity'] > 0 for s in stats) else 0
    
    results_table['Average Validation Response Cosine Similarity'][f'Iteration {iteration + 1}'] = val_cosine_similarity
    results_table['Training Speed (epochs per second)'][f'Iteration {iteration + 1}'] = 1 / avg_epoch_duration if avg_epoch_duration > 0 else 0
    results_table['Training Time (second/epoch)'][f'Iteration {iteration + 1}'] = avg_epoch_duration
    results_table['Total Training Time (second/iteration)'][f'Iteration {iteration + 1}'] = total_training_time
    results_table['Computational Resource Usage'][f'Iteration {iteration + 1}'] = avg_cpu_usage + avg_gpu_load
    results_table['Average CPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_cpu_usage
    results_table['Average GPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_gpu_load
    results_table['Average Memory (GB)'][f'Iteration {iteration + 1}'] = avg_memory
    results_table['Average GPU Memory (GB)'][f'Iteration {iteration + 1}'] = avg_gpu_memory
    results_table['Average FLOPS Estimate (GFLOPS)'][f'Iteration {iteration + 1}'] = avg_flops
    results_table['Expert NLU Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlu_stability']
    results_table['Expert NLG Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlg_stability']
    for i in range(8):
        if i < len(expert_stats['nlu_percentages']):
            results_table[f'Average NLU Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlu_percentages'][i]
        if i < len(expert_stats['nlg_percentages']):
            results_table[f'Average NLG Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlg_percentages'][i]
    results_table['Average Validation Entity Accuracy'][f'Iteration {iteration + 1}'] = val_entity_accuracy
    results_table['Average Intent Accuracy'][f'Iteration {iteration + 1}'] = val_intent_accuracy
    results_table['Average Validation Perplexity'][f'Iteration {iteration + 1}'] = val_perplexity
    results_table['Average Validation Response Cosine Similarity'][f'Iteration {iteration + 1}'] = val_cosine_similarity
    val_loss = min([s['val_loss'] for s in stats]) if stats else float('inf')
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        try:
            model.save(best_model_path, save_format='tf', include_optimizer=True)
        except Exception as e:
            print(f"\nFailed to save best model to {best_model_path}: {str(e)}")

for key in results_table:
    results_table[key]['Average'] = np.mean([results_table[key][f'Iteration {i+1}'] for i in range(10)])

final_result = pd.DataFrame(results_table)
final_result = final_result.T
final_result.to_excel('training_Deepseekbaseline_result.xlsx', index=True)
final_result

2025-07-04 13:07:30.028772: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 13:07:30.028908: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 13:07:30.103505: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 13:07:30.245306: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-04 13:07:31.526568: W tensorflow/compiler/tf2


Starting Training Iteration 1

Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 13:07:34.914445: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 13:07:35.152078: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 13:07:35.152322: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl


Epoch 1:   0%|          | 0/4428 [00:00<?, ?it/s]2025-07-04 13:08:03.625390: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f1a244cf020 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-04 13:08:03.625452: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2025-07-04 13:08:03.646323: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-04 13:08:03.691190: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1751634483.811700     680 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
Epoch 1: 100%|██████████| 4428/4428 [03:02<00:00, 24.31it/s]


INFO:tensorflow:Assets written to: best_Deepseekbaseline_moe_model/assets


INFO:tensorflow:Assets written to: best_Deepseekbaseline_moe_model/assets


,Iteration 1,Iteration 2,Iteration 3,Iteration 4,Iteration 5,Iteration 6,Iteration 7,Iteration 8,Iteration 9,Iteration 10,Average
Training Speed (epochs per second),0.005666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000567
Training Time (second/epoch),176.480350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.648035
Total Training Time (second/iteration),183.451998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.345200
Computational Resource Usage,64.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.410000
Average CPU Usage (percent),25.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.510000
Average GPU Usage (percent),39.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.900000
Average Memory (GB),7.495152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.749515
Average GPU Memory (GB),14.531799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.453180
Average FLOPS Estimate (GFLOPS),2.377917,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.237792
Expert NLU Load Distribution Stability,207.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.725000


# DeepSeekMoE baseline evaluation code

In [1]:
def load_tf_datasets_from_disk(load_path):
    print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
    try:
        with open(os.path.join(load_path, "moe_params.json"), "r") as f:
            loaded_moe_params = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"moe_params.json not found in {load_path}")

    element_spec = (
        {
            'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
            'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
            'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
        },
        {
            'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
            'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
        }
    )

    try:
        train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
        test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
    except Exception as e:
        raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

    raw_data = {}
    for dataset_name in ['train', 'test']:
        path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
        if os.path.exists(path):
            with open(path, 'rb') as f:
                raw_data[dataset_name] = pickle.load(f)
            print(f"Loaded raw data for {dataset_name} from {path}")
        else:
            print(f"Warning: Raw data file {path} not found.")
            raw_data[dataset_name] = []

    return {
        "train_dataset": train_tf_dataset,
        "test_dataset": test_tf_dataset,
        "moe_params": loaded_moe_params,
        "raw_data": raw_data
    }

tf_dataset_save_path = "tf_datasets"
loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
train_tf_dataset = loaded_data["train_dataset"]
test_tf_dataset = loaded_data["test_dataset"]
moe_params = loaded_data["moe_params"]
raw_data = loaded_data["raw_data"]
moe_params["num_experts"] = 4

class MoELayer(layers.Layer):
    def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.01, top_k=2, name=None):
        super(MoELayer, self).__init__(name=name)
        self.m = m
        self.k_s = k_s
        self.total_experts = num_experts * self.m
        self.routed_experts = self.total_experts - self.k_s
        self.top_k = top_k
        self.top_mk = self.m * self.top_k
        self.top_mk = min(self.top_mk, self.routed_experts)
        self.alpha = alpha
        self.input_dim = input_dim
        self.seq_length = None
        self.adjusted_expert_dim = expert_dim // self.m
        self.shared_experts = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
            ]) for i in range(self.k_s)
        ]
        self.routed_experts_list = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
            ]) for i in range(self.k_s, self.total_experts)
        ]
        self.experts = self.shared_experts + self.routed_experts_list
        self.gate = layers.Dense(self.routed_experts, activation='softmax')
        self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

    def call(self, inputs, training=False):
        input_rank = inputs.shape.rank
        if input_rank == 3:
            batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
            flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
            is_3d = True
        else:
            batch_size = tf.shape(inputs)[0]
            seq_length = 1
            flat_inputs = inputs
            is_3d = False

        flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
        shared_output = tf.zeros([tf.shape(flat_inputs)[0], self.input_dim], dtype=tf.float32)
        
        for i in range(self.k_s):
            shared_output += self.shared_experts[i](flat_inputs)

        gate_weights = self.gate(flat_inputs)
        top_mk_values, top_mk_indices = tf.nn.top_k(gate_weights, k=self.top_mk, sorted=True)
        top_mk_indices = top_mk_indices + self.k_s
        top_mk_weights = top_mk_values / (tf.reduce_sum(top_mk_values, axis=-1, keepdims=True) + tf.keras.backend.epsilon())
        routed_output = tf.zeros([tf.shape(flat_inputs)[0], self.input_dim], dtype=tf.float32)
        
        for k in range(self.top_mk):
            kth_weights = top_mk_weights[:, k]
            kth_indices = top_mk_indices[:, k]
            mask = tf.one_hot(kth_indices, depth=self.total_experts, dtype=tf.float32)
            expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
            kth_expert_output = tf.reduce_sum(expert_outputs * tf.expand_dims(mask, -1), axis=1)
            weighted_kth_output = kth_expert_output * tf.expand_dims(kth_weights, -1)
            routed_output += weighted_kth_output
        output_flat = shared_output + routed_output + flat_inputs

        if is_3d:
            output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
            if self.seq_length is not None:
                output.set_shape([None, self.seq_length, self.input_dim])
            gate_weights_reshaped = tf.reshape(gate_weights, [batch_size, seq_length, self.routed_experts])
            if self.seq_length is not None:
                gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
        else:
            output = output_flat
            gate_weights_reshaped = gate_weights

        f_i = tf.reduce_mean(tf.reduce_sum(tf.one_hot(tf.argmax(gate_weights, axis=-1), depth=self.routed_experts), axis=0), axis=0)
        P_i = tf.reduce_mean(gate_weights, axis=0)
        load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
        self.add_loss(load_balancing_loss)
        self.load_balancing_loss_metric.update_state(load_balancing_loss)

        return output, gate_weights_reshaped

    def get_metrics(self):
        return {f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric}

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_experts': self.total_experts // self.m,
            'expert_dim': self.adjusted_expert_dim * self.m,
            'input_dim': self.input_dim,
            'm': self.m,
            'k_s': self.k_s,
            'alpha': self.alpha,
            'top_k': self.top_k,
            'name': self.name
        })
        return config

class TransformerDecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.ffn = keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask=None):
        if look_ahead_mask is not None:
            look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
            look_ahead_mask = 1.0 - look_ahead_mask

        attn1_output = self.mha1(query=x, value=x, key=x, attention_mask=look_ahead_mask, training=training)
        attn1 = self.dropout1(attn1_output, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2_output = self.mha2(query=out1, value=enc_output, key=enc_output, attention_mask=None, training=training)
        attn2 = self.dropout2(attn2_output, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3

    def get_config(self):
        config = super().get_config()
        mha1_config = self.mha1.get_config()
        config.update({
            'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
            'num_heads': mha1_config['num_heads'],
            'dff': self.ffn.layers[0].units,
            'rate': self.dropout1.rate
        })
        return config

class MoEModel(models.Model):
    def __init__(self, moe_params):
        super(MoEModel, self).__init__()
        self.embedding_dim = moe_params["embedding_dim"]
        self.max_seq_length = moe_params["max_seq_length"]
        self.num_domains = moe_params["num_domains"]
        self.num_intents = moe_params["num_intents"]
        self.vocab_size = moe_params["vocab_size"]
        self.num_experts = moe_params["num_experts"]
        self.expert_dim = moe_params["expert_dim"]
        self.turn_id_dim = moe_params["turn_id_dim"]
        self.num_heads = 4
        self.dff = self.embedding_dim * 4
        self.dropout_rate = 0.1
        self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                             self.num_domains + self.turn_id_dim + self.num_intents)
        self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains
        self.embedding = layers.Embedding(
            self.vocab_size,
            self.embedding_dim,
            mask_zero=True,
            embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05)
        )
        self.embedding.build((None,))
        with tf.init_scope():
            embedding_weights = self.embedding.get_weights()
            if embedding_weights and len(embedding_weights) > 0:
                embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                self.embedding.set_weights(embedding_weights)

        self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
        self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
        self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
        self.decoder_dropout = layers.Dropout(self.dropout_rate)
        self.moe_nlu = MoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlu_input_dim, m=2, k_s=1, alpha=0.01, top_k=2, name='moe_nlu')
        self.moe_nlg = MoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlg_input_dim, m=2, k_s=1, alpha=0.01, top_k=2, name='moe_nlg')
        self.intent_output = layers.Dense(self.num_intents, activation='sigmoid')
        self.domain_output = layers.Dense(self.num_domains, activation='softmax')
        self.response_output = layers.TimeDistributed(layers.Dense(self.vocab_size, activation='softmax'))
        self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)

    def create_look_ahead_mask(self, size):
        mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
        return mask

    def call(self, inputs, training=False):
        user_utterance_tokens = inputs.get('user_utterance_tokens')
        prev_system_response_tokens = inputs.get('prev_system_response_tokens')
        decoder_input_tokens = inputs.get('decoder_input_tokens')
        domain_onehot_input = inputs.get('domain_onehot_input')
        turn_id_embedding = inputs.get('turn_id_embedding')
        ontology_multihot_input = inputs.get('ontology_multihot_input')
        if any(x is None for x in [user_utterance_tokens, prev_system_response_tokens, decoder_input_tokens,
                                  domain_onehot_input, turn_id_embedding, ontology_multihot_input]):
            raise ValueError("One or more required inputs are missing or None")

        pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
        user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
        prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
        decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)

        user_enc_out = user_embedded
        prev_system_enc_out = prev_system_embedded
        look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
        dec_output = decoder_embedded
        
        for i, layer in enumerate(self.decoder_layers):
            dec_output = layer(dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask)

        decoder_output = self.decoder_dropout(dec_output, training=training)

        user_attn_out = self.attn_layer(query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), 
                                      value=user_enc_out, key=user_enc_out, training=training)
        prev_system_attn_out = self.attn_layer(query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), 
                                             value=prev_system_enc_out, key=prev_system_enc_out, training=training)
        decoder_attn_out = self.attn_layer(query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), 
                                         value=decoder_output, key=decoder_output, training=training)

        combined_features = layers.Concatenate()([
            tf.squeeze(user_attn_out, 1),
            tf.squeeze(prev_system_attn_out, 1),
            tf.squeeze(decoder_attn_out, 1),
            domain_onehot_input,
            turn_id_embedding,
            ontology_multihot_input
        ])
        combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
        nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
        nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
        intent_out = self.intent_output(nlu_out)
        domain_out = self.domain_output(nlu_out)
        intent_out_expanded = tf.expand_dims(intent_out, axis=1) 
        domain_out_expanded = tf.expand_dims(domain_out, axis=1) 
        intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1]) 
        domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])
        nlg_input = layers.Concatenate(axis=-1)([
            decoder_output, 
            intent_out_tiled,  
            domain_out_tiled  
        ])
        nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])
        nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
        nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
        response_out = self.response_output(nlg_out)

        return {
            'intent_output': intent_out,
            'domain_output': domain_out,
            'response_output': response_out,
            'response_embeddings': decoder_output,
            'nlu_gate_weights': nlu_gate_weights,
            'nlg_gate_weights': nlg_gate_weights
        }

    def build_graph(self):
        inputs = {
            'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
            'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
            'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
            'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32),
            'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32),
            'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32),
        }
        return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

    def get_config(self):
        config = super().get_config()
        config.update({
            'moe_params': {
                'embedding_dim': self.embedding_dim,
                'max_seq_length': self.max_seq_length,
                'num_domains': self.num_domains,
                'num_intents': self.num_intents,
                'vocab_size': self.vocab_size,
                'num_experts': self.num_experts,
                'expert_dim': self.expert_dim,
                'turn_id_dim': self.turn_id_dim
            }
        })
        return config

class IntentAccuracy(metrics.Metric):
    def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros')
        self.total_preds = self.add_weight(name='total_preds', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
        self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
        self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

    def reset_state(self):
        self.correct_preds.assign(0.)
        self.total_preds.assign(0.)

class IntentPrecision(metrics.Metric):
    def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.predicted_positives = self.add_weight(name='pp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        predicted_positives = tf.reduce_sum(y_pred)
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)

    def result(self):
        return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.predicted_positives.assign(0.)

class IntentRecall(metrics.Metric):
    def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.actual_positives = self.add_weight(name='ap', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        actual_positives = tf.reduce_sum(y_true)
        self.true_positives.assign_add(true_positives)
        self.actual_positives.assign_add(actual_positives)

    def result(self):
        return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.actual_positives.assign(0.)

class IntentF1(metrics.Metric):
    def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = IntentPrecision(threshold=threshold)
        self.recall = IntentRecall(threshold=threshold)

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

class DomainAccuracy(metrics.Metric):
    def __init__(self, name='domain_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.accuracy = self.add_weight(name='acc', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
        pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
        matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
        self.accuracy.assign_add(tf.reduce_sum(matches))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.accuracy / (self.count + tf.keras.backend.epsilon())

    def reset_state(self):
        self.accuracy.assign(0.)
        self.count.assign(0.)

class Perplexity(metrics.Metric):
    def __init__(self, name='perplexity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
        self.total_loss = self.add_weight(name='total_loss', initializer='zeros')
        self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        mask = tf.cast(y_true != 0, dtype=tf.float32)
        loss = self.cross_entropy(y_true, y_pred)
        masked_loss = loss * mask
        sum_loss = tf.reduce_sum(masked_loss)
        num_non_padding_tokens = tf.reduce_sum(mask)
        self.total_loss.assign_add(sum_loss)
        self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

    def result(self):
        avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
        return tf.exp(avg_loss)

    def reset_state(self):
        self.total_loss.assign(0.)
        self.total_non_padding_tokens.assign(0.)

class ResponseEmbeddingCosineSimilarity(metrics.Metric):
    def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
        super().__init__(name=name)
        self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
        self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true_norm = y_true / (y_true_norm_val + epsilon)
        y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
        cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
        non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
        non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
        masked_cosine_sim = cosine_sim_per_token * non_padding_mask
        num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
        avg_cosine_sim_per_sample = tf.where(
            num_non_zero_tokens > 0,
            tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
            tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
        )
        self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
        self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

    def reset_state(self):
        self.total_cosine_sim.assign(0.)
        self.num_samples.assign(0.)

def contrastive_loss(y_true, y_pred, margin=1.0):
    epsilon = tf.keras.backend.epsilon()
    y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
    y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
    y_true = y_true / (y_true_norm + epsilon)
    y_pred = y_pred / (y_pred_norm + epsilon)
    cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
    positive_loss = 1.0 - cosine_sim
    mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
    masked_loss = positive_loss * mask
    total_loss = tf.reduce_sum(masked_loss)
    num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
    return total_loss / num_tokens

def calculate_flops(model, batch_size=moe_params["batch_size"]):
    flops = 0
    functional_model = model.build_graph()

    def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
        layer_flops_count = 0

        if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
            return 0

        if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
            for sub_layer in layer_instance.layers:
                layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
            return layer_flops_count

        if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
            return 0

        if isinstance(layer_instance, layers.Dense):
            input_dim = layer_instance.input_shape[-1]
            output_dim = layer_instance.output_shape[-1]
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            layer_flops_count += 2 * input_dim * output_dim * effective_batch_size

        elif isinstance(layer_instance, layers.LSTM):
            input_dim = layer_instance.input_shape[-1]
            hidden_dim = layer_instance.units
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
            layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size

        elif isinstance(layer_instance, layers.Embedding):
            pass

        elif isinstance(layer_instance, MoELayer):
            moe_input_dim = layer_instance.input_dim
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
            layer_flops_count += gate_flops
            for expert in layer_instance.experts:
                layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)

        elif isinstance(layer_instance, layers.MultiHeadAttention):
            config = layer_instance.get_config()
            num_heads = config['num_heads']
            key_dim = config['key_dim']
            d_model = num_heads * key_dim
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
            projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
            attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
            context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
            output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
            layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops

        elif isinstance(layer_instance, layers.TimeDistributed):
            inner_layer = layer_instance.layer
            td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
            layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)

        elif isinstance(layer_instance, TransformerDecoderLayer):
            layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
            layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
            ffn_effective_batch_size = current_batch_size * model.max_seq_length
            layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)

        return layer_flops_count

    for layer in functional_model.layers:
        flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)

    return flops / 1e9

def get_gpu_stats():
    if nvidia_smi is None:
        return 0, 0
    try:
        nvidia_smi.nvmlInit()
        handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
        gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
        mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
        gpu_memory = mem_info.used / (1024 ** 3)
        nvidia_smi.nvmlShutdown()
        return gpu_load, gpu_memory
    except Exception:
        return 0, 0

class ResourceMonitor(keras.callbacks.Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.resource_stats = []
        self.expert_load_history = []
        self.start_time = None
        self.epoch_start_time = None
        self.flops = None
        self.validation_data = validation_data
        self.nlu_expert_usage_counts = None
        self.nlg_expert_usage_counts = None
        self.nlu_gate_weights_history = []
        self.nlg_gate_weights_history = []

    def on_train_begin(self, logs=None):
        self.start_time = time.time()
        try:
            self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
            self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
            self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
        except Exception:
            self.flops = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory()
        gpu_load, gpu_memory = get_gpu_stats()
        epoch_time = time.time() - self.epoch_start_time
        self.resource_stats.append({
            'epoch': epoch + 1,
            'epoch_time': epoch_time,
            'cpu_usage': cpu_usage,
            'memory_used': memory.used / (1024 ** 3),
            'memory_total': memory.total / (1024 ** 3),
            'gpu_load': gpu_load,
            'gpu_memory': gpu_memory,
            'loss': logs.get('loss', 0),
            'val_loss': logs.get('val_loss', 0),
            'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
            'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
            'intent_precision': logs.get('intent_output_intent_precision', 0),
            'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
            'intent_recall': logs.get('intent_output_intent_recall', 0),
            'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
            'intent_f1': logs.get('intent_output_intent_f1', 0),
            'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
            'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
            'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
            'perplexity': logs.get('response_output_perplexity', 0),
            'val_perplexity': logs.get('val_response_output_perplexity', 0),
            'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
            'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
            'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
            'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
        })
        if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
            self.expert_load_history.append({
                'epoch': epoch + 1,
                'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
            })

        sample_size = 100
        for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
            inputs, _ = batch
            outputs = self.model(inputs, training=False)
            nlu_gate_weights = outputs['nlu_gate_weights']
            reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
            self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
            nlg_gate_weights = outputs['nlg_gate_weights']
            reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
            self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

    def on_train_end(self, logs=None):
        self.total_time = time.time() - self.start_time

        if len(self.nlu_gate_weights_history) > 0:
            all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
            nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
            for expert_id in nlu_top_indices:
                if expert_id < self.model.moe_nlu.total_experts:
                    self.nlu_expert_usage_counts[expert_id] += 1

        if len(self.nlg_gate_weights_history) > 0:
            all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
            nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
            for expert_id in nlg_top_indices:
                if expert_id < self.model.moe_nlg.total_experts:
                    self.nlg_expert_usage_counts[expert_id] += 1

    def visualize_expert_load_distribution(self):
        total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
        total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
        nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
        nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
        nlu_stability = np.var(nlu_percentages)
        nlg_stability = np.var(nlg_percentages)
        return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {"d_model": self.d_model.numpy(), "warmup_steps": self.warmup_steps}

word_to_id = {}
id_to_word = {}
try:
    possible_paths = [
        os.path.join("preprocessor_models", "preprocessor_params.json"),
        os.path.join("tf_datasets", "preprocessor_params.json"),
        "preprocessor_params.json"
    ]
    
    vocab_loaded = False
    for path in possible_paths:
        if os.path.exists(path):
            with open(path, "r") as f:
                preprocessor_params = json.load(f)
            word_to_id = {k: int(v) for k, v in preprocessor_params['word_to_id'].items()}
            id_to_word = {int(k): v for k, v in preprocessor_params['id_to_word'].items()}
            print(f"Loaded vocabulary from {path}")
            vocab_loaded = True
            break
    
    if not vocab_loaded:
        raise FileNotFoundError("Vocabulary file not found in any location")
except Exception as e:
    print(f"Warning: Could not load vocabulary from preprocessor: {str(e)}")
    word_to_id = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
    id_to_word = {0: '<pad>', 1: '<sos>', 2: '<eos>', 3: '<unk>'}


model_save_path = "best_deepseekbaseline_moe_model"
print(f"\nLoading model from: {model_save_path}")
custom_objects = {
    "MoEModel":MoEModel,
    "CustomLearningRateSchedule": CustomLearningRateSchedule,
    "MoELayer": MoELayer,
    "TransformerDecoderLayer": TransformerDecoderLayer,
    "IntentAccuracy": IntentAccuracy,
    "IntentPrecision": IntentPrecision,
    "IntentRecall": IntentRecall,
    "IntentF1": IntentF1,
    "DomainAccuracy": DomainAccuracy,
    "Perplexity": Perplexity,
    "ResponseEmbeddingCosineSimilarity": ResponseEmbeddingCosineSimilarity
}
try:
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
except Exception as e:
    print(f"Failed to load model from {model_save_path}: {str(e)}")
    raise

def compile_model(model, moe_params):
    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    losses_dict = {
        'intent_output': tf.keras.losses.BinaryCrossentropy(),
        'domain_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss,
    }

    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }

    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={
            'intent_output': 0.5,
            'domain_output': 0.5,
            'response_output': 1.0,
            'response_embeddings': 1.0
        }
    )
    return model

model = compile_model(model, moe_params)
print("\nModel loaded and compiled successfully!")
sample_input = next(iter(train_tf_dataset.take(1)))
model(sample_input[0])
model.summary()

class EvaluationMetrics:
    def __init__(self, model, test_dataset, moe_params, raw_data, word_to_id, id_to_word):
        self.model = model
        self.test_dataset = test_dataset
        self.moe_params = moe_params
        self.raw_data = raw_data
        self.word_to_id = word_to_id
        self.id_to_word = id_to_word
        self.nlu_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.nlg_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.resource_stats = {
            'cpu_usage': [],
            'memory_used': [],
            'gpu_load': [],
            'gpu_memory': [],
            'batch_times': []
        }
        self.special_tokens = {'<pad>', '<sos>', '<eos>', '<unk>'}
        self.nlu_top_k = model.get_layer('moe_nlu').get_config().get('top_k', 2)
        self.nlg_top_k = model.get_layer('moe_nlg').get_config().get('top_k', 2)

    def evaluate(self):
        print("\nEvaluating model on test set...")
        
        def add_response_embeddings(features, targets):
            response_tokens = targets['response_output']
            response_embeddings = self.model.embedding(response_tokens)
            targets['response_embeddings'] = response_embeddings
            return features, targets
        
        test_dataset_with_embeddings = self.test_dataset.map(add_response_embeddings, num_parallel_calls=tf.data.AUTOTUNE)

        start_time = time.time()
        test_results = self.model.evaluate(test_dataset_with_embeddings, verbose=1)
        eval_time = time.time() - start_time
        
        metric_names = self.model.metrics_names
        
        perplexity_value = None
        for name, value in zip(metric_names, test_results):
            if name == 'response_output_perplexity':
                perplexity_value = value
                break
        if perplexity_value is None:
            print("Warning: Could not find perplexity in standard evaluation results.")
            perplexity_value = 0.0
        
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory().used / (1024 ** 3)
        gpu_load, gpu_memory = get_gpu_stats()
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))

        total_tokens = 0
        total_time = 0
        num_batches = 0
        
        test_dataset_small = test_dataset_with_embeddings.unbatch().batch(moe_params["batch_size"])
        
        all_true_intents = []
        all_pred_intents = []
        all_true_domains = []
        all_pred_domains = []
        
        for batch_idx, batch in enumerate(test_dataset_small):
            inputs, targets = batch
            response_tokens = targets['response_output'].numpy()
            non_padding_mask = response_tokens != 0
            total_tokens += np.sum(non_padding_mask)
            
            start_time = time.time()
            outputs = self.model(inputs, training=False)
            batch_time = time.time() - start_time
            
            total_time += batch_time
            num_batches += tf.shape(inputs['user_utterance_tokens'])[0]
            
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            
            self.resource_stats['cpu_usage'].append(cpu_usage)
            self.resource_stats['memory_used'].append(memory.used / (1024 ** 3))
            self.resource_stats['gpu_load'].append(gpu_load)
            self.resource_stats['gpu_memory'].append(gpu_memory)
            self.resource_stats['batch_times'].append(batch_time)
            
            nlu_gate_weights = outputs.get('nlu_gate_weights', tf.zeros([0, self.model.moe_nlu.routed_experts])).numpy()
            nlu_top_indices = np.argsort(-nlu_gate_weights, axis=-1)[:, :self.nlu_top_k] + self.model.moe_nlu.k_s
            for idx in range(self.nlu_top_k):
                expert_ids = nlu_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlu_expert_usage_counts):
                        self.nlu_expert_usage_counts[expert_id] += 1
            
            nlg_gate_weights = outputs.get('nlg_gate_weights', tf.zeros([0, self.model.moe_nlg.routed_experts])).numpy()
            if len(nlg_gate_weights.shape) == 3:
                nlg_gate_weights = nlg_gate_weights.reshape(-1, nlg_gate_weights.shape[-1])
            nlg_top_indices = np.argsort(-nlg_gate_weights, axis=-1)[:, :self.nlg_top_k] + self.model.moe_nlg.k_s
            for idx in range(self.nlg_top_k):
                expert_ids = nlg_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlg_expert_usage_counts):
                        self.nlg_expert_usage_counts[expert_id] += 1
            
            true_intents = targets.get('intent_output', np.zeros((moe_params["batch_size"], self.model.num_intents))).numpy()
            pred_intents = (outputs.get('intent_output', np.zeros_like(true_intents)).numpy() > 0.5).astype(np.int32)
            all_true_intents.append(true_intents)
            all_pred_intents.append(pred_intents)
            
            true_domains = targets.get('domain_output', np.zeros((moe_params["batch_size"], 1))).numpy().flatten()
            pred_domains = np.argmax(outputs.get('domain_output', np.zeros((moe_params["batch_size"], self.model.num_domains))).numpy(), axis=-1)
            all_true_domains.append(true_domains)
            all_pred_domains.append(pred_domains)
            
            if batch_idx % 50 == 0:
                gc.collect()
                tf.keras.backend.clear_session()
        
        avg_time_per_token = (total_time / total_tokens) * 1000 if total_tokens > 0 else 0
        
        avg_cpu = np.mean(self.resource_stats['cpu_usage']) if self.resource_stats['cpu_usage'] else 0
        avg_memory = np.mean(self.resource_stats['memory_used']) if self.resource_stats['memory_used'] else 0
        avg_gpu_load = np.mean(self.resource_stats['gpu_load']) if self.resource_stats['gpu_load'] else 0
        avg_gpu_memory = np.mean(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        peak_gpu_memory = np.max(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))
        
        all_true_intents = np.vstack(all_true_intents)
        all_pred_intents = np.vstack(all_pred_intents)
        all_true_domains = np.concatenate(all_true_domains)
        all_pred_domains = np.concatenate(all_pred_domains)
        
        intent_f1_score = f1_score(all_true_intents, all_pred_intents, average='micro')
        intent_f1_score_macro = f1_score(all_true_intents, all_pred_intents, average='macro')
        intent_f1_score_weighted = f1_score(all_true_intents, all_pred_intents, average='weighted')
        intent_accuracy_score = accuracy_score(all_true_intents, all_pred_intents)
        
        domain_accuracy_score = accuracy_score(all_true_domains, all_pred_domains)
        domain_f1_score_macro = f1_score(all_true_domains, all_pred_domains, average='macro')

        hypotheses = []
        references = []
        meteor_scores = []
        rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        
        domain_metrics = defaultdict(lambda: {
            'bleu': [],
            'rouge1': [],
            'rouge2': [],
            'rougeL': [],
            'meteor': [],
            'count': 0
        })
        
        if not self.raw_data.get('test'):
            print("Warning: raw_data['test'] is empty. Skipping NLG metrics calculation.")
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0
            domain_metrics = {}
        else:
            for i, batch in enumerate(test_dataset_small):
                if i >= len(self.raw_data.get('test', [])):
                    print(f"Warning: raw_data['test'] has fewer items ({len(self.raw_data.get('test', []))}) than test dataset. Stopping NLG metrics calculation.")
                    break
                
                try:
                    inputs, targets = batch
                    with tf.device('/CPU:0'):
                        outputs = self.model(inputs, training=False)
                    
                    domain_id = targets['domain_output'].numpy()[0][0]
                    domain_name = f"domain_{domain_id}"
                    
                    response_probs = outputs['response_output'].numpy()
                    generated_tokens = np.argmax(response_probs, axis=-1)[0]
                    true_tokens = targets['response_output'].numpy()[0]
                    
                    raw_item = self.raw_data['test'][i]
                    ref_text = raw_item.get('raw_next_system_response', '')
                    if not ref_text:
                        continue
                    
                    gen_text = self.tokens_to_text(generated_tokens)
                    ref_tokens = self.tokens_to_text(true_tokens)
                    
                    if not gen_text.strip() or not ref_text.strip():
                        continue
                    
                    hypotheses.append(gen_text)
                    references.append([ref_text])
                    
                    ref_tokens = word_tokenize(ref_text.lower())
                    gen_tokens = word_tokenize(gen_text.lower())
                    
                    try:
                        meteor = meteor_score([ref_tokens], gen_tokens)
                        meteor_scores.append(meteor)
                        domain_metrics[domain_name]['meteor'].append(meteor)
                    except Exception as e:
                        print(f"METEOR calculation error for example {i}: {str(e)}")
                        meteor_scores.append(0)
                        domain_metrics[domain_name]['meteor'].append(0)
                    
                    try:
                        rouge_scores = rouge_scorer_instance.score(ref_text, gen_text)
                        domain_metrics[domain_name]['rouge1'].append(rouge_scores['rouge1'].fmeasure)
                        domain_metrics[domain_name]['rouge2'].append(rouge_scores['rouge2'].fmeasure)
                        domain_metrics[domain_name]['rougeL'].append(rouge_scores['rougeL'].fmeasure)
                    except Exception as e:
                        print(f"ROUGE calculation error for example {i}: {str(e)}")
                    
                    domain_metrics[domain_name]['count'] += 1
                    
                    if i % 50 == 0:
                        gc.collect()
                        tf.keras.backend.clear_session()
                    
                except Exception as e:
                    print(f"Error processing example {i}: {str(e)}")
                    continue
        
        if hypotheses and references:
            hypotheses_tok = []
            references_tok = []
            
            for hyp, ref_list in zip(hypotheses, references):
                if not hyp or not ref_list[0]:
                    continue
                hyp_tokens = word_tokenize(hyp.lower())
                ref_tokens = [word_tokenize(ref.lower()) for ref in ref_list]
                hypotheses_tok.append(hyp_tokens)
                references_tok.append(ref_tokens)
            
            chencherry = SmoothingFunction()
            try:
                bleu_score = corpus_bleu(references_tok, hypotheses_tok, smoothing_function=chencherry.method1)
            except Exception as e:
                print(f"BLEU calculation error: {str(e)}")
                bleu_score = 0.0
            
            rouge_scores = []
            for hyp, ref in zip(hypotheses, references):
                if hyp and ref[0]:
                    try:
                        rouge_scores.append(rouge_scorer_instance.score(ref[0], hyp))
                    except Exception as e:
                        print(f"ROUGE scoring error: {str(e)}")
                        continue
            
            avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_meteor = np.mean(meteor_scores) if meteor_scores else 0.0
        else:
            print("Warning: No valid hypotheses or references for NLG metrics. Setting to 0.0.")
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0

        return {
            'intent_f1_micro': intent_f1_score,
            'intent_f1_macro': intent_f1_score_macro,
            'intent_f1_weighted': intent_f1_score_weighted,
            'intent_accuracy': intent_accuracy_score,
            'domain_accuracy': domain_accuracy_score,
            'domain_f1_macro': domain_f1_score_macro,
            'bleu': bleu_score,
            'rouge1': avg_rouge1,
            'rouge2': avg_rouge2,
            'rougeL': avg_rougeL,
            'meteor': avg_meteor,
            'perplexity': perplexity_value,
            'pred_speed': avg_time_per_token,
            'domain_metrics': domain_metrics,
            'eval_time': eval_time
        }
    
    def tokens_to_text(self, tokens):
        words = []
        for token in tokens:
            if token == 0:
                continue
            word = self.id_to_word.get(token, '<unk>')
            if word not in self.special_tokens:
                words.append(word)
        return " ".join(words).strip()

results_table = {
    'Evaluation Time (seconds)': [],
    'Prediction Speed (ms/token)': [],
    'Average CPU Usage (percent)': [],
    'Average GPU Usage (percent)': [],
    'Average Memory (GB)': [],
    'Average GPU Memory (GB)': [],
    'Average FLOPs Estimate (GFLOPs)': [],
    'NLU Expert 1 (percent)': [],
    'NLU Expert 2 (percent)': [],
    'NLU Expert 3 (percent)': [],
    'NLU Expert 4 (percent)': [],
    'NLU Expert 5 (percent)': [],
    'NLU Expert 6 (percent)': [],
    'NLU Expert 7 (percent)': [],
    'NLU Expert 8 (percent)': [],
    'NLG Expert 1 (percent)': [],
    'NLG Expert 2 (percent)': [],
    'NLG Expert 3 (percent)': [],
    'NLG Expert 4 (percent)': [],
    'NLG Expert 5 (percent)': [],
    'NLG Expert 6 (percent)': [],
    'NLG Expert 7 (percent)': [],
    'NLG Expert 8 (percent)': [],
    'Intent F1-score (Micro)': [],
    'Intent F1-score (Macro)': [],
    'Intent F1-score (Weighted)': [],
    'Intent Accuracy': [],
    'Domain Accuracy': [],
    'Domain F1-score (Macro)': [],
    'BLEU Score': [],
    'ROUGE-1 F1': [],
    'ROUGE-2 F1': [],
    'ROUGE-L F1': [],
    'METEOR Score': [],
    'Perplexity': [],
    'NLU Expert Load Stability (Variance)': [],
    'NLG Expert Load Stability (Variance)': []
}

num_iterations = 5
for iteration in range(1, num_iterations + 1):
    print(f"\nStarting Iteration {iteration}")
    tf.keras.backend.clear_session()
    gc.collect()
    
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
    model = compile_model(model, moe_params)
    
    evaluator = EvaluationMetrics(model, test_tf_dataset, moe_params, raw_data, word_to_id, id_to_word)

    evaluation_results = evaluator.evaluate()
    
    results_table['Evaluation Time (seconds)'].append(evaluation_results['eval_time'])
    results_table['Prediction Speed (ms/token)'].append(evaluation_results['pred_speed'])
    results_table['Average CPU Usage (percent)'].append(np.mean(evaluator.resource_stats['cpu_usage']) if evaluator.resource_stats['cpu_usage'] else 0)
    results_table['Average GPU Usage (percent)'].append(np.mean(evaluator.resource_stats['gpu_load']) if evaluator.resource_stats['gpu_load'] else 0)
    results_table['Average Memory (GB)'].append(np.mean(evaluator.resource_stats['memory_used']) if evaluator.resource_stats['memory_used'] else 0)
    results_table['Average GPU Memory (GB)'].append(np.mean(evaluator.resource_stats['gpu_memory']) if evaluator.resource_stats['gpu_memory'] else 0)
    results_table['Average FLOPs Estimate (GFLOPs)'].append(calculate_flops(model))
    nlu_counts = evaluator.nlu_expert_usage_counts
    nlg_counts = evaluator.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) if np.sum(nlu_counts) > 0 else 1
    total_nlg = np.sum(nlg_counts) if np.sum(nlg_counts) > 0 else 1
    results_table['NLU Expert 1 (percent)'].append((nlu_counts[0] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 2 (percent)'].append((nlu_counts[1] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 3 (percent)'].append((nlu_counts[2] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 4 (percent)'].append((nlu_counts[3] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 5 (percent)'].append((nlu_counts[4] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 6 (percent)'].append((nlu_counts[5] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 7 (percent)'].append((nlu_counts[6] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 8 (percent)'].append((nlu_counts[7] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLG Expert 1 (percent)'].append((nlg_counts[0] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 2 (percent)'].append((nlg_counts[1] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 3 (percent)'].append((nlg_counts[2] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 4 (percent)'].append((nlg_counts[3] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 5 (percent)'].append((nlg_counts[4] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 6 (percent)'].append((nlg_counts[5] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 7 (percent)'].append((nlg_counts[6] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 8 (percent)'].append((nlg_counts[7] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['Intent F1-score (Micro)'].append(evaluation_results['intent_f1_micro'])
    results_table['Intent F1-score (Macro)'].append(evaluation_results['intent_f1_macro'])
    results_table['Intent F1-score (Weighted)'].append(evaluation_results['intent_f1_weighted'])
    results_table['Intent Accuracy'].append(evaluation_results['intent_accuracy'])
    results_table['Domain Accuracy'].append(evaluation_results['domain_accuracy'])
    results_table['Domain F1-score (Macro)'].append(evaluation_results['domain_f1_macro'])
    results_table['BLEU Score'].append(evaluation_results['bleu'])
    results_table['ROUGE-1 F1'].append(evaluation_results['rouge1'])
    results_table['ROUGE-2 F1'].append(evaluation_results['rouge2'])
    results_table['ROUGE-L F1'].append(evaluation_results['rougeL'])
    results_table['METEOR Score'].append(evaluation_results['meteor'])
    results_table['Perplexity'].append(evaluation_results['perplexity'])
    results_table['NLU Expert Load Stability (Variance)'].append(np.var(nlu_counts / total_nlu * 100) if total_nlu > 0 else 0)
    results_table['NLG Expert Load Stability (Variance)'].append(np.var(nlg_counts / total_nlg * 100) if total_nlg > 0 else 0)

for key in results_table:
    if results_table[key]:
        results_table[key].append(np.mean(results_table[key]))
    else:
        results_table[key].append(0) 
        
print("\nTraining results saved")
final_df = pd.DataFrame(results_table)
final_df = final_df.T
final_df.columns = [f'Iteration {i+1}' for i in range(num_iterations)] + ['Average']
final_df.to_excel('prediction_deepseekbaseline_results.xlsx', index=False) 
final_df

2025-07-04 13:11:25.337236: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 13:11:25.337365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 13:11:25.372273: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 13:11:25.481770: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-04 13:11:26.690291: W tensorflow/compiler/tf2


Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 13:11:29.952269: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 13:11:30.151026: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 13:11:30.151340: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl
Loaded vocabulary from preprocessor_models/preprocessor_params.json

Loading model from: best_deepseekbaseline_moe_model

Model loaded and compiled successfully!
Model: "mo_e_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  784896    
                                                                 
 embedding_1 (Embedding)     multiple                  8576      
                                                                 
 layer_normalization (Layer  multiple                  256       
 Normalization)                                                  
                                                                 
 transformer_decoder_layer   multiple                  264576    
 (TransformerDecoderLayer)          

,Iteration 1,Average
Evaluation Time (seconds),25.593836,25.593836
Prediction Speed (ms/token),3.054597,3.054597
Average CPU Usage (percent),25.551396,25.551396
Average GPU Usage (percent),1.942019,1.942019
Average Memory (GB),6.975417,6.975417
Average GPU Memory (GB),14.504456,14.504456
Average FLOPs Estimate (GFLOPs),2.377917,2.377917
NLU Expert 1 (percent),0.000000,0.000000
NLU Expert 2 (percent),6.818182,6.818182
NLU Expert 3 (percent),11.864710,11.864710


# DeepSeekMoE Top-P experiment code

In [2]:
def reset_kernel():
    tf.keras.backend.clear_session()
    import gc
    gc.collect()

results_table = {
    'Training Speed (epochs per second)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Training Time (second/epoch)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Total Training Time (second/iteration)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Computational Resource Usage': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average CPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average GPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average GPU Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average FLOPS Estimate (GFLOPS)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Expert NLU Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Expert NLG Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Entity Accuracy': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Intent Accuracy': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Perplexity': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Response Cosine Similarity': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0}
}

best_val_loss = float('inf')
best_model_path = "best_deepseektopp_moe_model"

for iteration in range(5):
    print(f"\nStarting Training Iteration {iteration + 1}")
    reset_kernel()

    def load_tf_datasets_from_disk(load_path):
        print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
        try:
            with open(os.path.join(load_path, "moe_params.json"), "r") as f:
                loaded_moe_params = json.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"moe_params.json not found in {load_path}")

        element_spec = (
            {
                'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
                'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
                'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            },
            {
                'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
                'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
                'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
            }
        )

        try:
            train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
            test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
        except Exception as e:
            raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

        raw_data = {}
        for dataset_name in ['train', 'test']:
            path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
            if os.path.exists(path):
                with open(path, 'rb') as f:
                    raw_data[dataset_name] = pickle.load(f)
                print(f"Loaded raw data for {dataset_name} from {path}")
            else:
                print(f"Warning: Raw data file {path} not found.")
                raw_data[dataset_name] = []

        return {
            "train_dataset": train_tf_dataset,
            "test_dataset": test_tf_dataset,
            "moe_params": loaded_moe_params,
            "raw_data": raw_data
        }

    tf_dataset_save_path = "tf_datasets"
    loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
    train_tf_dataset = loaded_data["train_dataset"]
    moe_params = loaded_data["moe_params"]
    raw_data = loaded_data["raw_data"]
    moe_params["num_experts"] = 4

    class TopPRouting:
        def __init__(self, p=0.9):
            self.p = p 

        def __call__(self, gate_logits):
            probs = tf.nn.softmax(gate_logits, axis=-1)
            sorted_probs, sorted_indices = tf.math.top_k(probs, k=tf.shape(probs)[-1])
            cum_probs = tf.cumsum(sorted_probs, axis=-1)
            p_mask = cum_probs >= self.p
            first_over_p = tf.argmax(tf.cast(p_mask, tf.int32), axis=-1, output_type=tf.int32)
            first_over_p = tf.maximum(first_over_p, 0)
            batch_size = tf.shape(gate_logits)[0]
            num_experts = tf.shape(gate_logits)[1]
            batch_range = tf.range(batch_size)
            batch_range = tf.expand_dims(batch_range, -1)
            selection_mask = tf.less_equal(tf.range(num_experts), tf.expand_dims(first_over_p, -1))
            selected_indices = tf.boolean_mask(sorted_indices, selection_mask)
            selected_probs = tf.boolean_mask(sorted_probs, selection_mask)
            num_selected = first_over_p + 1
            max_selected = tf.reduce_max(num_selected)
            selected_indices = tf.RaggedTensor.from_row_lengths(selected_indices, num_selected).to_tensor(shape=[batch_size, max_selected])
            selected_probs = tf.RaggedTensor.from_row_lengths(selected_probs, num_selected).to_tensor(shape=[batch_size, max_selected])
            weights = selected_probs / tf.reduce_sum(selected_probs, axis=-1, keepdims=True)

            return selected_indices, weights, num_selected

    class MoELayer(layers.Layer):
        def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.01, top_p=0.9, name=None):
            super(MoELayer, self).__init__(name=name)
            self.m = m 
            self.k_s = k_s 
            self.total_experts = num_experts * self.m  
            self.routed_experts = self.total_experts - self.k_s  
            self.top_p = top_p  
            self.alpha = alpha
            self.input_dim = input_dim
            self.seq_length = None
            self.adjusted_expert_dim = expert_dim // self.m
            self.shared_experts = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', 
                                kernel_regularizer=tf.keras.regularizers.l2(1e-4), 
                                name=f'shared_expert_{i}_dense1'),
                    layers.Dense(input_dim, activation='relu', 
                                kernel_regularizer=tf.keras.regularizers.l2(1e-4), 
                                name=f'shared_expert_{i}_dense2')
                ], name=f'shared_expert_{i}') for i in range(self.k_s)
            ]

            self.routed_experts_list = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', 
                                kernel_regularizer=tf.keras.regularizers.l2(1e-4), 
                                name=f'routed_expert_{i}_dense1'),
                    layers.Dense(input_dim, activation='relu', 
                                kernel_regularizer=tf.keras.regularizers.l2(1e-4), 
                                name=f'routed_expert_{i}_dense2')
                ], name=f'routed_expert_{i}') for i in range(self.k_s, self.total_experts)
            ]

            self.experts = self.shared_experts + self.routed_experts_list
            self.gate = layers.Dense(self.routed_experts, activation='softmax', name='gate_weights')
            self.top_p_router = TopPRouting(p=self.top_p)
            self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

        def call(self, inputs, training=False):
            input_rank = inputs.shape.rank
            if input_rank == 3:
                batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
                flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
                is_3d = True
            else:
                batch_size = tf.shape(inputs)[0]
                seq_length = 1
                flat_inputs = inputs
                is_3d = False

            flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
            shared_output = tf.zeros([tf.shape(flat_inputs)[0], self.input_dim], dtype=tf.float32)
            for i in range(self.k_s):
                shared_output += self.shared_experts[i](flat_inputs)
            gate_probs = self.gate(flat_inputs)
            expert_indices, expert_weights, num_selected = self.top_p_router(gate_probs)
            expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
            expert_indices_adjusted = expert_indices + self.k_s
            effective_batch_size = tf.shape(flat_inputs)[0]
            batch_indices = tf.tile(
                tf.expand_dims(tf.range(effective_batch_size), 1),
                [1, tf.shape(expert_indices)[1]]
            )
            gather_indices = tf.stack([batch_indices, expert_indices_adjusted], axis=-1)
            selected_outputs = tf.gather_nd(expert_outputs, gather_indices)
            weighted_outputs = selected_outputs * tf.expand_dims(expert_weights, -1)
            routed_output = tf.reduce_sum(weighted_outputs, axis=1)
            output_flat = shared_output + routed_output + flat_inputs

            if is_3d:
                output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
                if self.seq_length is not None:
                    output.set_shape([None, self.seq_length, self.input_dim])
                gate_weights_reshaped = tf.reshape(gate_probs, [batch_size, seq_length, self.routed_experts])
                if self.seq_length is not None:
                    gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
            else:
                output = output_flat
                gate_weights_reshaped = gate_probs
            expert_selection = tf.one_hot(expert_indices, depth=self.routed_experts, dtype=tf.float32)
            expert_selection_sum = tf.reduce_sum(expert_selection, axis=1)  
            f_i = tf.reduce_mean(expert_selection_sum, axis=0)  
            P_i = tf.reduce_mean(gate_probs, axis=0)  
            load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
            self.add_loss(load_balancing_loss)
            self.load_balancing_loss_metric.update_state(load_balancing_loss)

            return output, gate_weights_reshaped

        def get_metrics(self):
            return {f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric}

        def get_config(self):
            config = super().get_config()
            config.update({
                'num_experts': self.total_experts // self.m,
                'expert_dim': self.adjusted_expert_dim * self.m,
                'input_dim': self.input_dim,
                'm': self.m,
                'k_s': self.k_s,
                'alpha': self.alpha,
                'top_p': self.top_p,
                'name': self.name
            })
            return config

    class TransformerDecoderLayer(layers.Layer):
        def __init__(self, d_model, num_heads, dff, rate=0.1):
            super(TransformerDecoderLayer, self).__init__()
            self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.ffn = keras.Sequential([
                layers.Dense(dff, activation='relu'),
                layers.Dense(d_model)
            ])
            self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
            self.dropout1 = layers.Dropout(rate)
            self.dropout2 = layers.Dropout(rate)
            self.dropout3 = layers.Dropout(rate)

        def call(self, x, enc_output, training, look_ahead_mask=None):
            if look_ahead_mask is not None:
                look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
                look_ahead_mask = 1.0 - look_ahead_mask

            attn1_output = self.mha1(
                query=x,
                value=x,
                key=x,
                attention_mask=look_ahead_mask,
                training=training,
                return_attention_scores=True
            )
            if isinstance(attn1_output, tuple):
                attn1, attn_weights1 = attn1_output
            else:
                attn1, attn_weights1 = attn1_output, None

            attn1 = self.dropout1(attn1, training=training)
            out1 = self.layernorm1(attn1 + x)

            attn2_output = self.mha2(
                query=out1,
                value=enc_output,
                key=enc_output,
                attention_mask=None,
                training=training,
                return_attention_scores=True
            )
            if isinstance(attn2_output, tuple):
                attn2, attn_weights2 = attn2_output
            else:
                attn2, attn_weights2 = attn2_output, None

            attn2 = self.dropout2(attn2, training=training)
            out2 = self.layernorm2(attn2 + out1)

            ffn_output = self.ffn(out2)
            ffn_output = self.dropout3(ffn_output, training=training)
            out3 = self.layernorm3(ffn_output + out2)

            return out3, attn_weights1, attn_weights2

        def get_config(self):
            config = super().get_config()
            mha1_config = self.mha1.get_config()
            config.update({
                'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
                'num_heads': mha1_config['num_heads'],
                'dff': self.ffn.layers[0].units,
                'rate': self.dropout1.rate
            })
            return config

    class MoEModel(models.Model):
        def __init__(self, moe_params):
            super(MoEModel, self).__init__()
            self.embedding_dim = moe_params["embedding_dim"]
            self.max_seq_length = moe_params["max_seq_length"]
            self.num_domains = moe_params["num_domains"]
            self.num_intents = moe_params["num_intents"]
            self.vocab_size = moe_params["vocab_size"]
            self.num_experts = moe_params["num_experts"]
            self.expert_dim = moe_params["expert_dim"]
            self.turn_id_dim = moe_params["turn_id_dim"]
            self.num_heads = 4
            self.dff = self.embedding_dim * 4
            self.dropout_rate = 0.1
            self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                                 self.num_domains + self.turn_id_dim + self.num_intents)
            self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains
            self.embedding = layers.Embedding(
                self.vocab_size,
                self.embedding_dim,
                mask_zero=True,
                embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05),
                name="embedding"
            )
            self.embedding.build((None,))
            with tf.init_scope():
                embedding_weights = self.embedding.get_weights()
                if embedding_weights and len(embedding_weights) > 0:
                    embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                    self.embedding.set_weights(embedding_weights)

            self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
            self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
            self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
            self.decoder_dropout = layers.Dropout(self.dropout_rate)
            self.moe_nlu = MoELayer(
                num_experts=self.num_experts,
                expert_dim=self.expert_dim,
                input_dim=self.nlu_input_dim,
                m=2,
                k_s=2,
                alpha=0.01,
                top_p=0.9,
                name='moe_nlu'
            )
            self.moe_nlg = MoELayer(
                num_experts=self.num_experts,
                expert_dim=self.expert_dim,
                input_dim=self.nlg_input_dim,
                m=2,
                k_s=2,
                alpha=0.01,
                top_p=0.9,
                name='moe_nlg'
            )
            self.intent_output = layers.Dense(self.num_intents, activation='sigmoid', name='intent_output')
            self.domain_output = layers.Dense(self.num_domains, activation='softmax', name='domain_output')
            self.response_output = layers.TimeDistributed(
                layers.Dense(self.vocab_size, activation='softmax'), name='response_output'
            )
            self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)
            self.nlg_projection = layers.TimeDistributed(
                layers.Dense(self.embedding_dim, activation='relu', name='nlg_projection')
            )

        def create_look_ahead_mask(self, size):
            mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
            return mask

        def call(self, inputs, training=False):
            user_utterance_tokens = inputs['user_utterance_tokens']
            prev_system_response_tokens = inputs['prev_system_response_tokens']
            decoder_input_tokens = inputs['decoder_input_tokens']
            domain_onehot_input = inputs['domain_onehot_input']
            turn_id_embedding = inputs['turn_id_embedding']
            ontology_multihot_input = inputs['ontology_multihot_input']

            if any(x is None for x in [user_utterance_tokens, prev_system_response_tokens, decoder_input_tokens,
                                      domain_onehot_input, turn_id_embedding, ontology_multihot_input]):
                raise ValueError("One or more required inputs are missing or None")

            pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
            user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
            prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
            decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)

            user_enc_out = user_embedded
            prev_system_enc_out = prev_system_embedded
            look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
            dec_output = decoder_embedded
            attn_weights = {}
            
            for i, layer in enumerate(self.decoder_layers):
                dec_output, attn_w1, attn_w2 = layer(
                    dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask
                )
                attn_weights[f'decoder_layer{i+1}_attn1'] = attn_w1
                attn_weights[f'decoder_layer{i+1}_attn2'] = attn_w2

            decoder_output = self.decoder_dropout(dec_output, training=training)

            user_attn_out = self.attn_layer(
                query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), 
                value=user_enc_out, key=user_enc_out, training=training
            )
            prev_system_attn_out = self.attn_layer(
                query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), 
                value=prev_system_enc_out, key=prev_system_enc_out, training=training
            )
            decoder_attn_out = self.attn_layer(
                query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), 
                value=decoder_output, key=decoder_output, training=training
            )

            combined_features = layers.Concatenate()([
                tf.squeeze(user_attn_out, 1),
                tf.squeeze(prev_system_attn_out, 1),
                tf.squeeze(decoder_attn_out, 1),
                domain_onehot_input,
                turn_id_embedding,
                ontology_multihot_input
            ])
            combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
            nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
            nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
            intent_out = self.intent_output(nlu_out)
            domain_out = self.domain_output(nlu_out)

            intent_out_expanded = tf.expand_dims(intent_out, axis=1)
            domain_out_expanded = tf.expand_dims(domain_out, axis=1)
            intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
            domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])

            nlg_input = layers.Concatenate(axis=-1)([
                decoder_output,
                intent_out_tiled,
                domain_out_tiled
            ])
            nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])

            nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
            nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
            response_embeddings = self.nlg_projection(nlg_out)
            response_embeddings = tf.ensure_shape(response_embeddings, [None, self.max_seq_length, self.embedding_dim])
            response_out = self.response_output(nlg_out)

            return {
                'intent_output': intent_out,
                'domain_output': domain_out,
                'response_output': response_out,
                'response_embeddings': response_embeddings,
                'nlu_gate_weights': nlu_gate_weights,
                'nlg_gate_weights': nlg_gate_weights
            }

        def build_graph(self):
            inputs = {
                'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='user_utterance_tokens'),
                'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='prev_system_response_tokens'),
                'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='decoder_input_tokens'),
                'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32, name='domain_onehot_input'),
                'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32, name='turn_id_embedding'),
                'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32, name='ontology_multihot_input'),
            }
            return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

        def get_config(self):
            config = super().get_config()
            config.update({
                'moe_params': {
                    'embedding_dim': self.embedding_dim,
                    'max_seq_length': self.max_seq_length,
                    'num_domains': self.num_domains,
                    'num_intents': self.num_intents,
                    'vocab_size': self.vocab_size,
                    'num_experts': self.num_experts,
                    'expert_dim': self.expert_dim,
                    'turn_id_dim': self.turn_id_dim
                }
            })
            return config

    class IntentAccuracy(metrics.Metric):
        def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros', dtype=tf.float32)
            self.total_preds = self.add_weight(name='total_preds', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
            self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
            self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

        def reset_state(self):
            self.correct_preds.assign(0.)
            self.total_preds.assign(0.)

    class IntentPrecision(metrics.Metric):
        def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
            self.predicted_positives = self.add_weight(name='pp', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            predicted_positives = tf.reduce_sum(y_pred)
            self.true_positives.assign_add(true_positives)
            self.predicted_positives.assign_add(predicted_positives)

        def result(self):
            return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.predicted_positives.assign(0.)

    class IntentRecall(metrics.Metric):
        def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
            self.actual_positives = self.add_weight(name='ap', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            actual_positives = tf.reduce_sum(y_true)
            self.true_positives.assign_add(true_positives)
            self.actual_positives.assign_add(actual_positives)

        def result(self):
            return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.actual_positives.assign(0.)

    class IntentF1(metrics.Metric):
        def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.precision = IntentPrecision(threshold=threshold)
            self.recall = IntentRecall(threshold=threshold)

        def update_state(self, y_true, y_pred, sample_weight=None):
            self.precision.update_state(y_true, y_pred, sample_weight)
            self.recall.update_state(y_true, y_pred, sample_weight)

        def result(self):
            p = self.precision.result()
            r = self.recall.result()
            return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

        def reset_state(self):
            self.precision.reset_state()
            self.recall.reset_state()

    class DomainAccuracy(metrics.Metric):
        def __init__(self, name='domain_accuracy', **kwargs):
            super().__init__(name=name, **kwargs)
            self.accuracy = self.add_weight(name='acc', initializer='zeros', dtype=tf.float32)
            self.count = self.add_weight(name='count', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
            pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
            matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
            self.accuracy.assign_add(tf.reduce_sum(matches))
            self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.accuracy / (self.count + tf.keras.backend.epsilon())

        def reset_state(self):
            self.accuracy.assign(0.)
            self.count.assign(0.)

    class Perplexity(metrics.Metric):
        def __init__(self, name='perplexity', **kwargs):
            super().__init__(name=name, **kwargs)
            self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
            self.total_loss = self.add_weight(name='total_loss', initializer='zeros', dtype=tf.float32)
            self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            mask = tf.cast(y_true != 0, dtype=tf.float32)
            loss = self.cross_entropy(y_true, y_pred)
            masked_loss = loss * mask
            sum_loss = tf.reduce_sum(masked_loss)
            num_non_padding_tokens = tf.reduce_sum(mask)
            self.total_loss.assign_add(sum_loss)
            self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

        def result(self):
            avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
            return tf.exp(avg_loss)

        def reset_state(self):
            self.total_loss.assign(0.)
            self.total_non_padding_tokens.assign(0.)

    class ResponseEmbeddingCosineSimilarity(metrics.Metric):
        def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
            super().__init__(name=name)
            self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
            self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            epsilon = tf.keras.backend.epsilon()
            y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
            y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
            y_true_norm = y_true / (y_true_norm_val + epsilon)
            y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
            cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
            non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
            non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
            masked_cosine_sim = cosine_sim_per_token * non_padding_mask
            num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
            avg_cosine_sim_per_sample = tf.where(
                num_non_zero_tokens > 0,
                tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
                tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
            )
            self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
            self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

        def reset_state(self):
            self.total_cosine_sim.assign(0.)
            self.num_samples.assign(0.)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    def calculate_flops(model, batch_size=moe_params["batch_size"]):
        flops = 0
        functional_model = model.build_graph()

        def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
            layer_flops_count = 0

            if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
                return 0

            if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
                for sub_layer in layer_instance.layers:
                    layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
                return layer_flops_count

            if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
                print(f"Warning: Layer {layer_instance.name} still has no defined input shape after forward pass. Skipping FLOPs calculation for it.")
                return 0

            if isinstance(layer_instance, layers.Dense):
                input_dim = layer_instance.input_shape[-1]
                output_dim = layer_instance.output_shape[-1]
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                layer_flops_count += 2 * input_dim * output_dim * effective_batch_size

            elif isinstance(layer_instance, layers.LSTM):
                input_dim = layer_instance.input_shape[-1]
                hidden_dim = layer_instance.units
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
                layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size

            elif isinstance(layer_instance, layers.Embedding):
                pass

            elif isinstance(layer_instance, MoELayer):
                moe_input_dim = layer_instance.input_dim
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
                layer_flops_count += gate_flops
                for expert in layer_instance.experts:
                    layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)

            elif isinstance(layer_instance, layers.MultiHeadAttention):
                config = layer_instance.get_config()
                num_heads = config['num_heads']
                key_dim = config['key_dim']
                d_model = num_heads * key_dim
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
                projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
                attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
                context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
                output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
                layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops

            elif isinstance(layer_instance, layers.TimeDistributed):
                inner_layer = layer_instance.layer
                td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
                layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)

            elif isinstance(layer_instance, TransformerDecoderLayer):
                layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
                layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
                ffn_effective_batch_size = current_batch_size * model.max_seq_length
                layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)

            return layer_flops_count

        for layer in functional_model.layers:
            flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)

        return flops / 1e9

    def get_gpu_stats():
        if nvidia_smi is None:
            print("NVIDIA SMI not available. GPU stats will be reported as zero.")
            return 0, 0
        try:
            nvidia_smi.nvmlInit()
            handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
            gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
            mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
            gpu_memory = mem_info.used / (1024 ** 2)
            nvidia_smi.nvmlShutdown()
            return gpu_load, gpu_memory / 1024
        except Exception as e:
            print(f"GPU monitoring failed: {str(e)}. GPU stats will be reported as zero.")
            return 0, 0

    class ResourceMonitor(keras.callbacks.Callback):
        def __init__(self, validation_data):
            super().__init__()
            self.resource_stats = []
            self.expert_load_history = []
            self.start_time = None
            self.epoch_start_time = None
            self.flops = None
            self.validation_data = validation_data
            self.nlu_expert_usage_counts = None
            self.nlg_expert_usage_counts = None
            self.nlu_gate_weights_history = []
            self.nlg_gate_weights_history = []

        def on_train_begin(self, logs=None):
            self.start_time = time.time()
            try:
                self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
                self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
                self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
            except Exception:
                self.flops = 0

        def on_epoch_begin(self, epoch, logs=None):
            self.epoch_start_time = time.time()

        def on_epoch_end(self, epoch, logs=None):
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            epoch_time = time.time() - self.epoch_start_time
            self.resource_stats.append({
                'epoch': epoch + 1,
                'epoch_time': epoch_time,
                'cpu_usage': cpu_usage,
                'memory_used': memory.used / (1024 ** 3),
                'memory_total': memory.total / (1024 ** 3),
                'gpu_load': gpu_load,
                'gpu_memory': gpu_memory,
                'loss': logs.get('loss', 0),
                'val_loss': logs.get('val_loss', 0),
                'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
                'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
                'intent_precision': logs.get('intent_output_intent_precision', 0),
                'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
                'intent_recall': logs.get('intent_output_intent_recall', 0),
                'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
                'intent_f1': logs.get('intent_output_intent_f1', 0),
                'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
                'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
                'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
                'perplexity': logs.get('response_output_perplexity', 0),
                'val_perplexity': logs.get('val_response_output_perplexity', 0),
                'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
                'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
                'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
                'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
            })
            if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
                self.expert_load_history.append({
                    'epoch': epoch + 1,
                    'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                    'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
                })

            sample_size = 100
            for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
                inputs, _ = batch
                outputs = self.model(inputs, training=False)
                nlu_gate_weights = outputs['nlu_gate_weights']
                reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
                self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
                nlg_gate_weights = outputs['nlg_gate_weights']
                reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
                self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

        def on_train_end(self, logs=None):
            self.total_time = time.time() - self.start_time

            if len(self.nlu_gate_weights_history) > 0:
                all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
                nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
                for expert_id in nlu_top_indices:
                    if expert_id < self.model.moe_nlu.total_experts:
                        self.nlu_expert_usage_counts[expert_id] += 1

            if len(self.nlg_gate_weights_history) > 0:
                all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
                nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
                for expert_id in nlg_top_indices:
                    if expert_id < self.model.moe_nlg.total_experts:
                        self.nlg_expert_usage_counts[expert_id] += 1

        def visualize_expert_load_distribution(self):
            total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
            total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
            nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
            nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
            nlu_stability = np.var(nlu_percentages)
            nlg_stability = np.var(nlg_percentages)
            return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

    class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
        def __init__(self, d_model, warmup_steps=4000):
            super().__init__()
            self.d_model = tf.cast(d_model, tf.float32)
            self.warmup_steps = warmup_steps

        def __call__(self, step):
            step = tf.cast(step, tf.float32)
            arg1 = tf.math.rsqrt(step)
            arg2 = step * (self.warmup_steps ** -1.5)
            return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

        def get_config(self):
            return {
                "d_model": self.d_model.numpy(),
                "warmup_steps": self.warmup_steps
            }

    model = MoEModel(moe_params)
    embedding_layer = model.embedding

    def create_validation_split(dataset, embedding_layer, validation_split=0.2, batch_size=moe_params["batch_size"]):
        element_spec = dataset.element_spec
        data_list = [(features, targets) for features, targets in dataset.unbatch().as_numpy_iterator()]
        if not data_list:
            raise ValueError("Dataset is empty.")
        if len(data_list) < 2:
            raise ValueError("Dataset too small to split.")
        train_data, val_data = train_test_split(data_list, test_size=validation_split, shuffle=True)

        def create_dataset_from_list(data):
            if not data:
                raise ValueError("Empty data list provided.")
            features = {
                'user_utterance_tokens': np.array([d[0]['user_utterance_tokens'] for d in data], dtype=np.int32),
                'prev_system_response_tokens': np.array([d[0]['prev_system_response_tokens'] for d in data], dtype=np.int32),
                'decoder_input_tokens': np.array([d[0]['decoder_input_tokens'] for d in data], dtype=np.int32),
                'domain_onehot_input': np.array([d[0]['domain_onehot_input'] for d in data], dtype=np.float32),
                'turn_id_embedding': np.array([d[0]['turn_id_embedding'] for d in data], dtype=np.float32),
                'ontology_multihot_input': np.array([d[0]['ontology_multihot_input'] for d in data], dtype=np.float32),
            }
            response_output = np.array([d[1]['response_output'] for d in data], dtype=np.int32)
            response_embeddings = embedding_layer(response_output).numpy()
            targets = {
                'domain_output': np.array([d[1]['domain_output'] for d in data], dtype=np.int32),
                'intent_output': np.array([d[1]['intent_output'] for d in data], dtype=np.float32),
                'response_output': response_output,
                'response_embeddings': response_embeddings
            }
            return tf.data.Dataset.from_tensor_slices((features, targets))

        train_dataset = create_dataset_from_list(train_data).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        val_dataset = create_dataset_from_list(val_data).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return train_dataset, val_dataset

    train_tf_dataset, val_tf_dataset = create_validation_split(train_tf_dataset, embedding_layer)

    losses_dict = {
        'intent_output': losses.BinaryCrossentropy(),
        'domain_output': losses.SparseCategoricalCrossentropy(),
        'response_output': losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss
    }

    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }

    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={'intent_output': 0.5, 'domain_output': 0.5, 'response_output': 1.0, 'response_embeddings': 1.0}
    )

    sample_input = next(iter(train_tf_dataset.take(1)))
    model(sample_input[0])

    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=8, mode='min', restore_best_weights=True)
    resource_monitor = ResourceMonitor(val_tf_dataset)
    class TQDMProgressBar(callbacks.Callback):
        def on_epoch_begin(self, epoch, logs=None):
            self.progress_bar = tqdm(total=len(train_tf_dataset), desc=f'Epoch {epoch + 1}')

        def on_batch_end(self, batch, logs=None):
            self.progress_bar.update(1)

        def on_epoch_end(self, epoch, logs=None):
            self.progress_bar.close()

    tqdm_callback = TQDMProgressBar()

    history = model.fit(
        train_tf_dataset,
        epochs=100,
        validation_data=val_tf_dataset,
        callbacks=[early_stopping, resource_monitor, tqdm_callback],
        verbose=0
    )

    expert_stats = resource_monitor.visualize_expert_load_distribution()
    stats = resource_monitor.resource_stats
    avg_epoch_duration = np.mean([s['epoch_time'] for s in stats]) if stats else 0
    total_training_time = resource_monitor.total_time
    avg_cpu_usage = np.mean([s['cpu_usage'] for s in stats]) if stats else 0
    avg_memory = np.mean([s['memory_used'] for s in stats]) if stats else 0
    avg_gpu_load = np.mean([s['gpu_load'] for s in stats]) if stats else 0
    avg_gpu_memory = np.mean([s['gpu_memory'] for s in stats]) if stats else 0
    avg_flops = resource_monitor.flops if resource_monitor.flops else 0
    nlu_counts = resource_monitor.nlu_expert_usage_counts
    nlg_counts = resource_monitor.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) or 1
    total_nlg = np.sum(nlg_counts) or 1
    expert_nlu_percentages = [(nlu_counts[i] / total_nlu) * 100 for i in range(8)]
    expert_nlg_percentages = [(nlg_counts[i] / total_nlg) * 100 for i in range(8)]
    val_intent_accuracy = np.mean([s['val_intent_accuracy'] for s in stats if s['val_intent_accuracy'] > 0]) if stats else 0
    val_entity_accuracy = np.mean([s['val_domain_accuracy'] for s in stats if s['val_domain_accuracy'] > 0]) if stats else 0
    val_perplexity = np.mean([s['val_perplexity'] for s in stats if s['val_perplexity'] > 0]) if stats else 0
    val_cosine_similarity = np.mean([s['val_cosine_similarity'] for s in stats if s['val_cosine_similarity'] > 0]) if stats else 0

    results_table['Training Speed (epochs per second)'][f'Iteration {iteration + 1}'] = 1 / avg_epoch_duration if avg_epoch_duration > 0 else 0
    results_table['Training Time (second/epoch)'][f'Iteration {iteration + 1}'] = avg_epoch_duration
    results_table['Total Training Time (second/iteration)'][f'Iteration {iteration + 1}'] = total_training_time
    results_table['Computational Resource Usage'][f'Iteration {iteration + 1}'] = avg_cpu_usage + avg_gpu_load
    results_table['Average CPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_cpu_usage
    results_table['Average GPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_gpu_load
    results_table['Average Memory (GB)'][f'Iteration {iteration + 1}'] = avg_memory
    results_table['Average GPU Memory (GB)'][f'Iteration {iteration + 1}'] = avg_gpu_memory
    results_table['Average FLOPS Estimate (GFLOPS)'][f'Iteration {iteration + 1}'] = avg_flops
    results_table['Expert NLU Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlu_stability']
    results_table['Expert NLG Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlg_stability']
    for i in range(8):
        if i < len(expert_stats['nlu_percentages']):
            results_table[f'Average NLU Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlu_percentages'][i]
        if i < len(expert_stats['nlg_percentages']):
            results_table[f'Average NLG Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlg_percentages'][i]
    results_table['Average Validation Entity Accuracy'][f'Iteration {iteration + 1}'] = val_entity_accuracy
    results_table['Average Intent Accuracy'][f'Iteration {iteration + 1}'] = val_intent_accuracy
    results_table['Average Validation Perplexity'][f'Iteration {iteration + 1}'] = val_perplexity
    results_table['Average Validation Response Cosine Similarity'][f'Iteration {iteration + 1}'] = val_cosine_similarity

    val_loss = min([s['val_loss'] for s in stats]) if stats else float('inf')
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        try:
            model.save(best_model_path, save_format='tf', include_optimizer=True)
        except Exception as e:
            print(f"\nFailed to save best model to {best_model_path}: {str(e)}")

for key in results_table:
    results_table[key]['Average'] = np.mean([results_table[key][f'Iteration {i+1}'] for i in range(10)])

final_result = pd.DataFrame(results_table)
final_result = final_result.T
final_result.to_excel('training_deepseektopp_result.xlsx', index=True)
final_result


Starting Training Iteration 1

Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 13:45:07.222166: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 13:45:07.591959: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 13:45:07.592158: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl


Epoch 1:   0%|          | 0/4428 [00:00<?, ?it/s]2025-07-04 13:45:34.515705: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f7a0011f8d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-04 13:45:34.515755: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2025-07-04 13:45:34.536714: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-04 13:45:34.582022: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1751636734.671252    4748 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
Epoch 1: 100%|██████████| 4428/4428 [02:39<00:00, 27.69it/s]


INFO:tensorflow:Assets written to: best_deepseektopp_moe_model/assets


INFO:tensorflow:Assets written to: best_deepseektopp_moe_model/assets


,Iteration 1,Iteration 2,Iteration 3,Iteration 4,Iteration 5,Iteration 6,Iteration 7,Iteration 8,Iteration 9,Iteration 10,Average
Training Speed (epochs per second),0.006431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000643
Training Time (second/epoch),155.506772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.550677
Total Training Time (second/iteration),161.162587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.116259
Computational Resource Usage,45.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.540000
Average CPU Usage (percent),27.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.740000
Average GPU Usage (percent),18.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.800000
Average Memory (GB),7.522411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.752241
Average GPU Memory (GB),14.531799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.453180
Average FLOPS Estimate (GFLOPS),2.377871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.237787
Expert NLU Load Distribution Stability,1044.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.450000


# DeepSeekMoE Top-P evaluation code

In [2]:
def load_tf_datasets_from_disk(load_path):
    print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
    try:
        with open(os.path.join(load_path, "moe_params.json"), "r") as f:
            loaded_moe_params = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"moe_params.json not found in {load_path}")

    element_spec = (
        {
            'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
            'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
            'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
        },
        {
            'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
            'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
        }
    )

    try:
        train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
        test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
    except Exception as e:
        raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

    raw_data = {}
    for dataset_name in ['train', 'test']:
        path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
        if os.path.exists(path):
            with open(path, 'rb') as f:
                raw_data[dataset_name] = pickle.load(f)
            print(f"Loaded raw data for {dataset_name} from {path}")
        else:
            print(f"Warning: Raw data file {path} not found.")
            raw_data[dataset_name] = []

    return {
        "train_dataset": train_tf_dataset,
        "test_dataset": test_tf_dataset,
        "moe_params": loaded_moe_params,
        "raw_data": raw_data
    }

tf_dataset_save_path = "tf_datasets"
loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
train_tf_dataset = loaded_data["train_dataset"]
test_tf_dataset = loaded_data["test_dataset"]
moe_params = loaded_data["moe_params"]
raw_data = loaded_data["raw_data"]
moe_params["num_experts"] = 4

class TopPRouting:
    def __init__(self, p=0.9):
        self.p = p 

    def __call__(self, gate_logits):
        probs = tf.nn.softmax(gate_logits, axis=-1)
        sorted_probs, sorted_indices = tf.math.top_k(probs, k=tf.shape(probs)[-1])
        cum_probs = tf.cumsum(sorted_probs, axis=-1)
        p_mask = cum_probs >= self.p
        first_over_p = tf.argmax(tf.cast(p_mask, tf.int32), axis=-1, output_type=tf.int32)
        first_over_p = tf.maximum(first_over_p, 0)
        batch_size = tf.shape(gate_logits)[0]
        num_experts = tf.shape(gate_logits)[1]
        batch_range = tf.range(batch_size)
        batch_range = tf.expand_dims(batch_range, -1)
        selection_mask = tf.less_equal(tf.range(num_experts), tf.expand_dims(first_over_p, -1))
        selected_indices = tf.boolean_mask(sorted_indices, selection_mask)
        selected_probs = tf.boolean_mask(sorted_probs, selection_mask)
        num_selected = first_over_p + 1
        max_selected = tf.reduce_max(num_selected)
        selected_indices = tf.RaggedTensor.from_row_lengths(selected_indices, num_selected).to_tensor(shape=[batch_size, max_selected])
        selected_probs = tf.RaggedTensor.from_row_lengths(selected_probs, num_selected).to_tensor(shape=[batch_size, max_selected])
        weights = selected_probs / tf.reduce_sum(selected_probs, axis=-1, keepdims=True)

        return selected_indices, weights, num_selected

class MoELayer(layers.Layer):
    def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.01, top_p=0.9, name=None):
        super(MoELayer, self).__init__(name=name)
        self.m = m 
        self.k_s = k_s 
        self.total_experts = num_experts * self.m  
        self.routed_experts = self.total_experts - self.k_s  
        self.top_p = top_p  
        self.alpha = alpha
        self.input_dim = input_dim
        self.seq_length = None
        self.adjusted_expert_dim = expert_dim // self.m
        self.shared_experts = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', 
                            kernel_regularizer=tf.keras.regularizers.l2(1e-4), 
                            name=f'shared_expert_{i}_dense1'),
                layers.Dense(input_dim, activation='relu', 
                            kernel_regularizer=tf.keras.regularizers.l2(1e-4), 
                            name=f'shared_expert_{i}_dense2')
            ], name=f'shared_expert_{i}') for i in range(self.k_s)
        ]

        self.routed_experts_list = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', 
                            kernel_regularizer=tf.keras.regularizers.l2(1e-4), 
                            name=f'routed_expert_{i}_dense1'),
                layers.Dense(input_dim, activation='relu', 
                            kernel_regularizer=tf.keras.regularizers.l2(1e-4), 
                            name=f'routed_expert_{i}_dense2')
            ], name=f'routed_expert_{i}') for i in range(self.k_s, self.total_experts)
        ]

        self.experts = self.shared_experts + self.routed_experts_list
        self.gate = layers.Dense(self.routed_experts, activation='softmax', name='gate_weights')
        self.top_p_router = TopPRouting(p=self.top_p)
        self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

    def call(self, inputs, training=False):
        input_rank = inputs.shape.rank
        if input_rank == 3:
            batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
            flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
            is_3d = True
        else:
            batch_size = tf.shape(inputs)[0]
            seq_length = 1
            flat_inputs = inputs
            is_3d = False

        flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
        shared_output = tf.zeros([tf.shape(flat_inputs)[0], self.input_dim], dtype=tf.float32)
        for i in range(self.k_s):
            shared_output += self.shared_experts[i](flat_inputs)
        gate_probs = self.gate(flat_inputs)
        expert_indices, expert_weights, num_selected = self.top_p_router(gate_probs)
        expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
        expert_indices_adjusted = expert_indices + self.k_s
        effective_batch_size = tf.shape(flat_inputs)[0]
        batch_indices = tf.tile(
            tf.expand_dims(tf.range(effective_batch_size), 1),
            [1, tf.shape(expert_indices)[1]]
        )
        gather_indices = tf.stack([batch_indices, expert_indices_adjusted], axis=-1)
        selected_outputs = tf.gather_nd(expert_outputs, gather_indices)
        weighted_outputs = selected_outputs * tf.expand_dims(expert_weights, -1)
        routed_output = tf.reduce_sum(weighted_outputs, axis=1)
        output_flat = shared_output + routed_output + flat_inputs

        if is_3d:
            output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
            if self.seq_length is not None:
                output.set_shape([None, self.seq_length, self.input_dim])
            gate_weights_reshaped = tf.reshape(gate_probs, [batch_size, seq_length, self.routed_experts])
            if self.seq_length is not None:
                gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
        else:
            output = output_flat
            gate_weights_reshaped = gate_probs
        expert_selection = tf.one_hot(expert_indices, depth=self.routed_experts, dtype=tf.float32)
        expert_selection_sum = tf.reduce_sum(expert_selection, axis=1)  
        f_i = tf.reduce_mean(expert_selection_sum, axis=0)  
        P_i = tf.reduce_mean(gate_probs, axis=0)  
        load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
        self.add_loss(load_balancing_loss)
        self.load_balancing_loss_metric.update_state(load_balancing_loss)

        return output, gate_weights_reshaped

    def get_metrics(self):
        return {f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric}

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_experts': self.total_experts // self.m,
            'expert_dim': self.adjusted_expert_dim * self.m,
            'input_dim': self.input_dim,
            'm': self.m,
            'k_s': self.k_s,
            'alpha': self.alpha,
            'top_p': self.top_p,
            'name': self.name
        })
        return config

class TransformerDecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.ffn = keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask=None):
        if look_ahead_mask is not None:
            look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
            look_ahead_mask = 1.0 - look_ahead_mask

        attn1_output = self.mha1(
            query=x,
            value=x,
            key=x,
            attention_mask=look_ahead_mask,
            training=training,
            return_attention_scores=True
        )
        if isinstance(attn1_output, tuple):
            attn1, attn_weights1 = attn1_output
        else:
            attn1, attn_weights1 = attn1_output, None

        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2_output = self.mha2(
            query=out1,
            value=enc_output,
            key=enc_output,
            attention_mask=None,
            training=training,
            return_attention_scores=True
        )
        if isinstance(attn2_output, tuple):
            attn2, attn_weights2 = attn2_output
        else:
            attn2, attn_weights2 = attn2_output, None

        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights1, attn_weights2

    def get_config(self):
        config = super().get_config()
        mha1_config = self.mha1.get_config()
        config.update({
            'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
            'num_heads': mha1_config['num_heads'],
            'dff': self.ffn.layers[0].units,
            'rate': self.dropout1.rate
        })
        return config

class MoEModel(models.Model):
    def __init__(self, moe_params):
        super(MoEModel, self).__init__()
        self.embedding_dim = moe_params["embedding_dim"]
        self.max_seq_length = moe_params["max_seq_length"]
        self.num_domains = moe_params["num_domains"]
        self.num_intents = moe_params["num_intents"]
        self.vocab_size = moe_params["vocab_size"]
        self.num_experts = moe_params["num_experts"]
        self.expert_dim = moe_params["expert_dim"]
        self.turn_id_dim = moe_params["turn_id_dim"]
        self.num_heads = 4
        self.dff = self.embedding_dim * 4
        self.dropout_rate = 0.1
        self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                             self.num_domains + self.turn_id_dim + self.num_intents)
        self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains
        self.embedding = layers.Embedding(
            self.vocab_size,
            self.embedding_dim,
            mask_zero=True,
            embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05),
            name="embedding"
        )
        self.embedding.build((None,))
        with tf.init_scope():
            embedding_weights = self.embedding.get_weights()
            if embedding_weights and len(embedding_weights) > 0:
                embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                self.embedding.set_weights(embedding_weights)

        self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
        self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
        self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
        self.decoder_dropout = layers.Dropout(self.dropout_rate)
        self.moe_nlu = MoELayer(
            num_experts=self.num_experts,
            expert_dim=self.expert_dim,
            input_dim=self.nlu_input_dim,
            m=2,
            k_s=2,
            alpha=0.01,
            top_p=0.9,
            name='moe_nlu'
        )
        self.moe_nlg = MoELayer(
            num_experts=self.num_experts,
            expert_dim=self.expert_dim,
            input_dim=self.nlg_input_dim,
            m=2,
            k_s=2,
            alpha=0.01,
            top_p=0.9,
            name='moe_nlg'
        )
        self.intent_output = layers.Dense(self.num_intents, activation='sigmoid', name='intent_output')
        self.domain_output = layers.Dense(self.num_domains, activation='softmax', name='domain_output')
        self.response_output = layers.TimeDistributed(
            layers.Dense(self.vocab_size, activation='softmax'), name='response_output'
        )
        self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)
        self.nlg_projection = layers.TimeDistributed(
            layers.Dense(self.embedding_dim, activation='relu', name='nlg_projection')
        )

    def create_look_ahead_mask(self, size):
        mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
        return mask

    def call(self, inputs, training=False):
        user_utterance_tokens = inputs['user_utterance_tokens']
        prev_system_response_tokens = inputs['prev_system_response_tokens']
        decoder_input_tokens = inputs['decoder_input_tokens']
        domain_onehot_input = inputs['domain_onehot_input']
        turn_id_embedding = inputs['turn_id_embedding']
        ontology_multihot_input = inputs['ontology_multihot_input']

        if any(x is None for x in [user_utterance_tokens, prev_system_response_tokens, decoder_input_tokens,
                                  domain_onehot_input, turn_id_embedding, ontology_multihot_input]):
            raise ValueError("One or more required inputs are missing or None")

        pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
        user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
        prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
        decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)

        user_enc_out = user_embedded
        prev_system_enc_out = prev_system_embedded
        look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
        dec_output = decoder_embedded
        attn_weights = {}

        for i, layer in enumerate(self.decoder_layers):
            dec_output, attn_w1, attn_w2 = layer(
                dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask
            )
            attn_weights[f'decoder_layer{i+1}_attn1'] = attn_w1
            attn_weights[f'decoder_layer{i+1}_attn2'] = attn_w2

        decoder_output = self.decoder_dropout(dec_output, training=training)

        user_attn_out = self.attn_layer(
            query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), 
            value=user_enc_out, key=user_enc_out, training=training
        )
        prev_system_attn_out = self.attn_layer(
            query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), 
            value=prev_system_enc_out, key=prev_system_enc_out, training=training
        )
        decoder_attn_out = self.attn_layer(
            query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), 
            value=decoder_output, key=decoder_output, training=training
        )

        combined_features = layers.Concatenate()([
            tf.squeeze(user_attn_out, 1),
            tf.squeeze(prev_system_attn_out, 1),
            tf.squeeze(decoder_attn_out, 1),
            domain_onehot_input,
            turn_id_embedding,
            ontology_multihot_input
        ])
        combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
        nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
        nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
        intent_out = self.intent_output(nlu_out)
        domain_out = self.domain_output(nlu_out)

        intent_out_expanded = tf.expand_dims(intent_out, axis=1)
        domain_out_expanded = tf.expand_dims(domain_out, axis=1)
        intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
        domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])

        nlg_input = layers.Concatenate(axis=-1)([
            decoder_output,
            intent_out_tiled,
            domain_out_tiled
        ])
        nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])

        nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
        nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
        response_embeddings = self.nlg_projection(nlg_out)
        response_embeddings = tf.ensure_shape(response_embeddings, [None, self.max_seq_length, self.embedding_dim])
        response_out = self.response_output(nlg_out)

        return {
            'intent_output': intent_out,
            'domain_output': domain_out,
            'response_output': response_out,
            'response_embeddings': response_embeddings,
            'nlu_gate_weights': nlu_gate_weights,
            'nlg_gate_weights': nlg_gate_weights
        }

    def build_graph(self):
        inputs = {
            'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='user_utterance_tokens'),
            'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='prev_system_response_tokens'),
            'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='decoder_input_tokens'),
            'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32, name='domain_onehot_input'),
            'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32, name='turn_id_embedding'),
            'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32, name='ontology_multihot_input'),
        }
        return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

    def get_config(self):
        config = super().get_config()
        config.update({
            'moe_params': {
                'embedding_dim': self.embedding_dim,
                'max_seq_length': self.max_seq_length,
                'num_domains': self.num_domains,
                'num_intents': self.num_intents,
                'vocab_size': self.vocab_size,
                'num_experts': self.num_experts,
                'expert_dim': self.expert_dim,
                'turn_id_dim': self.turn_id_dim
            }
        })
        return config

class IntentAccuracy(metrics.Metric):
    def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros', dtype=tf.float32)
        self.total_preds = self.add_weight(name='total_preds', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
        self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
        self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

    def reset_state(self):
        self.correct_preds.assign(0.)
        self.total_preds.assign(0.)

class IntentPrecision(metrics.Metric):
    def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
        self.predicted_positives = self.add_weight(name='pp', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        predicted_positives = tf.reduce_sum(y_pred)
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)

    def result(self):
        return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.predicted_positives.assign(0.)

class IntentRecall(metrics.Metric):
    def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
        self.actual_positives = self.add_weight(name='ap', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        actual_positives = tf.reduce_sum(y_true)
        self.true_positives.assign_add(true_positives)
        self.actual_positives.assign_add(actual_positives)

    def result(self):
        return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.actual_positives.assign(0.)

class IntentF1(metrics.Metric):
    def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = IntentPrecision(threshold=threshold)
        self.recall = IntentRecall(threshold=threshold)

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

class DomainAccuracy(metrics.Metric):
    def __init__(self, name='domain_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.accuracy = self.add_weight(name='acc', initializer='zeros', dtype=tf.float32)
        self.count = self.add_weight(name='count', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
        pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
        matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
        self.accuracy.assign_add(tf.reduce_sum(matches))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.accuracy / (self.count + tf.keras.backend.epsilon())

    def reset_state(self):
        self.accuracy.assign(0.)
        self.count.assign(0.)

class Perplexity(metrics.Metric):
    def __init__(self, name='perplexity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
        self.total_loss = self.add_weight(name='total_loss', initializer='zeros', dtype=tf.float32)
        self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        mask = tf.cast(y_true != 0, dtype=tf.float32)
        loss = self.cross_entropy(y_true, y_pred)
        masked_loss = loss * mask
        sum_loss = tf.reduce_sum(masked_loss)
        num_non_padding_tokens = tf.reduce_sum(mask)
        self.total_loss.assign_add(sum_loss)
        self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

    def result(self):
        avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
        return tf.exp(avg_loss)

    def reset_state(self):
        self.total_loss.assign(0.)
        self.total_non_padding_tokens.assign(0.)

class ResponseEmbeddingCosineSimilarity(metrics.Metric):
    def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
        super().__init__(name=name)
        self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
        self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true_norm = y_true / (y_true_norm_val + epsilon)
        y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
        cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
        non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
        non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
        masked_cosine_sim = cosine_sim_per_token * non_padding_mask
        num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
        avg_cosine_sim_per_sample = tf.where(
            num_non_zero_tokens > 0,
            tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
            tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
        )
        self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
        self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

    def reset_state(self):
        self.total_cosine_sim.assign(0.)
        self.num_samples.assign(0.)

def contrastive_loss(y_true, y_pred, margin=1.0):
    epsilon = tf.keras.backend.epsilon()
    y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
    y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
    y_true = y_true / (y_true_norm + epsilon)
    y_pred = y_pred / (y_pred_norm + epsilon)
    cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
    positive_loss = 1.0 - cosine_sim
    mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
    masked_loss = positive_loss * mask
    total_loss = tf.reduce_sum(masked_loss)
    num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
    return total_loss / num_tokens

def calculate_flops(model, batch_size=moe_params["batch_size"]):
    flops = 0
    functional_model = model.build_graph()

    def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
        layer_flops_count = 0

        if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
            return 0

        if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
            for sub_layer in layer_instance.layers:
                layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
            return layer_flops_count

        if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
            print(f"Warning: Layer {layer_instance.name} still has no defined input shape after forward pass. Skipping FLOPs calculation for it.")
            return 0

        if isinstance(layer_instance, layers.Dense):
            input_dim = layer_instance.input_shape[-1]
            output_dim = layer_instance.output_shape[-1]
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            layer_flops_count += 2 * input_dim * output_dim * effective_batch_size

        elif isinstance(layer_instance, layers.LSTM):
            input_dim = layer_instance.input_shape[-1]
            hidden_dim = layer_instance.units
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
            layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size

        elif isinstance(layer_instance, layers.Embedding):
            pass

        elif isinstance(layer_instance, MoELayer):
            moe_input_dim = layer_instance.input_dim
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
            layer_flops_count += gate_flops
            for expert in layer_instance.experts:
                layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)

        elif isinstance(layer_instance, layers.MultiHeadAttention):
            config = layer_instance.get_config()
            num_heads = config['num_heads']
            key_dim = config['key_dim']
            d_model = num_heads * key_dim
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
            projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
            attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
            context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
            output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
            layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops

        elif isinstance(layer_instance, layers.TimeDistributed):
            inner_layer = layer_instance.layer
            td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
            layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)

        elif isinstance(layer_instance, TransformerDecoderLayer):
            layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
            layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
            ffn_effective_batch_size = current_batch_size * model.max_seq_length
            layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)

        return layer_flops_count

    for layer in functional_model.layers:
        flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)

    return flops / 1e9

def get_gpu_stats():
    if nvidia_smi is None:
        print("NVIDIA SMI not available. GPU stats will be reported as zero.")
        return 0, 0
    try:
        nvidia_smi.nvmlInit()
        handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
        gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
        mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
        gpu_memory = mem_info.used / (1024 ** 2)
        nvidia_smi.nvmlShutdown()
        return gpu_load, gpu_memory / 1024
    except Exception as e:
        print(f"GPU monitoring failed: {str(e)}. GPU stats will be reported as zero.")
        return 0, 0

class ResourceMonitor(keras.callbacks.Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.resource_stats = []
        self.expert_load_history = []
        self.start_time = None
        self.epoch_start_time = None
        self.flops = None
        self.validation_data = validation_data
        self.nlu_expert_usage_counts = None
        self.nlg_expert_usage_counts = None
        self.nlu_gate_weights_history = []
        self.nlg_gate_weights_history = []

    def on_train_begin(self, logs=None):
        self.start_time = time.time()
        try:
            self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
            self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
            self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
        except Exception:
            self.flops = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory()
        gpu_load, gpu_memory = get_gpu_stats()
        epoch_time = time.time() - self.epoch_start_time
        self.resource_stats.append({
            'epoch': epoch + 1,
            'epoch_time': epoch_time,
            'cpu_usage': cpu_usage,
            'memory_used': memory.used / (1024 ** 3),
            'memory_total': memory.total / (1024 ** 3),
            'gpu_load': gpu_load,
            'gpu_memory': gpu_memory,
            'loss': logs.get('loss', 0),
            'val_loss': logs.get('val_loss', 0),
            'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
            'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
            'intent_precision': logs.get('intent_output_intent_precision', 0),
            'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
            'intent_recall': logs.get('intent_output_intent_recall', 0),
            'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
            'intent_f1': logs.get('intent_output_intent_f1', 0),
            'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
            'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
            'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
            'perplexity': logs.get('response_output_perplexity', 0),
            'val_perplexity': logs.get('val_response_output_perplexity', 0),
            'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
            'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
            'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
            'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
        })
        if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
            self.expert_load_history.append({
                'epoch': epoch + 1,
                'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
            })

        sample_size = 100
        for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
            inputs, _ = batch
            outputs = self.model(inputs, training=False)
            nlu_gate_weights = outputs['nlu_gate_weights']
            reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
            self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
            nlg_gate_weights = outputs['nlg_gate_weights']
            reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
            self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

    def on_train_end(self, logs=None):
        self.total_time = time.time() - self.start_time

        if len(self.nlu_gate_weights_history) > 0:
            all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
            nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
            for expert_id in nlu_top_indices:
                if expert_id < self.model.moe_nlu.total_experts:
                    self.nlu_expert_usage_counts[expert_id] += 1

        if len(self.nlg_gate_weights_history) > 0:
            all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
            nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
            for expert_id in nlg_top_indices:
                if expert_id < self.model.moe_nlg.total_experts:
                    self.nlg_expert_usage_counts[expert_id] += 1

    def visualize_expert_load_distribution(self):
        total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
        total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
        nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
        nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
        nlu_stability = np.var(nlu_percentages)
        nlg_stability = np.var(nlg_percentages)
        return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {
            "d_model": self.d_model.numpy(),
            "warmup_steps": self.warmup_steps
        }


word_to_id = {}
id_to_word = {}
try:
    possible_paths = [
        os.path.join("preprocessor_models", "preprocessor_params.json"),
        os.path.join("tf_datasets", "preprocessor_params.json"),
        "preprocessor_params.json"
    ]
    
    vocab_loaded = False
    for path in possible_paths:
        if os.path.exists(path):
            with open(path, "r") as f:
                preprocessor_params = json.load(f)
            word_to_id = {k: int(v) for k, v in preprocessor_params['word_to_id'].items()}
            id_to_word = {int(k): v for k, v in preprocessor_params['id_to_word'].items()}
            print(f"Loaded vocabulary from {path}")
            vocab_loaded = True
            break
    
    if not vocab_loaded:
        raise FileNotFoundError("Vocabulary file not found in any location")
except Exception as e:
    print(f"Warning: Could not load vocabulary from preprocessor: {str(e)}")
    word_to_id = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
    id_to_word = {0: '<pad>', 1: '<sos>', 2: '<eos>', 3: '<unk>'}


model_save_path = "best_deepseektopp_moe_model"
print(f"\nLoading model from: {model_save_path}")
custom_objects = {
    "MoEModel":MoEModel,
    "CustomLearningRateSchedule": CustomLearningRateSchedule,
    "MoELayer": MoELayer,
    "TransformerDecoderLayer": TransformerDecoderLayer,
    "IntentAccuracy": IntentAccuracy,
    "IntentPrecision": IntentPrecision,
    "IntentRecall": IntentRecall,
    "IntentF1": IntentF1,
    "DomainAccuracy": DomainAccuracy,
    "Perplexity": Perplexity,
    "ResponseEmbeddingCosineSimilarity": ResponseEmbeddingCosineSimilarity
}
try:
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
except Exception as e:
    print(f"Failed to load model from {model_save_path}: {str(e)}")
    raise

def compile_model(model, moe_params):
    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    losses_dict = {
        'intent_output': tf.keras.losses.BinaryCrossentropy(),
        'domain_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss,
    }

    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }

    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={
            'intent_output': 0.5,
            'domain_output': 0.5,
            'response_output': 1.0,
            'response_embeddings': 1.0
        }
    )
    return model

model = compile_model(model, moe_params)
print("\nModel loaded and compiled successfully!")
sample_input = next(iter(train_tf_dataset.take(1)))
model(sample_input[0])
model.summary()

class EvaluationMetrics:
    def __init__(self, model, test_dataset, moe_params, raw_data, word_to_id, id_to_word):
        self.model = model
        self.test_dataset = test_dataset
        self.moe_params = moe_params
        self.raw_data = raw_data
        self.word_to_id = word_to_id
        self.id_to_word = id_to_word
        self.nlu_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.nlg_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.resource_stats = {
            'cpu_usage': [],
            'memory_used': [],
            'gpu_load': [],
            'gpu_memory': [],
            'batch_times': []
        }
        self.special_tokens = {'<pad>', '<sos>', '<eos>', '<unk>'}
        self.nlu_top_k = model.get_layer('moe_nlu').get_config().get('top_k', 2)
        self.nlg_top_k = model.get_layer('moe_nlg').get_config().get('top_k', 2)

    def evaluate(self):
        print("\nEvaluating model on test set...")
        
        def add_response_embeddings(features, targets):
            response_tokens = targets['response_output']
            response_embeddings = self.model.embedding(response_tokens)
            targets['response_embeddings'] = response_embeddings
            return features, targets
        
        test_dataset_with_embeddings = self.test_dataset.map(add_response_embeddings, num_parallel_calls=tf.data.AUTOTUNE)

        start_time = time.time()
        test_results = self.model.evaluate(test_dataset_with_embeddings, verbose=1)
        eval_time = time.time() - start_time
        
        metric_names = self.model.metrics_names
        
        perplexity_value = None
        for name, value in zip(metric_names, test_results):
            if name == 'response_output_perplexity':
                perplexity_value = value
                break
        if perplexity_value is None:
            print("Warning: Could not find perplexity in standard evaluation results.")
            perplexity_value = 0.0
        
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory().used / (1024 ** 3)
        gpu_load, gpu_memory = get_gpu_stats()
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))

        total_tokens = 0
        total_time = 0
        num_batches = 0
        
        test_dataset_small = test_dataset_with_embeddings.unbatch().batch(moe_params["batch_size"])
        
        all_true_intents = []
        all_pred_intents = []
        all_true_domains = []
        all_pred_domains = []
        
        for batch_idx, batch in enumerate(test_dataset_small):
            inputs, targets = batch
            response_tokens = targets['response_output'].numpy()
            non_padding_mask = response_tokens != 0
            total_tokens += np.sum(non_padding_mask)
            
            start_time = time.time()
            outputs = self.model(inputs, training=False)
            batch_time = time.time() - start_time
            
            total_time += batch_time
            num_batches += tf.shape(inputs['user_utterance_tokens'])[0]
            
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            
            self.resource_stats['cpu_usage'].append(cpu_usage)
            self.resource_stats['memory_used'].append(memory.used / (1024 ** 3))
            self.resource_stats['gpu_load'].append(gpu_load)
            self.resource_stats['gpu_memory'].append(gpu_memory)
            self.resource_stats['batch_times'].append(batch_time)
            
            nlu_gate_weights = outputs.get('nlu_gate_weights', tf.zeros([0, self.model.moe_nlu.routed_experts])).numpy()
            nlu_top_indices = np.argsort(-nlu_gate_weights, axis=-1)[:, :self.nlu_top_k] + self.model.moe_nlu.k_s
            for idx in range(self.nlu_top_k):
                expert_ids = nlu_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlu_expert_usage_counts):
                        self.nlu_expert_usage_counts[expert_id] += 1
            
            nlg_gate_weights = outputs.get('nlg_gate_weights', tf.zeros([0, self.model.moe_nlg.routed_experts])).numpy()
            if len(nlg_gate_weights.shape) == 3:
                nlg_gate_weights = nlg_gate_weights.reshape(-1, nlg_gate_weights.shape[-1])
            nlg_top_indices = np.argsort(-nlg_gate_weights, axis=-1)[:, :self.nlg_top_k] + self.model.moe_nlg.k_s
            for idx in range(self.nlg_top_k):
                expert_ids = nlg_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlg_expert_usage_counts):
                        self.nlg_expert_usage_counts[expert_id] += 1
            
            true_intents = targets.get('intent_output', np.zeros((moe_params["batch_size"], self.model.num_intents))).numpy()
            pred_intents = (outputs.get('intent_output', np.zeros_like(true_intents)).numpy() > 0.5).astype(np.int32)
            all_true_intents.append(true_intents)
            all_pred_intents.append(pred_intents)
            
            true_domains = targets.get('domain_output', np.zeros((moe_params["batch_size"], 1))).numpy().flatten()
            pred_domains = np.argmax(outputs.get('domain_output', np.zeros((moe_params["batch_size"], self.model.num_domains))).numpy(), axis=-1)
            all_true_domains.append(true_domains)
            all_pred_domains.append(pred_domains)
            
            if batch_idx % 50 == 0:
                gc.collect()
                tf.keras.backend.clear_session()
        
        avg_time_per_token = (total_time / total_tokens) * 1000 if total_tokens > 0 else 0
        
        avg_cpu = np.mean(self.resource_stats['cpu_usage']) if self.resource_stats['cpu_usage'] else 0
        avg_memory = np.mean(self.resource_stats['memory_used']) if self.resource_stats['memory_used'] else 0
        avg_gpu_load = np.mean(self.resource_stats['gpu_load']) if self.resource_stats['gpu_load'] else 0
        avg_gpu_memory = np.mean(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        peak_gpu_memory = np.max(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))
        
        all_true_intents = np.vstack(all_true_intents)
        all_pred_intents = np.vstack(all_pred_intents)
        all_true_domains = np.concatenate(all_true_domains)
        all_pred_domains = np.concatenate(all_pred_domains)
        
        intent_f1_score = f1_score(all_true_intents, all_pred_intents, average='micro')
        intent_f1_score_macro = f1_score(all_true_intents, all_pred_intents, average='macro')
        intent_f1_score_weighted = f1_score(all_true_intents, all_pred_intents, average='weighted')
        intent_accuracy_score = accuracy_score(all_true_intents, all_pred_intents)
        
        domain_accuracy_score = accuracy_score(all_true_domains, all_pred_domains)
        domain_f1_score_macro = f1_score(all_true_domains, all_pred_domains, average='macro')

        hypotheses = []
        references = []
        meteor_scores = []
        rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        
        domain_metrics = defaultdict(lambda: {
            'bleu': [],
            'rouge1': [],
            'rouge2': [],
            'rougeL': [],
            'meteor': [],
            'count': 0
        })
        
        if not self.raw_data.get('test'):
            print("Warning: raw_data['test'] is empty. Skipping NLG metrics calculation.")
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0
            domain_metrics = {}
        else:
            for i, batch in enumerate(test_dataset_small):
                if i >= len(self.raw_data.get('test', [])):
                    print(f"Warning: raw_data['test'] has fewer items ({len(self.raw_data.get('test', []))}) than test dataset. Stopping NLG metrics calculation.")
                    break
                
                try:
                    inputs, targets = batch
                    with tf.device('/CPU:0'):
                        outputs = self.model(inputs, training=False)
                    
                    domain_id = targets['domain_output'].numpy()[0][0]
                    domain_name = f"domain_{domain_id}"
                    
                    response_probs = outputs['response_output'].numpy()
                    generated_tokens = np.argmax(response_probs, axis=-1)[0]
                    true_tokens = targets['response_output'].numpy()[0]
                    
                    raw_item = self.raw_data['test'][i]
                    ref_text = raw_item.get('raw_next_system_response', '')
                    if not ref_text:
                        continue
                    
                    gen_text = self.tokens_to_text(generated_tokens)
                    ref_tokens = self.tokens_to_text(true_tokens)
                    
                    if not gen_text.strip() or not ref_text.strip():
                        continue
                    
                    hypotheses.append(gen_text)
                    references.append([ref_text])
                    
                    ref_tokens = word_tokenize(ref_text.lower())
                    gen_tokens = word_tokenize(gen_text.lower())
                    
                    try:
                        meteor = meteor_score([ref_tokens], gen_tokens)
                        meteor_scores.append(meteor)
                        domain_metrics[domain_name]['meteor'].append(meteor)
                    except Exception as e:
                        print(f"METEOR calculation error for example {i}: {str(e)}")
                        meteor_scores.append(0)
                        domain_metrics[domain_name]['meteor'].append(0)
                    
                    try:
                        rouge_scores = rouge_scorer_instance.score(ref_text, gen_text)
                        domain_metrics[domain_name]['rouge1'].append(rouge_scores['rouge1'].fmeasure)
                        domain_metrics[domain_name]['rouge2'].append(rouge_scores['rouge2'].fmeasure)
                        domain_metrics[domain_name]['rougeL'].append(rouge_scores['rougeL'].fmeasure)
                    except Exception as e:
                        print(f"ROUGE calculation error for example {i}: {str(e)}")
                    
                    domain_metrics[domain_name]['count'] += 1
                    
                    if i % 50 == 0:
                        gc.collect()
                        tf.keras.backend.clear_session()
                    
                except Exception as e:
                    print(f"Error processing example {i}: {str(e)}")
                    continue
        
        if hypotheses and references:
            hypotheses_tok = []
            references_tok = []
            
            for hyp, ref_list in zip(hypotheses, references):
                if not hyp or not ref_list[0]:
                    continue
                hyp_tokens = word_tokenize(hyp.lower())
                ref_tokens = [word_tokenize(ref.lower()) for ref in ref_list]
                hypotheses_tok.append(hyp_tokens)
                references_tok.append(ref_tokens)
            
            chencherry = SmoothingFunction()
            try:
                bleu_score = corpus_bleu(references_tok, hypotheses_tok, smoothing_function=chencherry.method1)
            except Exception as e:
                print(f"BLEU calculation error: {str(e)}")
                bleu_score = 0.0
            
            rouge_scores = []
            for hyp, ref in zip(hypotheses, references):
                if hyp and ref[0]:
                    try:
                        rouge_scores.append(rouge_scorer_instance.score(ref[0], hyp))
                    except Exception as e:
                        print(f"ROUGE scoring error: {str(e)}")
                        continue
            
            avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_meteor = np.mean(meteor_scores) if meteor_scores else 0.0
        else:
            print("Warning: No valid hypotheses or references for NLG metrics. Setting to 0.0.")
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0

        return {
            'intent_f1_micro': intent_f1_score,
            'intent_f1_macro': intent_f1_score_macro,
            'intent_f1_weighted': intent_f1_score_weighted,
            'intent_accuracy': intent_accuracy_score,
            'domain_accuracy': domain_accuracy_score,
            'domain_f1_macro': domain_f1_score_macro,
            'bleu': bleu_score,
            'rouge1': avg_rouge1,
            'rouge2': avg_rouge2,
            'rougeL': avg_rougeL,
            'meteor': avg_meteor,
            'perplexity': perplexity_value,
            'pred_speed': avg_time_per_token,
            'domain_metrics': domain_metrics,
            'eval_time': eval_time
        }
    
    def tokens_to_text(self, tokens):
        words = []
        for token in tokens:
            if token == 0:
                continue
            word = self.id_to_word.get(token, '<unk>')
            if word not in self.special_tokens:
                words.append(word)
        return " ".join(words).strip()

results_table = {
    'Evaluation Time (seconds)': [],
    'Prediction Speed (ms/token)': [],
    'Average CPU Usage (percent)': [],
    'Average GPU Usage (percent)': [],
    'Average Memory (GB)': [],
    'Average GPU Memory (GB)': [],
    'Average FLOPs Estimate (GFLOPs)': [],
    'NLU Expert 1 (percent)': [],
    'NLU Expert 2 (percent)': [],
    'NLU Expert 3 (percent)': [],
    'NLU Expert 4 (percent)': [],
    'NLU Expert 5 (percent)': [],
    'NLU Expert 6 (percent)': [],
    'NLU Expert 7 (percent)': [],
    'NLU Expert 8 (percent)': [],
    'NLG Expert 1 (percent)': [],
    'NLG Expert 2 (percent)': [],
    'NLG Expert 3 (percent)': [],
    'NLG Expert 4 (percent)': [],
    'NLG Expert 5 (percent)': [],
    'NLG Expert 6 (percent)': [],
    'NLG Expert 7 (percent)': [],
    'NLG Expert 8 (percent)': [],
    'Intent F1-score (Micro)': [],
    'Intent F1-score (Macro)': [],
    'Intent F1-score (Weighted)': [],
    'Intent Accuracy': [],
    'Domain Accuracy': [],
    'Domain F1-score (Macro)': [],
    'BLEU Score': [],
    'ROUGE-1 F1': [],
    'ROUGE-2 F1': [],
    'ROUGE-L F1': [],
    'METEOR Score': [],
    'Perplexity': [],
    'NLU Expert Load Stability (Variance)': [],
    'NLG Expert Load Stability (Variance)': []
}

num_iterations = 5
for iteration in range(1, num_iterations + 1):
    print(f"\nStarting Iteration {iteration}")
    tf.keras.backend.clear_session()
    gc.collect()
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
    model = compile_model(model, moe_params)
    
    evaluator = EvaluationMetrics(model, test_tf_dataset, moe_params, raw_data, word_to_id, id_to_word)

    evaluation_results = evaluator.evaluate()
    results_table['Evaluation Time (seconds)'].append(evaluation_results['eval_time'])
    results_table['Prediction Speed (ms/token)'].append(evaluation_results['pred_speed'])
    results_table['Average CPU Usage (percent)'].append(np.mean(evaluator.resource_stats['cpu_usage']) if evaluator.resource_stats['cpu_usage'] else 0)
    results_table['Average GPU Usage (percent)'].append(np.mean(evaluator.resource_stats['gpu_load']) if evaluator.resource_stats['gpu_load'] else 0)
    results_table['Average Memory (GB)'].append(np.mean(evaluator.resource_stats['memory_used']) if evaluator.resource_stats['memory_used'] else 0)
    results_table['Average GPU Memory (GB)'].append(np.mean(evaluator.resource_stats['gpu_memory']) if evaluator.resource_stats['gpu_memory'] else 0)
    results_table['Average FLOPs Estimate (GFLOPs)'].append(calculate_flops(model))
    nlu_counts = evaluator.nlu_expert_usage_counts
    nlg_counts = evaluator.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) if np.sum(nlu_counts) > 0 else 1
    total_nlg = np.sum(nlg_counts) if np.sum(nlg_counts) > 0 else 1
    results_table['NLU Expert 1 (percent)'].append((nlu_counts[0] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 2 (percent)'].append((nlu_counts[1] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 3 (percent)'].append((nlu_counts[2] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 4 (percent)'].append((nlu_counts[3] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 5 (percent)'].append((nlu_counts[4] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 6 (percent)'].append((nlu_counts[5] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 7 (percent)'].append((nlu_counts[6] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 8 (percent)'].append((nlu_counts[7] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLG Expert 1 (percent)'].append((nlg_counts[0] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 2 (percent)'].append((nlg_counts[1] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 3 (percent)'].append((nlg_counts[2] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 4 (percent)'].append((nlg_counts[3] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 5 (percent)'].append((nlg_counts[4] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 6 (percent)'].append((nlg_counts[5] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 7 (percent)'].append((nlg_counts[6] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 8 (percent)'].append((nlg_counts[7] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['Intent F1-score (Micro)'].append(evaluation_results['intent_f1_micro'])
    results_table['Intent F1-score (Macro)'].append(evaluation_results['intent_f1_macro'])
    results_table['Intent F1-score (Weighted)'].append(evaluation_results['intent_f1_weighted'])
    results_table['Intent Accuracy'].append(evaluation_results['intent_accuracy'])
    results_table['Domain Accuracy'].append(evaluation_results['domain_accuracy'])
    results_table['Domain F1-score (Macro)'].append(evaluation_results['domain_f1_macro'])
    results_table['BLEU Score'].append(evaluation_results['bleu'])
    results_table['ROUGE-1 F1'].append(evaluation_results['rouge1'])
    results_table['ROUGE-2 F1'].append(evaluation_results['rouge2'])
    results_table['ROUGE-L F1'].append(evaluation_results['rougeL'])
    results_table['METEOR Score'].append(evaluation_results['meteor'])
    results_table['Perplexity'].append(evaluation_results['perplexity'])
    results_table['NLU Expert Load Stability (Variance)'].append(np.var(nlu_counts / total_nlu * 100) if total_nlu > 0 else 0)
    results_table['NLG Expert Load Stability (Variance)'].append(np.var(nlg_counts / total_nlg * 100) if total_nlg > 0 else 0)

for key in results_table:
    if results_table[key]:
        results_table[key].append(np.mean(results_table[key]))
    else:
        results_table[key].append(0)

print("\nTraining results saved")
#final_df
final_df = pd.DataFrame(results_table)
final_df = final_df.T
final_df.columns = [f'Iteration {i+1}' for i in range(num_iterations)] + ['Average']
final_df.to_excel('prediction_deepseektopp_results.xlsx', index=False)
final_df


Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 06:34:34.785026: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 06:34:35.143446: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 06:34:35.143669: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl
Loaded vocabulary from preprocessor_models/preprocessor_params.json

Loading model from: best_deepseektopp_moe_model

Model loaded and compiled successfully!
Model: "mo_e_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  784896    
                                                                 
 embedding (Embedding)       multiple                  8576      
                                                                 
 layer_normalization (Layer  multiple                  256       
 Normalization)                                                  
                                                                 
 transformer_decoder_layer   multiple                  264576    
 (TransformerDecoderLayer)              

,Iteration 1,Average
Evaluation Time (seconds),22.119367,22.119367
Prediction Speed (ms/token),2.530680,2.530680
Average CPU Usage (percent),20.531353,20.531353
Average GPU Usage (percent),1.837509,1.837509
Average Memory (GB),5.678094,5.678094
Average GPU Memory (GB),14.504456,14.504456
Average FLOPs Estimate (GFLOPs),2.377871,2.377871
NLU Expert 1 (percent),0.000000,0.000000
NLU Expert 2 (percent),0.000000,0.000000
NLU Expert 3 (percent),9.824624,9.824624


# DeepSeekMoE Noisy Top-K Gating experiment code

In [1]:
def reset_kernel():
    tf.keras.backend.clear_session()
    import gc
    gc.collect()

results_table = {
    'Training Speed (epochs per second)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Training Time (second/epoch)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Total Training Time (second/iteration)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Computational Resource Usage': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average CPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average GPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average GPU Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average FLOPS Estimate (GFLOPS)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Expert NLU Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Expert NLG Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Entity Accuracy': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Intent Accuracy': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Perplexity': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Response Cosine Similarity': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0}
}

best_val_loss = float('inf')
best_model_path = "best_noisytop-K_moe_model"

for iteration in range(5):
    print(f"\nStarting Training Iteration {iteration + 1}")
    reset_kernel()

    def load_tf_datasets_from_disk(load_path):
        print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
        try:
            with open(os.path.join(load_path, "moe_params.json"), "r") as f:
                loaded_moe_params = json.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"moe_params.json not found in {load_path}")

        element_spec = (
            {
                'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
                'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
                'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            },
            {
                'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
                'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
                'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
            }
        )

        try:
            train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
            test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
        except Exception as e:
            raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

        raw_data = {}
        for dataset_name in ['train', 'test']:
            path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
            if os.path.exists(path):
                with open(path, 'rb') as f:
                    raw_data[dataset_name] = pickle.load(f)
                print(f"Loaded raw data for {dataset_name} from {path}")
            else:
                print(f"Warning: Raw data file {path} not found.")
                raw_data[dataset_name] = []

        return {
            "train_dataset": train_tf_dataset,
            "test_dataset": test_tf_dataset,
            "moe_params": loaded_moe_params,
            "raw_data": raw_data
        }

    tf_dataset_save_path = "tf_datasets"
    loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
    train_tf_dataset = loaded_data["train_dataset"]
    moe_params = loaded_data["moe_params"]
    raw_data = loaded_data["raw_data"]

    moe_params["num_experts"] = 4

    class NoisyTopKGating(layers.Layer):
        def __init__(self, num_experts, input_dim, k=2, noise_epsilon=1.0, name=None):
            super(NoisyTopKGating, self).__init__(name=name)
            self.num_experts = num_experts
            self.input_dim = input_dim
            self.k = k
            self.noise_epsilon = noise_epsilon
            self.w_g = self.add_weight(
                name='w_g',
                shape=(input_dim, num_experts),
                initializer='glorot_uniform',
                trainable=True
            )
            self.w_noise = self.add_weight(
                name='w_noise',
                shape=(input_dim, num_experts),
                initializer=keras.initializers.Constant(value=noise_epsilon),
                trainable=True
            )
            self.softmax = layers.Softmax(axis=-1)

        def call(self, inputs, training=False, usage_stats=None):
            gate_logits = tf.matmul(inputs, self.w_g)
            if training and usage_stats is not None:
                usage_mean = tf.reduce_mean(usage_stats)
                usage_std = tf.math.reduce_std(usage_stats)
                imbalance_factor = tf.clip_by_value(usage_std / (usage_mean + 1e-6), 0.0, 1.0)
                adjusted_k = tf.cast(
                    self.k * (1.0 + imbalance_factor * 2),
                    tf.int32
                )
                adjusted_k = tf.minimum(tf.maximum(adjusted_k, self.k), self.num_experts)
            else:
                adjusted_k = self.k

            if training:
                raw_noise = tf.random.normal(
                    shape=(tf.shape(inputs)[0], self.num_experts),
                    mean=0.0,
                    stddev=1.0
                )
                gate_std = tf.math.reduce_std(gate_logits, axis=-1, keepdims=True)
                noise_scale = tf.nn.softplus(tf.matmul(inputs, self.w_noise)) * (1.0 + gate_std)
                noisy_gate_logits = gate_logits + (raw_noise * noise_scale)
            else:
                noisy_gate_logits = gate_logits
            top_k_values, _ = tf.math.top_k(noisy_gate_logits, k=adjusted_k)
            min_values = tf.reduce_min(top_k_values, axis=-1, keepdims=True)
            top_k_mask = tf.cast(
                noisy_gate_logits >= min_values,
                dtype=tf.float32
            )
            keep_top_k_logits = tf.where(
                top_k_mask > 0,
                noisy_gate_logits,
                tf.ones_like(noisy_gate_logits) * -float('inf')
            )
            gate_outputs = self.softmax(keep_top_k_logits)
            return gate_outputs, gate_logits

    class MoELayer(layers.Layer):
        def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.5, top_k=2, name=None):
            super(MoELayer, self).__init__(name=name)
            self.m = m
            self.k_s = k_s
            self.total_experts = num_experts * self.m
            self.routed_experts = self.total_experts - self.k_s
            self.top_k = top_k
            self.top_mk = self.m * self.top_k
            self.top_mk = min(self.top_mk, self.routed_experts)
            self.alpha = alpha
            self.input_dim = input_dim
            self.seq_length = None
            self.usage_stats = tf.Variable(
                tf.ones(self.routed_experts) / self.routed_experts,
                trainable=False,
                dtype=tf.float32
            )
            self.adjusted_expert_dim = expert_dim // self.m
            self.shared_experts = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu'),
                    layers.Dense(input_dim, activation=None)
                ], name=f'shared_expert_{i}') for i in range(self.k_s)
            ]
            self.routed_experts_list = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu'),
                    layers.Dense(input_dim, activation=None)
                ], name=f'routed_expert_{i}') for i in range(self.k_s, self.total_experts)
            ]
            self.experts = self.shared_experts + self.routed_experts_list
            self.gate = NoisyTopKGating(
                num_experts=self.routed_experts,
                input_dim=self.input_dim,
                k=self.top_mk,
                noise_epsilon=1.0,
                name='noisy_topk_gate'
            )
            self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')
            self.expert_usage_metric = tf.keras.metrics.Mean(name=f'{name}_expert_usage')

        def call(self, inputs, training=False):
            input_rank = inputs.shape.rank
            if input_rank == 3:
                batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
                flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
                is_3d = True
            else:
                batch_size = tf.shape(inputs)[0]
                seq_length = 1
                flat_inputs = inputs
                is_3d = False

            flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
            shared_output = tf.zeros([tf.shape(flat_inputs)[0], self.input_dim], dtype=tf.float32)
            for i in range(self.k_s):
                shared_output += self.shared_experts[i](flat_inputs)
            gate_weights, clean_logits = self.gate(flat_inputs, training=training, usage_stats=self.usage_stats)
            top_mk_values, top_mk_indices = tf.nn.top_k(gate_weights, k=self.top_mk, sorted=True)
            top_mk_indices = top_mk_indices + self.k_s
            top_mk_weights = top_mk_values / (tf.reduce_sum(top_mk_values, axis=-1, keepdims=True) + tf.keras.backend.epsilon())
            routed_output = tf.zeros([tf.shape(flat_inputs)[0], self.input_dim], dtype=tf.float32)

            for k in range(self.top_mk):
                kth_weights = top_mk_weights[:, k]
                kth_indices = top_mk_indices[:, k]
                mask = tf.one_hot(kth_indices, depth=self.total_experts, dtype=tf.float32)
                expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
                kth_expert_output = tf.reduce_sum(expert_outputs * tf.expand_dims(mask, -1), axis=1)
                weighted_kth_output = kth_expert_output * tf.expand_dims(kth_weights, -1)
                routed_output += weighted_kth_output

            output_flat = shared_output + routed_output + flat_inputs

            if is_3d:
                output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
                gate_weights_reshaped = tf.reshape(gate_weights, [batch_size, seq_length, self.routed_experts])
            else:
                output = output_flat
                gate_weights_reshaped = gate_weights
            expert_assignments = tf.argmax(clean_logits, axis=-1)
            f_i = tf.reduce_mean(
                tf.one_hot(expert_assignments, depth=self.routed_experts, dtype=tf.float32),
                axis=0
            )
            P_i = tf.reduce_mean(gate_weights, axis=0)
            cov = tf.reduce_sum((f_i - tf.reduce_mean(f_i)) * (P_i - tf.reduce_mean(P_i)))
            load_balancing_loss = self.alpha * (tf.reduce_sum(f_i * P_i) + 0.1 * cov)
            self.add_loss(load_balancing_loss)
            self.load_balancing_loss_metric.update_state(load_balancing_loss)
            if training:
                current_usage = tf.reduce_mean(
                    tf.one_hot(expert_assignments, depth=self.routed_experts, dtype=tf.float32),
                    axis=0
                )
                decay_rate = tf.minimum(0.1, 1.0 / (tf.reduce_sum(self.usage_stats) + 1e-6))
                self.usage_stats.assign(
                    self.usage_stats * (1.0 - decay_rate) + current_usage * decay_rate
                )
            expert_usage = tf.reduce_mean(tf.reduce_sum(tf.cast(gate_weights > 0, tf.float32), axis=0))
            self.expert_usage_metric.update_state(expert_usage)
            return output, gate_weights_reshaped

        def get_metrics(self):
            return {
                f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric,
                f'{self.name}_expert_usage': self.expert_usage_metric
            }

        def get_config(self):
            config = super().get_config()
            config.update({
                'num_experts': self.total_experts // self.m,
                'expert_dim': self.adjusted_expert_dim * self.m,
                'input_dim': self.input_dim,
                'm': self.m,
                'k_s': self.k_s,
                'alpha': self.alpha,
                'top_k': self.top_k,
                'name': self.name
            })
            return config

    class TransformerDecoderLayer(layers.Layer):
        def __init__(self, d_model, num_heads, dff, rate=0.1):
            super(TransformerDecoderLayer, self).__init__()
            self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.ffn = keras.Sequential([
                layers.Dense(dff, activation='relu'),
                layers.Dense(d_model)
            ])
            self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
            self.dropout1 = layers.Dropout(rate)
            self.dropout2 = layers.Dropout(rate)
            self.dropout3 = layers.Dropout(rate)

        def call(self, x, enc_output, training, look_ahead_mask=None):
            if look_ahead_mask is not None:
                look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
                look_ahead_mask = 1.0 - look_ahead_mask
            attn1_output = self.mha1(
                query=x,
                value=x,
                key=x,
                attention_mask=look_ahead_mask,
                training=training,
                return_attention_scores=True
            )
            if isinstance(attn1_output, tuple):
                attn1, attn_weights1 = attn1_output
            else:
                attn1, attn_weights1 = attn1_output, None
            attn1 = self.dropout1(attn1, training=training)
            out1 = self.layernorm1(attn1 + x)
            attn2_output = self.mha2(
                query=out1,
                value=enc_output,
                key=enc_output,
                attention_mask=None,
                training=training,
                return_attention_scores=True
            )
            if isinstance(attn2_output, tuple):
                attn2, attn_weights2 = attn2_output
            else:
                attn2, attn_weights2 = attn2_output, None
            attn2 = self.dropout2(attn2, training=training)
            out2 = self.layernorm2(attn2 + out1)
            ffn_output = self.ffn(out2)
            ffn_output = self.dropout3(ffn_output, training=training)
            out3 = self.layernorm3(ffn_output + out2)
            return out3, attn_weights1, attn_weights2

        def get_config(self):
            config = super().get_config()
            mha1_config = self.mha1.get_config()
            config.update({
                'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
                'num_heads': mha1_config['num_heads'],
                'dff': self.ffn.layers[0].units,
                'rate': self.dropout1.rate
            })
            return config

    class MoEModel(models.Model):
        def __init__(self, moe_params):
            super(MoEModel, self).__init__()
            self.embedding_dim = moe_params["embedding_dim"]
            self.max_seq_length = moe_params["max_seq_length"]
            self.num_domains = moe_params["num_domains"]
            self.num_intents = moe_params["num_intents"]
            self.vocab_size = moe_params["vocab_size"]
            self.num_experts = moe_params["num_experts"]
            self.expert_dim = moe_params["expert_dim"]
            self.turn_id_dim = moe_params["turn_id_dim"]
            self.num_heads = 4
            self.dff = self.embedding_dim * 4
            self.dropout_rate = 0.1
            self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                                 self.num_domains + self.turn_id_dim + self.num_intents)
            self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains
            self.embedding = layers.Embedding(
                self.vocab_size,
                self.embedding_dim,
                mask_zero=True,
                embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05),
                name="embedding"
            )
            self.embedding.build((None,))
            with tf.init_scope():
                embedding_weights = self.embedding.get_weights()
                if embedding_weights and len(embedding_weights) > 0:
                    embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                    self.embedding.set_weights(embedding_weights)
            self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
            self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
            self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
            self.decoder_dropout = layers.Dropout(self.dropout_rate)
            self.moe_nlu = MoELayer(
                num_experts=self.num_experts,
                expert_dim=self.expert_dim,
                input_dim=self.nlu_input_dim,
                m=2,
                k_s=2,
                alpha=0.01,
                top_k=2,
                name='moe_nlu'
            )
            self.moe_nlg = MoELayer(
                num_experts=self.num_experts,
                expert_dim=self.expert_dim,
                input_dim=self.nlg_input_dim,
                m=2,
                k_s=2,
                alpha=0.01,
                top_k=2,
                name='moe_nlg'
            )
            self.intent_output = layers.Dense(self.num_intents, activation='sigmoid', name='intent_output')
            self.domain_output = layers.Dense(self.num_domains, activation='softmax', name='domain_output')
            self.response_output = layers.TimeDistributed(
                layers.Dense(self.vocab_size, activation='softmax'), name='response_output'
            )
            self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)
            self.nlg_projection = layers.TimeDistributed(
                layers.Dense(self.embedding_dim, activation='relu', name='nlg_projection')
            )

        def create_look_ahead_mask(self, size):
            mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
            return mask

        def call(self, inputs, training=False):
            user_utterance_tokens = inputs['user_utterance_tokens']
            prev_system_response_tokens = inputs['prev_system_response_tokens']
            decoder_input_tokens = inputs['decoder_input_tokens']
            domain_onehot_input = inputs['domain_onehot_input']
            turn_id_embedding = inputs['turn_id_embedding']
            ontology_multihot_input = inputs['ontology_multihot_input']
            pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
            user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
            prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
            decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)
            user_enc_out = user_embedded
            prev_system_enc_out = prev_system_embedded
            look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
            dec_output = decoder_embedded
            attn_weights = {}
            for i, layer in enumerate(self.decoder_layers):
                dec_output, attn_w1, attn_w2 = layer(
                    dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask
                )
                attn_weights[f'decoder_layer{i+1}_attn1'] = attn_w1
                attn_weights[f'decoder_layer{i+1}_attn2'] = attn_w2
            decoder_output = self.decoder_dropout(dec_output, training=training)
            user_attn_out = self.attn_layer(
                query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), 
                value=user_enc_out, key=user_enc_out, training=training
            )
            prev_system_attn_out = self.attn_layer(
                query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), 
                value=prev_system_enc_out, key=prev_system_enc_out, training=training
            )
            decoder_attn_out = self.attn_layer(
                query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), 
                value=decoder_output, key=decoder_output, training=training
            )
            combined_features = layers.Concatenate()([
                tf.squeeze(user_attn_out, 1),
                tf.squeeze(prev_system_attn_out, 1),
                tf.squeeze(decoder_attn_out, 1),
                domain_onehot_input,
                turn_id_embedding,
                ontology_multihot_input
            ])
            combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
            nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
            nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
            intent_out = self.intent_output(nlu_out)
            domain_out = self.domain_output(nlu_out)
            intent_out_expanded = tf.expand_dims(intent_out, axis=1)
            domain_out_expanded = tf.expand_dims(domain_out, axis=1)
            intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
            domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])
            nlg_input = layers.Concatenate(axis=-1)([
                decoder_output,
                intent_out_tiled,
                domain_out_tiled
            ])
            nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])
            nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
            nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
            response_embeddings = self.nlg_projection(nlg_out)
            response_embeddings = tf.ensure_shape(response_embeddings, [None, self.max_seq_length, self.embedding_dim])
            response_out = self.response_output(nlg_out)
            return {
                'intent_output': intent_out,
                'domain_output': domain_out,
                'response_output': response_out,
                'response_embeddings': response_embeddings,
                'nlu_gate_weights': nlu_gate_weights,
                'nlg_gate_weights': nlg_gate_weights
            }

        def build_graph(self):
            inputs = {
                'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='user_utterance_tokens'),
                'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='prev_system_response_tokens'),
                'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='decoder_input_tokens'),
                'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32, name='domain_onehot_input'),
                'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32, name='turn_id_embedding'),
                'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32, name='ontology_multihot_input'),
            }
            return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

        def get_config(self):
            config = super().get_config()
            config.update({
                'moe_params': {
                    'embedding_dim': self.embedding_dim,
                    'max_seq_length': self.max_seq_length,
                    'num_domains': self.num_domains,
                    'num_intents': self.num_intents,
                    'vocab_size': self.vocab_size,
                    'num_experts': self.num_experts,
                    'expert_dim': self.expert_dim,
                    'turn_id_dim': self.turn_id_dim
                }
            })
            return config

        def compile(self, **kwargs):
            super().compile(**kwargs)
            self.val_moe_nlu_load_balancing_loss_metric = tf.keras.metrics.Mean(name='val_moe_nlu_load_balancing_loss')
            self.val_moe_nlg_load_balancing_loss_metric = tf.keras.metrics.Mean(name='val_moe_nlg_load_balancing_loss')

        def test_step(self, data):
            x, y = data
            y_pred = self(x, training=False)
            self.compiled_loss(y, y_pred)
            self.compiled_metrics.update_state(y, y_pred)
            nlu_gate_weights = y_pred['nlu_gate_weights']
            nlg_gate_weights = y_pred['nlg_gate_weights']
            nlu_expert_assignments = tf.argmax(nlu_gate_weights, axis=-1)
            nlu_f_i = tf.reduce_mean(
                tf.one_hot(nlu_expert_assignments, depth=self.moe_nlu.routed_experts, dtype=tf.float32),
                axis=0
            )
            nlu_P_i = tf.reduce_mean(nlu_gate_weights, axis=0)
            nlu_load_balancing_loss = self.moe_nlu.alpha * tf.reduce_sum(nlu_f_i * nlu_P_i)
            self.val_moe_nlu_load_balancing_loss_metric.update_state(nlu_load_balancing_loss)
            nlg_expert_assignments = tf.argmax(nlg_gate_weights, axis=-1)
            nlg_f_i = tf.reduce_mean(
                tf.one_hot(nlg_expert_assignments, depth=self.moe_nlg.routed_experts, dtype=tf.float32),
                axis=0
            )
            nlg_P_i = tf.reduce_mean(nlg_gate_weights, axis=0)
            nlg_load_balancing_loss = self.moe_nlg.alpha * tf.reduce_sum(nlg_f_i * nlg_P_i)
            self.val_moe_nlg_load_balancing_loss_metric.update_state(nlg_load_balancing_loss)
            return {m.name: m.result() for m in self.metrics + [self.val_moe_nlu_load_balancing_loss_metric, self.val_moe_nlg_load_balancing_loss_metric]}

    class IntentAccuracy(metrics.Metric):
        def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros', dtype=tf.float32)
            self.total_preds = self.add_weight(name='total_preds', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
            self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
            self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

        def reset_state(self):
            self.correct_preds.assign(0.)
            self.total_preds.assign(0.)

    class IntentPrecision(metrics.Metric):
        def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
            self.predicted_positives = self.add_weight(name='pp', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            predicted_positives = tf.reduce_sum(y_pred)
            self.true_positives.assign_add(true_positives)
            self.predicted_positives.assign_add(predicted_positives)

        def result(self):
            return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.predicted_positives.assign(0.)

    class IntentRecall(metrics.Metric):
        def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
            self.actual_positives = self.add_weight(name='ap', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            actual_positives = tf.reduce_sum(y_true)
            self.true_positives.assign_add(true_positives)
            self.actual_positives.assign_add(actual_positives)

        def result(self):
            return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.actual_positives.assign(0.)

    class IntentF1(metrics.Metric):
        def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.precision = IntentPrecision(threshold=threshold)
            self.recall = IntentRecall(threshold=threshold)

        def update_state(self, y_true, y_pred, sample_weight=None):
            self.precision.update_state(y_true, y_pred, sample_weight)
            self.recall.update_state(y_true, y_pred, sample_weight)

        def result(self):
            p = self.precision.result()
            r = self.recall.result()
            return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

        def reset_state(self):
            self.precision.reset_state()
            self.recall.reset_state()

    class DomainAccuracy(metrics.Metric):
        def __init__(self, name='domain_accuracy', **kwargs):
            super().__init__(name=name, **kwargs)
            self.accuracy = self.add_weight(name='acc', initializer='zeros', dtype=tf.float32)
            self.count = self.add_weight(name='count', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
            pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
            matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
            self.accuracy.assign_add(tf.reduce_sum(matches))
            self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.accuracy / (self.count + tf.keras.backend.epsilon())

        def reset_state(self):
            self.accuracy.assign(0.)
            self.count.assign(0.)

    class Perplexity(metrics.Metric):
        def __init__(self, name='perplexity', **kwargs):
            super().__init__(name=name, **kwargs)
            self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
            self.total_loss = self.add_weight(name='total_loss', initializer='zeros', dtype=tf.float32)
            self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            mask = tf.cast(y_true != 0, dtype=tf.float32)
            loss = self.cross_entropy(y_true, y_pred)
            masked_loss = loss * mask
            sum_loss = tf.reduce_sum(masked_loss)
            num_non_padding_tokens = tf.reduce_sum(mask)
            self.total_loss.assign_add(sum_loss)
            self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

        def result(self):
            avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
            return tf.exp(avg_loss)

        def reset_state(self):
            self.total_loss.assign(0.)
            self.total_non_padding_tokens.assign(0.)

    class ResponseEmbeddingCosineSimilarity(metrics.Metric):
        def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
            super().__init__(name=name)
            self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
            self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            epsilon = tf.keras.backend.epsilon()
            y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
            y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
            y_true_norm = y_true / (y_true_norm_val + epsilon)
            y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
            cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
            non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
            non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
            masked_cosine_sim = cosine_sim_per_token * non_padding_mask
            num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
            avg_cosine_sim_per_sample = tf.where(
                num_non_zero_tokens > 0,
                tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
                tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
            )
            self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
            self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

        def reset_state(self):
            self.total_cosine_sim.assign(0.)
            self.num_samples.assign(0.)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    def calculate_flops(model, batch_size=moe_params["batch_size"]):
        flops = 0
        functional_model = model.build_graph()
        def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
            layer_flops_count = 0
            if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
                return 0
            if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
                for sub_layer in layer_instance.layers:
                    layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
                return layer_flops_count
            if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
                print(f"Warning: Layer {layer_instance.name} still has no defined input shape after forward pass. Skipping FLOPs calculation for it.")
                return 0
            if isinstance(layer_instance, layers.Dense):
                input_dim = layer_instance.input_shape[-1]
                output_dim = layer_instance.output_shape[-1]
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                layer_flops_count += 2 * input_dim * output_dim * effective_batch_size
            elif isinstance(layer_instance, layers.LSTM):
                input_dim = layer_instance.input_shape[-1]
                hidden_dim = layer_instance.units
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
                layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size
            elif isinstance(layer_instance, layers.Embedding):
                pass
            elif isinstance(layer_instance, MoELayer):
                moe_input_dim = layer_instance.input_dim
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
                layer_flops_count += gate_flops
                for expert in layer_instance.experts:
                    layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)
            elif isinstance(layer_instance, layers.MultiHeadAttention):
                config = layer_instance.get_config()
                num_heads = config['num_heads']
                key_dim = config['key_dim']
                d_model = num_heads * key_dim
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
                projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
                attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
                context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
                output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
                layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops
            elif isinstance(layer_instance, layers.TimeDistributed):
                inner_layer = layer_instance.layer
                td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
                layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)
            elif isinstance(layer_instance, TransformerDecoderLayer):
                layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
                layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
                ffn_effective_batch_size = current_batch_size * model.max_seq_length
                layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)
            return layer_flops_count
        for layer in functional_model.layers:
            flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)
        return flops / 1e9

    def get_gpu_stats():
        if nvidia_smi is None:
            print("NVIDIA SMI not available. GPU stats will be reported as zero.")
            return 0, 0
        try:
            nvidia_smi.nvmlInit()
            handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
            gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
            mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
            gpu_memory = mem_info.used / (1024 ** 2)
            nvidia_smi.nvmlShutdown()
            return gpu_load, gpu_memory / 1024
        except Exception as e:
            print(f"GPU monitoring failed: {str(e)}. GPU stats will be reported as zero.")
            return 0, 0

    class ResourceMonitor(keras.callbacks.Callback):
        def __init__(self, validation_data):
            super().__init__()
            self.resource_stats = []
            self.expert_load_history = []
            self.start_time = None
            self.epoch_start_time = None
            self.flops = None
            self.validation_data = validation_data
            self.nlu_expert_usage_counts = None
            self.nlg_expert_usage_counts = None
            self.nlu_gate_weights_history = []
            self.nlg_gate_weights_history = []

        def on_train_begin(self, logs=None):
            self.start_time = time.time()
            try:
                self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
                self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
                self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
            except Exception:
                self.flops = 0

        def on_epoch_begin(self, epoch, logs=None):
            self.epoch_start_time = time.time()

        def on_epoch_end(self, epoch, logs=None):
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            epoch_time = time.time() - self.epoch_start_time
            self.resource_stats.append({
                'epoch': epoch + 1,
                'epoch_time': epoch_time,
                'cpu_usage': cpu_usage,
                'memory_used': memory.used / (1024 ** 3),
                'memory_total': memory.total / (1024 ** 3),
                'gpu_load': gpu_load,
                'gpu_memory': gpu_memory,
                'loss': logs.get('loss', 0),
                'val_loss': logs.get('val_loss', 0),
                'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
                'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
                'intent_precision': logs.get('intent_output_intent_precision', 0),
                'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
                'intent_recall': logs.get('intent_output_intent_recall', 0),
                'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
                'intent_f1': logs.get('intent_output_intent_f1', 0),
                'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
                'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
                'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
                'perplexity': logs.get('response_output_perplexity', 0),
                'val_perplexity': logs.get('val_response_output_perplexity', 0),
                'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
                'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
                'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
                'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
            })
            if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
                self.expert_load_history.append({
                    'epoch': epoch + 1,
                    'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                    'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
                })
            sample_size = 100
            for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
                inputs, _ = batch
                outputs = self.model(inputs, training=False)
                nlu_gate_weights = outputs['nlu_gate_weights']
                reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
                self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
                nlg_gate_weights = outputs['nlg_gate_weights']
                reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
                self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

        def on_train_end(self, logs=None):
            self.total_time = time.time() - self.start_time
            if len(self.nlu_gate_weights_history) > 0:
                all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
                nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
                for expert_id in nlu_top_indices:
                    if expert_id < self.model.moe_nlu.total_experts:
                        self.nlu_expert_usage_counts[expert_id] += 1
            if len(self.nlg_gate_weights_history) > 0:
                all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
                nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
                for expert_id in nlg_top_indices:
                    if expert_id < self.model.moe_nlg.total_experts:
                        self.nlg_expert_usage_counts[expert_id] += 1

        def visualize_expert_load_distribution(self):
            total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
            total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
            nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
            nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
            nlu_stability = np.var(nlu_percentages)
            nlg_stability = np.var(nlg_percentages)
            return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

    class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
        def __init__(self, d_model, warmup_steps=4000):
            super().__init__()
            self.d_model = tf.cast(d_model, tf.float32)
            self.warmup_steps = warmup_steps

        def __call__(self, step):
            step = tf.cast(step, tf.float32)
            arg1 = tf.math.rsqrt(step)
            arg2 = step * (self.warmup_steps ** -1.5)
            return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

        def get_config(self):
            return {
                "d_model": self.d_model.numpy(),
                "warmup_steps": self.warmup_steps
            }

    model = MoEModel(moe_params)
    embedding_layer = model.embedding

    def create_validation_split(dataset, embedding_layer, validation_split=0.2, batch_size=moe_params["batch_size"]):
        element_spec = dataset.element_spec
        data_list = [(features, targets) for features, targets in dataset.unbatch().as_numpy_iterator()]
        if not data_list:
            raise ValueError("Dataset is empty.")
        if len(data_list) < 2:
            raise ValueError("Dataset too small to split.")
        train_data, val_data = train_test_split(data_list, test_size=validation_split, shuffle=True)
        def create_dataset_from_list(data):
            if not data:
                raise ValueError("Empty data list provided.")
            features = {
                'user_utterance_tokens': np.array([d[0]['user_utterance_tokens'] for d in data], dtype=np.int32),
                'prev_system_response_tokens': np.array([d[0]['prev_system_response_tokens'] for d in data], dtype=np.int32),
                'decoder_input_tokens': np.array([d[0]['decoder_input_tokens'] for d in data], dtype=np.int32),
                'domain_onehot_input': np.array([d[0]['domain_onehot_input'] for d in data], dtype=np.float32),
                'turn_id_embedding': np.array([d[0]['turn_id_embedding'] for d in data], dtype=np.float32),
                'ontology_multihot_input': np.array([d[0]['ontology_multihot_input'] for d in data], dtype=np.float32),
            }
            response_output = np.array([d[1]['response_output'] for d in data], dtype=np.int32)
            response_embeddings = embedding_layer(response_output).numpy()
            targets = {
                'domain_output': np.array([d[1]['domain_output'] for d in data], dtype=np.int32),
                'intent_output': np.array([d[1]['intent_output'] for d in data], dtype=np.float32),
                'response_output': response_output,
                'response_embeddings': response_embeddings
            }
            return tf.data.Dataset.from_tensor_slices((features, targets))
        train_dataset = create_dataset_from_list(train_data).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        val_dataset = create_dataset_from_list(val_data).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return train_dataset, val_dataset

    train_tf_dataset, val_tf_dataset = create_validation_split(train_tf_dataset, embedding_layer)
    losses_dict = {
        'intent_output': losses.BinaryCrossentropy(),
        'domain_output': losses.SparseCategoricalCrossentropy(),
        'response_output': losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss
    }
    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }
    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={'intent_output': 0.5, 'domain_output': 0.5, 'response_output': 1.0, 'response_embeddings': 1.0}
    )
    model.metrics.extend([
        model.moe_nlu.load_balancing_loss_metric,
        model.moe_nlg.load_balancing_loss_metric,
        model.val_moe_nlu_load_balancing_loss_metric,
        model.val_moe_nlg_load_balancing_loss_metric
    ])
    sample_input = next(iter(train_tf_dataset.take(1)))
    model(sample_input[0])
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=8, mode='min', restore_best_weights=True)
    resource_monitor = ResourceMonitor(val_tf_dataset)
    class TQDMProgressBar(callbacks.Callback):
        def on_epoch_begin(self, epoch, logs=None):
            self.progress_bar = tqdm(total=len(train_tf_dataset), desc=f'Epoch {epoch + 1}')
        def on_batch_end(self, batch, logs=None):
            self.progress_bar.update(1)
        def on_epoch_end(self, epoch, logs=None):
            self.progress_bar.close()
    tqdm_callback = TQDMProgressBar()
    history = model.fit(
        train_tf_dataset,
        epochs=100,
        validation_data=val_tf_dataset,
        callbacks=[early_stopping, resource_monitor, tqdm_callback],
        verbose=0
    )
    expert_stats = resource_monitor.visualize_expert_load_distribution()
    stats = resource_monitor.resource_stats
    avg_epoch_duration = np.mean([s['epoch_time'] for s in stats]) if stats else 0
    total_training_time = resource_monitor.total_time
    avg_cpu_usage = np.mean([s['cpu_usage'] for s in stats]) if stats else 0
    avg_memory = np.mean([s['memory_used'] for s in stats]) if stats else 0
    avg_gpu_load = np.mean([s['gpu_load'] for s in stats]) if stats else 0
    avg_gpu_memory = np.mean([s['gpu_memory'] for s in stats]) if stats else 0
    avg_flops = resource_monitor.flops if resource_monitor.flops else 0
    nlu_counts = resource_monitor.nlu_expert_usage_counts
    nlg_counts = resource_monitor.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) or 1
    total_nlg = np.sum(nlg_counts) or 1
    expert_nlu_percentages = [(nlu_counts[i] / total_nlu) * 100 for i in range(8)]
    expert_nlg_percentages = [(nlg_counts[i] / total_nlg) * 100 for i in range(8)]
    val_intent_accuracy = np.mean([s['val_intent_accuracy'] for s in stats if s['val_intent_accuracy'] > 0]) if stats and any(s['val_intent_accuracy'] > 0 for s in stats) else 0
    val_entity_accuracy = np.mean([s['val_domain_accuracy'] for s in stats if s['val_domain_accuracy'] > 0]) if stats and any(s['val_domain_accuracy'] > 0 for s in stats) else 0
    val_perplexity = np.mean([s['val_perplexity'] for s in stats if s['val_perplexity'] > 0]) if stats and any(s['val_perplexity'] > 0 for s in stats) else 0
    val_cosine_similarity = np.mean([s['val_cosine_similarity'] for s in stats if s['val_cosine_similarity'] > 0]) if stats and any(s['val_cosine_similarity'] > 0 for s in stats) else 0
    results_table['Training Speed (epochs per second)'][f'Iteration {iteration + 1}'] = 1 / avg_epoch_duration if avg_epoch_duration > 0 else 0
    results_table['Training Time (second/epoch)'][f'Iteration {iteration + 1}'] = avg_epoch_duration
    results_table['Total Training Time (second/iteration)'][f'Iteration {iteration + 1}'] = total_training_time
    results_table['Computational Resource Usage'][f'Iteration {iteration + 1}'] = avg_cpu_usage + avg_gpu_load
    results_table['Average CPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_cpu_usage
    results_table['Average GPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_gpu_load
    results_table['Average Memory (GB)'][f'Iteration {iteration + 1}'] = avg_memory
    results_table['Average GPU Memory (GB)'][f'Iteration {iteration + 1}'] = avg_gpu_memory
    results_table['Average FLOPS Estimate (GFLOPS)'][f'Iteration {iteration + 1}'] = avg_flops
    results_table['Expert NLU Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlu_stability']
    results_table['Expert NLG Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlg_stability']
    for i in range(8):
        if i < len(expert_stats['nlu_percentages']):
            results_table[f'Average NLU Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlu_percentages'][i]
        if i < len(expert_stats['nlg_percentages']):
            results_table[f'Average NLG Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlg_percentages'][i]
    results_table['Average Validation Entity Accuracy'][f'Iteration {iteration + 1}'] = val_entity_accuracy
    results_table['Average Intent Accuracy'][f'Iteration {iteration + 1}'] = val_intent_accuracy
    results_table['Average Validation Perplexity'][f'Iteration {iteration + 1}'] = val_perplexity
    results_table['Average Validation Response Cosine Similarity'][f'Iteration {iteration + 1}'] = val_cosine_similarity
    val_loss = min([s['val_loss'] for s in stats]) if stats else float('inf')
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        try:
            model.save(best_model_path, save_format='tf', include_optimizer=True)
        except Exception as e:
            print(f"\nFailed to save best model to {best_model_path}: {str(e)}")

for key in results_table:
    results_table[key]['Average'] = np.mean([results_table[key][f'Iteration {i+1}'] for i in range(10)])

final_result = pd.DataFrame(results_table)
final_result = final_result.T
final_result.to_excel('training_NoisyTop-K_result.xlsx', index=True)
final_result

2025-07-04 07:04:44.232945: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 07:04:44.726116: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 07:04:44.726256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 07:04:44.807736: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 07:04:44.982292: I tensorflow/core/platform/cpu_feature_guar


Starting Training Iteration 1

Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 07:04:49.941350: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 07:04:50.273054: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 07:04:50.273485: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl


Epoch 1:   0%|          | 0/4428 [00:00<?, ?it/s]2025-07-04 07:05:15.520267: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fd0dc12f160 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-04 07:05:15.520351: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2025-07-04 07:05:15.539822: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-04 07:05:15.587014: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1751612715.682121    7873 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
Epoch 1: 100%|██████████| 4428/4428 [03:20<00:00, 22.08it/s]


INFO:tensorflow:Assets written to: best_noisytop-K_moe_model/assets


INFO:tensorflow:Assets written to: best_noisytop-K_moe_model/assets


,Iteration 1,Iteration 2,Iteration 3,Iteration 4,Iteration 5,Iteration 6,Iteration 7,Iteration 8,Iteration 9,Iteration 10,Average
Training Speed (epochs per second),0.005138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000514
Training Time (second/epoch),194.615082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.461508
Total Training Time (second/iteration),201.887702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.188770
Computational Resource Usage,65.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.550000
Average CPU Usage (percent),23.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.350000
Average GPU Usage (percent),42.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.200000
Average Memory (GB),6.599609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.659961
Average GPU Memory (GB),14.531799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.453180
Average FLOPS Estimate (GFLOPS),2.377871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.237787
Expert NLU Load Distribution Stability,148.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.875000


# DeepSeekMoE Noisy Top-K Gating evaluation code

In [3]:
def load_tf_datasets_from_disk(load_path):
    try:
        with open(os.path.join(load_path, "moe_params.json"), "r") as f:
            loaded_moe_params = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"moe_params.json not found in {load_path}")

    element_spec = (
        {
            'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
            'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
            'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
        },
        {
            'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
            'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
        }
    )

    try:
        train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
        test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
    except Exception as e:
        raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

    raw_data = {}
    for dataset_name in ['train', 'test']:
        path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
        if os.path.exists(path):
            with open(path, 'rb') as f:
                raw_data[dataset_name] = pickle.load(f)
        else:
            raw_data[dataset_name] = []

    return {
        "train_dataset": train_tf_dataset,
        "test_dataset": test_tf_dataset,
        "moe_params": loaded_moe_params,
        "raw_data": raw_data
    }

tf_dataset_save_path = "tf_datasets"
loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
train_tf_dataset = loaded_data["train_dataset"]
test_tf_dataset = loaded_data["test_dataset"]
moe_params = loaded_data["moe_params"]
raw_data = loaded_data["raw_data"]

moe_params["num_experts"] = 4

class NoisyTopKGating(layers.Layer):
    def __init__(self, num_experts, input_dim, k=2, noise_epsilon=1.0, name=None):
        super(NoisyTopKGating, self).__init__(name=name)
        self.num_experts = num_experts
        self.input_dim = input_dim
        self.k = k
        self.noise_epsilon = noise_epsilon
        self.w_g = self.add_weight(
            name='w_g',
            shape=(input_dim, num_experts),
            initializer='glorot_uniform',
            trainable=True
        )
        self.w_noise = self.add_weight(
            name='w_noise',
            shape=(input_dim, num_experts),
            initializer=keras.initializers.Constant(value=noise_epsilon),
            trainable=True
        )
        self.softmax = layers.Softmax(axis=-1)

    def call(self, inputs, training=False, usage_stats=None):
        gate_logits = tf.matmul(inputs, self.w_g)
        if training and usage_stats is not None:
            usage_mean = tf.reduce_mean(usage_stats)
            usage_std = tf.math.reduce_std(usage_stats)
            imbalance_factor = tf.clip_by_value(usage_std / (usage_mean + 1e-6), 0.0, 1.0)
            adjusted_k = tf.cast(
                self.k * (1.0 + imbalance_factor * 2),
                tf.int32
            )
            adjusted_k = tf.minimum(tf.maximum(adjusted_k, self.k), self.num_experts)
        else:
            adjusted_k = self.k
        if training:
            raw_noise = tf.random.normal(
                shape=(tf.shape(inputs)[0], self.num_experts),
                mean=0.0,
                stddev=1.0
            )
            gate_std = tf.math.reduce_std(gate_logits, axis=-1, keepdims=True)
            noise_scale = tf.nn.softplus(tf.matmul(inputs, self.w_noise)) * (1.0 + gate_std)
            noisy_gate_logits = gate_logits + (raw_noise * noise_scale)
        else:
            noisy_gate_logits = gate_logits
        top_k_values, _ = tf.math.top_k(noisy_gate_logits, k=adjusted_k)
        min_values = tf.reduce_min(top_k_values, axis=-1, keepdims=True)
        top_k_mask = tf.cast(
            noisy_gate_logits >= min_values,
            dtype=tf.float32
        )
        keep_top_k_logits = tf.where(
            top_k_mask > 0,
            noisy_gate_logits,
            tf.ones_like(noisy_gate_logits) * -float('inf')
        )
        gate_outputs = self.softmax(keep_top_k_logits)
        return gate_outputs, gate_logits

class MoELayer(layers.Layer):
    def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.5, top_k=2, name=None):
        super(MoELayer, self).__init__(name=name)
        self.m = m
        self.k_s = k_s
        self.total_experts = num_experts * self.m
        self.routed_experts = self.total_experts - self.k_s
        self.top_k = top_k
        self.top_mk = self.m * self.top_k
        self.top_mk = min(self.top_mk, self.routed_experts)
        self.alpha = alpha
        self.input_dim = input_dim
        self.seq_length = None
        self.usage_stats = tf.Variable(
            tf.ones(self.routed_experts) / self.routed_experts,
            trainable=False,
            dtype=tf.float32
        )
        self.adjusted_expert_dim = expert_dim // self.m
        self.shared_experts = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu'),
                layers.Dense(input_dim, activation=None)
            ], name=f'shared_expert_{i}') for i in range(self.k_s)
        ]
        self.routed_experts_list = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu'),
                layers.Dense(input_dim, activation=None)
            ], name=f'routed_expert_{i}') for i in range(self.k_s, self.total_experts)
        ]
        self.experts = self.shared_experts + self.routed_experts_list
        self.gate = NoisyTopKGating(
            num_experts=self.routed_experts,
            input_dim=self.input_dim,
            k=self.top_mk,
            noise_epsilon=1.0,
            name='noisy_topk_gate'
        )
        self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')
        self.expert_usage_metric = tf.keras.metrics.Mean(name=f'{name}_expert_usage')

    def call(self, inputs, training=False):
        input_rank = inputs.shape.rank
        if input_rank == 3:
            batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
            flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
            is_3d = True
        else:
            batch_size = tf.shape(inputs)[0]
            seq_length = 1
            flat_inputs = inputs
            is_3d = False
        flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
        shared_output = tf.zeros([tf.shape(flat_inputs)[0], self.input_dim], dtype=tf.float32)
        for i in range(self.k_s):
            shared_output += self.shared_experts[i](flat_inputs)
        gate_weights, clean_logits = self.gate(flat_inputs, training=training, usage_stats=self.usage_stats)
        top_mk_values, top_mk_indices = tf.nn.top_k(gate_weights, k=self.top_mk, sorted=True)
        top_mk_indices = top_mk_indices + self.k_s
        top_mk_weights = top_mk_values / (tf.reduce_sum(top_mk_values, axis=-1, keepdims=True) + tf.keras.backend.epsilon())
        routed_output = tf.zeros([tf.shape(flat_inputs)[0], self.input_dim], dtype=tf.float32)
        for k in range(self.top_mk):
            kth_weights = top_mk_weights[:, k]
            kth_indices = top_mk_indices[:, k]
            mask = tf.one_hot(kth_indices, depth=self.total_experts, dtype=tf.float32)
            expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
            kth_expert_output = tf.reduce_sum(expert_outputs * tf.expand_dims(mask, -1), axis=1)
            weighted_kth_output = kth_expert_output * tf.expand_dims(kth_weights, -1)
            routed_output += weighted_kth_output
        output_flat = shared_output + routed_output + flat_inputs
        if is_3d:
            output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
            gate_weights_reshaped = tf.reshape(gate_weights, [batch_size, seq_length, self.routed_experts])
        else:
            output = output_flat
            gate_weights_reshaped = gate_weights
        expert_assignments = tf.argmax(clean_logits, axis=-1)
        f_i = tf.reduce_mean(
            tf.one_hot(expert_assignments, depth=self.routed_experts, dtype=tf.float32),
            axis=0
        )
        P_i = tf.reduce_mean(gate_weights, axis=0)
        cov = tf.reduce_sum((f_i - tf.reduce_mean(f_i)) * (P_i - tf.reduce_mean(P_i)))
        load_balancing_loss = self.alpha * (tf.reduce_sum(f_i * P_i) + 0.1 * cov)
        self.add_loss(load_balancing_loss)
        self.load_balancing_loss_metric.update_state(load_balancing_loss)
        if training:
            current_usage = tf.reduce_mean(
                tf.one_hot(expert_assignments, depth=self.routed_experts, dtype=tf.float32),
                axis=0
            )
            decay_rate = tf.minimum(0.1, 1.0 / (tf.reduce_sum(self.usage_stats) + 1e-6))
            self.usage_stats.assign(
                self.usage_stats * (1.0 - decay_rate) + current_usage * decay_rate
            )
        expert_usage = tf.reduce_mean(tf.reduce_sum(tf.cast(gate_weights > 0, tf.float32), axis=0))
        self.expert_usage_metric.update_state(expert_usage)
        return output, gate_weights_reshaped

    def get_metrics(self):
        return {
            f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric,
            f'{self.name}_expert_usage': self.expert_usage_metric
        }

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_experts': self.total_experts // self.m,
            'expert_dim': self.adjusted_expert_dim * self.m,
            'input_dim': self.input_dim,
            'm': self.m,
            'k_s': self.k_s,
            'alpha': self.alpha,
            'top_k': self.top_k,
            'name': self.name
        })
        return config

class TransformerDecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.ffn = keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask=None):
        if look_ahead_mask is not None:
            look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
            look_ahead_mask = 1.0 - look_ahead_mask
        attn1_output = self.mha1(
            query=x,
            value=x,
            key=x,
            attention_mask=look_ahead_mask,
            training=training,
            return_attention_scores=True
        )
        if isinstance(attn1_output, tuple):
            attn1, attn_weights1 = attn1_output
        else:
            attn1, attn_weights1 = attn1_output, None
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)
        attn2_output = self.mha2(
            query=out1,
            value=enc_output,
            key=enc_output,
            attention_mask=None,
            training=training,
            return_attention_scores=True
        )
        if isinstance(attn2_output, tuple):
            attn2, attn_weights2 = attn2_output
        else:
            attn2, attn_weights2 = attn2_output, None
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)
        return out3, attn_weights1, attn_weights2

    def get_config(self):
        config = super().get_config()
        mha1_config = self.mha1.get_config()
        config.update({
            'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
            'num_heads': mha1_config['num_heads'],
            'dff': self.ffn.layers[0].units,
            'rate': self.dropout1.rate
        })
        return config

class MoEModel(models.Model):
    def __init__(self, moe_params):
        super(MoEModel, self).__init__()
        self.embedding_dim = moe_params["embedding_dim"]
        self.max_seq_length = moe_params["max_seq_length"]
        self.num_domains = moe_params["num_domains"]
        self.num_intents = moe_params["num_intents"]
        self.vocab_size = moe_params["vocab_size"]
        self.num_experts = moe_params["num_experts"]
        self.expert_dim = moe_params["expert_dim"]
        self.turn_id_dim = moe_params["turn_id_dim"]
        self.num_heads = 4
        self.dff = self.embedding_dim * 4
        self.dropout_rate = 0.1
        self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                             self.num_domains + self.turn_id_dim + self.num_intents)
        self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains
        self.embedding = layers.Embedding(
            self.vocab_size,
            self.embedding_dim,
            mask_zero=True,
            embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05),
            name="embedding"
        )
        self.embedding.build((None,))
        with tf.init_scope():
            embedding_weights = self.embedding.get_weights()
            if embedding_weights and len(embedding_weights) > 0:
                embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                self.embedding.set_weights(embedding_weights)
        self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
        self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
        self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
        self.decoder_dropout = layers.Dropout(self.dropout_rate)
        self.moe_nlu = MoELayer(
            num_experts=self.num_experts,
            expert_dim=self.expert_dim,
            input_dim=self.nlu_input_dim,
            m=2,
            k_s=2,
            alpha=0.01,
            top_k=2,
            name='moe_nlu'
        )
        self.moe_nlg = MoELayer(
            num_experts=self.num_experts,
            expert_dim=self.expert_dim,
            input_dim=self.nlg_input_dim,
            m=2,
            k_s=2,
            alpha=0.01,
            top_k=2,
            name='moe_nlg'
        )
        self.intent_output = layers.Dense(self.num_intents, activation='sigmoid', name='intent_output')
        self.domain_output = layers.Dense(self.num_domains, activation='softmax', name='domain_output')
        self.response_output = layers.TimeDistributed(
            layers.Dense(self.vocab_size, activation='softmax'), name='response_output'
        )
        self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)
        self.nlg_projection = layers.TimeDistributed(
            layers.Dense(self.embedding_dim, activation='relu', name='nlg_projection')
        )

    def create_look_ahead_mask(self, size):
        mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
        return mask

    def call(self, inputs, training=False):
        user_utterance_tokens = inputs['user_utterance_tokens']
        prev_system_response_tokens = inputs['prev_system_response_tokens']
        decoder_input_tokens = inputs['decoder_input_tokens']
        domain_onehot_input = inputs['domain_onehot_input']
        turn_id_embedding = inputs['turn_id_embedding']
        ontology_multihot_input = inputs['ontology_multihot_input']
        pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
        user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
        prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
        decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)
        user_enc_out = user_embedded
        prev_system_enc_out = prev_system_embedded
        look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
        dec_output = decoder_embedded
        attn_weights = {}
        for i, layer in enumerate(self.decoder_layers):
            dec_output, attn_w1, attn_w2 = layer(
                dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask
            )
            attn_weights[f'decoder_layer{i+1}_attn1'] = attn_w1
            attn_weights[f'decoder_layer{i+1}_attn2'] = attn_w2
        decoder_output = self.decoder_dropout(dec_output, training=training)
        user_attn_out = self.attn_layer(
            query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), 
            value=user_enc_out, key=user_enc_out, training=training
        )
        prev_system_attn_out = self.attn_layer(
            query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), 
            value=prev_system_enc_out, key=prev_system_enc_out, training=training
        )
        decoder_attn_out = self.attn_layer(
            query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), 
            value=decoder_output, key=decoder_output, training=training
        )
        combined_features = layers.Concatenate()([
            tf.squeeze(user_attn_out, 1),
            tf.squeeze(prev_system_attn_out, 1),
            tf.squeeze(decoder_attn_out, 1),
            domain_onehot_input,
            turn_id_embedding,
            ontology_multihot_input
        ])
        combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
        nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
        nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
        intent_out = self.intent_output(nlu_out)
        domain_out = self.domain_output(nlu_out)
        intent_out_expanded = tf.expand_dims(intent_out, axis=1)
        domain_out_expanded = tf.expand_dims(domain_out, axis=1)
        intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
        domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])
        nlg_input = layers.Concatenate(axis=-1)([
            decoder_output,
            intent_out_tiled,
            domain_out_tiled
        ])
        nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])
        nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
        nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
        response_embeddings = self.nlg_projection(nlg_out)
        response_embeddings = tf.ensure_shape(response_embeddings, [None, self.max_seq_length, self.embedding_dim])
        response_out = self.response_output(nlg_out)
        return {
            'intent_output': intent_out,
            'domain_output': domain_out,
            'response_output': response_out,
            'response_embeddings': response_embeddings,
            'nlu_gate_weights': nlu_gate_weights,
            'nlg_gate_weights': nlg_gate_weights
        }

    def build_graph(self):
        inputs = {
            'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='user_utterance_tokens'),
            'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='prev_system_response_tokens'),
            'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='decoder_input_tokens'),
            'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32, name='domain_onehot_input'),
            'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32, name='turn_id_embedding'),
            'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32, name='ontology_multihot_input'),
        }
        return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

    def get_config(self):
        config = super().get_config()
        config.update({
            'moe_params': {
                'embedding_dim': self.embedding_dim,
                'max_seq_length': self.max_seq_length,
                'num_domains': self.num_domains,
                'num_intents': self.num_intents,
                'vocab_size': self.vocab_size,
                'num_experts': self.num_experts,
                'expert_dim': self.expert_dim,
                'turn_id_dim': self.turn_id_dim
            }
        })
        return config

    def compile(self, **kwargs):
        super().compile(**kwargs)
        self.val_moe_nlu_load_balancing_loss_metric = tf.keras.metrics.Mean(name='val_moe_nlu_load_balancing_loss')
        self.val_moe_nlg_load_balancing_loss_metric = tf.keras.metrics.Mean(name='val_moe_nlg_load_balancing_loss')

    def test_step(self, data):
        x, y = data
        y_pred = self(x, training=False)
        self.compiled_loss(y, y_pred)
        self.compiled_metrics.update_state(y, y_pred)
        nlu_gate_weights = y_pred['nlu_gate_weights']
        nlg_gate_weights = y_pred['nlg_gate_weights']
        nlu_expert_assignments = tf.argmax(nlu_gate_weights, axis=-1)
        nlu_f_i = tf.reduce_mean(
            tf.one_hot(nlu_expert_assignments, depth=self.moe_nlu.routed_experts, dtype=tf.float32),
            axis=0
        )
        nlu_P_i = tf.reduce_mean(nlu_gate_weights, axis=0)
        nlu_load_balancing_loss = self.moe_nlu.alpha * tf.reduce_sum(nlu_f_i * nlu_P_i)
        self.val_moe_nlu_load_balancing_loss_metric.update_state(nlu_load_balancing_loss)
        nlg_expert_assignments = tf.argmax(nlg_gate_weights, axis=-1)
        nlg_f_i = tf.reduce_mean(
            tf.one_hot(nlg_expert_assignments, depth=self.moe_nlg.routed_experts, dtype=tf.float32),
            axis=0
        )
        nlg_P_i = tf.reduce_mean(nlg_gate_weights, axis=0)
        nlg_load_balancing_loss = self.moe_nlg.alpha * tf.reduce_sum(nlg_f_i * nlg_P_i)
        self.val_moe_nlg_load_balancing_loss_metric.update_state(nlg_load_balancing_loss)
        return {m.name: m.result() for m in self.metrics + [self.val_moe_nlu_load_balancing_loss_metric, self.val_moe_nlg_load_balancing_loss_metric]}

class IntentAccuracy(metrics.Metric):
    def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros', dtype=tf.float32)
        self.total_preds = self.add_weight(name='total_preds', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
        self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
        self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

    def reset_state(self):
        self.correct_preds.assign(0.)
        self.total_preds.assign(0.)

class IntentPrecision(metrics.Metric):
    def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
        self.predicted_positives = self.add_weight(name='pp', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        predicted_positives = tf.reduce_sum(y_pred)
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)

    def result(self):
        return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.predicted_positives.assign(0.)

class IntentRecall(metrics.Metric):
    def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
        self.actual_positives = self.add_weight(name='ap', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        actual_positives = tf.reduce_sum(y_true)
        self.true_positives.assign_add(true_positives)
        self.actual_positives.assign_add(actual_positives)

    def result(self):
        return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.actual_positives.assign(0.)

class IntentF1(metrics.Metric):
    def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = IntentPrecision(threshold=threshold)
        self.recall = IntentRecall(threshold=threshold)

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

class DomainAccuracy(metrics.Metric):
    def __init__(self, name='domain_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.accuracy = self.add_weight(name='acc', initializer='zeros', dtype=tf.float32)
        self.count = self.add_weight(name='count', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
        pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
        matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
        self.accuracy.assign_add(tf.reduce_sum(matches))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.accuracy / (self.count + tf.keras.backend.epsilon())

    def reset_state(self):
        self.accuracy.assign(0.)
        self.count.assign(0.)

class Perplexity(metrics.Metric):
    def __init__(self, name='perplexity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
        self.total_loss = self.add_weight(name='total_loss', initializer='zeros', dtype=tf.float32)
        self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        mask = tf.cast(y_true != 0, dtype=tf.float32)
        loss = self.cross_entropy(y_true, y_pred)
        masked_loss = loss * mask
        sum_loss = tf.reduce_sum(masked_loss)
        num_non_padding_tokens = tf.reduce_sum(mask)
        self.total_loss.assign_add(sum_loss)
        self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

    def result(self):
        avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
        return tf.exp(avg_loss)

    def reset_state(self):
        self.total_loss.assign(0.)
        self.total_non_padding_tokens.assign(0.)

class ResponseEmbeddingCosineSimilarity(metrics.Metric):
    def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
        super().__init__(name=name)
        self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
        self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true_norm = y_true / (y_true_norm_val + epsilon)
        y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
        cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
        non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
        non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
        masked_cosine_sim = cosine_sim_per_token * non_padding_mask
        num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
        avg_cosine_sim_per_sample = tf.where(
            num_non_zero_tokens > 0,
            tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
            tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
        )
        self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
        self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

    def reset_state(self):
        self.total_cosine_sim.assign(0.)
        self.num_samples.assign(0.)

def contrastive_loss(y_true, y_pred, margin=1.0):
    epsilon = tf.keras.backend.epsilon()
    y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
    y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
    y_true = y_true / (y_true_norm + epsilon)
    y_pred = y_pred / (y_pred_norm + epsilon)
    cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
    positive_loss = 1.0 - cosine_sim
    mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
    masked_loss = positive_loss * mask
    total_loss = tf.reduce_sum(masked_loss)
    num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
    return total_loss / num_tokens

def calculate_flops(model, batch_size=moe_params["batch_size"]):
    flops = 0
    functional_model = model.build_graph()
    def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
        layer_flops_count = 0
        if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
            return 0
        if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
            for sub_layer in layer_instance.layers:
                layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
            return layer_flops_count
        if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
            return 0
        if isinstance(layer_instance, layers.Dense):
            input_dim = layer_instance.input_shape[-1]
            output_dim = layer_instance.output_shape[-1]
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            layer_flops_count += 2 * input_dim * output_dim * effective_batch_size
        elif isinstance(layer_instance, layers.LSTM):
            input_dim = layer_instance.input_shape[-1]
            hidden_dim = layer_instance.units
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
            layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size
        elif isinstance(layer_instance, layers.Embedding):
            pass
        elif isinstance(layer_instance, MoELayer):
            moe_input_dim = layer_instance.input_dim
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
            layer_flops_count += gate_flops
            for expert in layer_instance.experts:
                layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)
        elif isinstance(layer_instance, layers.MultiHeadAttention):
            config = layer_instance.get_config()
            num_heads = config['num_heads']
            key_dim = config['key_dim']
            d_model = num_heads * key_dim
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
            projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
            attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
            context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
            output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
            layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops
        elif isinstance(layer_instance, layers.TimeDistributed):
            inner_layer = layer_instance.layer
            td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
            layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)
        elif isinstance(layer_instance, TransformerDecoderLayer):
            layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
            layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
            ffn_effective_batch_size = current_batch_size * model.max_seq_length
            layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)
        return layer_flops_count
    for layer in functional_model.layers:
        flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)
    return flops / 1e9

def get_gpu_stats():
    if nvidia_smi is None:
        return 0, 0
    try:
        nvidia_smi.nvmlInit()
        handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
        gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
        mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
        gpu_memory = mem_info.used / (1024 ** 2)
        nvidia_smi.nvmlShutdown()
        return gpu_load, gpu_memory / 1024
    except Exception as e:
        return 0, 0

class ResourceMonitor(keras.callbacks.Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.resource_stats = []
        self.expert_load_history = []
        self.start_time = None
        self.epoch_start_time = None
        self.flops = None
        self.validation_data = validation_data
        self.nlu_expert_usage_counts = None
        self.nlg_expert_usage_counts = None
        self.nlu_gate_weights_history = []
        self.nlg_gate_weights_history = []

    def on_train_begin(self, logs=None):
        self.start_time = time.time()
        try:
            self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
            self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
            self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
        except Exception:
            self.flops = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory()
        gpu_load, gpu_memory = get_gpu_stats()
        epoch_time = time.time() - self.epoch_start_time
        self.resource_stats.append({
            'epoch': epoch + 1,
            'epoch_time': epoch_time,
            'cpu_usage': cpu_usage,
            'memory_used': memory.used / (1024 ** 3),
            'memory_total': memory.total / (1024 ** 3),
            'gpu_load': gpu_load,
            'gpu_memory': gpu_memory,
            'loss': logs.get('loss', 0),
            'val_loss': logs.get('val_loss', 0),
            'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
            'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
            'intent_precision': logs.get('intent_output_intent_precision', 0),
            'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
            'intent_recall': logs.get('intent_output_intent_recall', 0),
            'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
            'intent_f1': logs.get('intent_output_intent_f1', 0),
            'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
            'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
            'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
            'perplexity': logs.get('response_output_perplexity', 0),
            'val_perplexity': logs.get('val_response_output_perplexity', 0),
            'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
            'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
            'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
            'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
        })
        if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
            self.expert_load_history.append({
                'epoch': epoch + 1,
                'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
            })
        sample_size = 100
        for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
            inputs, _ = batch
            outputs = self.model(inputs, training=False)
            nlu_gate_weights = outputs['nlu_gate_weights']
            reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
            self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
            nlg_gate_weights = outputs['nlg_gate_weights']
            reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
            self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

    def on_train_end(self, logs=None):
        self.total_time = time.time() - self.start_time
        if len(self.nlu_gate_weights_history) > 0:
            all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
            nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
            for expert_id in nlu_top_indices:
                if expert_id < self.model.moe_nlu.total_experts:
                    self.nlu_expert_usage_counts[expert_id] += 1
        if len(self.nlg_gate_weights_history) > 0:
            all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
            nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
            for expert_id in nlg_top_indices:
                if expert_id < self.model.moe_nlg.total_experts:
                    self.nlg_expert_usage_counts[expert_id] += 1

    def visualize_expert_load_distribution(self):
        total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
        total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
        nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
        nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
        nlu_stability = np.var(nlu_percentages)
        nlg_stability = np.var(nlg_percentages)
        return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {
            "d_model": self.d_model.numpy(),
            "warmup_steps": self.warmup_steps
        }

word_to_id = {}
id_to_word = {}
try:
    possible_paths = [
        os.path.join("preprocessor_models", "preprocessor_params.json"),
        os.path.join("tf_datasets", "preprocessor_params.json"),
        "preprocessor_params.json"
    ]
    vocab_loaded = False
    for path in possible_paths:
        if os.path.exists(path):
            with open(path, "r") as f:
                preprocessor_params = json.load(f)
            word_to_id = {k: int(v) for k, v in preprocessor_params['word_to_id'].items()}
            id_to_word = {int(k): v for k, v in preprocessor_params['id_to_word'].items()}
            vocab_loaded = True
            break
    if not vocab_loaded:
        raise FileNotFoundError("Vocabulary file not found in any location")
except Exception as e:
    word_to_id = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
    id_to_word = {0: '<pad>', 1: '<sos>', 2: '<eos>', 3: '<unk>'}

model_save_path = "best_noisytop-K_moe_model"
custom_objects = {
    "NoisyTopKGating":NoisyTopKGating,
    "MoEModel":MoEModel,
    "CustomLearningRateSchedule": CustomLearningRateSchedule,
    "MoELayer": MoELayer,
    "TransformerDecoderLayer": TransformerDecoderLayer,
    "IntentAccuracy": IntentAccuracy,
    "IntentPrecision": IntentPrecision,
    "IntentRecall": IntentRecall,
    "IntentF1": IntentF1,
    "DomainAccuracy": DomainAccuracy,
    "Perplexity": Perplexity,
    "ResponseEmbeddingCosineSimilarity": ResponseEmbeddingCosineSimilarity
}
try:
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
except Exception as e:
    raise

def compile_model(model, moe_params):
    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens
    losses_dict = {
        'intent_output': tf.keras.losses.BinaryCrossentropy(),
        'domain_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss,
    }
    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }
    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={
            'intent_output': 0.5,
            'domain_output': 0.5,
            'response_output': 1.0,
            'response_embeddings': 1.0
        }
    )
    return model

model = compile_model(model, moe_params)
sample_input = next(iter(train_tf_dataset.take(1)))
model(sample_input[0])
model.summary()

class EvaluationMetrics:
    def __init__(self, model, test_dataset, moe_params, raw_data, word_to_id, id_to_word):
        self.model = model
        self.test_dataset = test_dataset
        self.moe_params = moe_params
        self.raw_data = raw_data
        self.word_to_id = word_to_id
        self.id_to_word = id_to_word
        self.nlu_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.nlg_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.resource_stats = {
            'cpu_usage': [],
            'memory_used': [],
            'gpu_load': [],
            'gpu_memory': [],
            'batch_times': []
        }
        self.special_tokens = {'<pad>', '<sos>', '<eos>', '<unk>'}
        self.nlu_top_k = model.get_layer('moe_nlu').get_config().get('top_k', 2)
        self.nlg_top_k = model.get_layer('moe_nlg').get_config().get('top_k', 2)

    def evaluate(self):
        def add_response_embeddings(features, targets):
            response_tokens = targets['response_output']
            response_embeddings = self.model.embedding(response_tokens)
            targets['response_embeddings'] = response_embeddings
            return features, targets
        test_dataset_with_embeddings = self.test_dataset.map(add_response_embeddings, num_parallel_calls=tf.data.AUTOTUNE)
        start_time = time.time()
        test_results = self.model.evaluate(test_dataset_with_embeddings, verbose=1)
        eval_time = time.time() - start_time
        metric_names = self.model.metrics_names
        perplexity_value = None
        for name, value in zip(metric_names, test_results):
            if name == 'response_output_perplexity':
                perplexity_value = value
                break
        if perplexity_value is None:
            perplexity_value = 0.0
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory().used / (1024 ** 3)
        gpu_load, gpu_memory = get_gpu_stats()
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))
        total_tokens = 0
        total_time = 0
        num_batches = 0
        test_dataset_small = test_dataset_with_embeddings.unbatch().batch(moe_params["batch_size"])
        all_true_intents = []
        all_pred_intents = []
        all_true_domains = []
        all_pred_domains = []
        for batch_idx, batch in enumerate(test_dataset_small):
            inputs, targets = batch
            response_tokens = targets['response_output'].numpy()
            non_padding_mask = response_tokens != 0
            total_tokens += np.sum(non_padding_mask)
            start_time = time.time()
            outputs = self.model(inputs, training=False)
            batch_time = time.time() - start_time
            total_time += batch_time
            num_batches += tf.shape(inputs['user_utterance_tokens'])[0]
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            self.resource_stats['cpu_usage'].append(cpu_usage)
            self.resource_stats['memory_used'].append(memory.used / (1024 ** 3))
            self.resource_stats['gpu_load'].append(gpu_load)
            self.resource_stats['gpu_memory'].append(gpu_memory)
            self.resource_stats['batch_times'].append(batch_time)
            nlu_gate_weights = outputs.get('nlu_gate_weights', tf.zeros([0, self.model.moe_nlu.routed_experts])).numpy()
            nlu_top_indices = np.argsort(-nlu_gate_weights, axis=-1)[:, :self.nlu_top_k] + self.model.moe_nlu.k_s
            for idx in range(self.nlu_top_k):
                expert_ids = nlu_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlu_expert_usage_counts):
                        self.nlu_expert_usage_counts[expert_id] += 1
            nlg_gate_weights = outputs.get('nlg_gate_weights', tf.zeros([0, self.model.moe_nlg.routed_experts])).numpy()
            if len(nlg_gate_weights.shape) == 3:
                nlg_gate_weights = nlg_gate_weights.reshape(-1, nlg_gate_weights.shape[-1])
            nlg_top_indices = np.argsort(-nlg_gate_weights, axis=-1)[:, :self.nlg_top_k] + self.model.moe_nlg.k_s
            for idx in range(self.nlg_top_k):
                expert_ids = nlg_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlg_expert_usage_counts):
                        self.nlg_expert_usage_counts[expert_id] += 1
            true_intents = targets.get('intent_output', np.zeros((moe_params["batch_size"], self.model.num_intents))).numpy()
            pred_intents = (outputs.get('intent_output', np.zeros_like(true_intents)).numpy() > 0.5).astype(np.int32)
            all_true_intents.append(true_intents)
            all_pred_intents.append(pred_intents)
            true_domains = targets.get('domain_output', np.zeros((moe_params["batch_size"], 1))).numpy().flatten()
            pred_domains = np.argmax(outputs.get('domain_output', np.zeros((moe_params["batch_size"], self.model.num_domains))).numpy(), axis=-1)
            all_true_domains.append(true_domains)
            all_pred_domains.append(pred_domains)
            if batch_idx % 50 == 0:
                gc.collect()
                tf.keras.backend.clear_session()
        avg_time_per_token = (total_time / total_tokens) * 1000 if total_tokens > 0 else 0
        avg_cpu = np.mean(self.resource_stats['cpu_usage']) if self.resource_stats['cpu_usage'] else 0
        avg_memory = np.mean(self.resource_stats['memory_used']) if self.resource_stats['memory_used'] else 0
        avg_gpu_load = np.mean(self.resource_stats['gpu_load']) if self.resource_stats['gpu_load'] else 0
        avg_gpu_memory = np.mean(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        peak_gpu_memory = np.max(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))
        all_true_intents = np.vstack(all_true_intents)
        all_pred_intents = np.vstack(all_pred_intents)
        all_true_domains = np.concatenate(all_true_domains)
        all_pred_domains = np.concatenate(all_pred_domains)
        intent_f1_score = f1_score(all_true_intents, all_pred_intents, average='micro')
        intent_f1_score_macro = f1_score(all_true_intents, all_pred_intents, average='macro')
        intent_f1_score_weighted = f1_score(all_true_intents, all_pred_intents, average='weighted')
        intent_accuracy_score = accuracy_score(all_true_intents, all_pred_intents)
        domain_accuracy_score = accuracy_score(all_true_domains, all_pred_domains)
        domain_f1_score_macro = f1_score(all_true_domains, all_pred_domains, average='macro')
        hypotheses = []
        references = []
        meteor_scores = []
        rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        domain_metrics = defaultdict(lambda: {
            'bleu': [],
            'rouge1': [],
            'rouge2': [],
            'rougeL': [],
            'meteor': [],
            'count': 0
        })
        if not self.raw_data.get('test'):
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0
            domain_metrics = {}
        else:
            for i, batch in enumerate(test_dataset_small):
                if i >= len(self.raw_data.get('test', [])):
                    break
                try:
                    inputs, targets = batch
                    with tf.device('/CPU:0'):
                        outputs = self.model(inputs, training=False)
                    domain_id = targets['domain_output'].numpy()[0][0]
                    domain_name = f"domain_{domain_id}"
                    response_probs = outputs['response_output'].numpy()
                    generated_tokens = np.argmax(response_probs, axis=-1)[0]
                    true_tokens = targets['response_output'].numpy()[0]
                    raw_item = self.raw_data['test'][i]
                    ref_text = raw_item.get('raw_next_system_response', '')
                    if not ref_text:
                        continue
                    gen_text = self.tokens_to_text(generated_tokens)
                    ref_tokens = self.tokens_to_text(true_tokens)
                    if not gen_text.strip() or not ref_text.strip():
                        continue
                    hypotheses.append(gen_text)
                    references.append([ref_text])
                    ref_tokens = word_tokenize(ref_text.lower())
                    gen_tokens = word_tokenize(gen_text.lower())
                    try:
                        meteor = meteor_score([ref_tokens], gen_tokens)
                        meteor_scores.append(meteor)
                        domain_metrics[domain_name]['meteor'].append(meteor)
                    except Exception as e:
                        meteor_scores.append(0)
                        domain_metrics[domain_name]['meteor'].append(0)
                    try:
                        rouge_scores = rouge_scorer_instance.score(ref_text, gen_text)
                        domain_metrics[domain_name]['rouge1'].append(rouge_scores['rouge1'].fmeasure)
                        domain_metrics[domain_name]['rouge2'].append(rouge_scores['rouge2'].fmeasure)
                        domain_metrics[domain_name]['rougeL'].append(rouge_scores['rougeL'].fmeasure)
                    except Exception as e:
                        pass
                    domain_metrics[domain_name]['count'] += 1
                    if i % 50 == 0:
                        gc.collect()
                        tf.keras.backend.clear_session()
                except Exception as e:
                    continue
        if hypotheses and references:
            hypotheses_tok = []
            references_tok = []
            for hyp, ref_list in zip(hypotheses, references):
                if not hyp or not ref_list[0]:
                    continue
                hyp_tokens = word_tokenize(hyp.lower())
                ref_tokens = [word_tokenize(ref.lower()) for ref in ref_list]
                hypotheses_tok.append(hyp_tokens)
                references_tok.append(ref_tokens)
            chencherry = SmoothingFunction()
            try:
                bleu_score = corpus_bleu(references_tok, hypotheses_tok, smoothing_function=chencherry.method1)
            except Exception as e:
                bleu_score = 0.0
            rouge_scores = []
            for hyp, ref in zip(hypotheses, references):
                if hyp and ref[0]:
                    try:
                        rouge_scores.append(rouge_scorer_instance.score(ref[0], hyp))
                    except Exception as e:
                        continue
            avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_meteor = np.mean(meteor_scores) if meteor_scores else 0.0
        else:
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0
        return {
            'intent_f1_micro': intent_f1_score,
            'intent_f1_macro': intent_f1_score_macro,
            'intent_f1_weighted': intent_f1_score_weighted,
            'intent_accuracy': intent_accuracy_score,
            'domain_accuracy': domain_accuracy_score,
            'domain_f1_macro': domain_f1_score_macro,
            'bleu': bleu_score,
            'rouge1': avg_rouge1,
            'rouge2': avg_rouge2,
            'rougeL': avg_rougeL,
            'meteor': avg_meteor,
            'perplexity': perplexity_value,
            'pred_speed': avg_time_per_token,
            'domain_metrics': domain_metrics,
            'eval_time': eval_time
        }
    
    def tokens_to_text(self, tokens):
        words = []
        for token in tokens:
            if token == 0:
                continue
            word = self.id_to_word.get(token, '<unk>')
            if word not in self.special_tokens:
                words.append(word)
        return " ".join(words).strip()

results_table = {
    'Evaluation Time (seconds)': [],
    'Prediction Speed (ms/token)': [],
    'Average CPU Usage (percent)': [],
    'Average GPU Usage (percent)': [],
    'Average Memory (GB)': [],
    'Average GPU Memory (GB)': [],
    'Average FLOPs Estimate (GFLOPs)': [],
    'NLU Expert 1 (percent)': [],
    'NLU Expert 2 (percent)': [],
    'NLU Expert 3 (percent)': [],
    'NLU Expert 4 (percent)': [],
    'NLU Expert 5 (percent)': [],
    'NLU Expert 6 (percent)': [],
    'NLU Expert 7 (percent)': [],
    'NLU Expert 8 (percent)': [],
    'NLG Expert 1 (percent)': [],
    'NLG Expert 2 (percent)': [],
    'NLG Expert 3 (percent)': [],
    'NLG Expert 4 (percent)': [],
    'NLG Expert 5 (percent)': [],
    'NLG Expert 6 (percent)': [],
    'NLG Expert 7 (percent)': [],
    'NLG Expert 8 (percent)': [],
    'Intent F1-score (Micro)': [],
    'Intent F1-score (Macro)': [],
    'Intent F1-score (Weighted)': [],
    'Intent Accuracy': [],
    'Domain Accuracy': [],
    'Domain F1-score (Macro)': [],
    'BLEU Score': [],
    'ROUGE-1 F1': [],
    'ROUGE-2 F1': [],
    'ROUGE-L F1': [],
    'METEOR Score': [],
    'Perplexity': [],
    'NLU Expert Load Stability (Variance)': [],
    'NLG Expert Load Stability (Variance)': []
}

num_iterations = 5
for iteration in range(1, num_iterations + 1):
    tf.keras.backend.clear_session()
    gc.collect()
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
    model = compile_model(model, moe_params)
    evaluator = EvaluationMetrics(model, test_tf_dataset, moe_params, raw_data, word_to_id, id_to_word)
    evaluation_results = evaluator.evaluate()
    results_table['Evaluation Time (seconds)'].append(evaluation_results['eval_time'])
    results_table['Prediction Speed (ms/token)'].append(evaluation_results['pred_speed'])
    results_table['Average CPU Usage (percent)'].append(np.mean(evaluator.resource_stats['cpu_usage']) if evaluator.resource_stats['cpu_usage'] else 0)
    results_table['Average GPU Usage (percent)'].append(np.mean(evaluator.resource_stats['gpu_load']) if evaluator.resource_stats['gpu_load'] else 0)
    results_table['Average Memory (GB)'].append(np.mean(evaluator.resource_stats['memory_used']) if evaluator.resource_stats['memory_used'] else 0)
    results_table['Average GPU Memory (GB)'].append(np.mean(evaluator.resource_stats['gpu_memory']) if evaluator.resource_stats['gpu_memory'] else 0)
    results_table['Average FLOPs Estimate (GFLOPs)'].append(calculate_flops(model))
    nlu_counts = evaluator.nlu_expert_usage_counts
    nlg_counts = evaluator.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) if np.sum(nlu_counts) > 0 else 1
    total_nlg = np.sum(nlg_counts) if np.sum(nlg_counts) > 0 else 1
    results_table['NLU Expert 1 (percent)'].append((nlu_counts[0] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 2 (percent)'].append((nlu_counts[1] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 3 (percent)'].append((nlu_counts[2] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 4 (percent)'].append((nlu_counts[3] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 5 (percent)'].append((nlu_counts[4] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 6 (percent)'].append((nlu_counts[5] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 7 (percent)'].append((nlu_counts[6] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 8 (percent)'].append((nlu_counts[7] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLG Expert 1 (percent)'].append((nlg_counts[0] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 2 (percent)'].append((nlg_counts[1] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 3 (percent)'].append((nlg_counts[2] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 4 (percent)'].append((nlg_counts[3] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 5 (percent)'].append((nlg_counts[4] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 6 (percent)'].append((nlg_counts[5] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 7 (percent)'].append((nlg_counts[6] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 8 (percent)'].append((nlg_counts[7] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['Intent F1-score (Micro)'].append(evaluation_results['intent_f1_micro'])
    results_table['Intent F1-score (Macro)'].append(evaluation_results['intent_f1_macro'])
    results_table['Intent F1-score (Weighted)'].append(evaluation_results['intent_f1_weighted'])
    results_table['Intent Accuracy'].append(evaluation_results['intent_accuracy'])
    results_table['Domain Accuracy'].append(evaluation_results['domain_accuracy'])
    results_table['Domain F1-score (Macro)'].append(evaluation_results['domain_f1_macro'])
    results_table['BLEU Score'].append(evaluation_results['bleu'])
    results_table['ROUGE-1 F1'].append(evaluation_results['rouge1'])
    results_table['ROUGE-2 F1'].append(evaluation_results['rouge2'])
    results_table['ROUGE-L F1'].append(evaluation_results['rougeL'])
    results_table['METEOR Score'].append(evaluation_results['meteor'])
    results_table['Perplexity'].append(evaluation_results['perplexity'])
    results_table['NLU Expert Load Stability (Variance)'].append(np.var(nlu_counts / total_nlu * 100) if total_nlu > 0 else 0)
    results_table['NLG Expert Load Stability (Variance)'].append(np.var(nlg_counts / total_nlg * 100) if total_nlg > 0 else 0)

for key in results_table:
    if results_table[key]:
        results_table[key].append(np.mean(results_table[key]))
    else:
        results_table[key].append(0)

final_df = pd.DataFrame(results_table)
final_df = final_df.T
final_df.columns = [f'Iteration {i+1}' for i in range(num_iterations)] + ['Average']
final_df.to_excel('prediction_noisytop-kgating_results.xlsx', index=False)
final_df

2025-07-04 06:46:12.446085: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 06:46:12.753432: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 06:46:12.753627: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "mo_e_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  784896    
                                                                 
 embedding (Embedding)       multiple                  8576      
                                                                 
 layer_normalization (Layer  multiple                  256       
 Normalization)                                                  
                                                                 
 transformer_decoder_layer   multiple                  264576    
 (TransformerDecoderLayer)                                       
                                                                 
 dropout_3 (Dropout)         multiple                  0         
                                                                 
 moe_nlu (MoELayer)          multiple                  2

,Iteration 1,Average
Evaluation Time (seconds),23.712171,23.712171
Prediction Speed (ms/token),3.400129,3.400129
Average CPU Usage (percent),22.791625,22.791625
Average GPU Usage (percent),1.934860,1.934860
Average Memory (GB),5.661268,5.661268
Average GPU Memory (GB),14.504456,14.504456
Average FLOPs Estimate (GFLOPs),2.377871,2.377871
NLU Expert 1 (percent),0.000000,0.000000
NLU Expert 2 (percent),0.000000,0.000000
NLU Expert 3 (percent),6.460272,6.460272


# DeepSeekMoE with Efficient Distributed Sparse Routing experiment code

In [1]:
def reset_kernel():
    tf.keras.backend.clear_session()
    import gc
    gc.collect()

results_table = {
    'Training Speed (epochs per second)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Training Time (second/epoch)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Total Training Time (second/iteration)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Computational Resource Usage': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average CPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average GPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average GPU Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average FLOPS Estimate (GFLOPS)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Expert NLU Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Expert NLG Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Entity Accuracy': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Intent Accuracy': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Perplexity': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Response Cosine Similarity': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0}
}

best_val_loss = float('inf')
best_model_path = "best_deepseekefficientdis_model"

for iteration in range(5):
    print(f"\nStarting Training Iteration {iteration + 1}")
    reset_kernel()

    def load_tf_datasets_from_disk(load_path):
        print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
        try:
            with open(os.path.join(load_path, "moe_params.json"), "r") as f:
                loaded_moe_params = json.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"moe_params.json not found in {load_path}")

        element_spec = (
            {
                'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
                'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
                'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            },
            {
                'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
                'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
                'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
            }
        )

        try:
            train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
            test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
        except Exception as e:
            raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

        raw_data = {}
        for dataset_name in ['train', 'test']:
            path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
            if os.path.exists(path):
                with open(path, 'rb') as f:
                    raw_data[dataset_name] = pickle.load(f)
                print(f"Loaded raw data for {dataset_name} from {path}")
            else:
                print(f"Warning: Raw data file {path} not found.")
                raw_data[dataset_name] = []

        return {
            "train_dataset": train_tf_dataset,
            "test_dataset": test_tf_dataset,
            "moe_params": loaded_moe_params,
            "raw_data": raw_data
        }

    tf_dataset_save_path = "tf_datasets"
    loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
    train_tf_dataset = loaded_data["train_dataset"]
    moe_params = loaded_data["moe_params"]
    raw_data = loaded_data["raw_data"]

    class DeepSeekMoELayer(layers.Layer):
        def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.01, top_k=2, name=None):
            super(DeepSeekMoELayer, self).__init__(name=name)
            self.m = m
            self.k_s = k_s
            self.total_experts = num_experts * self.m
            self.routed_experts = self.total_experts - self.k_s
            self.top_k = top_k
            self.top_mk = self.m * self.top_k
            self.top_mk = min(self.top_mk, self.routed_experts)
            self.alpha = alpha
            self.input_dim = input_dim
            self.seq_length = None
            self.adjusted_expert_dim = expert_dim // self.m
            self.shared_experts = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                    layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
                ]) for _ in range(self.k_s)
            ]
            self.routed_experts_list = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                    layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
                ]) for _ in range(self.routed_experts)
            ]
            self.experts = self.shared_experts + self.routed_experts_list
            self.gate = layers.Dense(self.routed_experts, activation='softmax')
            self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

        def call(self, inputs, training=False):
            input_rank = inputs.shape.rank
            if input_rank == 3:
                batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
                flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
                is_3d = True
            else:
                batch_size = tf.shape(inputs)[0]
                seq_length = 1
                flat_inputs = inputs
                is_3d = False

            flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
            shared_output = tf.zeros([tf.shape(flat_inputs)[0], self.input_dim], dtype=tf.float32)
            for i in range(self.k_s):
                shared_output += self.shared_experts[i](flat_inputs)
            gate_weights = self.gate(flat_inputs)
            top_mk_values, top_mk_indices = tf.nn.top_k(gate_weights, k=self.top_mk, sorted=True)
            top_mk_indices = top_mk_indices + self.k_s
            top_mk_weights = top_mk_values / (tf.reduce_sum(top_mk_values, axis=-1, keepdims=True) + tf.keras.backend.epsilon())
            routed_output = tf.zeros([tf.shape(flat_inputs)[0], self.input_dim], dtype=tf.float32)
            for k in range(self.top_mk):
                kth_weights = top_mk_weights[:, k]
                kth_indices = top_mk_indices[:, k]
                mask = tf.one_hot(kth_indices, depth=self.total_experts, dtype=tf.float32)
                expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
                kth_expert_output = tf.reduce_sum(expert_outputs * tf.expand_dims(mask, -1), axis=1)
                weighted_kth_output = kth_expert_output * tf.expand_dims(kth_weights, -1)
                routed_output += weighted_kth_output
            output_flat = shared_output + routed_output + flat_inputs
            if is_3d:
                output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
                if self.seq_length is not None:
                    output.set_shape([None, self.seq_length, self.input_dim])
                gate_weights_reshaped = tf.reshape(gate_weights, [batch_size, seq_length, self.routed_experts])
                if self.seq_length is not None:
                    gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
            else:
                output = output_flat
                gate_weights_reshaped = gate_weights
            f_i = tf.reduce_mean(tf.reduce_sum(tf.one_hot(tf.argmax(gate_weights, axis=-1), depth=self.routed_experts), axis=0), axis=0)
            P_i = tf.reduce_mean(gate_weights, axis=0)
            load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
            self.add_loss(load_balancing_loss)
            self.load_balancing_loss_metric.update_state(load_balancing_loss)
            return output, gate_weights_reshaped

        def get_metrics(self):
            return {f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric}

        def get_config(self):
            config = super().get_config()
            config.update({
                'num_experts': self.total_experts // self.m,
                'expert_dim': self.adjusted_expert_dim * self.m,
                'input_dim': self.input_dim,
                'm': self.m,
                'k_s': self.k_s,
                'alpha': self.alpha,
                'top_k': self.top_k,
                'name': self.name
            })
            return config

    class TransformerDecoderLayer(layers.Layer):
        def __init__(self, d_model, num_heads, dff, rate=0.1):
            super(TransformerDecoderLayer, self).__init__()
            self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.ffn = keras.Sequential([
                layers.Dense(dff, activation='relu'),
                layers.Dense(d_model)
            ])
            self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
            self.dropout1 = layers.Dropout(rate)
            self.dropout2 = layers.Dropout(rate)
            self.dropout3 = layers.Dropout(rate)

        def call(self, x, enc_output, training, look_ahead_mask=None):
            if look_ahead_mask is not None:
                look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
                look_ahead_mask = 1.0 - look_ahead_mask
            attn1_output = self.mha1(query=x, value=x, key=x, attention_mask=look_ahead_mask, training=training)
            attn1 = self.dropout1(attn1_output, training=training)
            out1 = self.layernorm1(attn1 + x)
            attn2_output = self.mha2(query=out1, value=enc_output, key=enc_output, training=training)
            attn2 = self.dropout2(attn2_output, training=training)
            out2 = self.layernorm2(attn2 + out1)
            ffn_output = self.ffn(out2)
            ffn_output = self.dropout3(ffn_output, training=training)
            out3 = self.layernorm3(ffn_output + out2)
            return out3

        def get_config(self):
            config = super().get_config()
            mha1_config = self.mha1.get_config()
            config.update({
                'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
                'num_heads': mha1_config['num_heads'],
                'dff': self.ffn.layers[0].units,
                'rate': self.dropout1.rate
            })
            return config

    class MoEModel(models.Model):
        def __init__(self, moe_params):
            super(MoEModel, self).__init__()
            self.embedding_dim = moe_params["embedding_dim"]
            self.max_seq_length = moe_params["max_seq_length"]
            self.num_domains = moe_params["num_domains"]
            self.num_intents = moe_params["num_intents"]
            self.vocab_size = moe_params["vocab_size"]
            self.num_experts = moe_params["num_experts"]
            self.expert_dim = moe_params["expert_dim"]
            self.turn_id_dim = moe_params["turn_id_dim"]
            self.num_heads = 4
            self.dff = self.embedding_dim * 4
            self.dropout_rate = 0.1
            self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                                 self.num_domains + self.turn_id_dim + self.num_intents)
            self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains
            self.embedding = layers.Embedding(
                self.vocab_size,
                self.embedding_dim,
                mask_zero=True,
                embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05)
            )
            self.embedding.build((None,))
            with tf.init_scope():
                embedding_weights = self.embedding.get_weights()
                if embedding_weights and len(embedding_weights) > 0:
                    embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                    self.embedding.set_weights(embedding_weights)
            self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
            self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
            self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
            self.decoder_dropout = layers.Dropout(self.dropout_rate)
            self.moe_nlu = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlu_input_dim, m=2, k_s=2, alpha=0.01, top_k=2, name='moe_nlu')
            self.moe_nlg = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlg_input_dim, m=2, k_s=2, alpha=0.01, top_k=2, name='moe_nlg')
            self.intent_output = layers.Dense(self.num_intents, activation='sigmoid')
            self.domain_output = layers.Dense(self.num_domains, activation='softmax')
            self.response_output = layers.TimeDistributed(layers.Dense(self.vocab_size, activation='softmax'))
            self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)

        def create_look_ahead_mask(self, size):
            mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
            return mask

        def call(self, inputs, training=False):
            user_utterance_tokens = inputs['user_utterance_tokens']
            prev_system_response_tokens = inputs['prev_system_response_tokens']
            decoder_input_tokens = inputs['decoder_input_tokens']
            domain_onehot_input = inputs['domain_onehot_input']
            turn_id_embedding = inputs['turn_id_embedding']
            ontology_multihot_input = inputs['ontology_multihot_input']
            pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
            user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
            prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
            decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)
            user_enc_out = user_embedded
            prev_system_enc_out = prev_system_embedded
            look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
            dec_output = decoder_embedded
            for i, layer in enumerate(self.decoder_layers):
                dec_output = layer(dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask)
            decoder_output = self.decoder_dropout(dec_output, training=training)
            user_attn_out = self.attn_layer(query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), value=user_enc_out, key=user_enc_out, training=training)
            prev_system_attn_out = self.attn_layer(query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), value=prev_system_enc_out, key=prev_system_enc_out, training=training)
            decoder_attn_out = self.attn_layer(query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), value=decoder_output, key=decoder_output, training=training)
            combined_features = layers.Concatenate()([
                tf.squeeze(user_attn_out, 1),
                tf.squeeze(prev_system_attn_out, 1),
                tf.squeeze(decoder_attn_out, 1),
                domain_onehot_input,
                turn_id_embedding,
                ontology_multihot_input
            ])
            combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
            nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
            nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
            intent_out = self.intent_output(nlu_out)
            domain_out = self.domain_output(nlu_out)
            intent_out_expanded = tf.expand_dims(intent_out, axis=1)
            domain_out_expanded = tf.expand_dims(domain_out, axis=1)
            intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
            domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])
            nlg_input = layers.Concatenate(axis=-1)([
                decoder_output,
                intent_out_tiled,
                domain_out_tiled
            ])
            nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])
            nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
            nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
            response_out = self.response_output(nlg_out)
            return {
                'intent_output': intent_out,
                'domain_output': domain_out,
                'response_output': response_out,
                'response_embeddings': decoder_output,
                'nlu_gate_weights': nlu_gate_weights,
                'nlg_gate_weights': nlg_gate_weights
            }

        def build_graph(self):
            inputs = {
                'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
                'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
                'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
                'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32),
                'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32),
                'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32),
            }
            return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

        def get_config(self):
            config = super().get_config()
            config.update({
                'moe_params': {
                    'embedding_dim': self.embedding_dim,
                    'max_seq_length': self.max_seq_length,
                    'num_domains': self.num_domains,
                    'num_intents': self.num_intents,
                    'vocab_size': self.vocab_size,
                    'num_experts': self.num_experts,
                    'expert_dim': self.expert_dim,
                    'turn_id_dim': self.turn_id_dim
                }
            })
            return config

    class IntentAccuracy(metrics.Metric):
        def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros')
            self.total_preds = self.add_weight(name='total_preds', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
            self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
            self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

        def reset_state(self):
            self.correct_preds.assign(0.)
            self.total_preds.assign(0.)

    class IntentPrecision(metrics.Metric):
        def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros')
            self.predicted_positives = self.add_weight(name='pp', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            predicted_positives = tf.reduce_sum(y_pred)
            self.true_positives.assign_add(true_positives)
            self.predicted_positives.assign_add(predicted_positives)

        def result(self):
            return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.predicted_positives.assign(0.)

    class IntentRecall(metrics.Metric):
        def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros')
            self.actual_positives = self.add_weight(name='ap', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            actual_positives = tf.reduce_sum(y_true)
            self.true_positives.assign_add(true_positives)
            self.actual_positives.assign_add(actual_positives)

        def result(self):
            return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.actual_positives.assign(0.)

    class IntentF1(metrics.Metric):
        def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.precision = IntentPrecision(threshold=threshold)
            self.recall = IntentRecall(threshold=threshold)

        def update_state(self, y_true, y_pred, sample_weight=None):
            self.precision.update_state(y_true, y_pred, sample_weight)
            self.recall.update_state(y_true, y_pred, sample_weight)

        def result(self):
            p = self.precision.result()
            r = self.recall.result()
            return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

        def reset_state(self):
            self.precision.reset_state()
            self.recall.reset_state()

    class DomainAccuracy(metrics.Metric):
        def __init__(self, name='domain_accuracy', **kwargs):
            super().__init__(name=name, **kwargs)
            self.accuracy = self.add_weight(name='acc', initializer='zeros')
            self.count = self.add_weight(name='count', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
            pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
            matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
            self.accuracy.assign_add(tf.reduce_sum(matches))
            self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.accuracy / (self.count + tf.keras.backend.epsilon())

        def reset_state(self):
            self.accuracy.assign(0.)
            self.count.assign(0.)

    class Perplexity(metrics.Metric):
        def __init__(self, name='perplexity', **kwargs):
            super().__init__(name=name, **kwargs)
            self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
            self.total_loss = self.add_weight(name='total_loss', initializer='zeros')
            self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            mask = tf.cast(y_true != 0, dtype=tf.float32)
            loss = self.cross_entropy(y_true, y_pred)
            masked_loss = loss * mask
            sum_loss = tf.reduce_sum(masked_loss)
            num_non_padding_tokens = tf.reduce_sum(mask)
            self.total_loss.assign_add(sum_loss)
            self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

        def result(self):
            avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
            return tf.exp(avg_loss)

        def reset_state(self):
            self.total_loss.assign(0.)
            self.total_non_padding_tokens.assign(0.)

    class ResponseEmbeddingCosineSimilarity(metrics.Metric):
        def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
            super().__init__(name=name)
            self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
            self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            epsilon = tf.keras.backend.epsilon()
            y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
            y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
            y_true_norm = y_true / (y_true_norm_val + epsilon)
            y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
            cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
            non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
            non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
            masked_cosine_sim = cosine_sim_per_token * non_padding_mask
            num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
            avg_cosine_sim_per_sample = tf.where(
                num_non_zero_tokens > 0,
                tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
                tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
            )
            self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
            self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

        def reset_state(self):
            self.total_cosine_sim.assign(0.)
            self.num_samples.assign(0.)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    def calculate_flops(model, batch_size=moe_params["batch_size"]):
        flops = 0
        functional_model = model.build_graph()
        def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
            layer_flops_count = 0
            if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
                return 0
            if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
                for sub_layer in layer_instance.layers:
                    layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
                return layer_flops_count
            if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
                return 0
            if isinstance(layer_instance, layers.Dense):
                input_dim = layer_instance.input_shape[-1]
                output_dim = layer_instance.output_shape[-1]
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                layer_flops_count += 2 * input_dim * output_dim * effective_batch_size
            elif isinstance(layer_instance, layers.LSTM):
                input_dim = layer_instance.input_shape[-1]
                hidden_dim = layer_instance.units
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
                layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size
            elif isinstance(layer_instance, layers.Embedding):
                pass
            elif isinstance(layer_instance, DeepSeekMoELayer):
                moe_input_dim = layer_instance.input_dim
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
                layer_flops_count += gate_flops
                for expert in layer_instance.experts:
                    layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)
            elif isinstance(layer_instance, layers.MultiHeadAttention):
                config = layer_instance.get_config()
                num_heads = config['num_heads']
                key_dim = config['key_dim']
                d_model = num_heads * key_dim
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
                projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
                attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
                context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
                output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
                layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops
            elif isinstance(layer_instance, layers.TimeDistributed):
                inner_layer = layer_instance.layer
                td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
                layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)
            elif isinstance(layer_instance, TransformerDecoderLayer):
                layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
                layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
                ffn_effective_batch_size = current_batch_size * model.max_seq_length
                layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)
            return layer_flops_count
        for layer in functional_model.layers:
            flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)
        return flops / 1e9

    def get_gpu_stats():
        if nvidia_smi is None:
            return 0, 0
        try:
            nvidia_smi.nvmlInit()
            handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
            gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
            mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
            gpu_memory = mem_info.used / (1024 ** 3)
            nvidia_smi.nvmlShutdown()
            return gpu_load, gpu_memory
        except Exception:
            return 0, 0

    class ResourceMonitor(keras.callbacks.Callback):
        def __init__(self, validation_data):
            super().__init__()
            self.resource_stats = []
            self.expert_load_history = []
            self.start_time = None
            self.epoch_start_time = None
            self.flops = None
            self.validation_data = validation_data
            self.nlu_expert_usage_counts = None
            self.nlg_expert_usage_counts = None
            self.nlu_gate_weights_history = []
            self.nlg_gate_weights_history = []

        def on_train_begin(self, logs=None):
            self.start_time = time.time()
            try:
                self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
                self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
                self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
            except Exception:
                self.flops = 0

        def on_epoch_begin(self, epoch, logs=None):
            self.epoch_start_time = time.time()

        def on_epoch_end(self, epoch, logs=None):
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            epoch_time = time.time() - self.epoch_start_time
            self.resource_stats.append({
                'epoch': epoch + 1,
                'epoch_time': epoch_time,
                'cpu_usage': cpu_usage,
                'memory_used': memory.used / (1024 ** 3),
                'memory_total': memory.total / (1024 ** 3),
                'gpu_load': gpu_load,
                'gpu_memory': gpu_memory,
                'loss': logs.get('loss', 0),
                'val_loss': logs.get('val_loss', 0),
                'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
                'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
                'intent_precision': logs.get('intent_output_intent_precision', 0),
                'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
                'intent_recall': logs.get('intent_output_intent_recall', 0),
                'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
                'intent_f1': logs.get('intent_output_intent_f1', 0),
                'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
                'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
                'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
                'perplexity': logs.get('response_output_perplexity', 0),
                'val_perplexity': logs.get('val_response_output_perplexity', 0),
                'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
                'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
                'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
                'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
            })
            if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
                self.expert_load_history.append({
                    'epoch': epoch + 1,
                    'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                    'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
                })
            sample_size = 100
            for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
                inputs, _ = batch
                outputs = self.model(inputs, training=False)
                nlu_gate_weights = outputs['nlu_gate_weights']
                reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
                self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
                nlg_gate_weights = outputs['nlg_gate_weights']
                reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
                self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

        def on_train_end(self, logs=None):
            self.total_time = time.time() - self.start_time
            if len(self.nlu_gate_weights_history) > 0:
                all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
                nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
                for expert_id in nlu_top_indices:
                    if expert_id < self.model.moe_nlu.total_experts:
                        self.nlu_expert_usage_counts[expert_id] += 1
            if len(self.nlg_gate_weights_history) > 0:
                all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
                nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
                for expert_id in nlg_top_indices:
                    if expert_id < self.model.moe_nlg.total_experts:
                        self.nlg_expert_usage_counts[expert_id] += 1

        def visualize_expert_load_distribution(self):
            total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
            total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
            nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
            nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
            nlu_stability = np.var(nlu_percentages)
            nlg_stability = np.var(nlg_percentages)
            return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

    class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
        def __init__(self, d_model, warmup_steps=4000):
            super().__init__()
            self.d_model = tf.cast(d_model, tf.float32)
            self.warmup_steps = warmup_steps

        def __call__(self, step):
            step = tf.cast(step, tf.float32)
            arg1 = tf.math.rsqrt(step)
            arg2 = step * (self.warmup_steps ** -1.5)
            return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

        def get_config(self):
            return {"d_model": self.d_model.numpy(), "warmup_steps": self.warmup_steps}

    model = MoEModel(moe_params)
    embedding_layer = model.embedding

    def create_validation_split(dataset, embedding_layer, validation_split=0.2, batch_size=moe_params["batch_size"]):
        element_spec = dataset.element_spec
        data_list = [(features, targets) for features, targets in dataset.unbatch().as_numpy_iterator()]
        if not data_list:
            raise ValueError("Dataset is empty.")
        if len(data_list) < 2:
            raise ValueError("Dataset too small to split.")
        train_data, val_data = train_test_split(data_list, test_size=validation_split, shuffle=True)
        def create_dataset_from_list(data):
            if not data:
                raise ValueError("Empty data list provided.")
            features = {
                'user_utterance_tokens': np.array([d[0]['user_utterance_tokens'] for d in data], dtype=np.int32),
                'prev_system_response_tokens': np.array([d[0]['prev_system_response_tokens'] for d in data], dtype=np.int32),
                'decoder_input_tokens': np.array([d[0]['decoder_input_tokens'] for d in data], dtype=np.int32),
                'domain_onehot_input': np.array([d[0]['domain_onehot_input'] for d in data], dtype=np.float32),
                'turn_id_embedding': np.array([d[0]['turn_id_embedding'] for d in data], dtype=np.float32),
                'ontology_multihot_input': np.array([d[0]['ontology_multihot_input'] for d in data], dtype=np.float32),
            }
            response_output = np.array([d[1]['response_output'] for d in data], dtype=np.int32)
            response_embeddings = embedding_layer(response_output).numpy()
            targets = {
                'domain_output': np.array([d[1]['domain_output'] for d in data], dtype=np.int32),
                'intent_output': np.array([d[1]['intent_output'] for d in data], dtype=np.float32),
                'response_output': response_output,
                'response_embeddings': response_embeddings
            }
            return tf.data.Dataset.from_tensor_slices((features, targets))
        train_dataset = create_dataset_from_list(train_data).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        val_dataset = create_dataset_from_list(val_data).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return train_dataset, val_dataset

    train_tf_dataset, val_tf_dataset = create_validation_split(train_tf_dataset, embedding_layer)
    losses_dict = {
        'intent_output': losses.BinaryCrossentropy(),
        'domain_output': losses.SparseCategoricalCrossentropy(),
        'response_output': losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss
    }
    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }
    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={'intent_output': 0.5, 'domain_output': 0.5, 'response_output': 1.0, 'response_embeddings': 1.0}
    )
    sample_input = next(iter(train_tf_dataset.take(1)))
    model(sample_input[0])
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=8, mode='min', restore_best_weights=True)
    resource_monitor = ResourceMonitor(val_tf_dataset)
    class TQDMProgressBar(callbacks.Callback):
        def on_epoch_begin(self, epoch, logs=None):
            self.progress_bar = tqdm(total=len(train_tf_dataset), desc=f'Epoch {epoch + 1}')
        def on_batch_end(self, batch, logs=None):
            self.progress_bar.update(1)
        def on_epoch_end(self, epoch, logs=None):
            self.progress_bar.close()
    tqdm_callback = TQDMProgressBar()
    history = model.fit(
        train_tf_dataset,
        epochs=100,
        validation_data=val_tf_dataset,
        callbacks=[early_stopping, resource_monitor, tqdm_callback],
        verbose=0
    )
    expert_stats = resource_monitor.visualize_expert_load_distribution()
    stats = resource_monitor.resource_stats
    avg_epoch_duration = np.mean([s['epoch_time'] for s in stats]) if stats else 0
    total_training_time = resource_monitor.total_time
    avg_cpu_usage = np.mean([s['cpu_usage'] for s in stats]) if stats else 0
    avg_memory = np.mean([s['memory_used'] for s in stats]) if stats else 0
    avg_gpu_load = np.mean([s['gpu_load'] for s in stats]) if stats else 0
    avg_gpu_memory = np.mean([s['gpu_memory'] for s in stats]) if stats else 0
    avg_flops = resource_monitor.flops if resource_monitor.flops else 0
    nlu_counts = resource_monitor.nlu_expert_usage_counts
    nlg_counts = resource_monitor.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) or 1
    total_nlg = np.sum(nlg_counts) or 1
    expert_nlu_percentages = [(nlu_counts[i] / total_nlu) * 100 for i in range(8)]
    expert_nlg_percentages = [(nlg_counts[i] / total_nlg) * 100 for i in range(8)]
    val_intent_accuracy = np.mean([s['val_intent_accuracy'] for s in stats if s['val_intent_accuracy'] > 0]) if stats else 0
    val_entity_accuracy = np.mean([s['val_domain_accuracy'] for s in stats if s['val_domain_accuracy'] > 0]) if stats else 0
    val_perplexity = np.mean([s['val_perplexity'] for s in stats if s['val_perplexity'] > 0]) if stats else 0
    val_cosine_similarity = np.mean([s['val_cosine_similarity'] for s in stats if s['val_cosine_similarity'] > 0]) if stats and any(s['val_cosine_similarity'] > 0 for s in stats) else 0
    results_table['Average Validation Response Cosine Similarity'][f'Iteration {iteration + 1}'] = val_cosine_similarity
    results_table['Training Speed (epochs per second)'][f'Iteration {iteration + 1}'] = 1 / avg_epoch_duration if avg_epoch_duration > 0 else 0
    results_table['Training Time (second/epoch)'][f'Iteration {iteration + 1}'] = avg_epoch_duration
    results_table['Total Training Time (second/iteration)'][f'Iteration {iteration + 1}'] = total_training_time
    results_table['Computational Resource Usage'][f'Iteration {iteration + 1}'] = avg_cpu_usage + avg_gpu_load
    results_table['Average CPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_cpu_usage
    results_table['Average GPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_gpu_load
    results_table['Average Memory (GB)'][f'Iteration {iteration + 1}'] = avg_memory
    results_table['Average GPU Memory (GB)'][f'Iteration {iteration + 1}'] = avg_gpu_memory
    results_table['Average FLOPS Estimate (GFLOPS)'][f'Iteration {iteration + 1}'] = avg_flops
    results_table['Expert NLU Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlu_stability']
    results_table['Expert NLG Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlg_stability']
    for i in range(8):
        if i < len(expert_stats['nlu_percentages']):
            results_table[f'Average NLU Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlu_percentages'][i]
        if i < len(expert_stats['nlg_percentages']):
            results_table[f'Average NLG Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlg_percentages'][i]
    results_table['Average Validation Entity Accuracy'][f'Iteration {iteration + 1}'] = val_entity_accuracy
    results_table['Average Intent Accuracy'][f'Iteration {iteration + 1}'] = val_intent_accuracy
    results_table['Average Validation Perplexity'][f'Iteration {iteration + 1}'] = val_perplexity
    results_table['Average Validation Response Cosine Similarity'][f'Iteration {iteration + 1}'] = val_cosine_similarity
    val_loss = min([s['val_loss'] for s in stats]) if stats else float('inf')
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        try:
            model.save(best_model_path, save_format='tf', include_optimizer=True)
        except Exception as e:
            print(f"\nFailed to save best model to {best_model_path}: {str(e)}")

for key in results_table:
    results_table[key]['Average'] = np.mean([results_table[key][f'Iteration {i+1}'] for i in range(10)])

final_result = pd.DataFrame(results_table)
final_result = final_result.T
final_result.to_excel('training_deepseekefficientdis_result.xlsx', index=True)
final_result

2025-07-04 07:17:03.965878: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 07:17:04.381808: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 07:17:04.381961: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 07:17:04.466307: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 07:17:04.613763: I tensorflow/core/platform/cpu_feature_guar


Starting Training Iteration 1

Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 07:17:09.674440: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 07:17:09.970538: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 07:17:09.970824: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl


Epoch 1:   0%|          | 0/4428 [00:00<?, ?it/s]2025-07-04 07:17:35.240570: I external/local_xla/xla/service/service.cc:168] XLA service 0xb54c840 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-04 07:17:35.240648: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2025-07-04 07:17:35.258485: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-04 07:17:35.297694: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1751613455.416348    9833 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
Epoch 1: 100%|██████████| 4428/4428 [03:23<00:00, 21.76it/s]


INFO:tensorflow:Assets written to: best_deepseekefficientdis_model/assets


INFO:tensorflow:Assets written to: best_deepseekefficientdis_model/assets


,Iteration 1,Iteration 2,Iteration 3,Iteration 4,Iteration 5,Iteration 6,Iteration 7,Iteration 8,Iteration 9,Iteration 10,Average
Training Speed (epochs per second),0.005058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000506
Training Time (second/epoch),197.697094,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.769709
Total Training Time (second/iteration),205.429722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.542972
Computational Resource Usage,57.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.760000
Average CPU Usage (percent),26.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.660000
Average GPU Usage (percent),31.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.100000
Average Memory (GB),6.583912,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.658391
Average GPU Memory (GB),14.531799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.453180
Average FLOPS Estimate (GFLOPS),2.377871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.237787
Expert NLU Load Distribution Stability,1093.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,109.375000


# DeepSeekMoE with Efficient Distributed Sparse Routing evaluation code

In [1]:
def load_tf_datasets_from_disk(load_path):
    print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
    try:
        with open(os.path.join(load_path, "moe_params.json"), "r") as f:
            loaded_moe_params = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"moe_params.json not found in {load_path}")

    element_spec = (
        {
            'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
            'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
            'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
        },
        {
            'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
            'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
        }
    )

    try:
        train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
        test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
    except Exception as e:
        raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

    raw_data = {}
    for dataset_name in ['train', 'test']:
        path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
        if os.path.exists(path):
            with open(path, 'rb') as f:
                raw_data[dataset_name] = pickle.load(f)
            print(f"Loaded raw data for {dataset_name} from {path}")
        else:
            print(f"Warning: Raw data file {path} not found.")
            raw_data[dataset_name] = []

    return {
        "train_dataset": train_tf_dataset,
        "test_dataset": test_tf_dataset,
        "moe_params": loaded_moe_params,
        "raw_data": raw_data
    }

tf_dataset_save_path = "tf_datasets"
loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
train_tf_dataset = loaded_data["train_dataset"]
test_tf_dataset = loaded_data["test_dataset"]
moe_params = loaded_data["moe_params"]
raw_data = loaded_data["raw_data"]

moe_params["num_experts"] = 4

class DeepSeekMoELayer(layers.Layer):
    def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.01, top_k=2, name=None):
        super(DeepSeekMoELayer, self).__init__(name=name)
        self.m = m
        self.k_s = k_s
        self.total_experts = num_experts * self.m
        self.routed_experts = self.total_experts - self.k_s
        self.top_k = top_k
        self.top_mk = self.m * self.top_k
        self.top_mk = min(self.top_mk, self.routed_experts)
        self.alpha = alpha
        self.input_dim = input_dim
        self.seq_length = None
        self.adjusted_expert_dim = expert_dim // self.m
        self.shared_experts = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
            ]) for _ in range(self.k_s)
        ]
        self.routed_experts_list = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
            ]) for _ in range(self.routed_experts)
        ]
        self.experts = self.shared_experts + self.routed_experts_list
        self.gate = layers.Dense(self.routed_experts, activation='softmax')
        self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

    def call(self, inputs, training=False):
        input_rank = inputs.shape.rank
        if input_rank == 3:
            batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
            flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
            is_3d = True
        else:
            batch_size = tf.shape(inputs)[0]
            seq_length = 1
            flat_inputs = inputs
            is_3d = False

        flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
        shared_output = tf.zeros([tf.shape(flat_inputs)[0], self.input_dim], dtype=tf.float32)
        for i in range(self.k_s):
            shared_output += self.shared_experts[i](flat_inputs)
        gate_weights = self.gate(flat_inputs)
        top_mk_values, top_mk_indices = tf.nn.top_k(gate_weights, k=self.top_mk, sorted=True)
        top_mk_indices = top_mk_indices + self.k_s
        top_mk_weights = top_mk_values / (tf.reduce_sum(top_mk_values, axis=-1, keepdims=True) + tf.keras.backend.epsilon())
        routed_output = tf.zeros([tf.shape(flat_inputs)[0], self.input_dim], dtype=tf.float32)
        for k in range(self.top_mk):
            kth_weights = top_mk_weights[:, k]
            kth_indices = top_mk_indices[:, k]
            mask = tf.one_hot(kth_indices, depth=self.total_experts, dtype=tf.float32)
            expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
            kth_expert_output = tf.reduce_sum(expert_outputs * tf.expand_dims(mask, -1), axis=1)
            weighted_kth_output = kth_expert_output * tf.expand_dims(kth_weights, -1)
            routed_output += weighted_kth_output
        output_flat = shared_output + routed_output + flat_inputs
        if is_3d:
            output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
            if self.seq_length is not None:
                output.set_shape([None, self.seq_length, self.input_dim])
            gate_weights_reshaped = tf.reshape(gate_weights, [batch_size, seq_length, self.routed_experts])
            if self.seq_length is not None:
                gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
        else:
            output = output_flat
            gate_weights_reshaped = gate_weights
        f_i = tf.reduce_mean(tf.reduce_sum(tf.one_hot(tf.argmax(gate_weights, axis=-1), depth=self.routed_experts), axis=0), axis=0)
        P_i = tf.reduce_mean(gate_weights, axis=0)
        load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
        self.add_loss(load_balancing_loss)
        self.load_balancing_loss_metric.update_state(load_balancing_loss)
        return output, gate_weights_reshaped

    def get_metrics(self):
        return {f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric}

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_experts': self.total_experts // self.m,
            'expert_dim': self.adjusted_expert_dim * self.m,
            'input_dim': self.input_dim,
            'm': self.m,
            'k_s': self.k_s,
            'alpha': self.alpha,
            'top_k': self.top_k,
            'name': self.name
        })
        return config

class TransformerDecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.ffn = keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask=None):
        if look_ahead_mask is not None:
            look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
            look_ahead_mask = 1.0 - look_ahead_mask
        attn1_output = self.mha1(query=x, value=x, key=x, attention_mask=look_ahead_mask, training=training)
        attn1 = self.dropout1(attn1_output, training=training)
        out1 = self.layernorm1(attn1 + x)
        attn2_output = self.mha2(query=out1, value=enc_output, key=enc_output, training=training)
        attn2 = self.dropout2(attn2_output, training=training)
        out2 = self.layernorm2(attn2 + out1)
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)
        return out3

    def get_config(self):
        config = super().get_config()
        mha1_config = self.mha1.get_config()
        config.update({
            'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
            'num_heads': mha1_config['num_heads'],
            'dff': self.ffn.layers[0].units,
            'rate': self.dropout1.rate
        })
        return config

class MoEModel(models.Model):
    def __init__(self, moe_params):
        super(MoEModel, self).__init__()
        self.embedding_dim = moe_params["embedding_dim"]
        self.max_seq_length = moe_params["max_seq_length"]
        self.num_domains = moe_params["num_domains"]
        self.num_intents = moe_params["num_intents"]
        self.vocab_size = moe_params["vocab_size"]
        self.num_experts = moe_params["num_experts"]
        self.expert_dim = moe_params["expert_dim"]
        self.turn_id_dim = moe_params["turn_id_dim"]
        self.num_heads = 4
        self.dff = self.embedding_dim * 4
        self.dropout_rate = 0.1
        self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                             self.num_domains + self.turn_id_dim + self.num_intents)
        self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains
        self.embedding = layers.Embedding(
            self.vocab_size,
            self.embedding_dim,
            mask_zero=True,
            embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05)
        )
        self.embedding.build((None,))
        with tf.init_scope():
            embedding_weights = self.embedding.get_weights()
            if embedding_weights and len(embedding_weights) > 0:
                embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                self.embedding.set_weights(embedding_weights)
        self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
        self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
        self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
        self.decoder_dropout = layers.Dropout(self.dropout_rate)
        self.moe_nlu = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlu_input_dim, m=2, k_s=2, alpha=0.01, top_k=2, name='moe_nlu')
        self.moe_nlg = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlg_input_dim, m=2, k_s=2, alpha=0.01, top_k=2, name='moe_nlg')
        self.intent_output = layers.Dense(self.num_intents, activation='sigmoid')
        self.domain_output = layers.Dense(self.num_domains, activation='softmax')
        self.response_output = layers.TimeDistributed(layers.Dense(self.vocab_size, activation='softmax'))
        self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)

    def create_look_ahead_mask(self, size):
        mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
        return mask

    def call(self, inputs, training=False):
        user_utterance_tokens = inputs['user_utterance_tokens']
        prev_system_response_tokens = inputs['prev_system_response_tokens']
        decoder_input_tokens = inputs['decoder_input_tokens']
        domain_onehot_input = inputs['domain_onehot_input']
        turn_id_embedding = inputs['turn_id_embedding']
        ontology_multihot_input = inputs['ontology_multihot_input']
        pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
        user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
        prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
        decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)
        user_enc_out = user_embedded
        prev_system_enc_out = prev_system_embedded
        look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
        dec_output = decoder_embedded
        for i, layer in enumerate(self.decoder_layers):
            dec_output = layer(dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask)
        decoder_output = self.decoder_dropout(dec_output, training=training)
        user_attn_out = self.attn_layer(query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), value=user_enc_out, key=user_enc_out, training=training)
        prev_system_attn_out = self.attn_layer(query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), value=prev_system_enc_out, key=prev_system_enc_out, training=training)
        decoder_attn_out = self.attn_layer(query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), value=decoder_output, key=decoder_output, training=training)
        combined_features = layers.Concatenate()([
            tf.squeeze(user_attn_out, 1),
            tf.squeeze(prev_system_attn_out, 1),
            tf.squeeze(decoder_attn_out, 1),
            domain_onehot_input,
            turn_id_embedding,
            ontology_multihot_input
        ])
        combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
        nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
        nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
        intent_out = self.intent_output(nlu_out)
        domain_out = self.domain_output(nlu_out)
        intent_out_expanded = tf.expand_dims(intent_out, axis=1)
        domain_out_expanded = tf.expand_dims(domain_out, axis=1)
        intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
        domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])
        nlg_input = layers.Concatenate(axis=-1)([
            decoder_output,
            intent_out_tiled,
            domain_out_tiled
        ])
        nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])
        nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
        nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
        response_out = self.response_output(nlg_out)
        return {
            'intent_output': intent_out,
            'domain_output': domain_out,
            'response_output': response_out,
            'response_embeddings': decoder_output,
            'nlu_gate_weights': nlu_gate_weights,
            'nlg_gate_weights': nlg_gate_weights
        }

    def build_graph(self):
        inputs = {
            'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
            'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
            'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
            'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32),
            'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32),
            'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32),
        }
        return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

    def get_config(self):
        config = super().get_config()
        config.update({
            'moe_params': {
                'embedding_dim': self.embedding_dim,
                'max_seq_length': self.max_seq_length,
                'num_domains': self.num_domains,
                'num_intents': self.num_intents,
                'vocab_size': self.vocab_size,
                'num_experts': self.num_experts,
                'expert_dim': self.expert_dim,
                'turn_id_dim': self.turn_id_dim
            }
        })
        return config

class IntentAccuracy(metrics.Metric):
    def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros')
        self.total_preds = self.add_weight(name='total_preds', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
        self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
        self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

    def reset_state(self):
        self.correct_preds.assign(0.)
        self.total_preds.assign(0.)

class IntentPrecision(metrics.Metric):
    def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.predicted_positives = self.add_weight(name='pp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        predicted_positives = tf.reduce_sum(y_pred)
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)

    def result(self):
        return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.predicted_positives.assign(0.)

class IntentRecall(metrics.Metric):
    def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.actual_positives = self.add_weight(name='ap', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        actual_positives = tf.reduce_sum(y_true)
        self.true_positives.assign_add(true_positives)
        self.actual_positives.assign_add(actual_positives)

    def result(self):
        return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.actual_positives.assign(0.)

class IntentF1(metrics.Metric):
    def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = IntentPrecision(threshold=threshold)
        self.recall = IntentRecall(threshold=threshold)

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

class DomainAccuracy(metrics.Metric):
    def __init__(self, name='domain_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.accuracy = self.add_weight(name='acc', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
        pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
        matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
        self.accuracy.assign_add(tf.reduce_sum(matches))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.accuracy / (self.count + tf.keras.backend.epsilon())

    def reset_state(self):
        self.accuracy.assign(0.)
        self.count.assign(0.)

class Perplexity(metrics.Metric):
    def __init__(self, name='perplexity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
        self.total_loss = self.add_weight(name='total_loss', initializer='zeros')
        self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        mask = tf.cast(y_true != 0, dtype=tf.float32)
        loss = self.cross_entropy(y_true, y_pred)
        masked_loss = loss * mask
        sum_loss = tf.reduce_sum(masked_loss)
        num_non_padding_tokens = tf.reduce_sum(mask)
        self.total_loss.assign_add(sum_loss)
        self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

    def result(self):
        avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
        return tf.exp(avg_loss)

    def reset_state(self):
        self.total_loss.assign(0.)
        self.total_non_padding_tokens.assign(0.)

class ResponseEmbeddingCosineSimilarity(metrics.Metric):
    def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
        super().__init__(name=name)
        self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
        self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true_norm = y_true / (y_true_norm_val + epsilon)
        y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
        cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
        non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
        non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
        masked_cosine_sim = cosine_sim_per_token * non_padding_mask
        num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
        avg_cosine_sim_per_sample = tf.where(
            num_non_zero_tokens > 0,
            tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
            tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
        )
        self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
        self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

    def reset_state(self):
        self.total_cosine_sim.assign(0.)
        self.num_samples.assign(0.)

def contrastive_loss(y_true, y_pred, margin=1.0):
    epsilon = tf.keras.backend.epsilon()
    y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
    y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
    y_true = y_true / (y_true_norm + epsilon)
    y_pred = y_pred / (y_pred_norm + epsilon)
    cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
    positive_loss = 1.0 - cosine_sim
    mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
    masked_loss = positive_loss * mask
    total_loss = tf.reduce_sum(masked_loss)
    num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
    return total_loss / num_tokens

def calculate_flops(model, batch_size=moe_params["batch_size"]):
    flops = 0
    functional_model = model.build_graph()
    def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
        layer_flops_count = 0
        if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
            return 0
        if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
            for sub_layer in layer_instance.layers:
                layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
            return layer_flops_count
        if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
            return 0
        if isinstance(layer_instance, layers.Dense):
            input_dim = layer_instance.input_shape[-1]
            output_dim = layer_instance.output_shape[-1]
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            layer_flops_count += 2 * input_dim * output_dim * effective_batch_size
        elif isinstance(layer_instance, layers.LSTM):
            input_dim = layer_instance.input_shape[-1]
            hidden_dim = layer_instance.units
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
            layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size
        elif isinstance(layer_instance, layers.Embedding):
            pass
        elif isinstance(layer_instance, DeepSeekMoELayer):
            moe_input_dim = layer_instance.input_dim
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
            layer_flops_count += gate_flops
            for expert in layer_instance.experts:
                layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)
        elif isinstance(layer_instance, layers.MultiHeadAttention):
            config = layer_instance.get_config()
            num_heads = config['num_heads']
            key_dim = config['key_dim']
            d_model = num_heads * key_dim
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
            projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
            attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
            context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
            output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
            layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops
        elif isinstance(layer_instance, layers.TimeDistributed):
            inner_layer = layer_instance.layer
            td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
            layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)
        elif isinstance(layer_instance, TransformerDecoderLayer):
            layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
            layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
            ffn_effective_batch_size = current_batch_size * model.max_seq_length
            layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)
        return layer_flops_count
    for layer in functional_model.layers:
        flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)
    return flops / 1e9

def get_gpu_stats():
    if nvidia_smi is None:
        return 0, 0
    try:
        nvidia_smi.nvmlInit()
        handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
        gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
        mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
        gpu_memory = mem_info.used / (1024 ** 3)
        nvidia_smi.nvmlShutdown()
        return gpu_load, gpu_memory
    except Exception:
        return 0, 0

class ResourceMonitor(keras.callbacks.Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.resource_stats = []
        self.expert_load_history = []
        self.start_time = None
        self.epoch_start_time = None
        self.flops = None
        self.validation_data = validation_data
        self.nlu_expert_usage_counts = None
        self.nlg_expert_usage_counts = None
        self.nlu_gate_weights_history = []
        self.nlg_gate_weights_history = []

    def on_train_begin(self, logs=None):
        self.start_time = time.time()
        try:
            self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
            self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
            self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
        except Exception:
            self.flops = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory()
        gpu_load, gpu_memory = get_gpu_stats()
        epoch_time = time.time() - self.epoch_start_time
        self.resource_stats.append({
            'epoch': epoch + 1,
            'epoch_time': epoch_time,
            'cpu_usage': cpu_usage,
            'memory_used': memory.used / (1024 ** 3),
            'memory_total': memory.total / (1024 ** 3),
            'gpu_load': gpu_load,
            'gpu_memory': gpu_memory,
            'loss': logs.get('loss', 0),
            'val_loss': logs.get('val_loss', 0),
            'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
            'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
            'intent_precision': logs.get('intent_output_intent_precision', 0),
            'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
            'intent_recall': logs.get('intent_output_intent_recall', 0),
            'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
            'intent_f1': logs.get('intent_output_intent_f1', 0),
            'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
            'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
            'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
            'perplexity': logs.get('response_output_perplexity', 0),
            'val_perplexity': logs.get('val_response_output_perplexity', 0),
            'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
            'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
            'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
            'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
        })
        if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
            self.expert_load_history.append({
                'epoch': epoch + 1,
                'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
            })
        sample_size = 100
        for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
            inputs, _ = batch
            outputs = self.model(inputs, training=False)
            nlu_gate_weights = outputs['nlu_gate_weights']
            reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
            self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
            nlg_gate_weights = outputs['nlg_gate_weights']
            reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
            self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

    def on_train_end(self, logs=None):
        self.total_time = time.time() - self.start_time
        if len(self.nlu_gate_weights_history) > 0:
            all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
            nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
            for expert_id in nlu_top_indices:
                if expert_id < self.model.moe_nlu.total_experts:
                    self.nlu_expert_usage_counts[expert_id] += 1
        if len(self.nlg_gate_weights_history) > 0:
            all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
            nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
            for expert_id in nlg_top_indices:
                if expert_id < self.model.moe_nlg.total_experts:
                    self.nlg_expert_usage_counts[expert_id] += 1

    def visualize_expert_load_distribution(self):
        total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
        total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
        nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
        nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
        nlu_stability = np.var(nlu_percentages)
        nlg_stability = np.var(nlg_percentages)
        return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {"d_model": self.d_model.numpy(), "warmup_steps": self.warmup_steps}

word_to_id = {}
id_to_word = {}
try:
    possible_paths = [
        os.path.join("preprocessor_models", "preprocessor_params.json"),
        os.path.join("tf_datasets", "preprocessor_params.json"),
        "preprocessor_params.json"
    ]
    vocab_loaded = False
    for path in possible_paths:
        if os.path.exists(path):
            with open(path, "r") as f:
                preprocessor_params = json.load(f)
            word_to_id = {k: int(v) for k, v in preprocessor_params['word_to_id'].items()}
            id_to_word = {int(k): v for k, v in preprocessor_params['id_to_word'].items()}
            print(f"Loaded vocabulary from {path}")
            vocab_loaded = True
            break
    if not vocab_loaded:
        raise FileNotFoundError("Vocabulary file not found in any location")
except Exception as e:
    print(f"Warning: Could not load vocabulary from preprocessor: {str(e)}")
    word_to_id = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
    id_to_word = {0: '<pad>', 1: '<sos>', 2: '<eos>', 3: '<unk>'}

model_save_path = "best_deepseekefficientdis_model"
print(f"\nLoading model from: {model_save_path}")
custom_objects = {
    "MoEModel":MoEModel,
    "CustomLearningRateSchedule": CustomLearningRateSchedule,
    "DeepSeekMoELayer": DeepSeekMoELayer,
    "TransformerDecoderLayer": TransformerDecoderLayer,
    "IntentAccuracy": IntentAccuracy,
    "IntentPrecision": IntentPrecision,
    "IntentRecall": IntentRecall,
    "IntentF1": IntentF1,
    "DomainAccuracy": DomainAccuracy,
    "Perplexity": Perplexity,
    "ResponseEmbeddingCosineSimilarity": ResponseEmbeddingCosineSimilarity
}
try:
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
except Exception as e:
    print(f"Failed to load model from {model_save_path}: {str(e)}")
    raise

def compile_model(model, moe_params):
    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens
    losses_dict = {
        'intent_output': tf.keras.losses.BinaryCrossentropy(),
        'domain_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss,
    }
    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }
    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={
            'intent_output': 0.5,
            'domain_output': 0.5,
            'response_output': 1.0,
            'response_embeddings': 1.0
        }
    )
    return model

model = compile_model(model, moe_params)
print("\nModel loaded and compiled successfully!")
sample_input = next(iter(train_tf_dataset.take(1)))
model(sample_input[0])
model.summary()

class EvaluationMetrics:
    def __init__(self, model, test_dataset, moe_params, raw_data, word_to_id, id_to_word):
        self.model = model
        self.test_dataset = test_dataset
        self.moe_params = moe_params
        self.raw_data = raw_data
        self.word_to_id = word_to_id
        self.id_to_word = id_to_word
        self.nlu_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.nlg_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.resource_stats = {
            'cpu_usage': [],
            'memory_used': [],
            'gpu_load': [],
            'gpu_memory': [],
            'batch_times': []
        }
        self.special_tokens = {'<pad>', '<sos>', '<eos>', '<unk>'}
        self.nlu_top_k = model.get_layer('moe_nlu').get_config().get('top_k', 2)
        self.nlg_top_k = model.get_layer('moe_nlg').get_config().get('top_k', 2)

    def evaluate(self):
        print("\nEvaluating model on test set...")
        def add_response_embeddings(features, targets):
            response_tokens = targets['response_output']
            response_embeddings = self.model.embedding(response_tokens)
            targets['response_embeddings'] = response_embeddings
            return features, targets
        test_dataset_with_embeddings = self.test_dataset.map(add_response_embeddings, num_parallel_calls=tf.data.AUTOTUNE)
        start_time = time.time()
        test_results = self.model.evaluate(test_dataset_with_embeddings, verbose=1)
        eval_time = time.time() - start_time
        metric_names = self.model.metrics_names
        perplexity_value = None
        for name, value in zip(metric_names, test_results):
            if name == 'response_output_perplexity':
                perplexity_value = value
                break
        if perplexity_value is None:
            print("Warning: Could not find perplexity in standard evaluation results.")
            perplexity_value = 0.0
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory().used / (1024 ** 3)
        gpu_load, gpu_memory = get_gpu_stats()
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))
        total_tokens = 0
        total_time = 0
        num_batches = 0
        test_dataset_small = test_dataset_with_embeddings.unbatch().batch(moe_params["batch_size"])
        all_true_intents = []
        all_pred_intents = []
        all_true_domains = []
        all_pred_domains = []
        for batch_idx, batch in enumerate(test_dataset_small):
            inputs, targets = batch
            response_tokens = targets['response_output'].numpy()
            non_padding_mask = response_tokens != 0
            total_tokens += np.sum(non_padding_mask)
            start_time = time.time()
            outputs = self.model(inputs, training=False)
            batch_time = time.time() - start_time
            total_time += batch_time
            num_batches += tf.shape(inputs['user_utterance_tokens'])[0]
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            self.resource_stats['cpu_usage'].append(cpu_usage)
            self.resource_stats['memory_used'].append(memory.used / (1024 ** 3))
            self.resource_stats['gpu_load'].append(gpu_load)
            self.resource_stats['gpu_memory'].append(gpu_memory)
            self.resource_stats['batch_times'].append(batch_time)
            nlu_gate_weights = outputs.get('nlu_gate_weights', tf.zeros([0, self.model.moe_nlu.routed_experts])).numpy()
            nlu_top_indices = np.argsort(-nlu_gate_weights, axis=-1)[:, :self.nlu_top_k] + self.model.moe_nlu.k_s
            for idx in range(self.nlu_top_k):
                expert_ids = nlu_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlu_expert_usage_counts):
                        self.nlu_expert_usage_counts[expert_id] += 1
            nlg_gate_weights = outputs.get('nlg_gate_weights', tf.zeros([0, self.model.moe_nlg.routed_experts])).numpy()
            if len(nlg_gate_weights.shape) == 3:
                nlg_gate_weights = nlg_gate_weights.reshape(-1, nlg_gate_weights.shape[-1])
            nlg_top_indices = np.argsort(-nlg_gate_weights, axis=-1)[:, :self.nlg_top_k] + self.model.moe_nlg.k_s
            for idx in range(self.nlg_top_k):
                expert_ids = nlg_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlg_expert_usage_counts):
                        self.nlg_expert_usage_counts[expert_id] += 1
            true_intents = targets.get('intent_output', np.zeros((moe_params["batch_size"], self.model.num_intents))).numpy()
            pred_intents = (outputs.get('intent_output', np.zeros_like(true_intents)).numpy() > 0.5).astype(np.int32)
            all_true_intents.append(true_intents)
            all_pred_intents.append(pred_intents)
            true_domains = targets.get('domain_output', np.zeros((moe_params["batch_size"], 1))).numpy().flatten()
            pred_domains = np.argmax(outputs.get('domain_output', np.zeros((moe_params["batch_size"], self.model.num_domains))).numpy(), axis=-1)
            all_true_domains.append(true_domains)
            all_pred_domains.append(pred_domains)
            if batch_idx % 50 == 0:
                gc.collect()
                tf.keras.backend.clear_session()
        avg_time_per_token = (total_time / total_tokens) * 1000 if total_tokens > 0 else 0
        avg_cpu = np.mean(self.resource_stats['cpu_usage']) if self.resource_stats['cpu_usage'] else 0
        avg_memory = np.mean(self.resource_stats['memory_used']) if self.resource_stats['memory_used'] else 0
        avg_gpu_load = np.mean(self.resource_stats['gpu_load']) if self.resource_stats['gpu_load'] else 0
        avg_gpu_memory = np.mean(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        peak_gpu_memory = np.max(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))
        all_true_intents = np.vstack(all_true_intents)
        all_pred_intents = np.vstack(all_pred_intents)
        all_true_domains = np.concatenate(all_true_domains)
        all_pred_domains = np.concatenate(all_pred_domains)
        intent_f1_score = f1_score(all_true_intents, all_pred_intents, average='micro')
        intent_f1_score_macro = f1_score(all_true_intents, all_pred_intents, average='macro')
        intent_f1_score_weighted = f1_score(all_true_intents, all_pred_intents, average='weighted')
        intent_accuracy_score = accuracy_score(all_true_intents, all_pred_intents)
        domain_accuracy_score = accuracy_score(all_true_domains, all_pred_domains)
        domain_f1_score_macro = f1_score(all_true_domains, all_pred_domains, average='macro')
        hypotheses = []
        references = []
        meteor_scores = []
        rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        domain_metrics = defaultdict(lambda: {
            'bleu': [],
            'rouge1': [],
            'rouge2': [],
            'rougeL': [],
            'meteor': [],
            'count': 0
        })
        if not self.raw_data.get('test'):
            print("Warning: raw_data['test'] is empty. Skipping NLG metrics calculation.")
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0
            domain_metrics = {}
        else:
            for i, batch in enumerate(test_dataset_small):
                if i >= len(self.raw_data.get('test', [])):
                    print(f"Warning: raw_data['test'] has fewer items ({len(self.raw_data.get('test', []))}) than test dataset. Stopping NLG metrics calculation.")
                    break
                try:
                    inputs, targets = batch
                    with tf.device('/CPU:0'):
                        outputs = self.model(inputs, training=False)
                    domain_id = targets['domain_output'].numpy()[0][0]
                    domain_name = f"domain_{domain_id}"
                    response_probs = outputs['response_output'].numpy()
                    generated_tokens = np.argmax(response_probs, axis=-1)[0]
                    true_tokens = targets['response_output'].numpy()[0]
                    raw_item = self.raw_data['test'][i]
                    ref_text = raw_item.get('raw_next_system_response', '')
                    if not ref_text:
                        continue
                    gen_text = self.tokens_to_text(generated_tokens)
                    ref_tokens = self.tokens_to_text(true_tokens)
                    if not gen_text.strip() or not ref_text.strip():
                        continue
                    hypotheses.append(gen_text)
                    references.append([ref_text])
                    ref_tokens = word_tokenize(ref_text.lower())
                    gen_tokens = word_tokenize(gen_text.lower())
                    try:
                        meteor = meteor_score([ref_tokens], gen_tokens)
                        meteor_scores.append(meteor)
                        domain_metrics[domain_name]['meteor'].append(meteor)
                    except Exception as e:
                        print(f"METEOR calculation error for example {i}: {str(e)}")
                        meteor_scores.append(0)
                        domain_metrics[domain_name]['meteor'].append(0)
                    try:
                        rouge_scores = rouge_scorer_instance.score(ref_text, gen_text)
                        domain_metrics[domain_name]['rouge1'].append(rouge_scores['rouge1'].fmeasure)
                        domain_metrics[domain_name]['rouge2'].append(rouge_scores['rouge2'].fmeasure)
                        domain_metrics[domain_name]['rougeL'].append(rouge_scores['rougeL'].fmeasure)
                    except Exception as e:
                        print(f"ROUGE calculation error for example {i}: {str(e)}")
                    domain_metrics[domain_name]['count'] += 1
                    if i % 50 == 0:
                        gc.collect()
                        tf.keras.backend.clear_session()
                except Exception as e:
                    print(f"Error processing example {i}: {str(e)}")
                    continue
        if hypotheses and references:
            hypotheses_tok = []
            references_tok = []
            for hyp, ref_list in zip(hypotheses, references):
                if not hyp or not ref_list[0]:
                    continue
                hyp_tokens = word_tokenize(hyp.lower())
                ref_tokens = [word_tokenize(ref.lower()) for ref in ref_list]
                hypotheses_tok.append(hyp_tokens)
                references_tok.append(ref_tokens)
            chencherry = SmoothingFunction()
            try:
                bleu_score = corpus_bleu(references_tok, hypotheses_tok, smoothing_function=chencherry.method1)
            except Exception as e:
                print(f"BLEU calculation error: {str(e)}")
                bleu_score = 0.0
            rouge_scores = []
            for hyp, ref in zip(hypotheses, references):
                if hyp and ref[0]:
                    try:
                        rouge_scores.append(rouge_scorer_instance.score(ref[0], hyp))
                    except Exception as e:
                        print(f"ROUGE scoring error: {str(e)}")
                        continue
            avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_meteor = np.mean(meteor_scores) if meteor_scores else 0.0
        else:
            print("Warning: No valid hypotheses or references for NLG metrics. Setting to 0.0.")
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0
        return {
            'intent_f1_micro': intent_f1_score,
            'intent_f1_macro': intent_f1_score_macro,
            'intent_f1_weighted': intent_f1_score_weighted,
            'intent_accuracy': intent_accuracy_score,
            'domain_accuracy': domain_accuracy_score,
            'domain_f1_macro': domain_f1_score_macro,
            'bleu': bleu_score,
            'rouge1': avg_rouge1,
            'rouge2': avg_rouge2,
            'rougeL': avg_rougeL,
            'meteor': avg_meteor,
            'perplexity': perplexity_value,
            'pred_speed': avg_time_per_token,
            'domain_metrics': domain_metrics,
            'eval_time': eval_time
        }
    
    def tokens_to_text(self, tokens):
        words = []
        for token in tokens:
            if token == 0:
                continue
            word = self.id_to_word.get(token, '<unk>')
            if word not in self.special_tokens:
                words.append(word)
        return " ".join(words).strip()

results_table = {
    'Evaluation Time (seconds)': [],
    'Prediction Speed (ms/token)': [],
    'Average CPU Usage (percent)': [],
    'Average GPU Usage (percent)': [],
    'Average Memory (GB)': [],
    'Average GPU Memory (GB)': [],
    'Average FLOPs Estimate (GFLOPs)': [],
    'NLU Expert 1 (percent)': [],
    'NLU Expert 2 (percent)': [],
    'NLU Expert 3 (percent)': [],
    'NLU Expert 4 (percent)': [],
    'NLU Expert 5 (percent)': [],
    'NLU Expert 6 (percent)': [],
    'NLU Expert 7 (percent)': [],
    'NLU Expert 8 (percent)': [],
    'NLG Expert 1 (percent)': [],
    'NLG Expert 2 (percent)': [],
    'NLG Expert 3 (percent)': [],
    'NLG Expert 4 (percent)': [],
    'NLG Expert 5 (percent)': [],
    'NLG Expert 6 (percent)': [],
    'NLG Expert 7 (percent)': [],
    'NLG Expert 8 (percent)': [],
    'Intent F1-score (Micro)': [],
    'Intent F1-score (Macro)': [],
    'Intent F1-score (Weighted)': [],
    'Intent Accuracy': [],
    'Domain Accuracy': [],
    'Domain F1-score (Macro)': [],
    'BLEU Score': [],
    'ROUGE-1 F1': [],
    'ROUGE-2 F1': [],
    'ROUGE-L F1': [],
    'METEOR Score': [],
    'Perplexity': [],
    'NLU Expert Load Stability (Variance)': [],
    'NLG Expert Load Stability (Variance)': []
}

num_iterations = 5
for iteration in range(1, num_iterations + 1):
    print(f"\nStarting Iteration {iteration}")
    tf.keras.backend.clear_session()
    gc.collect()
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
    model = compile_model(model, moe_params)
    evaluator = EvaluationMetrics(model, test_tf_dataset, moe_params, raw_data, word_to_id, id_to_word)
    evaluation_results = evaluator.evaluate()
    results_table['Evaluation Time (seconds)'].append(evaluation_results['eval_time'])
    results_table['Prediction Speed (ms/token)'].append(evaluation_results['pred_speed'])
    results_table['Average CPU Usage (percent)'].append(np.mean(evaluator.resource_stats['cpu_usage']) if evaluator.resource_stats['cpu_usage'] else 0)
    results_table['Average GPU Usage (percent)'].append(np.mean(evaluator.resource_stats['gpu_load']) if evaluator.resource_stats['gpu_load'] else 0)
    results_table['Average Memory (GB)'].append(np.mean(evaluator.resource_stats['memory_used']) if evaluator.resource_stats['memory_used'] else 0)
    results_table['Average GPU Memory (GB)'].append(np.mean(evaluator.resource_stats['gpu_memory']) if evaluator.resource_stats['gpu_memory'] else 0)
    results_table['Average FLOPs Estimate (GFLOPs)'].append(calculate_flops(model))
    nlu_counts = evaluator.nlu_expert_usage_counts
    nlg_counts = evaluator.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) if np.sum(nlu_counts) > 0 else 1
    total_nlg = np.sum(nlg_counts) if np.sum(nlg_counts) > 0 else 1
    results_table['NLU Expert 1 (percent)'].append((nlu_counts[0] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 2 (percent)'].append((nlu_counts[1] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 3 (percent)'].append((nlu_counts[2] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 4 (percent)'].append((nlu_counts[3] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 5 (percent)'].append((nlu_counts[4] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 6 (percent)'].append((nlu_counts[5] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 7 (percent)'].append((nlu_counts[6] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 8 (percent)'].append((nlu_counts[7] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLG Expert 1 (percent)'].append((nlg_counts[0] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 2 (percent)'].append((nlg_counts[1] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 3 (percent)'].append((nlg_counts[2] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 4 (percent)'].append((nlg_counts[3] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 5 (percent)'].append((nlg_counts[4] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 6 (percent)'].append((nlg_counts[5] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 7 (percent)'].append((nlg_counts[6] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 8 (percent)'].append((nlg_counts[7] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['Intent F1-score (Micro)'].append(evaluation_results['intent_f1_micro'])
    results_table['Intent F1-score (Macro)'].append(evaluation_results['intent_f1_macro'])
    results_table['Intent F1-score (Weighted)'].append(evaluation_results['intent_f1_weighted'])
    results_table['Intent Accuracy'].append(evaluation_results['intent_accuracy'])
    results_table['Domain Accuracy'].append(evaluation_results['domain_accuracy'])
    results_table['Domain F1-score (Macro)'].append(evaluation_results['domain_f1_macro'])
    results_table['BLEU Score'].append(evaluation_results['bleu'])
    results_table['ROUGE-1 F1'].append(evaluation_results['rouge1'])
    results_table['ROUGE-2 F1'].append(evaluation_results['rouge2'])
    results_table['ROUGE-L F1'].append(evaluation_results['rougeL'])
    results_table['METEOR Score'].append(evaluation_results['meteor'])
    results_table['Perplexity'].append(evaluation_results['perplexity'])
    results_table['NLU Expert Load Stability (Variance)'].append(np.var(nlu_counts / total_nlu * 100) if total_nlu > 0 else 0)
    results_table['NLG Expert Load Stability (Variance)'].append(np.var(nlg_counts / total_nlg * 100) if total_nlg > 0 else 0)

for key in results_table:
    if results_table[key]:
        results_table[key].append(np.mean(results_table[key]))
    else:
        results_table[key].append(0)

print("\nTraining results saved")
final_df = pd.DataFrame(results_table)
final_df = final_df.T
final_df.columns = [f'Iteration {i+1}' for i in range(num_iterations)] + ['Average']
final_df.to_excel('prediction_deepseekefficientdis_results.xlsx', index=False)
final_df

2025-07-04 07:25:14.806654: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 07:25:15.101313: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 07:25:15.101438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 07:25:15.155743: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 07:25:15.263202: I tensorflow/core/platform/cpu_feature_guar


Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 07:25:19.628719: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 07:25:19.768384: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 07:25:19.768674: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl
Loaded vocabulary from preprocessor_models/preprocessor_params.json

Loading model from: best_deepseekefficientdis_model

Model loaded and compiled successfully!
Model: "mo_e_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  784896    
                                                                 
 embedding_1 (Embedding)     multiple                  8576      
                                                                 
 layer_normalization (Layer  multiple                  256       
 Normalization)                                                  
                                                                 
 transformer_decoder_layer   multiple                  264576    
 (TransformerDecoderLayer)          

,Iteration 1,Average
Evaluation Time (seconds),24.118228,24.118228
Prediction Speed (ms/token),3.358257,3.358257
Average CPU Usage (percent),23.522763,23.522763
Average GPU Usage (percent),1.901933,1.901933
Average Memory (GB),5.702466,5.702466
Average GPU Memory (GB),14.504456,14.504456
Average FLOPs Estimate (GFLOPs),2.377871,2.377871
NLU Expert 1 (percent),0.000000,0.000000
NLU Expert 2 (percent),0.000000,0.000000
NLU Expert 3 (percent),1.252684,1.252684


# DeepSeekMoE with Adaptive Gating experiment code

In [1]:
def reset_kernel():
    tf.keras.backend.clear_session()
    import gc
    gc.collect()

results_table = {
    'Training Speed (epochs per second)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Training Time (second/epoch)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Total Training Time (second/iteration)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Computational Resource Usage': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average CPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average GPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average GPU Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average FLOPS Estimate (GFLOPS)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Expert NLU Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)},
    'Expert NLG Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average Validation Entity Accuracy': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average Intent Accuracy': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average Validation Perplexity': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average Validation Response Cosine Similarity': {f'Iteration {i+1}': 0 for i in range(10)}
}
for key in results_table:
    results_table[key]['Average'] = 0

best_val_loss = float('inf')
best_model_path = "best_deepseekmoe_adaptive_gating_model"

for iteration in range(5):
    print(f"\nStarting Training Iteration {iteration + 1}")
    reset_kernel()

    def load_tf_datasets_from_disk(load_path):
        print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
        try:
            with open(os.path.join(load_path, "moe_params.json"), "r") as f:
                loaded_moe_params = json.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"moe_params.json not found in {load_path}")

        element_spec = (
            {
                'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
                'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
                'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            },
            {
                'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
                'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
                'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
            }
        )

        try:
            train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
            test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
        except Exception as e:
            raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

        raw_data = {}
        for dataset_name in ['train', 'test']:
            path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
            if os.path.exists(path):
                with open(path, 'rb') as f:
                    raw_data[dataset_name] = pickle.load(f)
                print(f"Loaded raw data for {dataset_name} from {path}")
            else:
                print(f"Warning: Raw data file {path} not found.")
                raw_data[dataset_name] = []

        return {
            "train_dataset": train_tf_dataset,
            "test_dataset": test_tf_dataset,
            "moe_params": loaded_moe_params,
            "raw_data": raw_data
        }

    tf_dataset_save_path = "tf_datasets"
    loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
    train_tf_dataset = loaded_data["train_dataset"]
    moe_params = loaded_data["moe_params"]
    raw_data = loaded_data["raw_data"]

    class DeepSeekMoELayer(layers.Layer):
        def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.01, tau=0.1, name=None):
            super(DeepSeekMoELayer, self).__init__(name=name)
            self.m = m
            self.k_s = k_s
            self.total_experts = num_experts * self.m
            self.routed_experts = self.total_experts - self.k_s
            self.alpha = alpha
            self.tau = tau
            self.input_dim = input_dim
            self.seq_length = None
            self.adjusted_expert_dim = expert_dim // self.m

            self.shared_experts = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                    layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
                ], name=f'shared_expert_{i}') for i in range(self.k_s)
            ]
            self.routed_experts_list = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                    layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
                ], name=f'routed_expert_{i}') for i in range(self.routed_experts)
            ]
            self.experts = self.shared_experts + self.routed_experts_list
            self.gate = layers.Dense(self.routed_experts, activation=None, name='gate_weights')
            self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

        def call(self, inputs, training=False):
            input_rank = inputs.shape.rank
            if input_rank == 3:
                batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
                flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
                is_3d = True
            else:
                batch_size = tf.shape(inputs)[0]
                seq_length = 1
                flat_inputs = inputs
                is_3d = False

            flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
            num_tokens = tf.shape(flat_inputs)[0]

            shared_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
            for i in range(self.k_s):
                shared_output += self.shared_experts[i](flat_inputs)

            gate_logits = self.gate(flat_inputs)
            gate_weights = tf.nn.softmax(gate_logits, axis=-1)
            sorted_weights = tf.sort(gate_weights, axis=-1, direction='DESCENDING')
            p1 = sorted_weights[:, 0]
            p2 = sorted_weights[:, 1]
            use_top_1 = tf.cast(p1 - p2 >= self.tau, tf.float32)
            k_per_token = tf.where(use_top_1 > 0, 1, 2)

            max_k = 2
            top_k_values, top_k_indices = tf.nn.top_k(gate_weights, k=max_k, sorted=True)
            top_k_indices = top_k_indices + self.k_s
            top_k_weights = top_k_values / (tf.reduce_sum(top_k_values, axis=-1, keepdims=True) + tf.keras.backend.epsilon())

            k_mask = tf.range(max_k, dtype=tf.int32) < tf.expand_dims(k_per_token, -1)
            k_mask = tf.cast(k_mask, tf.float32)
            top_k_weights = top_k_weights * k_mask
            top_k_weights = top_k_weights / (tf.reduce_sum(top_k_weights, axis=-1, keepdims=True) + tf.keras.backend.epsilon())

            expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
            routed_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
            for k in range(max_k):
                kth_weights = top_k_weights[:, k]
                kth_indices = top_k_indices[:, k]
                mask = tf.one_hot(kth_indices, depth=self.total_experts, dtype=tf.float32)
                kth_expert_output = tf.reduce_sum(expert_outputs * tf.expand_dims(mask, -1), axis=1)
                weighted_kth_output = kth_expert_output * tf.expand_dims(kth_weights, -1)
                routed_output += weighted_kth_output

            output_flat = shared_output + routed_output + flat_inputs

            if is_3d:
                output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
                if self.seq_length is not None:
                    output.set_shape([None, self.seq_length, self.input_dim])
                gate_weights_reshaped = tf.reshape(gate_weights, [batch_size, seq_length, self.routed_experts])
                if self.seq_length is not None:
                    gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
            else:
                output = output_flat
                gate_weights_reshaped = gate_weights

            f_i = tf.reduce_mean(tf.reduce_sum(tf.one_hot(tf.argmax(gate_weights, axis=-1), depth=self.routed_experts), axis=0))
            P_i = tf.reduce_mean(gate_weights, axis=0)
            load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
            self.add_loss(load_balancing_loss)
            self.load_balancing_loss_metric.update_state(load_balancing_loss)

            return output, gate_weights_reshaped

        def get_metrics(self):
            return {f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric}

        def get_config(self):
            config = super().get_config()
            config.update({
                'num_experts': self.total_experts // self.m,
                'expert_dim': self.adjusted_expert_dim * self.m,
                'input_dim': self.input_dim,
                'm': self.m,
                'k_s': self.k_s,
                'alpha': self.alpha,
                'tau': self.tau,
                'name': self.name
            })
            return config

    class TransformerDecoderLayer(layers.Layer):
        def __init__(self, d_model, num_heads, dff, rate=0.1):
            super(TransformerDecoderLayer, self).__init__()
            self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.ffn = keras.Sequential([
                layers.Dense(dff, activation='relu'),
                layers.Dense(d_model)
            ])
            self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
            self.dropout1 = layers.Dropout(rate)
            self.dropout2 = layers.Dropout(rate)
            self.dropout3 = layers.Dropout(rate)

        def call(self, x, enc_output, training, look_ahead_mask=None):
            if look_ahead_mask is not None:
                look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
                look_ahead_mask = 1.0 - look_ahead_mask
            attn1_output = self.mha1(query=x, value=x, key=x, attention_mask=look_ahead_mask, training=training)
            attn1 = self.dropout1(attn1_output, training=training)
            out1 = self.layernorm1(attn1 + x)
            attn2_output = self.mha2(query=out1, value=enc_output, key=enc_output, training=training)
            attn2 = self.dropout2(attn2_output, training=training)
            out2 = self.layernorm2(attn2 + out1)
            ffn_output = self.ffn(out2)
            ffn_output = self.dropout3(ffn_output, training=training)
            out3 = self.layernorm3(ffn_output + out2)
            return out3

        def get_config(self):
            config = super().get_config()
            mha1_config = self.mha1.get_config()
            config.update({
                'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
                'num_heads': mha1_config['num_heads'],
                'dff': self.ffn.layers[0].units,
                'rate': self.dropout1.rate
            })
            return config

    class MoEModel(models.Model):
        def __init__(self, moe_params):
            super(MoEModel, self).__init__()
            self.embedding_dim = moe_params["embedding_dim"]
            self.max_seq_length = moe_params["max_seq_length"]
            self.num_domains = moe_params["num_domains"]
            self.num_intents = moe_params["num_intents"]
            self.vocab_size = moe_params["vocab_size"]
            self.num_experts = moe_params["num_experts"]
            self.expert_dim = moe_params["expert_dim"]
            self.turn_id_dim = moe_params["turn_id_dim"]
            self.num_heads = 4
            self.dff = self.embedding_dim * 4
            self.dropout_rate = 0.1
            self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                                 self.num_domains + self.turn_id_dim + self.num_intents)
            self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains
            self.embedding = layers.Embedding(
                self.vocab_size,
                self.embedding_dim,
                mask_zero=True,
                embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05),
                name="embedding"
            )
            self.embedding.build((None,))
            with tf.init_scope():
                embedding_weights = self.embedding.get_weights()
                if embedding_weights and len(embedding_weights) > 0:
                    embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                    self.embedding.set_weights(embedding_weights)
            self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
            self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
            self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
            self.decoder_dropout = layers.Dropout(self.dropout_rate)
            self.moe_nlu = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlu_input_dim, m=2, k_s=2, alpha=0.01, tau=0.1, name='moe_nlu')
            self.moe_nlg = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlg_input_dim, m=2, k_s=2, alpha=0.01, tau=0.1, name='moe_nlg')
            self.intent_output = layers.Dense(self.num_intents, activation='sigmoid', name='intent_output')
            self.domain_output = layers.Dense(self.num_domains, activation='softmax', name='domain_output')
            self.response_output = layers.TimeDistributed(layers.Dense(self.vocab_size, activation='softmax'), name='response_output')
            self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)
            self.nlg_projection = layers.TimeDistributed(layers.Dense(self.embedding_dim, activation='relu'), name='nlg_projection')

        def create_look_ahead_mask(self, size):
            mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
            return mask

        def call(self, inputs, training=False):
            user_utterance_tokens = inputs['user_utterance_tokens']
            prev_system_response_tokens = inputs['prev_system_response_tokens']
            decoder_input_tokens = inputs['decoder_input_tokens']
            domain_onehot_input = inputs['domain_onehot_input']
            turn_id_embedding = inputs['turn_id_embedding']
            ontology_multihot_input = inputs['ontology_multihot_input']
            pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
            user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
            prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
            decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)
            user_enc_out = user_embedded
            prev_system_enc_out = prev_system_embedded
            look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
            dec_output = decoder_embedded
            for i, layer in enumerate(self.decoder_layers):
                dec_output = layer(dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask)
            decoder_output = self.decoder_dropout(dec_output, training=training)
            user_attn_out = self.attn_layer(query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), value=user_enc_out, key=user_enc_out, training=training)
            prev_system_attn_out = self.attn_layer(query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), value=prev_system_enc_out, key=prev_system_enc_out, training=training)
            decoder_attn_out = self.attn_layer(query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), value=decoder_output, key=decoder_output, training=training)
            combined_features = layers.Concatenate()([
                tf.squeeze(user_attn_out, 1),
                tf.squeeze(prev_system_attn_out, 1),
                tf.squeeze(decoder_attn_out, 1),
                domain_onehot_input,
                turn_id_embedding,
                ontology_multihot_input
            ])
            combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
            nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
            nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
            intent_out = self.intent_output(nlu_out)
            domain_out = self.domain_output(nlu_out)
            intent_out_expanded = tf.expand_dims(intent_out, axis=1)
            domain_out_expanded = tf.expand_dims(domain_out, axis=1)
            intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
            domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])
            nlg_input = layers.Concatenate(axis=-1)([
                decoder_output,
                intent_out_tiled,
                domain_out_tiled
            ])
            nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])
            nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
            nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
            response_embeddings = self.nlg_projection(nlg_out)
            response_embeddings = tf.ensure_shape(response_embeddings, [None, self.max_seq_length, self.embedding_dim])
            response_out = self.response_output(nlg_out)
            return {
                'intent_output': intent_out,
                'domain_output': domain_out,
                'response_output': response_out,
                'response_embeddings': response_embeddings,
                'nlu_gate_weights': nlu_gate_weights,
                'nlg_gate_weights': nlg_gate_weights
            }

        def build_graph(self):
            inputs = {
                'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='user_utterance_tokens'),
                'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='prev_system_response_tokens'),
                'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='decoder_input_tokens'),
                'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32, name='domain_onehot_input'),
                'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32, name='turn_id_embedding'),
                'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32, name='ontology_multihot_input'),
            }
            return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

        def get_config(self):
            config = super().get_config()
            config.update({
                'moe_params': {
                    'embedding_dim': self.embedding_dim,
                    'max_seq_length': self.max_seq_length,
                    'num_domains': self.num_domains,
                    'num_intents': self.num_intents,
                    'vocab_size': self.vocab_size,
                    'num_experts': self.num_experts,
                    'expert_dim': self.expert_dim,
                    'turn_id_dim': self.turn_id_dim
                }
            })
            return config

    class IntentAccuracy(metrics.Metric):
        def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros')
            self.total_preds = self.add_weight(name='total_preds', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
            self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
            self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

        def reset_state(self):
            self.correct_preds.assign(0.)
            self.total_preds.assign(0.)

    class IntentPrecision(metrics.Metric):
        def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros')
            self.predicted_positives = self.add_weight(name='pp', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            predicted_positives = tf.reduce_sum(y_pred)
            self.true_positives.assign_add(true_positives)
            self.predicted_positives.assign_add(predicted_positives)

        def result(self):
            return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.predicted_positives.assign(0.)

    class IntentRecall(metrics.Metric):
        def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros')
            self.actual_positives = self.add_weight(name='ap', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            actual_positives = tf.reduce_sum(y_true)
            self.true_positives.assign_add(true_positives)
            self.actual_positives.assign_add(actual_positives)

        def result(self):
            return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.actual_positives.assign(0.)

    class IntentF1(metrics.Metric):
        def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.precision = IntentPrecision(threshold=threshold)
            self.recall = IntentRecall(threshold=threshold)

        def update_state(self, y_true, y_pred, sample_weight=None):
            self.precision.update_state(y_true, y_pred, sample_weight)
            self.recall.update_state(y_true, y_pred, sample_weight)

        def result(self):
            p = self.precision.result()
            r = self.recall.result()
            return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

        def reset_state(self):
            self.precision.reset_state()
            self.recall.reset_state()

    class DomainAccuracy(metrics.Metric):
        def __init__(self, name='domain_accuracy', **kwargs):
            super().__init__(name=name, **kwargs)
            self.accuracy = self.add_weight(name='acc', initializer='zeros')
            self.count = self.add_weight(name='count', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
            pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
            matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
            self.accuracy.assign_add(tf.reduce_sum(matches))
            self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.accuracy / (self.count + tf.keras.backend.epsilon())

        def reset_state(self):
            self.accuracy.assign(0.)
            self.count.assign(0.)

    class Perplexity(metrics.Metric):
        def __init__(self, name='perplexity', **kwargs):
            super().__init__(name=name, **kwargs)
            self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
            self.total_loss = self.add_weight(name='total_loss', initializer='zeros')
            self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            mask = tf.cast(y_true != 0, dtype=tf.float32)
            loss = self.cross_entropy(y_true, y_pred)
            masked_loss = loss * mask
            sum_loss = tf.reduce_sum(masked_loss)
            num_non_padding_tokens = tf.reduce_sum(mask)
            self.total_loss.assign_add(sum_loss)
            self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

        def result(self):
            avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
            return tf.exp(avg_loss)

        def reset_state(self):
            self.total_loss.assign(0.)
            self.total_non_padding_tokens.assign(0.)

    class ResponseEmbeddingCosineSimilarity(metrics.Metric):
        def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
            super().__init__(name=name, **kwargs)
            self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
            self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            epsilon = tf.keras.backend.epsilon()
            y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
            y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
            y_true_norm = y_true / (y_true_norm_val + epsilon)
            y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
            cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
            non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
            non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
            masked_cosine_sim = cosine_sim_per_token * non_padding_mask
            num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
            avg_cosine_sim_per_sample = tf.where(
                num_non_zero_tokens > 0,
                tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
                tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
            )
            self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
            self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

        def reset_state(self):
            self.total_cosine_sim.assign(0.)
            self.num_samples.assign(0.)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    def calculate_flops(model, batch_size=moe_params["batch_size"]):
        flops = 0
        functional_model = model.build_graph()

        def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
            layer_flops_count = 0

            if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
                return 0

            if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
                for sub_layer in layer_instance.layers:
                    layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
                return layer_flops_count

            if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
                return 0

            if isinstance(layer_instance, layers.Dense):
                input_dim = layer_instance.input_shape[-1]
                output_dim = layer_instance.output_shape[-1]
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                layer_flops_count += 2 * input_dim * output_dim * effective_batch_size

            elif isinstance(layer_instance, layers.LSTM):
                input_dim = layer_instance.input_shape[-1]
                hidden_dim = layer_instance.units
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
                layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size

            elif isinstance(layer_instance, layers.Embedding):
                pass

            elif isinstance(layer_instance, DeepSeekMoELayer):
                moe_input_dim = layer_instance.input_dim
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
                layer_flops_count += gate_flops
                for expert in layer_instance.experts:
                    layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)

            elif isinstance(layer_instance, layers.MultiHeadAttention):
                config = layer_instance.get_config()
                num_heads = config['num_heads']
                key_dim = config['key_dim']
                d_model = num_heads * key_dim
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
                projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
                attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
                context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
                output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
                layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops

            elif isinstance(layer_instance, layers.TimeDistributed):
                inner_layer = layer_instance.layer
                td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
                layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)

            elif isinstance(layer_instance, TransformerDecoderLayer):
                layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
                layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
                ffn_effective_batch_size = current_batch_size * model.max_seq_length
                layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)

            return layer_flops_count

        for layer in functional_model.layers:
            flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)

        return flops / 1e9

    def get_gpu_stats():
        if nvidia_smi is None:
            return 0, 0
        try:
            nvidia_smi.nvmlInit()
            handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
            gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
            mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
            gpu_memory = mem_info.used / (1024 ** 3)
            nvidia_smi.nvmlShutdown()
            return gpu_load, gpu_memory
        except Exception:
            return 0, 0

    class ResourceMonitor(keras.callbacks.Callback):
        def __init__(self, validation_data):
            super().__init__()
            self.resource_stats = []
            self.expert_load_history = []
            self.start_time = None
            self.epoch_start_time = None
            self.flops = None
            self.validation_data = validation_data
            self.nlu_expert_usage_counts = None
            self.nlg_expert_usage_counts = None
            self.nlu_gate_weights_history = []
            self.nlg_gate_weights_history = []

        def on_train_begin(self, logs=None):
            self.start_time = time.time()
            try:
                self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
                self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
                self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
            except Exception:
                self.flops = 0

        def on_epoch_begin(self, epoch, logs=None):
            self.epoch_start_time = time.time()

        def on_epoch_end(self, epoch, logs=None):
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            epoch_time = time.time() - self.epoch_start_time
            self.resource_stats.append({
                'epoch': epoch + 1,
                'epoch_time': epoch_time,
                'cpu_usage': cpu_usage,
                'memory_used': memory.used / (1024 ** 3),
                'memory_total': memory.total / (1024 ** 3),
                'gpu_load': gpu_load,
                'gpu_memory': gpu_memory,
                'loss': logs.get('loss', 0),
                'val_loss': logs.get('val_loss', 0),
                'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
                'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
                'intent_precision': logs.get('intent_output_intent_precision', 0),
                'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
                'intent_recall': logs.get('intent_output_intent_recall', 0),
                'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
                'intent_f1': logs.get('intent_output_intent_f1', 0),
                'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
                'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
                'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
                'perplexity': logs.get('response_output_perplexity', 0),
                'val_perplexity': logs.get('val_response_output_perplexity', 0),
                'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
                'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
                'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
                'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
            })
            if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
                self.expert_load_history.append({
                    'epoch': epoch + 1,
                    'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                    'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
                })

            sample_size = 100
            for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
                inputs, _ = batch
                outputs = self.model(inputs, training=False)
                nlu_gate_weights = outputs['nlu_gate_weights']
                reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
                self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
                nlg_gate_weights = outputs['nlg_gate_weights']
                reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
                self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

        def on_train_end(self, logs=None):
            self.total_time = time.time() - self.start_time

            if len(self.nlu_gate_weights_history) > 0:
                all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
                nlu_usage = np.mean(all_nlu_weights, axis=0)
                total_nlu_usage = np.sum(nlu_usage)
                self.nlu_expert_usage_counts = np.array([usage * 100 / total_nlu_usage for usage in nlu_usage]) if total_nlu_usage > 0 else np.zeros(self.model.moe_nlu.routed_experts)

            if len(self.nlg_gate_weights_history) > 0:
                all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
                nlg_usage = np.mean(all_nlg_weights, axis=0)
                total_nlg_usage = np.sum(nlg_usage)
                self.nlg_expert_usage_counts = np.array([usage * 100 / total_nlg_usage for usage in nlg_usage]) if total_nlg_usage > 0 else np.zeros(self.model.moe_nlg.routed_experts)

        def visualize_expert_load_distribution(self):
            nlu_stability = np.var(self.nlu_expert_usage_counts)
            nlg_stability = np.var(self.nlg_expert_usage_counts)
            return {'nlu_percentages': self.nlu_expert_usage_counts[:8], 'nlg_percentages': self.nlg_expert_usage_counts[:8], 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

    class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
        def __init__(self, d_model, warmup_steps=4000):
            super().__init__()
            self.d_model = tf.cast(d_model, tf.float32)
            self.warmup_steps = warmup_steps

        def __call__(self, step):
            step = tf.cast(step, tf.float32)
            arg1 = tf.math.rsqrt(step)
            arg2 = step * (self.warmup_steps ** -1.5)
            return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

        def get_config(self):
            return {"d_model": self.d_model.numpy(), "warmup_steps": self.warmup_steps}

    model = MoEModel(moe_params)
    embedding_layer = model.embedding

    def create_validation_split(dataset, embedding_layer, validation_split=0.2, batch_size=moe_params["batch_size"]):
        element_spec = dataset.element_spec
        data_list = [(features, targets) for features, targets in dataset.unbatch().as_numpy_iterator()]
        if not data_list:
            raise ValueError("Dataset is empty.")
        if len(data_list) < 2:
            raise ValueError("Dataset too small to split.")
        train_data, val_data = train_test_split(data_list, test_size=validation_split, shuffle=True)

        def create_dataset_from_list(data):
            if not data:
                raise ValueError("Empty data list provided.")
            features = {
                'user_utterance_tokens': np.array([d[0]['user_utterance_tokens'] for d in data], dtype=np.int32),
                'prev_system_response_tokens': np.array([d[0]['prev_system_response_tokens'] for d in data], dtype=np.int32),
                'decoder_input_tokens': np.array([d[0]['decoder_input_tokens'] for d in data], dtype=np.int32),
                'domain_onehot_input': np.array([d[0]['domain_onehot_input'] for d in data], dtype=np.float32),
                'turn_id_embedding': np.array([d[0]['turn_id_embedding'] for d in data], dtype=np.float32),
                'ontology_multihot_input': np.array([d[0]['ontology_multihot_input'] for d in data], dtype=np.float32),
            }
            response_output = np.array([d[1]['response_output'] for d in data], dtype=np.int32)
            response_embeddings = embedding_layer(response_output).numpy()
            targets = {
                'domain_output': np.array([d[1]['domain_output'] for d in data], dtype=np.int32),
                'intent_output': np.array([d[1]['intent_output'] for d in data], dtype=np.float32),
                'response_output': response_output,
                'response_embeddings': response_embeddings
            }
            return tf.data.Dataset.from_tensor_slices((features, targets)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

        train_dataset = create_dataset_from_list(train_data)
        val_dataset = create_dataset_from_list(val_data)
        return train_dataset, val_dataset

    train_tf_dataset, val_tf_dataset = create_validation_split(train_tf_dataset, embedding_layer)

    losses_dict = {
        'intent_output': losses.BinaryCrossentropy(),
        'domain_output': losses.SparseCategoricalCrossentropy(),
        'response_output': losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss
    }

    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }

    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={'intent_output': 0.5, 'domain_output': 0.5, 'response_output': 1.0, 'response_embeddings': 1.0}
    )

    sample_input = next(iter(train_tf_dataset.take(1)))
    model(sample_input[0])

    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=8, mode='min', restore_best_weights=True)
    resource_monitor = ResourceMonitor(val_tf_dataset)

    class TQDMProgressBar(callbacks.Callback):
        def on_epoch_begin(self, epoch, logs=None):
            self.progress_bar = tqdm(total=len(train_tf_dataset), desc=f'Epoch {epoch + 1}')

        def on_batch_end(self, batch, logs=None):
            self.progress_bar.update(1)

        def on_epoch_end(self, epoch, logs=None):
            self.progress_bar.close()

    tqdm_callback = TQDMProgressBar()

    history = model.fit(
        train_tf_dataset,
        epochs=100,
        validation_data=val_tf_dataset,
        callbacks=[early_stopping, resource_monitor, tqdm_callback],
        verbose=0
    )

    expert_stats = resource_monitor.visualize_expert_load_distribution()
    stats = resource_monitor.resource_stats
    avg_epoch_duration = np.mean([s['epoch_time'] for s in stats]) if stats else 0
    total_training_time = resource_monitor.total_time
    avg_cpu_usage = np.mean([s['cpu_usage'] for s in stats]) if stats else 0
    avg_memory = np.mean([s['memory_used'] for s in stats]) if stats else 0
    avg_gpu_load = np.mean([s['gpu_load'] for s in stats]) if stats else 0
    avg_gpu_memory = np.mean([s['gpu_memory'] for s in stats]) if stats else 0
    avg_flops = resource_monitor.flops if resource_monitor.flops else 0
    nlu_counts = resource_monitor.nlu_expert_usage_counts
    nlg_counts = resource_monitor.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) or 1
    total_nlg = np.sum(nlg_counts) or 1
    expert_nlu_percentages = resource_monitor.visualize_expert_load_distribution()['nlu_percentages']
    expert_nlg_percentages = resource_monitor.visualize_expert_load_distribution()['nlg_percentages']
    val_intent_accuracy = np.mean([s['val_intent_accuracy'] for s in stats if s['val_intent_accuracy'] > 0]) if stats else 0
    val_entity_accuracy = np.mean([s['val_domain_accuracy'] for s in stats if s['val_domain_accuracy'] > 0]) if stats else 0
    val_perplexity = np.mean([s['val_perplexity'] for s in stats if s['val_perplexity'] > 0]) if stats else 0
    val_cosine_similarity = np.mean([s['val_cosine_similarity'] for s in stats if s['val_cosine_similarity'] > 0]) if stats and any(s['val_cosine_similarity'] > 0 for s in stats) else 0
    
    results_table['Training Speed (epochs per second)'][f'Iteration {iteration + 1}'] = 1 / avg_epoch_duration if avg_epoch_duration > 0 else 0
    results_table['Training Time (second/epoch)'][f'Iteration {iteration + 1}'] = avg_epoch_duration
    results_table['Total Training Time (second/iteration)'][f'Iteration {iteration + 1}'] = total_training_time
    results_table['Computational Resource Usage'][f'Iteration {iteration + 1}'] = avg_cpu_usage + avg_gpu_load
    results_table['Average CPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_cpu_usage
    results_table['Average GPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_gpu_load
    results_table['Average Memory (GB)'][f'Iteration {iteration + 1}'] = avg_memory
    results_table['Average GPU Memory (GB)'][f'Iteration {iteration + 1}'] = avg_gpu_memory
    results_table['Average FLOPS Estimate (GFLOPS)'][f'Iteration {iteration + 1}'] = avg_flops
    results_table['Expert NLU Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlu_stability']
    results_table['Expert NLG Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlg_stability']
    for i in range(8):
        if i < len(expert_stats['nlu_percentages']):
            results_table[f'Average NLU Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlu_percentages'][i]
        if i < len(expert_stats['nlg_percentages']):
            results_table[f'Average NLG Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlg_percentages'][i]
    results_table['Average Validation Entity Accuracy'][f'Iteration {iteration + 1}'] = val_entity_accuracy
    results_table['Average Intent Accuracy'][f'Iteration {iteration + 1}'] = val_intent_accuracy
    results_table['Average Validation Perplexity'][f'Iteration {iteration + 1}'] = val_perplexity
    results_table['Average Validation Response Cosine Similarity'][f'Iteration {iteration + 1}'] = val_cosine_similarity

for key in results_table:
    results_table[key]['Average'] = np.mean([results_table[key][f'Iteration {i+1}'] for i in range(10)])

final_result = pd.DataFrame(results_table)
final_result = final_result.T
final_result.to_excel('training_deepseekmoe_adaptive_gating_model.xlsx', index=True)
final_result

2025-07-04 07:45:18.064145: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 07:45:18.525698: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 07:45:18.525836: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 07:45:18.607107: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 07:45:18.788772: I tensorflow/core/platform/cpu_feature_guar


Starting Training Iteration 1

Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 07:45:24.242955: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 07:45:24.533699: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 07:45:24.533929: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl


Epoch 1:   0%|          | 0/4428 [00:00<?, ?it/s]2025-07-04 07:45:47.801211: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f9930001a50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-04 07:45:47.801252: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2025-07-04 07:45:47.820690: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-04 07:45:47.859124: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1751615147.988813   12261 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
Epoch 1: 100%|██████████| 4428/4428 [02:47<00:00, 26.42it/s]


,Iteration 1,Iteration 2,Iteration 3,Iteration 4,Iteration 5,Iteration 6,Iteration 7,Iteration 8,Iteration 9,Iteration 10,Average
Training Speed (epochs per second),0.006109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000611
Training Time (second/epoch),163.685818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.368582
Total Training Time (second/iteration),169.437418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.943742
Computational Resource Usage,49.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.920000
Average CPU Usage (percent),26.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.620000
Average GPU Usage (percent),23.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.300000
Average Memory (GB),6.552757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.655276
Average GPU Memory (GB),14.531799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.453180
Average FLOPS Estimate (GFLOPS),2.377871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.237787
Expert NLU Load Distribution Stability,16.736093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.673609


# DeepSeekMoE with Adaptive Gating evaluation code

In [1]:
def load_tf_datasets_from_disk(load_path):
    print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
    try:
        with open(os.path.join(load_path, "moe_params.json"), "r") as f:
            loaded_moe_params = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"moe_params.json not found in {load_path}")

    element_spec = (
        {
            'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
            'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
            'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
        },
        {
            'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
            'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
        }
    )

    try:
        train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
        test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
    except Exception as e:
        raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

    raw_data = {}
    for dataset_name in ['train', 'test']:
        path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
        if os.path.exists(path):
            with open(path, 'rb') as f:
                raw_data[dataset_name] = pickle.load(f)
            print(f"Loaded raw data for {dataset_name} from {path}")
        else:
            print(f"Warning: Raw data file {path} not found.")
            raw_data[dataset_name] = []

    return {
        "train_dataset": train_tf_dataset,
        "test_dataset": test_tf_dataset,
        "moe_params": loaded_moe_params,
        "raw_data": raw_data
    }

tf_dataset_save_path = "tf_datasets"
loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
train_tf_dataset = loaded_data["train_dataset"]
test_tf_dataset = loaded_data["test_dataset"]
moe_params = loaded_data["moe_params"]
raw_data = loaded_data["raw_data"]

moe_params["num_experts"] = 4

class DeepSeekMoELayer(layers.Layer):
    def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.01, tau=0.1, name=None):
        super(DeepSeekMoELayer, self).__init__(name=name)
        self.m = m
        self.k_s = k_s
        self.total_experts = num_experts * self.m
        self.routed_experts = self.total_experts - self.k_s
        self.alpha = alpha
        self.tau = tau
        self.input_dim = input_dim
        self.seq_length = None
        self.adjusted_expert_dim = expert_dim // self.m

        self.shared_experts = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
            ], name=f'shared_expert_{i}') for i in range(self.k_s)
        ]
        self.routed_experts_list = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
            ], name=f'routed_expert_{i}') for i in range(self.routed_experts)
        ]
        self.experts = self.shared_experts + self.routed_experts_list
        self.gate = layers.Dense(self.routed_experts, activation=None, name='gate_weights')
        self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

    def call(self, inputs, training=False):
        input_rank = inputs.shape.rank
        if input_rank == 3:
            batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
            flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
            is_3d = True
        else:
            batch_size = tf.shape(inputs)[0]
            seq_length = 1
            flat_inputs = inputs
            is_3d = False

        flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
        num_tokens = tf.shape(flat_inputs)[0]

        shared_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
        for i in range(self.k_s):
            shared_output += self.shared_experts[i](flat_inputs)

        gate_logits = self.gate(flat_inputs)
        gate_weights = tf.nn.softmax(gate_logits, axis=-1)
        sorted_weights = tf.sort(gate_weights, axis=-1, direction='DESCENDING')
        p1 = sorted_weights[:, 0]
        p2 = sorted_weights[:, 1]
        use_top_1 = tf.cast(p1 - p2 >= self.tau, tf.float32)
        k_per_token = tf.where(use_top_1 > 0, 1, 2)

        max_k = 2
        top_k_values, top_k_indices = tf.nn.top_k(gate_weights, k=max_k, sorted=True)
        top_k_indices = top_k_indices + self.k_s
        top_k_weights = top_k_values / (tf.reduce_sum(top_k_values, axis=-1, keepdims=True) + tf.keras.backend.epsilon())

        k_mask = tf.range(max_k, dtype=tf.int32) < tf.expand_dims(k_per_token, -1)
        k_mask = tf.cast(k_mask, tf.float32)
        top_k_weights = top_k_weights * k_mask
        top_k_weights = top_k_weights / (tf.reduce_sum(top_k_weights, axis=-1, keepdims=True) + tf.keras.backend.epsilon())

        expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
        routed_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
        for k in range(max_k):
            kth_weights = top_k_weights[:, k]
            kth_indices = top_k_indices[:, k]
            mask = tf.one_hot(kth_indices, depth=self.total_experts, dtype=tf.float32)
            kth_expert_output = tf.reduce_sum(expert_outputs * tf.expand_dims(mask, -1), axis=1)
            weighted_kth_output = kth_expert_output * tf.expand_dims(kth_weights, -1)
            routed_output += weighted_kth_output

        output_flat = shared_output + routed_output + flat_inputs

        if is_3d:
            output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
            if self.seq_length is not None:
                output.set_shape([None, self.seq_length, self.input_dim])
            gate_weights_reshaped = tf.reshape(gate_weights, [batch_size, seq_length, self.routed_experts])
            if self.seq_length is not None:
                gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
        else:
            output = output_flat
            gate_weights_reshaped = gate_weights

        f_i = tf.reduce_mean(tf.reduce_sum(tf.one_hot(tf.argmax(gate_weights, axis=-1), depth=self.routed_experts), axis=0))
        P_i = tf.reduce_mean(gate_weights, axis=0)
        load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
        self.add_loss(load_balancing_loss)
        self.load_balancing_loss_metric.update_state(load_balancing_loss)

        return output, gate_weights_reshaped

    def get_metrics(self):
        return {f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric}

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_experts': self.total_experts // self.m,
            'expert_dim': self.adjusted_expert_dim * self.m,
            'input_dim': self.input_dim,
            'm': self.m,
            'k_s': self.k_s,
            'alpha': self.alpha,
            'tau': self.tau,
            'name': self.name
        })
        return config

class TransformerDecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.ffn = keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask=None):
        if look_ahead_mask is not None:
            look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
            look_ahead_mask = 1.0 - look_ahead_mask
        attn1_output = self.mha1(query=x, value=x, key=x, attention_mask=look_ahead_mask, training=training)
        attn1 = self.dropout1(attn1_output, training=training)
        out1 = self.layernorm1(attn1 + x)
        attn2_output = self.mha2(query=out1, value=enc_output, key=enc_output, training=training)
        attn2 = self.dropout2(attn2_output, training=training)
        out2 = self.layernorm2(attn2 + out1)
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)
        return out3

    def get_config(self):
        config = super().get_config()
        mha1_config = self.mha1.get_config()
        config.update({
            'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
            'num_heads': mha1_config['num_heads'],
            'dff': self.ffn.layers[0].units,
            'rate': self.dropout1.rate
        })
        return config

class MoEModel(models.Model):
    def __init__(self, moe_params):
        super(MoEModel, self).__init__()
        self.embedding_dim = moe_params["embedding_dim"]
        self.max_seq_length = moe_params["max_seq_length"]
        self.num_domains = moe_params["num_domains"]
        self.num_intents = moe_params["num_intents"]
        self.vocab_size = moe_params["vocab_size"]
        self.num_experts = moe_params["num_experts"]
        self.expert_dim = moe_params["expert_dim"]
        self.turn_id_dim = moe_params["turn_id_dim"]
        self.num_heads = 4
        self.dff = self.embedding_dim * 4
        self.dropout_rate = 0.1
        self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                             self.num_domains + self.turn_id_dim + self.num_intents)
        self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains
        self.embedding = layers.Embedding(
            self.vocab_size,
            self.embedding_dim,
            mask_zero=True,
            embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05),
            name="embedding"
        )
        self.embedding.build((None,))
        with tf.init_scope():
            embedding_weights = self.embedding.get_weights()
            if embedding_weights and len(embedding_weights) > 0:
                embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                self.embedding.set_weights(embedding_weights)
        self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
        self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
        self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
        self.decoder_dropout = layers.Dropout(self.dropout_rate)
        self.moe_nlu = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlu_input_dim, m=2, k_s=2, alpha=0.01, tau=0.1, name='moe_nlu')
        self.moe_nlg = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlg_input_dim, m=2, k_s=2, alpha=0.01, tau=0.1, name='moe_nlg')
        self.intent_output = layers.Dense(self.num_intents, activation='sigmoid', name='intent_output')
        self.domain_output = layers.Dense(self.num_domains, activation='softmax', name='domain_output')
        self.response_output = layers.TimeDistributed(layers.Dense(self.vocab_size, activation='softmax'), name='response_output')
        self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)
        self.nlg_projection = layers.TimeDistributed(layers.Dense(self.embedding_dim, activation='relu'), name='nlg_projection')

    def create_look_ahead_mask(self, size):
        mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
        return mask

    def call(self, inputs, training=False):
        user_utterance_tokens = inputs['user_utterance_tokens']
        prev_system_response_tokens = inputs['prev_system_response_tokens']
        decoder_input_tokens = inputs['decoder_input_tokens']
        domain_onehot_input = inputs['domain_onehot_input']
        turn_id_embedding = inputs['turn_id_embedding']
        ontology_multihot_input = inputs['ontology_multihot_input']
        pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
        user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
        prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
        decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)
        user_enc_out = user_embedded
        prev_system_enc_out = prev_system_embedded
        look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
        dec_output = decoder_embedded
        for i, layer in enumerate(self.decoder_layers):
            dec_output = layer(dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask)
        decoder_output = self.decoder_dropout(dec_output, training=training)
        user_attn_out = self.attn_layer(query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), value=user_enc_out, key=user_enc_out, training=training)
        prev_system_attn_out = self.attn_layer(query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), value=prev_system_enc_out, key=prev_system_enc_out, training=training)
        decoder_attn_out = self.attn_layer(query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), value=decoder_output, key=decoder_output, training=training)
        combined_features = layers.Concatenate()([
            tf.squeeze(user_attn_out, 1),
            tf.squeeze(prev_system_attn_out, 1),
            tf.squeeze(decoder_attn_out, 1),
            domain_onehot_input,
            turn_id_embedding,
            ontology_multihot_input
        ])
        combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
        nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
        nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
        intent_out = self.intent_output(nlu_out)
        domain_out = self.domain_output(nlu_out)
        intent_out_expanded = tf.expand_dims(intent_out, axis=1)
        domain_out_expanded = tf.expand_dims(domain_out, axis=1)
        intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
        domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])
        nlg_input = layers.Concatenate(axis=-1)([
            decoder_output,
            intent_out_tiled,
            domain_out_tiled
        ])
        nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])
        nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
        nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
        response_embeddings = self.nlg_projection(nlg_out)
        response_embeddings = tf.ensure_shape(response_embeddings, [None, self.max_seq_length, self.embedding_dim])
        response_out = self.response_output(nlg_out)
        return {
            'intent_output': intent_out,
            'domain_output': domain_out,
            'response_output': response_out,
            'response_embeddings': response_embeddings,
            'nlu_gate_weights': nlu_gate_weights,
            'nlg_gate_weights': nlg_gate_weights
        }

    def build_graph(self):
        inputs = {
            'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='user_utterance_tokens'),
            'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='prev_system_response_tokens'),
            'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='decoder_input_tokens'),
            'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32, name='domain_onehot_input'),
            'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32, name='turn_id_embedding'),
            'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32, name='ontology_multihot_input'),
        }
        return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

    def get_config(self):
        config = super().get_config()
        config.update({
            'moe_params': {
                'embedding_dim': self.embedding_dim,
                'max_seq_length': self.max_seq_length,
                'num_domains': self.num_domains,
                'num_intents': self.num_intents,
                'vocab_size': self.vocab_size,
                'num_experts': self.num_experts,
                'expert_dim': self.expert_dim,
                'turn_id_dim': self.turn_id_dim
            }
        })
        return config

class IntentAccuracy(metrics.Metric):
    def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros')
        self.total_preds = self.add_weight(name='total_preds', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
        self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
        self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

    def reset_state(self):
        self.correct_preds.assign(0.)
        self.total_preds.assign(0.)

class IntentPrecision(metrics.Metric):
    def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.predicted_positives = self.add_weight(name='pp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        predicted_positives = tf.reduce_sum(y_pred)
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)

    def result(self):
        return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.predicted_positives.assign(0.)

class IntentRecall(metrics.Metric):
    def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.actual_positives = self.add_weight(name='ap', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        actual_positives = tf.reduce_sum(y_true)
        self.true_positives.assign_add(true_positives)
        self.actual_positives.assign_add(actual_positives)

    def result(self):
        return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.actual_positives.assign(0.)

class IntentF1(metrics.Metric):
    def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = IntentPrecision(threshold=threshold)
        self.recall = IntentRecall(threshold=threshold)

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

class DomainAccuracy(metrics.Metric):
    def __init__(self, name='domain_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.accuracy = self.add_weight(name='acc', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
        pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
        matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
        self.accuracy.assign_add(tf.reduce_sum(matches))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.accuracy / (self.count + tf.keras.backend.epsilon())

    def reset_state(self):
        self.accuracy.assign(0.)
        self.count.assign(0.)

class Perplexity(metrics.Metric):
    def __init__(self, name='perplexity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
        self.total_loss = self.add_weight(name='total_loss', initializer='zeros')
        self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        mask = tf.cast(y_true != 0, dtype=tf.float32)
        loss = self.cross_entropy(y_true, y_pred)
        masked_loss = loss * mask
        sum_loss = tf.reduce_sum(masked_loss)
        num_non_padding_tokens = tf.reduce_sum(mask)
        self.total_loss.assign_add(sum_loss)
        self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

    def result(self):
        avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
        return tf.exp(avg_loss)

    def reset_state(self):
        self.total_loss.assign(0.)
        self.total_non_padding_tokens.assign(0.)

class ResponseEmbeddingCosineSimilarity(metrics.Metric):
    def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
        self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true_norm = y_true / (y_true_norm_val + epsilon)
        y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
        cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
        non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
        non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
        masked_cosine_sim = cosine_sim_per_token * non_padding_mask
        num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
        avg_cosine_sim_per_sample = tf.where(
            num_non_zero_tokens > 0,
            tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
            tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
        )
        self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
        self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

    def reset_state(self):
        self.total_cosine_sim.assign(0.)
        self.num_samples.assign(0.)

def contrastive_loss(y_true, y_pred, margin=1.0):
    epsilon = tf.keras.backend.epsilon()
    y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
    y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
    y_true = y_true / (y_true_norm + epsilon)
    y_pred = y_pred / (y_pred_norm + epsilon)
    cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
    positive_loss = 1.0 - cosine_sim
    mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
    masked_loss = positive_loss * mask
    total_loss = tf.reduce_sum(masked_loss)
    num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
    return total_loss / num_tokens

def calculate_flops(model, batch_size=moe_params["batch_size"]):
    flops = 0
    functional_model = model.build_graph()

    def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
        layer_flops_count = 0

        if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
            return 0

        if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
            for sub_layer in layer_instance.layers:
                layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
            return layer_flops_count

        if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
            return 0

        if isinstance(layer_instance, layers.Dense):
            input_dim = layer_instance.input_shape[-1]
            output_dim = layer_instance.output_shape[-1]
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            layer_flops_count += 2 * input_dim * output_dim * effective_batch_size

        elif isinstance(layer_instance, layers.LSTM):
            input_dim = layer_instance.input_shape[-1]
            hidden_dim = layer_instance.units
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
            layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size

        elif isinstance(layer_instance, layers.Embedding):
            pass

        elif isinstance(layer_instance, DeepSeekMoELayer):
            moe_input_dim = layer_instance.input_dim
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
            layer_flops_count += gate_flops
            for expert in layer_instance.experts:
                layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)

        elif isinstance(layer_instance, layers.MultiHeadAttention):
            config = layer_instance.get_config()
            num_heads = config['num_heads']
            key_dim = config['key_dim']
            d_model = num_heads * key_dim
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
            projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
            attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
            context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
            output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
            layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops

        elif isinstance(layer_instance, layers.TimeDistributed):
            inner_layer = layer_instance.layer
            td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
            layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)

        elif isinstance(layer_instance, TransformerDecoderLayer):
            layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
            layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
            ffn_effective_batch_size = current_batch_size * model.max_seq_length
            layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)

        return layer_flops_count

    for layer in functional_model.layers:
        flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)

    return flops / 1e9

def get_gpu_stats():
    if nvidia_smi is None:
        return 0, 0
    try:
        nvidia_smi.nvmlInit()
        handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
        gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
        mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
        gpu_memory = mem_info.used / (1024 ** 3)
        nvidia_smi.nvmlShutdown()
        return gpu_load, gpu_memory
    except Exception:
        return 0, 0

class ResourceMonitor(keras.callbacks.Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.resource_stats = []
        self.expert_load_history = []
        self.start_time = None
        self.epoch_start_time = None
        self.flops = None
        self.validation_data = validation_data
        self.nlu_expert_usage_counts = None
        self.nlg_expert_usage_counts = None
        self.nlu_gate_weights_history = []
        self.nlg_gate_weights_history = []

    def on_train_begin(self, logs=None):
        self.start_time = time.time()
        try:
            self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
            self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
            self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
        except Exception:
            self.flops = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory()
        gpu_load, gpu_memory = get_gpu_stats()
        epoch_time = time.time() - self.epoch_start_time
        self.resource_stats.append({
            'epoch': epoch + 1,
            'epoch_time': epoch_time,
            'cpu_usage': cpu_usage,
            'memory_used': memory.used / (1024 ** 3),
            'memory_total': memory.total / (1024 ** 3),
            'gpu_load': gpu_load,
            'gpu_memory': gpu_memory,
            'loss': logs.get('loss', 0),
            'val_loss': logs.get('val_loss', 0),
            'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
            'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
            'intent_precision': logs.get('intent_output_intent_precision', 0),
            'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
            'intent_recall': logs.get('intent_output_intent_recall', 0),
            'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
            'intent_f1': logs.get('intent_output_intent_f1', 0),
            'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
            'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
            'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
            'perplexity': logs.get('response_output_perplexity', 0),
            'val_perplexity': logs.get('val_response_output_perplexity', 0),
            'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
            'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
            'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
            'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
        })
        if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
            self.expert_load_history.append({
                'epoch': epoch + 1,
                'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
            })

        sample_size = 100
        for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
            inputs, _ = batch
            outputs = self.model(inputs, training=False)
            nlu_gate_weights = outputs['nlu_gate_weights']
            reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
            self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
            nlg_gate_weights = outputs['nlg_gate_weights']
            reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
            self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

    def on_train_end(self, logs=None):
        self.total_time = time.time() - self.start_time

        if len(self.nlu_gate_weights_history) > 0:
            all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
            nlu_usage = np.mean(all_nlu_weights, axis=0)
            total_nlu_usage = np.sum(nlu_usage)
            self.nlu_expert_usage_counts = np.array([usage * 100 / total_nlu_usage for usage in nlu_usage]) if total_nlu_usage > 0 else np.zeros(self.model.moe_nlu.routed_experts)

        if len(self.nlg_gate_weights_history) > 0:
            all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
            nlg_usage = np.mean(all_nlg_weights, axis=0)
            total_nlg_usage = np.sum(nlg_usage)
            self.nlg_expert_usage_counts = np.array([usage * 100 / total_nlg_usage for usage in nlg_usage]) if total_nlg_usage > 0 else np.zeros(self.model.moe_nlg.routed_experts)

    def visualize_expert_load_distribution(self):
        nlu_stability = np.var(self.nlu_expert_usage_counts)
        nlg_stability = np.var(self.nlg_expert_usage_counts)
        return {'nlu_percentages': self.nlu_expert_usage_counts[:8], 'nlg_percentages': self.nlg_expert_usage_counts[:8], 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {"d_model": self.d_model.numpy(), "warmup_steps": self.warmup_steps}

word_to_id = {}
id_to_word = {}
try:
    possible_paths = [
        os.path.join("preprocessor_models", "preprocessor_params.json"),
        os.path.join("tf_datasets", "preprocessor_params.json"),
        "preprocessor_params.json"
    ]
    
    vocab_loaded = False
    for path in possible_paths:
        if os.path.exists(path):
            with open(path, "r") as f:
                preprocessor_params = json.load(f)
            word_to_id = {k: int(v) for k, v in preprocessor_params['word_to_id'].items()}
            id_to_word = {int(k): v for k, v in preprocessor_params['id_to_word'].items()}
            print(f"Loaded vocabulary from {path}")
            vocab_loaded = True
            break
    
    if not vocab_loaded:
        raise FileNotFoundError("Vocabulary file not found in any location")
except Exception as e:
    print(f"Warning: Could not load vocabulary from preprocessor: {str(e)}")
    word_to_id = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
    id_to_word = {0: '<pad>', 1: '<sos>', 2: '<eos>', 3: '<unk>'}

model_save_path = "best_deepseekmoe_adaptive_gating_model"
print(f"\nLoading model from: {model_save_path}")
custom_objects = {
    "MoEModel":MoEModel,
    "CustomLearningRateSchedule": CustomLearningRateSchedule,
    "DeepSeekMoELayer": DeepSeekMoELayer,
    "TransformerDecoderLayer": TransformerDecoderLayer,
    "IntentAccuracy": IntentAccuracy,
    "IntentPrecision": IntentPrecision,
    "IntentRecall": IntentRecall,
    "IntentF1": IntentF1,
    "DomainAccuracy": DomainAccuracy,
    "Perplexity": Perplexity,
    "ResponseEmbeddingCosineSimilarity": ResponseEmbeddingCosineSimilarity
}
try:
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
except Exception as e:
    print(f"Failed to load model from {model_save_path}: {str(e)}")
    raise

def compile_model(model, moe_params):
    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    losses_dict = {
        'intent_output': tf.keras.losses.BinaryCrossentropy(),
        'domain_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss,
    }

    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }

    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={
            'intent_output': 0.5,
            'domain_output': 0.5,
            'response_output': 1.0,
            'response_embeddings': 1.0
        }
    )
    return model

model = compile_model(model, moe_params)
print("\nModel loaded and compiled successfully!")
sample_input = next(iter(train_tf_dataset.take(1)))
model(sample_input[0])
model.summary()

class EvaluationMetrics:
    def __init__(self, model, test_dataset, moe_params, raw_data, word_to_id, id_to_word):
        self.model = model
        self.test_dataset = test_dataset
        self.moe_params = moe_params
        self.raw_data = raw_data
        self.word_to_id = word_to_id
        self.id_to_word = id_to_word
        self.nlu_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.nlg_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.resource_stats = {
            'cpu_usage': [],
            'memory_used': [],
            'gpu_load': [],
            'gpu_memory': [],
            'batch_times': []
        }
        self.special_tokens = {'<pad>', '<sos>', '<eos>', '<unk>'}
        self.nlu_top_k = model.get_layer('moe_nlu').get_config().get('top_k', 2)
        self.nlg_top_k = model.get_layer('moe_nlg').get_config().get('top_k', 2)

    def evaluate(self):
        print("\nEvaluating model on test set...")
        
        def add_response_embeddings(features, targets):
            response_tokens = targets['response_output']
            response_embeddings = self.model.embedding(response_tokens)
            targets['response_embeddings'] = response_embeddings
            return features, targets
        
        test_dataset_with_embeddings = self.test_dataset.map(add_response_embeddings, num_parallel_calls=tf.data.AUTOTUNE)

        start_time = time.time()
        test_results = self.model.evaluate(test_dataset_with_embeddings, verbose=1)
        eval_time = time.time() - start_time
        
        metric_names = self.model.metrics_names
        
        perplexity_value = None
        for name, value in zip(metric_names, test_results):
            if name == 'response_output_perplexity':
                perplexity_value = value
                break
        if perplexity_value is None:
            print("Warning: Could not find perplexity in standard evaluation results.")
            perplexity_value = 0.0
        
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory().used / (1024 ** 3)
        gpu_load, gpu_memory = get_gpu_stats()
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))

        total_tokens = 0
        total_time = 0
        num_batches = 0
        
        test_dataset_small = test_dataset_with_embeddings.unbatch().batch(moe_params["batch_size"])
        
        all_true_intents = []
        all_pred_intents = []
        all_true_domains = []
        all_pred_domains = []
        
        for batch_idx, batch in enumerate(test_dataset_small):
            inputs, targets = batch
            response_tokens = targets['response_output'].numpy()
            non_padding_mask = response_tokens != 0
            total_tokens += np.sum(non_padding_mask)
            
            start_time = time.time()
            outputs = self.model(inputs, training=False)
            batch_time = time.time() - start_time
            
            total_time += batch_time
            num_batches += tf.shape(inputs['user_utterance_tokens'])[0]
            
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            
            self.resource_stats['cpu_usage'].append(cpu_usage)
            self.resource_stats['memory_used'].append(memory.used / (1024 ** 3))
            self.resource_stats['gpu_load'].append(gpu_load)
            self.resource_stats['gpu_memory'].append(gpu_memory)
            self.resource_stats['batch_times'].append(batch_time)
            
            nlu_gate_weights = outputs.get('nlu_gate_weights', tf.zeros([0, self.model.moe_nlu.routed_experts])).numpy()
            nlu_top_indices = np.argsort(-nlu_gate_weights, axis=-1)[:, :self.nlu_top_k] + self.model.moe_nlu.k_s
            for idx in range(self.nlu_top_k):
                expert_ids = nlu_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlu_expert_usage_counts):
                        self.nlu_expert_usage_counts[expert_id] += 1
            
            nlg_gate_weights = outputs.get('nlg_gate_weights', tf.zeros([0, self.model.moe_nlg.routed_experts])).numpy()
            if len(nlg_gate_weights.shape) == 3:
                nlg_gate_weights = nlg_gate_weights.reshape(-1, nlg_gate_weights.shape[-1])
            nlg_top_indices = np.argsort(-nlg_gate_weights, axis=-1)[:, :self.nlg_top_k] + self.model.moe_nlg.k_s
            for idx in range(self.nlg_top_k):
                expert_ids = nlg_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlg_expert_usage_counts):
                        self.nlg_expert_usage_counts[expert_id] += 1
            
            true_intents = targets.get('intent_output', np.zeros((moe_params["batch_size"], self.model.num_intents))).numpy()
            pred_intents = (outputs.get('intent_output', np.zeros_like(true_intents)).numpy() > 0.5).astype(np.int32)
            all_true_intents.append(true_intents)
            all_pred_intents.append(pred_intents)
            
            true_domains = targets.get('domain_output', np.zeros((moe_params["batch_size"], 1))).numpy().flatten()
            pred_domains = np.argmax(outputs.get('domain_output', np.zeros((moe_params["batch_size"], self.model.num_domains))).numpy(), axis=-1)
            all_true_domains.append(true_domains)
            all_pred_domains.append(pred_domains)
            
            if batch_idx % 50 == 0:
                gc.collect()
                tf.keras.backend.clear_session()
        
        avg_time_per_token = (total_time / total_tokens) * 1000 if total_tokens > 0 else 0
        
        avg_cpu = np.mean(self.resource_stats['cpu_usage']) if self.resource_stats['cpu_usage'] else 0
        avg_memory = np.mean(self.resource_stats['memory_used']) if self.resource_stats['memory_used'] else 0
        avg_gpu_load = np.mean(self.resource_stats['gpu_load']) if self.resource_stats['gpu_load'] else 0
        avg_gpu_memory = np.mean(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        peak_gpu_memory = np.max(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))
        
        all_true_intents = np.vstack(all_true_intents)
        all_pred_intents = np.vstack(all_pred_intents)
        all_true_domains = np.concatenate(all_true_domains)
        all_pred_domains = np.concatenate(all_pred_domains)
        
        intent_f1_score = f1_score(all_true_intents, all_pred_intents, average='micro')
        intent_f1_score_macro = f1_score(all_true_intents, all_pred_intents, average='macro')
        intent_f1_score_weighted = f1_score(all_true_intents, all_pred_intents, average='weighted')
        intent_accuracy_score = accuracy_score(all_true_intents, all_pred_intents)
        
        domain_accuracy_score = accuracy_score(all_true_domains, all_pred_domains)
        domain_f1_score_macro = f1_score(all_true_domains, all_pred_domains, average='macro')

        hypotheses = []
        references = []
        meteor_scores = []
        rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        
        domain_metrics = defaultdict(lambda: {
            'bleu': [],
            'rouge1': [],
            'rouge2': [],
            'rougeL': [],
            'meteor': [],
            'count': 0
        })
        
        if not self.raw_data.get('test'):
            print("Warning: raw_data['test'] is empty. Skipping NLG metrics calculation.")
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0
            domain_metrics = {}
        else:
            for i, batch in enumerate(test_dataset_small):
                if i >= len(self.raw_data.get('test', [])):
                    print(f"Warning: raw_data['test'] has fewer items ({len(self.raw_data.get('test', []))}) than test dataset. Stopping NLG metrics calculation.")
                    break
                
                try:
                    inputs, targets = batch
                    with tf.device('/CPU:0'):
                        outputs = self.model(inputs, training=False)
                    
                    domain_id = targets['domain_output'].numpy()[0][0]
                    domain_name = f"domain_{domain_id}"
                    
                    response_probs = outputs['response_output'].numpy()
                    generated_tokens = np.argmax(response_probs, axis=-1)[0]
                    true_tokens = targets['response_output'].numpy()[0]
                    
                    raw_item = self.raw_data['test'][i]
                    ref_text = raw_item.get('raw_next_system_response', '')
                    if not ref_text:
                        continue
                    
                    gen_text = self.tokens_to_text(generated_tokens)
                    ref_tokens = self.tokens_to_text(true_tokens)
                    
                    if not gen_text.strip() or not ref_text.strip():
                        continue
                    
                    hypotheses.append(gen_text)
                    references.append([ref_text])
                    
                    ref_tokens = word_tokenize(ref_text.lower())
                    gen_tokens = word_tokenize(gen_text.lower())
                    
                    try:
                        meteor = meteor_score([ref_tokens], gen_tokens)
                        meteor_scores.append(meteor)
                        domain_metrics[domain_name]['meteor'].append(meteor)
                    except Exception as e:
                        print(f"METEOR calculation error for example {i}: {str(e)}")
                        meteor_scores.append(0)
                        domain_metrics[domain_name]['meteor'].append(0)
                    
                    try:
                        rouge_scores = rouge_scorer_instance.score(ref_text, gen_text)
                        domain_metrics[domain_name]['rouge1'].append(rouge_scores['rouge1'].fmeasure)
                        domain_metrics[domain_name]['rouge2'].append(rouge_scores['rouge2'].fmeasure)
                        domain_metrics[domain_name]['rougeL'].append(rouge_scores['rougeL'].fmeasure)
                    except Exception as e:
                        print(f"ROUGE calculation error for example {i}: {str(e)}")
                    
                    domain_metrics[domain_name]['count'] += 1
                    
                    if i % 50 == 0:
                        gc.collect()
                        tf.keras.backend.clear_session()
                    
                except Exception as e:
                    print(f"Error processing example {i}: {str(e)}")
                    continue
        
        if hypotheses and references:
            hypotheses_tok = []
            references_tok = []
            
            for hyp, ref_list in zip(hypotheses, references):
                if not hyp or not ref_list[0]:
                    continue
                hyp_tokens = word_tokenize(hyp.lower())
                ref_tokens = [word_tokenize(ref.lower()) for ref in ref_list]
                hypotheses_tok.append(hyp_tokens)
                references_tok.append(ref_tokens)
            
            chencherry = SmoothingFunction()
            try:
                bleu_score = corpus_bleu(references_tok, hypotheses_tok, smoothing_function=chencherry.method1)
            except Exception as e:
                print(f"BLEU calculation error: {str(e)}")
                bleu_score = 0.0
            
            rouge_scores = []
            for hyp, ref in zip(hypotheses, references):
                if hyp and ref[0]:
                    try:
                        rouge_scores.append(rouge_scorer_instance.score(ref[0], hyp))
                    except Exception as e:
                        print(f"ROUGE scoring error: {str(e)}")
                        continue
            
            avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_meteor = np.mean(meteor_scores) if meteor_scores else 0.0
        else:
            print("Warning: No valid hypotheses or references for NLG metrics. Setting to 0.0.")
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0

        return {
            'intent_f1_micro': intent_f1_score,
            'intent_f1_macro': intent_f1_score_macro,
            'intent_f1_weighted': intent_f1_score_weighted,
            'intent_accuracy': intent_accuracy_score,
            'domain_accuracy': domain_accuracy_score,
            'domain_f1_macro': domain_f1_score_macro,
            'bleu': bleu_score,
            'rouge1': avg_rouge1,
            'rouge2': avg_rouge2,
            'rougeL': avg_rougeL,
            'meteor': avg_meteor,
            'perplexity': perplexity_value,
            'pred_speed': avg_time_per_token,
            'domain_metrics': domain_metrics,
            'eval_time': eval_time
        }
    
    def tokens_to_text(self, tokens):
        words = []
        for token in tokens:
            if token == 0:
                continue
            word = self.id_to_word.get(token, '<unk>')
            if word not in self.special_tokens:
                words.append(word)
        return " ".join(words).strip()

results_table = {
    'Evaluation Time (seconds)': [],
    'Prediction Speed (ms/token)': [],
    'Average CPU Usage (percent)': [],
    'Average GPU Usage (percent)': [],
    'Average Memory (GB)': [],
    'Average GPU Memory (GB)': [],
    'Average FLOPs Estimate (GFLOPs)': [],
    'NLU Expert 1 (percent)': [],
    'NLU Expert 2 (percent)': [],
    'NLU Expert 3 (percent)': [],
    'NLU Expert 4 (percent)': [],
    'NLU Expert 5 (percent)': [],
    'NLU Expert 6 (percent)': [],
    'NLU Expert 7 (percent)': [],
    'NLU Expert 8 (percent)': [],
    'NLG Expert 1 (percent)': [],
    'NLG Expert 2 (percent)': [],
    'NLG Expert 3 (percent)': [],
    'NLG Expert 4 (percent)': [],
    'NLG Expert 5 (percent)': [],
    'NLG Expert 6 (percent)': [],
    'NLG Expert 7 (percent)': [],
    'NLG Expert 8 (percent)': [],
    'Intent F1-score (Micro)': [],
    'Intent F1-score (Macro)': [],
    'Intent F1-score (Weighted)': [],
    'Intent Accuracy': [],
    'Domain Accuracy': [],
    'Domain F1-score (Macro)': [],
    'BLEU Score': [],
    'ROUGE-1 F1': [],
    'ROUGE-2 F1': [],
    'ROUGE-L F1': [],
    'METEOR Score': [],
    'Perplexity': [],
    'NLU Expert Load Stability (Variance)': [],
    'NLG Expert Load Stability (Variance)': []
}

num_iterations = 5
for iteration in range(1, num_iterations + 1):
    print(f"\nStarting Iteration {iteration}")
    tf.keras.backend.clear_session()
    gc.collect()
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
    model = compile_model(model, moe_params)
    evaluator = EvaluationMetrics(model, test_tf_dataset, moe_params, raw_data, word_to_id, id_to_word)
    evaluation_results = evaluator.evaluate()
    results_table['Evaluation Time (seconds)'].append(evaluation_results['eval_time'])
    results_table['Prediction Speed (ms/token)'].append(evaluation_results['pred_speed'])
    results_table['Average CPU Usage (percent)'].append(np.mean(evaluator.resource_stats['cpu_usage']) if evaluator.resource_stats['cpu_usage'] else 0)
    results_table['Average GPU Usage (percent)'].append(np.mean(evaluator.resource_stats['gpu_load']) if evaluator.resource_stats['gpu_load'] else 0)
    results_table['Average Memory (GB)'].append(np.mean(evaluator.resource_stats['memory_used']) if evaluator.resource_stats['memory_used'] else 0)
    results_table['Average GPU Memory (GB)'].append(np.mean(evaluator.resource_stats['gpu_memory']) if evaluator.resource_stats['gpu_memory'] else 0)
    results_table['Average FLOPs Estimate (GFLOPs)'].append(calculate_flops(model))
    nlu_counts = evaluator.nlu_expert_usage_counts
    nlg_counts = evaluator.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) if np.sum(nlu_counts) > 0 else 1
    total_nlg = np.sum(nlg_counts) if np.sum(nlg_counts) > 0 else 1
    results_table['NLU Expert 1 (percent)'].append((nlu_counts[0] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 2 (percent)'].append((nlu_counts[1] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 3 (percent)'].append((nlu_counts[2] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 4 (percent)'].append((nlu_counts[3] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 5 (percent)'].append((nlu_counts[4] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 6 (percent)'].append((nlu_counts[5] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 7 (percent)'].append((nlu_counts[6] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 8 (percent)'].append((nlu_counts[7] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLG Expert 1 (percent)'].append((nlg_counts[0] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 2 (percent)'].append((nlg_counts[1] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 3 (percent)'].append((nlg_counts[2] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 4 (percent)'].append((nlg_counts[3] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 5 (percent)'].append((nlg_counts[4] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 6 (percent)'].append((nlg_counts[5] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 7 (percent)'].append((nlg_counts[6] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 8 (percent)'].append((nlg_counts[7] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['Intent F1-score (Micro)'].append(evaluation_results['intent_f1_micro'])
    results_table['Intent F1-score (Macro)'].append(evaluation_results['intent_f1_macro'])
    results_table['Intent F1-score (Weighted)'].append(evaluation_results['intent_f1_weighted'])
    results_table['Intent Accuracy'].append(evaluation_results['intent_accuracy'])
    results_table['Domain Accuracy'].append(evaluation_results['domain_accuracy'])
    results_table['Domain F1-score (Macro)'].append(evaluation_results['domain_f1_macro'])
    results_table['BLEU Score'].append(evaluation_results['bleu'])
    results_table['ROUGE-1 F1'].append(evaluation_results['rouge1'])
    results_table['ROUGE-2 F1'].append(evaluation_results['rouge2'])
    results_table['ROUGE-L F1'].append(evaluation_results['rougeL'])
    results_table['METEOR Score'].append(evaluation_results['meteor'])
    results_table['Perplexity'].append(evaluation_results['perplexity'])
    results_table['NLU Expert Load Stability (Variance)'].append(np.var(nlu_counts / total_nlu * 100) if total_nlu > 0 else 0)
    results_table['NLG Expert Load Stability (Variance)'].append(np.var(nlg_counts / total_nlg * 100) if total_nlg > 0 else 0)

for key in results_table:
    if results_table[key]:
        results_table[key].append(np.mean(results_table[key]))
    else:
        results_table[key].append(0)

print("\nTraining results saved")
final_df = pd.DataFrame(results_table)
final_df = final_df.T
final_df.columns = [f'Iteration {i+1}' for i in range(num_iterations)] + ['Average']
final_df.to_excel('prediction_deepseekefficientdis_results.xlsx', index=False)
final_df

2025-07-04 07:59:04.188956: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 07:59:04.663638: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 07:59:04.663738: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 07:59:04.747540: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 07:59:04.914094: I tensorflow/core/platform/cpu_feature_guar


Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 07:59:10.452379: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 07:59:10.784964: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 07:59:10.785152: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl
Loaded vocabulary from preprocessor_models/preprocessor_params.json

Loading model from: best_deepseekmoe_adaptive_gating_model

Model loaded and compiled successfully!
Model: "mo_e_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  784896    
                                                                 
 embedding (Embedding)       multiple                  8576      
                                                                 
 layer_normalization (Layer  multiple                  256       
 Normalization)                                                  
                                                                 
 transformer_decoder_layer   multiple                  264576    
 (TransformerDecoderLayer)   

,Iteration 1,Average
Evaluation Time (seconds),19.700496,19.700496
Prediction Speed (ms/token),2.034601,2.034601
Average CPU Usage (percent),21.487688,21.487688
Average GPU Usage (percent),1.944882,1.944882
Average Memory (GB),5.635143,5.635143
Average GPU Memory (GB),14.504456,14.504456
Average FLOPs Estimate (GFLOPs),2.377871,2.377871
NLU Expert 1 (percent),0.000000,0.000000
NLU Expert 2 (percent),0.000000,0.000000
NLU Expert 3 (percent),25.143164,25.143164


# DeepSeekMoE with Expert Choice Routing experiment code

In [1]:
def reset_kernel():
    tf.keras.backend.clear_session()
    import gc
    gc.collect()

results_table = {
    'Training Speed (epochs per second)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Training Time (second/epoch)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Total Training Time (second/iteration)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Computational Resource Usage': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average CPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average GPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average GPU Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average FLOPS Estimate (GFLOPS)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Expert NLU Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Expert NLG Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Entity Accuracy': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Intent Accuracy': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Perplexity': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Response Cosine Similarity': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0}
}

best_val_loss = float('inf')
best_model_path = "best_deepseekmoe_expert_choice_model"

for iteration in range(5):
    print(f"\nStarting Training Iteration {iteration + 1}")
    reset_kernel()

    def load_tf_datasets_from_disk(load_path):
        print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
        try:
            with open(os.path.join(load_path, "moe_params.json"), "r") as f:
                loaded_moe_params = json.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"moe_params.json not found in {load_path}")

        element_spec = (
            {
                'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
                'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
                'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            },
            {
                'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
                'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
                'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
            }
        )

        try:
            train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
            test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
        except Exception as e:
            raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

        raw_data = {}
        for dataset_name in ['train', 'test']:
            path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
            if os.path.exists(path):
                with open(path, 'rb') as f:
                    raw_data[dataset_name] = pickle.load(f)
                print(f"Loaded raw data for {dataset_name} from {path}")
            else:
                print(f"Warning: Raw data file {path} not found.")
                raw_data[dataset_name] = []

        return {
            "train_dataset": train_tf_dataset,
            "test_dataset": test_tf_dataset,
            "moe_params": loaded_moe_params,
            "raw_data": raw_data
        }

    tf_dataset_save_path = "tf_datasets"
    loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
    train_tf_dataset = loaded_data["train_dataset"]
    moe_params = loaded_data["moe_params"]
    raw_data = loaded_data["raw_data"]

    class DeepSeekMoELayer(layers.Layer):
        def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.01, capacity_factor=1.0, name=None):
            super(DeepSeekMoELayer, self).__init__(name=name)
            self.m = m
            self.k_s = k_s
            self.total_experts = num_experts * self.m
            self.routed_experts = self.total_experts - self.k_s
            self.alpha = alpha
            self.capacity_factor = capacity_factor
            self.input_dim = input_dim
            self.seq_length = None
            self.adjusted_expert_dim = expert_dim // self.m
            self.max_k = 2

            self.shared_experts = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                    layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
                ], name=f'shared_expert_{i}') for i in range(self.k_s)
            ]
            self.routed_experts_list = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                    layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
                ], name=f'routed_expert_{i}') for i in range(self.routed_experts)
            ]
            self.experts = self.shared_experts + self.routed_experts_list
            self.gate = layers.Dense(self.routed_experts, activation=None, name='gate_weights')
            self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

        def call(self, inputs, training=False):
            input_rank = inputs.shape.rank
            if input_rank == 3:
                batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
                flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
                is_3d = True
            else:
                batch_size = tf.shape(inputs)[0]
                seq_length = 1
                flat_inputs = inputs
                is_3d = False

            num_tokens = tf.shape(flat_inputs)[0]
            flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])

            shared_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
            for i in range(self.k_s):
                shared_output += self.shared_experts[i](flat_inputs)

            gate_logits = self.gate(flat_inputs)
            gate_weights = tf.nn.softmax(gate_logits, axis=-1)

            k = self.max_k
            gate_weights_transposed = tf.transpose(gate_weights)
            top_k_values, top_k_indices = tf.nn.top_k(gate_weights_transposed, k=k, sorted=True)
            gating_matrix = top_k_values / (tf.reduce_sum(top_k_values, axis=-1, keepdims=True) + tf.keras.backend.epsilon())
            index_matrix = top_k_indices + self.k_s

            one_hot_matrix = tf.one_hot(index_matrix, depth=tf.cast(num_tokens, tf.int32), dtype=tf.float32)
            inputs_per_expert = tf.matmul(one_hot_matrix, flat_inputs)

            expert_outputs = []
            for i, expert in enumerate(self.routed_experts_list):
                expert_output = expert(inputs_per_expert[i])
                expert_outputs.append(expert_output)
            expert_outputs = tf.stack(expert_outputs, axis=0)

            routed_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
            for i in range(self.routed_experts):
                for j in range(k):
                    token_idx = index_matrix[i, j]
                    weight = gating_matrix[i, j]
                    expert_output = expert_outputs[i, j]
                    update = tf.expand_dims(expert_output * weight, 0)
                    scatter_indices = tf.expand_dims([token_idx], 1)
                    routed_output += tf.scatter_nd(scatter_indices, update, [num_tokens, self.input_dim])

            output_flat = shared_output + routed_output + flat_inputs

            if is_3d:
                output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
                if self.seq_length is not None:
                    output.set_shape([None, self.seq_length, self.input_dim])
                gate_weights_reshaped = tf.reshape(gate_weights, [batch_size, seq_length, self.routed_experts])
                if self.seq_length is not None:
                    gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
            else:
                output = output_flat
                gate_weights_reshaped = gate_weights

            f_i = tf.reduce_mean(tf.reduce_sum(tf.one_hot(tf.argmax(gate_weights, axis=-1), depth=self.routed_experts), axis=0))
            P_i = tf.reduce_mean(gate_weights, axis=0)
            load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
            self.add_loss(load_balancing_loss)
            self.load_balancing_loss_metric.update_state(load_balancing_loss)

            return output, gate_weights_reshaped

        def get_metrics(self):
            return {f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric}

        def get_config(self):
            config = super().get_config()
            config.update({
                'num_experts': self.total_experts // self.m,
                'expert_dim': self.adjusted_expert_dim * self.m,
                'input_dim': self.input_dim,
                'm': self.m,
                'k_s': self.k_s,
                'alpha': self.alpha,
                'capacity_factor': self.capacity_factor,
                'name': self.name
            })
            return config

    class TransformerDecoderLayer(layers.Layer):
        def __init__(self, d_model, num_heads, dff, rate=0.1):
            super(TransformerDecoderLayer, self).__init__()
            self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.ffn = keras.Sequential([
                layers.Dense(dff, activation='relu'),
                layers.Dense(d_model)
            ])
            self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
            self.dropout1 = layers.Dropout(rate)
            self.dropout2 = layers.Dropout(rate)
            self.dropout3 = layers.Dropout(rate)

        def call(self, x, enc_output, training, look_ahead_mask=None):
            if look_ahead_mask is not None:
                look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
                look_ahead_mask = 1.0 - look_ahead_mask

            attn1_output = self.mha1(query=x, value=x, key=x, attention_mask=look_ahead_mask, training=training)
            attn1 = self.dropout1(attn1_output, training=training)
            out1 = self.layernorm1(attn1 + x)

            attn2_output = self.mha2(query=out1, value=enc_output, key=enc_output, training=training)
            attn2 = self.dropout2(attn2_output, training=training)
            out2 = self.layernorm2(attn2 + out1)

            ffn_output = self.ffn(out2)
            ffn_output = self.dropout3(ffn_output, training=training)
            out3 = self.layernorm3(ffn_output + out2)

            return out3

        def get_config(self):
            config = super().get_config()
            mha1_config = self.mha1.get_config()
            config.update({
                'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
                'num_heads': mha1_config['num_heads'],
                'dff': self.ffn.layers[0].units,
                'rate': self.dropout1.rate
            })
            return config

    class MoEModel(models.Model):
        def __init__(self, moe_params):
            super(MoEModel, self).__init__()
            self.embedding_dim = moe_params["embedding_dim"]
            self.max_seq_length = moe_params["max_seq_length"]
            self.num_domains = moe_params["num_domains"]
            self.num_intents = moe_params["num_intents"]
            self.vocab_size = moe_params["vocab_size"]
            self.num_experts = moe_params["num_experts"]
            self.expert_dim = moe_params["expert_dim"]
            self.turn_id_dim = moe_params["turn_id_dim"]
            self.num_heads = 4
            self.dff = self.embedding_dim * 4
            self.dropout_rate = 0.1
            self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                                 self.num_domains + self.turn_id_dim + self.num_intents)
            self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains

            self.embedding = layers.Embedding(
                self.vocab_size,
                self.embedding_dim,
                mask_zero=True,
                embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05),
                name="embedding"
            )
            self.embedding.build((None,))
            with tf.init_scope():
                embedding_weights = self.embedding.get_weights()
                if embedding_weights and len(embedding_weights) > 0:
                    embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                    self.embedding.set_weights(embedding_weights)

            self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
            self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
            self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
            self.decoder_dropout = layers.Dropout(self.dropout_rate)
            self.moe_nlu = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlu_input_dim, m=2, k_s=2, alpha=0.01, capacity_factor=1.0, name='moe_nlu')
            self.moe_nlg = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlg_input_dim, m=2, k_s=2, alpha=0.01, capacity_factor=1.0, name='moe_nlg')
            self.intent_output = layers.Dense(self.num_intents, activation='sigmoid', name='intent_output')
            self.domain_output = layers.Dense(self.num_domains, activation='softmax', name='domain_output')
            self.response_output = layers.TimeDistributed(layers.Dense(self.vocab_size, activation='softmax'), name='response_output')
            self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)
            self.nlg_projection = layers.TimeDistributed(layers.Dense(self.embedding_dim, activation='relu'), name='nlg_projection')

        def create_look_ahead_mask(self, size):
            mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
            return mask

        def call(self, inputs, training=False):
            user_utterance_tokens = inputs['user_utterance_tokens']
            prev_system_response_tokens = inputs['prev_system_response_tokens']
            decoder_input_tokens = inputs['decoder_input_tokens']
            domain_onehot_input = inputs['domain_onehot_input']
            turn_id_embedding = inputs['turn_id_embedding']
            ontology_multihot_input = inputs['ontology_multihot_input']
            pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
            user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
            prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
            decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)
            user_enc_out = user_embedded
            prev_system_enc_out = prev_system_embedded
            look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
            dec_output = decoder_embedded
            for i, layer in enumerate(self.decoder_layers):
                dec_output = layer(dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask)
            decoder_output = self.decoder_dropout(dec_output, training=training)
            user_attn_out = self.attn_layer(query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), value=user_enc_out, key=user_enc_out, training=training)
            prev_system_attn_out = self.attn_layer(query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), value=prev_system_enc_out, key=prev_system_enc_out, training=training)
            decoder_attn_out = self.attn_layer(query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), value=decoder_output, key=decoder_output, training=training)
            combined_features = layers.Concatenate()([
                tf.squeeze(user_attn_out, 1),
                tf.squeeze(prev_system_attn_out, 1),
                tf.squeeze(decoder_attn_out, 1),
                domain_onehot_input,
                turn_id_embedding,
                ontology_multihot_input
            ])
            combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
            nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
            nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
            intent_out = self.intent_output(nlu_out)
            domain_out = self.domain_output(nlu_out)
            intent_out_expanded = tf.expand_dims(intent_out, axis=1)
            domain_out_expanded = tf.expand_dims(domain_out, axis=1)
            intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
            domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])
            nlg_input = layers.Concatenate(axis=-1)([
                decoder_output,
                intent_out_tiled,
                domain_out_tiled
            ])
            nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])
            nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
            nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
            response_embeddings = self.nlg_projection(nlg_out)
            response_out = self.response_output(nlg_out)
            return {
                'intent_output': intent_out,
                'domain_output': domain_out,
                'response_output': response_out,
                'response_embeddings': response_embeddings,
                'nlu_gate_weights': nlu_gate_weights,
                'nlg_gate_weights': nlg_gate_weights
            }

        def build_graph(self):
            inputs = {
                'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='user_utterance_tokens'),
                'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='prev_system_response_tokens'),
                'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='decoder_input_tokens'),
                'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32, name='domain_onehot_input'),
                'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32, name='turn_id_embedding'),
                'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32, name='ontology_multihot_input'),
            }
            return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

        def get_config(self):
            config = super().get_config()
            config.update({
                'moe_params': {
                    'embedding_dim': self.embedding_dim,
                    'max_seq_length': self.max_seq_length,
                    'num_domains': self.num_domains,
                    'num_intents': self.num_intents,
                    'vocab_size': self.vocab_size,
                    'num_experts': self.num_experts,
                    'expert_dim': self.expert_dim,
                    'turn_id_dim': self.turn_id_dim
                }
            })
            return config
    
    class IntentAccuracy(metrics.Metric):
        def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros')
            self.total_preds = self.add_weight(name='total_preds', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
            self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
            self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

        def reset_state(self):
            self.correct_preds.assign(0.)
            self.total_preds.assign(0.)

    class IntentPrecision(metrics.Metric):
        def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros')
            self.predicted_positives = self.add_weight(name='pp', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            predicted_positives = tf.reduce_sum(y_pred)
            self.true_positives.assign_add(true_positives)
            self.predicted_positives.assign_add(predicted_positives)

        def result(self):
            return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.predicted_positives.assign(0.)

    class IntentRecall(metrics.Metric):
        def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros')
            self.actual_positives = self.add_weight(name='ap', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            actual_positives = tf.reduce_sum(y_true)
            self.true_positives.assign_add(true_positives)
            self.actual_positives.assign_add(actual_positives)

        def result(self):
            return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.actual_positives.assign(0.)

    class IntentF1(metrics.Metric):
        def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.precision = IntentPrecision(threshold=threshold)
            self.recall = IntentRecall(threshold=threshold)

        def update_state(self, y_true, y_pred, sample_weight=None):
            self.precision.update_state(y_true, y_pred, sample_weight)
            self.recall.update_state(y_true, y_pred, sample_weight)

        def result(self):
            p = self.precision.result()
            r = self.recall.result()
            return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

        def reset_state(self):
            self.precision.reset_state()
            self.recall.reset_state()

    class DomainAccuracy(metrics.Metric):
        def __init__(self, name='domain_accuracy', **kwargs):
            super().__init__(name=name, **kwargs)
            self.accuracy = self.add_weight(name='acc', initializer='zeros')
            self.count = self.add_weight(name='count', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
            pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
            matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
            self.accuracy.assign_add(tf.reduce_sum(matches))
            self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.accuracy / (self.count + tf.keras.backend.epsilon())

        def reset_state(self):
            self.accuracy.assign(0.)
            self.count.assign(0.)

    class Perplexity(metrics.Metric):
        def __init__(self, name='perplexity', **kwargs):
            super().__init__(name=name, **kwargs)
            self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
            self.total_loss = self.add_weight(name='total_loss', initializer='zeros')
            self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            mask = tf.cast(y_true != 0, dtype=tf.float32)
            loss = self.cross_entropy(y_true, y_pred)
            masked_loss = loss * mask
            sum_loss = tf.reduce_sum(masked_loss)
            num_non_padding_tokens = tf.reduce_sum(mask)
            self.total_loss.assign_add(sum_loss)
            self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

        def result(self):
            avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
            return tf.exp(avg_loss)

        def reset_state(self):
            self.total_loss.assign(0.)
            self.total_non_padding_tokens.assign(0.)

    class ResponseEmbeddingCosineSimilarity(metrics.Metric):
        def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
            super().__init__(name=name, **kwargs)
            self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
            self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            epsilon = tf.keras.backend.epsilon()
            y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
            y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
            y_true_norm = y_true / (y_true_norm_val + epsilon)
            y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
            cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
            non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
            non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
            masked_cosine_sim = cosine_sim_per_token * non_padding_mask
            num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
            avg_cosine_sim_per_sample = tf.where(
                num_non_zero_tokens > 0,
                tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
                tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
            )
            self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
            self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

        def reset_state(self):
            self.total_cosine_sim.assign(0.)
            self.num_samples.assign(0.)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    def calculate_flops(model, batch_size=moe_params["batch_size"]):
        flops = 0
        functional_model = model.build_graph()

        def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
            layer_flops_count = 0

            if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
                return 0

            if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
                for sub_layer in layer_instance.layers:
                    layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
                return layer_flops_count

            if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
                return 0

            if isinstance(layer_instance, layers.Dense):
                input_dim = layer_instance.input_shape[-1]
                output_dim = layer_instance.output_shape[-1]
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                layer_flops_count += 2 * input_dim * output_dim * effective_batch_size

            elif isinstance(layer_instance, layers.LSTM):
                input_dim = layer_instance.input_shape[-1]
                hidden_dim = layer_instance.units
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
                layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size

            elif isinstance(layer_instance, layers.Embedding):
                pass

            elif isinstance(layer_instance, DeepSeekMoELayer):
                moe_input_dim = layer_instance.input_dim
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
                layer_flops_count += gate_flops
                for expert in layer_instance.experts:
                    layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)

            elif isinstance(layer_instance, layers.MultiHeadAttention):
                config = layer_instance.get_config()
                num_heads = config['num_heads']
                key_dim = config['key_dim']
                d_model = num_heads * key_dim
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
                projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
                attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
                context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
                output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
                layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops

            elif isinstance(layer_instance, layers.TimeDistributed):
                inner_layer = layer_instance.layer
                td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
                layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)

            elif isinstance(layer_instance, TransformerDecoderLayer):
                layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
                layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
                ffn_effective_batch_size = current_batch_size * model.max_seq_length
                layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)

            return layer_flops_count

        for layer in functional_model.layers:
            flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)

        return flops / 1e9

    def get_gpu_stats():
        if nvidia_smi is None:
            return 0, 0
        try:
            nvidia_smi.nvmlInit()
            handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
            gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
            mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
            gpu_memory = mem_info.used / (1024 ** 3)
            nvidia_smi.nvmlShutdown()
            return gpu_load, gpu_memory
        except Exception:
            return 0, 0

    class ResourceMonitor(keras.callbacks.Callback):
        def __init__(self, validation_data):
            super().__init__()
            self.resource_stats = []
            self.expert_load_history = []
            self.start_time = None
            self.epoch_start_time = None
            self.flops = None
            self.validation_data = validation_data
            self.nlu_expert_usage_counts = None
            self.nlg_expert_usage_counts = None
            self.nlu_gate_weights_history = []
            self.nlg_gate_weights_history = []

        def on_train_begin(self, logs=None):
            self.start_time = time.time()
            try:
                self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
                self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
                self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
            except Exception:
                self.flops = 0

        def on_epoch_begin(self, epoch, logs=None):
            self.epoch_start_time = time.time()

        def on_epoch_end(self, epoch, logs=None):
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            epoch_time = time.time() - self.epoch_start_time
            self.resource_stats.append({
                'epoch': epoch + 1,
                'epoch_time': epoch_time,
                'cpu_usage': cpu_usage,
                'memory_used': memory.used / (1024 ** 3),
                'memory_total': memory.total / (1024 ** 3),
                'gpu_load': gpu_load,
                'gpu_memory': gpu_memory,
                'loss': logs.get('loss', 0),
                'val_loss': logs.get('val_loss', 0),
                'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
                'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
                'intent_precision': logs.get('intent_output_intent_precision', 0),
                'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
                'intent_recall': logs.get('intent_output_intent_recall', 0),
                'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
                'intent_f1': logs.get('intent_output_intent_f1', 0),
                'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
                'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
                'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
                'perplexity': logs.get('response_output_perplexity', 0),
                'val_perplexity': logs.get('val_response_output_perplexity', 0),
                'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
                'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
                'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
                'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
            })
            if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
                self.expert_load_history.append({
                    'epoch': epoch + 1,
                    'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                    'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
                })

            sample_size = 100
            for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
                inputs, _ = batch
                outputs = self.model(inputs, training=False)
                nlu_gate_weights = outputs['nlu_gate_weights']
                reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
                self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
                nlg_gate_weights = outputs['nlg_gate_weights']
                reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
                self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

        def on_train_end(self, logs=None):
            self.total_time = time.time() - self.start_time

            if len(self.nlu_gate_weights_history) > 0:
                all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
                nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
                for expert_id in nlu_top_indices:
                    if expert_id < self.model.moe_nlu.total_experts:
                        self.nlu_expert_usage_counts[expert_id] += 1

            if len(self.nlg_gate_weights_history) > 0:
                all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
                nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
                for expert_id in nlg_top_indices:
                    if expert_id < self.model.moe_nlg.total_experts:
                        self.nlg_expert_usage_counts[expert_id] += 1

        def visualize_expert_load_distribution(self):
            total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
            total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
            nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
            nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
            nlu_stability = np.var(nlu_percentages)
            nlg_stability = np.var(nlg_percentages)
            return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

    class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
        def __init__(self, d_model, warmup_steps=4000):
            super().__init__()
            self.d_model = tf.cast(d_model, tf.float32)
            self.warmup_steps = warmup_steps

        def __call__(self, step):
            step = tf.cast(step, tf.float32)
            arg1 = tf.math.rsqrt(step)
            arg2 = step * (self.warmup_steps ** -1.5)
            return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

        def get_config(self):
            return {"d_model": self.d_model.numpy(), "warmup_steps": self.warmup_steps}

    model = MoEModel(moe_params)
    embedding_layer = model.embedding

    def create_validation_split(dataset, embedding_layer, validation_split=0.2, batch_size=moe_params["batch_size"]):
        element_spec = dataset.element_spec
        data_list = [(features, targets) for features, targets in dataset.unbatch().as_numpy_iterator()]
        if not data_list:
            raise ValueError("Dataset is empty.")
        if len(data_list) < 2:
            raise ValueError("Dataset too small to split.")
        train_data, val_data = train_test_split(data_list, test_size=validation_split, shuffle=True)

        def create_dataset_from_list(data):
            if not data:
                raise ValueError("Empty data list provided.")
            features = {
                'user_utterance_tokens': np.array([d[0]['user_utterance_tokens'] for d in data], dtype=np.int32),
                'prev_system_response_tokens': np.array([d[0]['prev_system_response_tokens'] for d in data], dtype=np.int32),
                'decoder_input_tokens': np.array([d[0]['decoder_input_tokens'] for d in data], dtype=np.int32),
                'domain_onehot_input': np.array([d[0]['domain_onehot_input'] for d in data], dtype=np.float32),
                'turn_id_embedding': np.array([d[0]['turn_id_embedding'] for d in data], dtype=np.float32),
                'ontology_multihot_input': np.array([d[0]['ontology_multihot_input'] for d in data], dtype=np.float32),
            }
            response_output = np.array([d[1]['response_output'] for d in data], dtype=np.int32)
            response_embeddings = embedding_layer(response_output).numpy()
            targets = {
                'domain_output': np.array([d[1]['domain_output'] for d in data], dtype=np.int32),
                'intent_output': np.array([d[1]['intent_output'] for d in data], dtype=np.float32),
                'response_output': response_output,
                'response_embeddings': response_embeddings
            }
            return tf.data.Dataset.from_tensor_slices((features, targets)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

        train_dataset = create_dataset_from_list(train_data)
        val_dataset = create_dataset_from_list(val_data)
        return train_dataset, val_dataset

    train_tf_dataset, val_tf_dataset = create_validation_split(train_tf_dataset, embedding_layer)

    losses_dict = {
        'intent_output': losses.BinaryCrossentropy(),
        'domain_output': losses.SparseCategoricalCrossentropy(),
        'response_output': losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss
    }

    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }

    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={'intent_output': 0.5, 'domain_output': 0.5, 'response_output': 1.0, 'response_embeddings': 1.0}
    )

    sample_input = next(iter(train_tf_dataset.take(1)))
    model(sample_input[0])

    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=8, mode='min', restore_best_weights=True)
    resource_monitor = ResourceMonitor(val_tf_dataset)
    class TQDMProgressBar(callbacks.Callback):
        def on_epoch_begin(self, epoch, logs=None):
            self.progress_bar = tqdm(total=len(train_tf_dataset), desc=f'Epoch {epoch + 1}')

        def on_batch_end(self, batch, logs=None):
            self.progress_bar.update(1)

        def on_epoch_end(self, epoch, logs=None):
            self.progress_bar.close()

    tqdm_callback = TQDMProgressBar()

    history = model.fit(
        train_tf_dataset,
        epochs=100,
        validation_data=val_tf_dataset,
        callbacks=[early_stopping, resource_monitor, tqdm_callback],
        verbose=0
    )
    
    val_batch = next(iter(val_tf_dataset.take(1)))
    model(val_batch[0], training=False)

    expert_stats = resource_monitor.visualize_expert_load_distribution()
    stats = resource_monitor.resource_stats
    avg_epoch_duration = np.mean([s['epoch_time'] for s in stats]) if stats else 0
    total_training_time = resource_monitor.total_time
    avg_cpu_usage = np.mean([s['cpu_usage'] for s in stats]) if stats else 0
    avg_memory = np.mean([s['memory_used'] for s in stats]) if stats else 0
    avg_gpu_load = np.mean([s['gpu_load'] for s in stats]) if stats else 0
    avg_gpu_memory = np.mean([s['gpu_memory'] for s in stats]) if stats else 0
    avg_flops = resource_monitor.flops if resource_monitor.flops else 0
    nlu_counts = resource_monitor.nlu_expert_usage_counts
    nlg_counts = resource_monitor.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) or 1
    total_nlg = np.sum(nlg_counts) or 1
    expert_nlu_percentages = [(nlu_counts[i] / total_nlu) * 100 for i in range(8)]
    expert_nlg_percentages = [(nlg_counts[i] / total_nlg) * 100 for i in range(8)]
    val_intent_accuracy = np.mean([s['val_intent_accuracy'] for s in stats if s['val_intent_accuracy'] > 0]) if stats else 0
    val_entity_accuracy = np.mean([s['val_domain_accuracy'] for s in stats if s['val_domain_accuracy'] > 0]) if stats else 0
    val_perplexity = np.mean([s['val_perplexity'] for s in stats if s['val_perplexity'] > 0]) if stats else 0
    val_cosine_similarity = np.mean([s['val_cosine_similarity'] for s in stats if s['val_cosine_similarity'] > 0]) if stats and any(s['val_cosine_similarity'] > 0 for s in stats) else 0
    
    results_table['Average Validation Response Cosine Similarity'][f'Iteration {iteration + 1}'] = val_cosine_similarity
    results_table['Training Speed (epochs per second)'][f'Iteration {iteration + 1}'] = 1 / avg_epoch_duration if avg_epoch_duration > 0 else 0
    results_table['Training Time (second/epoch)'][f'Iteration {iteration + 1}'] = avg_epoch_duration
    results_table['Total Training Time (second/iteration)'][f'Iteration {iteration + 1}'] = total_training_time
    results_table['Computational Resource Usage'][f'Iteration {iteration + 1}'] = avg_cpu_usage + avg_gpu_load
    results_table['Average CPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_cpu_usage
    results_table['Average GPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_gpu_load
    results_table['Average Memory (GB)'][f'Iteration {iteration + 1}'] = avg_memory
    results_table['Average GPU Memory (GB)'][f'Iteration {iteration + 1}'] = avg_gpu_memory
    results_table['Average FLOPS Estimate (GFLOPS)'][f'Iteration {iteration + 1}'] = avg_flops
    results_table['Expert NLU Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlu_stability']
    results_table['Expert NLG Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlg_stability']
    for i in range(8):
        if i < len(expert_stats['nlu_percentages']):
            results_table[f'Average NLU Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlu_percentages'][i]
        if i < len(expert_stats['nlg_percentages']):
            results_table[f'Average NLG Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlg_percentages'][i]
    results_table['Average Validation Entity Accuracy'][f'Iteration {iteration + 1}'] = val_entity_accuracy
    results_table['Average Intent Accuracy'][f'Iteration {iteration + 1}'] = val_intent_accuracy
    results_table['Average Validation Perplexity'][f'Iteration {iteration + 1}'] = val_perplexity
    results_table['Average Validation Response Cosine Similarity'][f'Iteration {iteration + 1}'] = val_cosine_similarity
    val_loss = min([s['val_loss'] for s in stats]) if stats else float('inf')
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        try:
            model.save(best_model_path, save_format='tf', include_optimizer=True)
        except Exception as e:
            print(f"\nFailed to save best model to {best_model_path}: {str(e)}")

for key in results_table:
    results_table[key]['Average'] = np.mean([results_table[key][f'Iteration {i+1}'] for i in range(10)])

final_result = pd.DataFrame(results_table)
final_result = final_result.T
final_result.to_excel('training_deepseekmoe_expert_choice_result.xlsx', index=True)
final_result

2025-07-04 08:05:57.798234: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 08:05:58.283565: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 08:05:58.283791: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 08:05:58.371756: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 08:05:58.540853: I tensorflow/core/platform/cpu_feature_guar


Starting Training Iteration 1

Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 08:06:03.401302: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 08:06:03.721094: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 08:06:03.721281: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl


Epoch 1:   0%|          | 0/4428 [00:00<?, ?it/s]2025-07-04 08:06:28.617508: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f719405e6e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-04 08:06:28.617548: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2025-07-04 08:06:28.642013: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-04 08:06:28.691690: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1751616388.788481   15049 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
Epoch 1: 100%|██████████| 4428/4428 [02:54<00:00, 25.38it/s]


INFO:tensorflow:Assets written to: best_deepseekmoe_expert_choice_model/assets


INFO:tensorflow:Assets written to: best_deepseekmoe_expert_choice_model/assets


,Iteration 1,Iteration 2,Iteration 3,Iteration 4,Iteration 5,Iteration 6,Iteration 7,Iteration 8,Iteration 9,Iteration 10,Average
Training Speed (epochs per second),0.005900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000590
Training Time (second/epoch),169.477412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.947741
Total Training Time (second/iteration),175.517203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.551720
Computational Resource Usage,54.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.440000
Average CPU Usage (percent),24.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.440000
Average GPU Usage (percent),30.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.000000
Average Memory (GB),6.612045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.661205
Average GPU Memory (GB),14.531799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.453180
Average FLOPS Estimate (GFLOPS),2.377871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.237787
Expert NLU Load Distribution Stability,176.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.600000


# DeepSeekMoE with Expert Choice Routing evaluation code

In [1]:
def load_tf_datasets_from_disk(load_path):
    print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
    try:
        with open(os.path.join(load_path, "moe_params.json"), "r") as f:
            loaded_moe_params = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"moe_params.json not found in {load_path}")

    element_spec = (
        {
            'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
            'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
            'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
        },
        {
            'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
            'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
        }
    )

    try:
        train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
        test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
    except Exception as e:
        raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

    raw_data = {}
    for dataset_name in ['train', 'test']:
        path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
        if os.path.exists(path):
            with open(path, 'rb') as f:
                raw_data[dataset_name] = pickle.load(f)
            print(f"Loaded raw data for {dataset_name} from {path}")
        else:
            print(f"Warning: Raw data file {path} not found.")
            raw_data[dataset_name] = []

    return {
        "train_dataset": train_tf_dataset,
        "test_dataset": test_tf_dataset,
        "moe_params": loaded_moe_params,
        "raw_data": raw_data
    }

tf_dataset_save_path = "tf_datasets"
loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
train_tf_dataset = loaded_data["train_dataset"]
test_tf_dataset = loaded_data["test_dataset"]
moe_params = loaded_data["moe_params"]
raw_data = loaded_data["raw_data"]

moe_params["num_experts"] = 4

class DeepSeekMoELayer(layers.Layer):
    def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.01, capacity_factor=1.0, name=None):
        super(DeepSeekMoELayer, self).__init__(name=name)
        self.m = m
        self.k_s = k_s
        self.total_experts = num_experts * self.m
        self.routed_experts = self.total_experts - self.k_s
        self.alpha = alpha
        self.capacity_factor = capacity_factor
        self.input_dim = input_dim
        self.adjusted_expert_dim = expert_dim // self.m
        self.max_k = min(2, self.routed_experts)

        self.shared_experts = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
            ], name=f'shared_expert_{i}') for i in range(self.k_s)
        ]
        self.routed_experts_list = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
            ], name=f'routed_expert_{i}') for i in range(self.routed_experts)
        ]
        self.experts = self.shared_experts + self.routed_experts_list
        self.gate = layers.Dense(self.routed_experts, activation=None, name='gate_weights')
        self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

    def call(self, inputs, training=False):
        input_rank = inputs.shape.rank
        if input_rank == 3:
            batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
            flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
            is_3d = True
        else:
            batch_size = tf.shape(inputs)[0]
            seq_length = 1
            flat_inputs = inputs
            is_3d = False

        num_tokens = tf.shape(flat_inputs)[0]
        flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])

        shared_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
        for i in range(self.k_s):
            shared_output += self.shared_experts[i](flat_inputs)

        gate_logits = self.gate(flat_inputs)
        gate_weights = tf.nn.softmax(gate_logits, axis=-1)

        k = min(self.max_k, self.routed_experts)
        top_k_values, top_k_indices = tf.nn.top_k(gate_weights, k=k, sorted=True)
        top_k_weights = top_k_values / (tf.reduce_sum(top_k_values, axis=-1, keepdims=True) + tf.keras.backend.epsilon())
        top_k_indices = top_k_indices + self.k_s

        routed_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
        for i in range(k):
            expert_indices = top_k_indices[:, i]
            weights = top_k_weights[:, i]
            valid_mask = tf.logical_and(expert_indices >= self.k_s, expert_indices < self.total_experts)
            valid_indices = tf.where(valid_mask, expert_indices - self.k_s, 0)
            valid_weights = tf.where(valid_mask, weights, 0.0)

            expert_outputs = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
            for j in range(self.routed_experts):
                expert_mask = tf.equal(valid_indices, j)
                if tf.reduce_any(expert_mask):
                    expert_input = tf.where(tf.expand_dims(expert_mask, -1), flat_inputs, tf.zeros_like(flat_inputs))
                    expert_output = self.routed_experts_list[j](expert_input)
                    expert_outputs += tf.where(tf.expand_dims(expert_mask, -1), expert_output, tf.zeros_like(expert_output))

            weighted_output = expert_outputs * tf.expand_dims(valid_weights, -1)
            routed_output += weighted_output

        output_flat = shared_output + routed_output + flat_inputs

        if is_3d:
            output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
        else:
            output = output_flat

        f_i = tf.reduce_mean(tf.reduce_sum(tf.one_hot(tf.argmax(gate_weights, axis=-1), depth=self.routed_experts), axis=0))
        P_i = tf.reduce_mean(gate_weights, axis=0)
        load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
        self.add_loss(load_balancing_loss)
        self.load_balancing_loss_metric.update_state(load_balancing_loss)

        gate_weights_reshaped = tf.reshape(gate_weights, [batch_size, seq_length, self.routed_experts]) if is_3d else gate_weights
        return output, gate_weights_reshaped

    def get_metrics(self):
        return {f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric}

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_experts': self.total_experts // self.m,
            'expert_dim': self.adjusted_expert_dim * self.m,
            'input_dim': self.input_dim,
            'm': self.m,
            'k_s': self.k_s,
            'alpha': self.alpha,
            'capacity_factor': self.capacity_factor,
            'name': self.name
        })
        return config

class TransformerDecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.ffn = keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask=None):
        if look_ahead_mask is not None:
            look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
            look_ahead_mask = 1.0 - look_ahead_mask

        attn1_output = self.mha1(query=x, value=x, key=x, attention_mask=look_ahead_mask, training=training)
        attn1 = self.dropout1(attn1_output, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2_output = self.mha2(query=out1, value=enc_output, key=enc_output, training=training)
        attn2 = self.dropout2(attn2_output, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3

    def get_config(self):
        config = super().get_config()
        mha1_config = self.mha1.get_config()
        config.update({
            'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
            'num_heads': mha1_config['num_heads'],
            'dff': self.ffn.layers[0].units,
            'rate': self.dropout1.rate
        })
        return config

class MoEModel(models.Model):
    def __init__(self, moe_params):
        super(MoEModel, self).__init__()
        self.embedding_dim = moe_params["embedding_dim"]
        self.max_seq_length = moe_params["max_seq_length"]
        self.num_domains = moe_params["num_domains"]
        self.num_intents = moe_params["num_intents"]
        self.vocab_size = moe_params["vocab_size"]
        self.num_experts = moe_params["num_experts"]
        self.expert_dim = moe_params["expert_dim"]
        self.turn_id_dim = moe_params["turn_id_dim"]
        self.num_heads = 4
        self.dff = self.embedding_dim * 4
        self.dropout_rate = 0.1
        self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                             self.num_domains + self.turn_id_dim + self.num_intents)
        self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains

        self.embedding = layers.Embedding(
            self.vocab_size,
            self.embedding_dim,
            mask_zero=True,
            embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05),
            name="embedding"
        )
        self.embedding.build((None,))
        with tf.init_scope():
            embedding_weights = self.embedding.get_weights()
            if embedding_weights and len(embedding_weights) > 0:
                embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                self.embedding.set_weights(embedding_weights)

        self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
        self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
        self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
        self.decoder_dropout = layers.Dropout(self.dropout_rate)
        self.moe_nlu = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlu_input_dim, m=2, k_s=2, alpha=0.01, capacity_factor=1.0, name='moe_nlu')
        self.moe_nlg = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlg_input_dim, m=2, k_s=2, alpha=0.01, capacity_factor=1.0, name='moe_nlg')
        self.intent_output = layers.Dense(self.num_intents, activation='sigmoid', name='intent_output')
        self.domain_output = layers.Dense(self.num_domains, activation='softmax', name='domain_output')
        self.response_output = layers.TimeDistributed(layers.Dense(self.vocab_size, activation='softmax'), name='response_output')
        self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)
        self.nlg_projection = layers.TimeDistributed(layers.Dense(self.embedding_dim, activation='relu'), name='nlg_projection')

    def create_look_ahead_mask(self, size):
        mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
        return mask

    def call(self, inputs, training=False):
        user_utterance_tokens = inputs['user_utterance_tokens']
        prev_system_response_tokens = inputs['prev_system_response_tokens']
        decoder_input_tokens = inputs['decoder_input_tokens']
        domain_onehot_input = inputs['domain_onehot_input']
        turn_id_embedding = inputs['turn_id_embedding']
        ontology_multihot_input = inputs['ontology_multihot_input']
        pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
        user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
        prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
        decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)
        user_enc_out = user_embedded
        prev_system_enc_out = prev_system_embedded
        look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
        dec_output = decoder_embedded
        for i, layer in enumerate(self.decoder_layers):
            dec_output = layer(dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask)
        decoder_output = self.decoder_dropout(dec_output, training=training)
        user_attn_out = self.attn_layer(query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), value=user_enc_out, key=user_enc_out, training=training)
        prev_system_attn_out = self.attn_layer(query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), value=prev_system_enc_out, key=prev_system_enc_out, training=training)
        decoder_attn_out = self.attn_layer(query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), value=decoder_output, key=decoder_output, training=training)
        combined_features = layers.Concatenate()([
            tf.squeeze(user_attn_out, 1),
            tf.squeeze(prev_system_attn_out, 1),
            tf.squeeze(decoder_attn_out, 1),
            domain_onehot_input,
            turn_id_embedding,
            ontology_multihot_input
        ])
        combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
        nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
        nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
        intent_out = self.intent_output(nlu_out)
        domain_out = self.domain_output(nlu_out)
        intent_out_expanded = tf.expand_dims(intent_out, axis=1)
        domain_out_expanded = tf.expand_dims(domain_out, axis=1)
        intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
        domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])
        nlg_input = layers.Concatenate(axis=-1)([
            decoder_output,
            intent_out_tiled,
            domain_out_tiled
        ])
        nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])
        nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
        nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
        response_embeddings = self.nlg_projection(nlg_out)
        response_out = self.response_output(nlg_out)
        return {
            'intent_output': intent_out,
            'domain_output': domain_out,
            'response_output': response_out,
            'response_embeddings': response_embeddings,
            'nlu_gate_weights': nlu_gate_weights,
            'nlg_gate_weights': nlg_gate_weights
        }

    def build_graph(self):
        inputs = {
            'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='user_utterance_tokens'),
            'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='prev_system_response_tokens'),
            'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='decoder_input_tokens'),
            'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32, name='domain_onehot_input'),
            'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32, name='turn_id_embedding'),
            'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32, name='ontology_multihot_input'),
        }
        return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

    def get_config(self):
        config = super().get_config()
        config.update({
            'moe_params': {
                'embedding_dim': self.embedding_dim,
                'max_seq_length': self.max_seq_length,
                'num_domains': self.num_domains,
                'num_intents': self.num_intents,
                'vocab_size': self.vocab_size,
                'num_experts': self.num_experts,
                'expert_dim': self.expert_dim,
                'turn_id_dim': self.turn_id_dim
            }
        })
        return config

class IntentAccuracy(metrics.Metric):
    def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros')
        self.total_preds = self.add_weight(name='total_preds', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
        self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
        self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

    def reset_state(self):
        self.correct_preds.assign(0.)
        self.total_preds.assign(0.)

class IntentPrecision(metrics.Metric):
    def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.predicted_positives = self.add_weight(name='pp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        predicted_positives = tf.reduce_sum(y_pred)
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)

    def result(self):
        return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.predicted_positives.assign(0.)

class IntentRecall(metrics.Metric):
    def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.actual_positives = self.add_weight(name='ap', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        actual_positives = tf.reduce_sum(y_true)
        self.true_positives.assign_add(true_positives)
        self.actual_positives.assign_add(actual_positives)

    def result(self):
        return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.actual_positives.assign(0.)

class IntentF1(metrics.Metric):
    def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = IntentPrecision(threshold=threshold)
        self.recall = IntentRecall(threshold=threshold)

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

class DomainAccuracy(metrics.Metric):
    def __init__(self, name='domain_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.accuracy = self.add_weight(name='acc', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
        pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
        matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
        self.accuracy.assign_add(tf.reduce_sum(matches))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.accuracy / (self.count + tf.keras.backend.epsilon())

    def reset_state(self):
        self.accuracy.assign(0.)
        self.count.assign(0.)

class Perplexity(metrics.Metric):
    def __init__(self, name='perplexity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
        self.total_loss = self.add_weight(name='total_loss', initializer='zeros')
        self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        mask = tf.cast(y_true != 0, dtype=tf.float32)
        loss = self.cross_entropy(y_true, y_pred)
        masked_loss = loss * mask
        sum_loss = tf.reduce_sum(masked_loss)
        num_non_padding_tokens = tf.reduce_sum(mask)
        self.total_loss.assign_add(sum_loss)
        self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

    def result(self):
        avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
        return tf.exp(avg_loss)

    def reset_state(self):
        self.total_loss.assign(0.)
        self.total_non_padding_tokens.assign(0.)

class ResponseEmbeddingCosineSimilarity(metrics.Metric):
    def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
        self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true_norm = y_true / (y_true_norm_val + epsilon)
        y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
        cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
        non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
        non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
        masked_cosine_sim = cosine_sim_per_token * non_padding_mask
        num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
        avg_cosine_sim_per_sample = tf.where(
            num_non_zero_tokens > 0,
            tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
            tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
        )
        self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
        self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

    def reset_state(self):
        self.total_cosine_sim.assign(0.)
        self.num_samples.assign(0.)

def contrastive_loss(y_true, y_pred, margin=1.0):
    epsilon = tf.keras.backend.epsilon()
    y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
    y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
    y_true = y_true / (y_true_norm + epsilon)
    y_pred = y_pred / (y_pred_norm + epsilon)
    cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
    positive_loss = 1.0 - cosine_sim
    mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
    masked_loss = positive_loss * mask
    total_loss = tf.reduce_sum(masked_loss)
    num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
    return total_loss / num_tokens

def calculate_flops(model, batch_size=moe_params["batch_size"]):
    flops = 0
    functional_model = model.build_graph()

    def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
        layer_flops_count = 0

        if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
            return 0

        if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
            for sub_layer in layer_instance.layers:
                layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
            return layer_flops_count

        if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
            return 0

        if isinstance(layer_instance, layers.Dense):
            input_dim = layer_instance.input_shape[-1]
            output_dim = layer_instance.output_shape[-1]
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            layer_flops_count += 2 * input_dim * output_dim * effective_batch_size

        elif isinstance(layer_instance, layers.LSTM):
            input_dim = layer_instance.input_shape[-1]
            hidden_dim = layer_instance.units
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
            layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size

        elif isinstance(layer_instance, layers.Embedding):
            pass

        elif isinstance(layer_instance, DeepSeekMoELayer):
            moe_input_dim = layer_instance.input_dim
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
            layer_flops_count += gate_flops
            for expert in layer_instance.experts:
                layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)

        elif isinstance(layer_instance, layers.MultiHeadAttention):
            config = layer_instance.get_config()
            num_heads = config['num_heads']
            key_dim = config['key_dim']
            d_model = num_heads * key_dim
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
            projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
            attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
            context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
            output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
            layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops

        elif isinstance(layer_instance, layers.TimeDistributed):
            inner_layer = layer_instance.layer
            td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
            layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)

        elif isinstance(layer_instance, TransformerDecoderLayer):
            layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
            layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
            ffn_effective_batch_size = current_batch_size * model.max_seq_length
            layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)

        return layer_flops_count

    for layer in functional_model.layers:
        flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)

    return flops / 1e9

def get_gpu_stats():
    if nvidia_smi is None:
        return 0, 0
    try:
        nvidia_smi.nvmlInit()
        handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
        gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
        mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
        gpu_memory = mem_info.used / (1024 ** 3)
        nvidia_smi.nvmlShutdown()
        return gpu_load, gpu_memory
    except Exception:
        return 0, 0

class ResourceMonitor(keras.callbacks.Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.resource_stats = []
        self.expert_load_history = []
        self.start_time = None
        self.epoch_start_time = None
        self.flops = None
        self.validation_data = validation_data
        self.nlu_expert_usage_counts = None
        self.nlg_expert_usage_counts = None
        self.nlu_gate_weights_history = []
        self.nlg_gate_weights_history = []

    def on_train_begin(self, logs=None):
        self.start_time = time.time()
        try:
            self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
            self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
            self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
        except Exception:
            self.flops = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory()
        gpu_load, gpu_memory = get_gpu_stats()
        epoch_time = time.time() - self.epoch_start_time
        self.resource_stats.append({
            'epoch': epoch + 1,
            'epoch_time': epoch_time,
            'cpu_usage': cpu_usage,
            'memory_used': memory.used / (1024 ** 3),
            'memory_total': memory.total / (1024 ** 3),
            'gpu_load': gpu_load,
            'gpu_memory': gpu_memory,
            'loss': logs.get('loss', 0),
            'val_loss': logs.get('val_loss', 0),
            'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
            'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
            'intent_precision': logs.get('intent_output_intent_precision', 0),
            'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
            'intent_recall': logs.get('intent_output_intent_recall', 0),
            'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
            'intent_f1': logs.get('intent_output_intent_f1', 0),
            'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
            'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
            'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
            'perplexity': logs.get('response_output_perplexity', 0),
            'val_perplexity': logs.get('val_response_output_perplexity', 0),
            'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
            'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
            'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
            'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
        })
        if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
            self.expert_load_history.append({
                'epoch': epoch + 1,
                'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
            })

        sample_size = 100
        for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
            inputs, _ = batch
            outputs = self.model(inputs, training=False)
            nlu_gate_weights = outputs['nlu_gate_weights']
            reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
            self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
            nlg_gate_weights = outputs['nlg_gate_weights']
            reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
            self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

    def on_train_end(self, logs=None):
        self.total_time = time.time() - self.start_time

        if len(self.nlu_gate_weights_history) > 0:
            all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
            nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
            for expert_id in nlu_top_indices:
                if expert_id < self.model.moe_nlu.total_experts:
                    self.nlu_expert_usage_counts[expert_id] += 1

        if len(self.nlg_gate_weights_history) > 0:
            all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
            nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
            for expert_id in nlg_top_indices:
                if expert_id < self.model.moe_nlg.total_experts:
                    self.nlg_expert_usage_counts[expert_id] += 1

    def visualize_expert_load_distribution(self):
        total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
        total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
        nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
        nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
        nlu_stability = np.var(nlu_percentages)
        nlg_stability = np.var(nlg_percentages)
        return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {"d_model": self.d_model.numpy(), "warmup_steps": self.warmup_steps}

word_to_id = {}
id_to_word = {}
try:
    possible_paths = [
        os.path.join("preprocessor_models", "preprocessor_params.json"),
        os.path.join("tf_datasets", "preprocessor_params.json"),
        "preprocessor_params.json"
    ]
    
    vocab_loaded = False
    for path in possible_paths:
        if os.path.exists(path):
            with open(path, "r") as f:
                preprocessor_params = json.load(f)
            word_to_id = {k: int(v) for k, v in preprocessor_params['word_to_id'].items()}
            id_to_word = {int(k): v for k, v in preprocessor_params['id_to_word'].items()}
            print(f"Loaded vocabulary from {path}")
            vocab_loaded = True
            break
    
    if not vocab_loaded:
        raise FileNotFoundError("Vocabulary file not found in any location")
except Exception as e:
    print(f"Warning: Could not load vocabulary from preprocessor: {str(e)}")
    word_to_id = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
    id_to_word = {0: '<pad>', 1: '<sos>', 2: '<eos>', 3: '<unk>'}

model_save_path = "best_deepseekmoe_expert_choice_model"
print(f"\nLoading model from: {model_save_path}")
custom_objects = {
    "MoEModel":MoEModel,
    "CustomLearningRateSchedule": CustomLearningRateSchedule,
    "DeepSeekMoELayer": DeepSeekMoELayer,
    "TransformerDecoderLayer": TransformerDecoderLayer,
    "IntentAccuracy": IntentAccuracy,
    "IntentPrecision": IntentPrecision,
    "IntentRecall": IntentRecall,
    "IntentF1": IntentF1,
    "DomainAccuracy": DomainAccuracy,
    "Perplexity": Perplexity,
    "ResponseEmbeddingCosineSimilarity": ResponseEmbeddingCosineSimilarity
}
try:
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
except Exception as e:
    print(f"Failed to load model from {model_save_path}: {str(e)}")
    raise

def compile_model(model, moe_params):
    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    losses_dict = {
        'intent_output': tf.keras.losses.BinaryCrossentropy(),
        'domain_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss,
    }

    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }

    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={
            'intent_output': 0.5,
            'domain_output': 0.5,
            'response_output': 1.0,
            'response_embeddings': 1.0
        }
    )
    return model

model = compile_model(model, moe_params)
print("\nModel loaded and compiled successfully!")
sample_input = next(iter(train_tf_dataset.take(1)))
model(sample_input[0])
model.summary()

class EvaluationMetrics:
    def __init__(self, model, test_dataset, moe_params, raw_data, word_to_id, id_to_word):
        self.model = model
        self.test_dataset = test_dataset
        self.moe_params = moe_params
        self.raw_data = raw_data
        self.word_to_id = word_to_id
        self.id_to_word = id_to_word
        self.nlu_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.nlg_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.resource_stats = {
            'cpu_usage': [],
            'memory_used': [],
            'gpu_load': [],
            'gpu_memory': [],
            'batch_times': []
        }
        self.special_tokens = {'<pad>', '<sos>', '<eos>', '<unk>'}
        self.nlu_top_k = model.get_layer('moe_nlu').get_config().get('top_k', 2)
        self.nlg_top_k = model.get_layer('moe_nlg').get_config().get('top_k', 2)

    def evaluate(self):
        print("\nEvaluating model on test set...")
        
        def add_response_embeddings(features, targets):
            response_tokens = targets['response_output']
            response_embeddings = self.model.embedding(response_tokens)
            targets['response_embeddings'] = response_embeddings
            return features, targets
        
        test_dataset_with_embeddings = self.test_dataset.map(add_response_embeddings, num_parallel_calls=tf.data.AUTOTUNE)

        start_time = time.time()
        test_results = self.model.evaluate(test_dataset_with_embeddings, verbose=1)
        eval_time = time.time() - start_time
        
        metric_names = self.model.metrics_names
        
        perplexity_value = None
        for name, value in zip(metric_names, test_results):
            if name == 'response_output_perplexity':
                perplexity_value = value
                break
        if perplexity_value is None:
            print("Warning: Could not find perplexity in standard evaluation results.")
            perplexity_value = 0.0
        
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory().used / (1024 ** 3)
        gpu_load, gpu_memory = get_gpu_stats()
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))

        total_tokens = 0
        total_time = 0
        num_batches = 0
        
        test_dataset_small = test_dataset_with_embeddings.unbatch().batch(moe_params["batch_size"])
        
        all_true_intents = []
        all_pred_intents = []
        all_true_domains = []
        all_pred_domains = []
        
        for batch_idx, batch in enumerate(test_dataset_small):
            inputs, targets = batch
            response_tokens = targets['response_output'].numpy()
            non_padding_mask = response_tokens != 0
            total_tokens += np.sum(non_padding_mask)
            
            start_time = time.time()
            outputs = self.model(inputs, training=False)
            batch_time = time.time() - start_time
            
            total_time += batch_time
            num_batches += tf.shape(inputs['user_utterance_tokens'])[0]
            
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            
            self.resource_stats['cpu_usage'].append(cpu_usage)
            self.resource_stats['memory_used'].append(memory.used / (1024 ** 3))
            self.resource_stats['gpu_load'].append(gpu_load)
            self.resource_stats['gpu_memory'].append(gpu_memory)
            self.resource_stats['batch_times'].append(batch_time)
            
            nlu_gate_weights = outputs.get('nlu_gate_weights', tf.zeros([0, self.model.moe_nlu.routed_experts])).numpy()
            nlu_top_indices = np.argsort(-nlu_gate_weights, axis=-1)[:, :self.nlu_top_k] + self.model.moe_nlu.k_s
            for idx in range(self.nlu_top_k):
                expert_ids = nlu_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlu_expert_usage_counts):
                        self.nlu_expert_usage_counts[expert_id] += 1
            
            nlg_gate_weights = outputs.get('nlg_gate_weights', tf.zeros([0, self.model.moe_nlg.routed_experts])).numpy()
            if len(nlg_gate_weights.shape) == 3:
                nlg_gate_weights = nlg_gate_weights.reshape(-1, nlg_gate_weights.shape[-1])
            nlg_top_indices = np.argsort(-nlg_gate_weights, axis=-1)[:, :self.nlg_top_k] + self.model.moe_nlg.k_s
            for idx in range(self.nlg_top_k):
                expert_ids = nlg_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlg_expert_usage_counts):
                        self.nlg_expert_usage_counts[expert_id] += 1
            
            true_intents = targets.get('intent_output', np.zeros((moe_params["batch_size"], self.model.num_intents))).numpy()
            pred_intents = (outputs.get('intent_output', np.zeros_like(true_intents)).numpy() > 0.5).astype(np.int32)
            all_true_intents.append(true_intents)
            all_pred_intents.append(pred_intents)
            
            true_domains = targets.get('domain_output', np.zeros((moe_params["batch_size"], 1))).numpy().flatten()
            pred_domains = np.argmax(outputs.get('domain_output', np.zeros((moe_params["batch_size"], self.model.num_domains))).numpy(), axis=-1)
            all_true_domains.append(true_domains)
            all_pred_domains.append(pred_domains)
            
            if batch_idx % 50 == 0:
                gc.collect()
                tf.keras.backend.clear_session()
        
        avg_time_per_token = (total_time / total_tokens) * 1000 if total_tokens > 0 else 0
        
        avg_cpu = np.mean(self.resource_stats['cpu_usage']) if self.resource_stats['cpu_usage'] else 0
        avg_memory = np.mean(self.resource_stats['memory_used']) if self.resource_stats['memory_used'] else 0
        avg_gpu_load = np.mean(self.resource_stats['gpu_load']) if self.resource_stats['gpu_load'] else 0
        avg_gpu_memory = np.mean(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        peak_gpu_memory = np.max(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))
        
        all_true_intents = np.vstack(all_true_intents)
        all_pred_intents = np.vstack(all_pred_intents)
        all_true_domains = np.concatenate(all_true_domains)
        all_pred_domains = np.concatenate(all_pred_domains)
        
        intent_f1_score = f1_score(all_true_intents, all_pred_intents, average='micro')
        intent_f1_score_macro = f1_score(all_true_intents, all_pred_intents, average='macro')
        intent_f1_score_weighted = f1_score(all_true_intents, all_pred_intents, average='weighted')
        intent_accuracy_score = accuracy_score(all_true_intents, all_pred_intents)
        
        domain_accuracy_score = accuracy_score(all_true_domains, all_pred_domains)
        domain_f1_score_macro = f1_score(all_true_domains, all_pred_domains, average='macro')

        hypotheses = []
        references = []
        meteor_scores = []
        rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        
        domain_metrics = defaultdict(lambda: {
            'bleu': [],
            'rouge1': [],
            'rouge2': [],
            'rougeL': [],
            'meteor': [],
            'count': 0
        })
        
        if not self.raw_data.get('test'):
            print("Warning: raw_data['test'] is empty. Skipping NLG metrics calculation.")
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0
            domain_metrics = {}
        else:
            for i, batch in enumerate(test_dataset_small):
                if i >= len(self.raw_data.get('test', [])):
                    print(f"Warning: raw_data['test'] has fewer items ({len(self.raw_data.get('test', []))}) than test dataset. Stopping NLG metrics calculation.")
                    break
                
                try:
                    inputs, targets = batch
                    with tf.device('/CPU:0'):
                        outputs = self.model(inputs, training=False)
                    
                    domain_id = targets['domain_output'].numpy()[0][0]
                    domain_name = f"domain_{domain_id}"
                    
                    response_probs = outputs['response_output'].numpy()
                    generated_tokens = np.argmax(response_probs, axis=-1)[0]
                    true_tokens = targets['response_output'].numpy()[0]
                    
                    raw_item = self.raw_data['test'][i]
                    ref_text = raw_item.get('raw_next_system_response', '')
                    if not ref_text:
                        continue
                    
                    gen_text = self.tokens_to_text(generated_tokens)
                    ref_tokens = self.tokens_to_text(true_tokens)
                    
                    if not gen_text.strip() or not ref_text.strip():
                        continue
                    
                    hypotheses.append(gen_text)
                    references.append([ref_text])
                    
                    ref_tokens = word_tokenize(ref_text.lower())
                    gen_tokens = word_tokenize(gen_text.lower())
                    
                    try:
                        meteor = meteor_score([ref_tokens], gen_tokens)
                        meteor_scores.append(meteor)
                        domain_metrics[domain_name]['meteor'].append(meteor)
                    except Exception as e:
                        print(f"METEOR calculation error for example {i}: {str(e)}")
                        meteor_scores.append(0)
                        domain_metrics[domain_name]['meteor'].append(0)
                    
                    try:
                        rouge_scores = rouge_scorer_instance.score(ref_text, gen_text)
                        domain_metrics[domain_name]['rouge1'].append(rouge_scores['rouge1'].fmeasure)
                        domain_metrics[domain_name]['rouge2'].append(rouge_scores['rouge2'].fmeasure)
                        domain_metrics[domain_name]['rougeL'].append(rouge_scores['rougeL'].fmeasure)
                    except Exception as e:
                        print(f"ROUGE calculation error for example {i}: {str(e)}")
                    
                    domain_metrics[domain_name]['count'] += 1
                    
                    if i % 50 == 0:
                        gc.collect()
                        tf.keras.backend.clear_session()
                    
                except Exception as e:
                    print(f"Error processing example {i}: {str(e)}")
                    continue
        
        if hypotheses and references:
            hypotheses_tok = []
            references_tok = []
            
            for hyp, ref_list in zip(hypotheses, references):
                if not hyp or not ref_list[0]:
                    continue
                hyp_tokens = word_tokenize(hyp.lower())
                ref_tokens = [word_tokenize(ref.lower()) for ref in ref_list]
                hypotheses_tok.append(hyp_tokens)
                references_tok.append(ref_tokens)
            
            chencherry = SmoothingFunction()
            try:
                bleu_score = corpus_bleu(references_tok, hypotheses_tok, smoothing_function=chencherry.method1)
            except Exception as e:
                print(f"BLEU calculation error: {str(e)}")
                bleu_score = 0.0
            
            rouge_scores = []
            for hyp, ref in zip(hypotheses, references):
                if hyp and ref[0]:
                    try:
                        rouge_scores.append(rouge_scorer_instance.score(ref[0], hyp))
                    except Exception as e:
                        print(f"ROUGE scoring error: {str(e)}")
                        continue
            
            avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_meteor = np.mean(meteor_scores) if meteor_scores else 0.0
        else:
            print("Warning: No valid hypotheses or references for NLG metrics. Setting to 0.0.")
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0

        return {
            'intent_f1_micro': intent_f1_score,
            'intent_f1_macro': intent_f1_score_macro,
            'intent_f1_weighted': intent_f1_score_weighted,
            'intent_accuracy': intent_accuracy_score,
            'domain_accuracy': domain_accuracy_score,
            'domain_f1_macro': domain_f1_score_macro,
            'bleu': bleu_score,
            'rouge1': avg_rouge1,
            'rouge2': avg_rouge2,
            'rougeL': avg_rougeL,
            'meteor': avg_meteor,
            'perplexity': perplexity_value,
            'pred_speed': avg_time_per_token,
            'domain_metrics': domain_metrics,
            'eval_time': eval_time
        }
    
    def tokens_to_text(self, tokens):
        words = []
        for token in tokens:
            if token == 0:
                continue
            word = self.id_to_word.get(token, '<unk>')
            if word not in self.special_tokens:
                words.append(word)
        return " ".join(words).strip()

results_table = {
    'Evaluation Time (seconds)': [],
    'Prediction Speed (ms/token)': [],
    'Average CPU Usage (percent)': [],
    'Average GPU Usage (percent)': [],
    'Average Memory (GB)': [],
    'Average GPU Memory (GB)': [],
    'Average FLOPs Estimate (GFLOPs)': [],
    'NLU Expert 1 (percent)': [],
    'NLU Expert 2 (percent)': [],
    'NLU Expert 3 (percent)': [],
    'NLU Expert 4 (percent)': [],
    'NLU Expert 5 (percent)': [],
    'NLU Expert 6 (percent)': [],
    'NLU Expert 7 (percent)': [],
    'NLU Expert 8 (percent)': [],
    'NLG Expert 1 (percent)': [],
    'NLG Expert 2 (percent)': [],
    'NLG Expert 3 (percent)': [],
    'NLG Expert 4 (percent)': [],
    'NLG Expert 5 (percent)': [],
    'NLG Expert 6 (percent)': [],
    'NLG Expert 7 (percent)': [],
    'NLG Expert 8 (percent)': [],
    'Intent F1-score (Micro)': [],
    'Intent F1-score (Macro)': [],
    'Intent F1-score (Weighted)': [],
    'Intent Accuracy': [],
    'Domain Accuracy': [],
    'Domain F1-score (Macro)': [],
    'BLEU Score': [],
    'ROUGE-1 F1': [],
    'ROUGE-2 F1': [],
    'ROUGE-L F1': [],
    'METEOR Score': [],
    'Perplexity': [],
    'NLU Expert Load Stability (Variance)': [],
    'NLG Expert Load Stability (Variance)': []
}

num_iterations = 5
for iteration in range(1, num_iterations + 1):
    print(f"\nStarting Iteration {iteration}")
    tf.keras.backend.clear_session()
    gc.collect()
    
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
    model = compile_model(model, moe_params)
    
    evaluator = EvaluationMetrics(model, test_tf_dataset, moe_params, raw_data, word_to_id, id_to_word)

    evaluation_results = evaluator.evaluate()
    results_table['Evaluation Time (seconds)'].append(evaluation_results['eval_time'])
    results_table['Prediction Speed (ms/token)'].append(evaluation_results['pred_speed'])
    results_table['Average CPU Usage (percent)'].append(np.mean(evaluator.resource_stats['cpu_usage']) if evaluator.resource_stats['cpu_usage'] else 0)
    results_table['Average GPU Usage (percent)'].append(np.mean(evaluator.resource_stats['gpu_load']) if evaluator.resource_stats['gpu_load'] else 0)
    results_table['Average Memory (GB)'].append(np.mean(evaluator.resource_stats['memory_used']) if evaluator.resource_stats['memory_used'] else 0)
    results_table['Average GPU Memory (GB)'].append(np.mean(evaluator.resource_stats['gpu_memory']) if evaluator.resource_stats['gpu_memory'] else 0)
    results_table['Average FLOPs Estimate (GFLOPs)'].append(calculate_flops(model))
    nlu_counts = evaluator.nlu_expert_usage_counts
    nlg_counts = evaluator.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) if np.sum(nlu_counts) > 0 else 1
    total_nlg = np.sum(nlg_counts) if np.sum(nlg_counts) > 0 else 1
    results_table['NLU Expert 1 (percent)'].append((nlu_counts[0] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 2 (percent)'].append((nlu_counts[1] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 3 (percent)'].append((nlu_counts[2] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 4 (percent)'].append((nlu_counts[3] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 5 (percent)'].append((nlu_counts[4] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 6 (percent)'].append((nlu_counts[5] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 7 (percent)'].append((nlu_counts[6] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 8 (percent)'].append((nlu_counts[7] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLG Expert 1 (percent)'].append((nlg_counts[0] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 2 (percent)'].append((nlg_counts[1] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 3 (percent)'].append((nlg_counts[2] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 4 (percent)'].append((nlg_counts[3] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 5 (percent)'].append((nlg_counts[4] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 6 (percent)'].append((nlg_counts[5] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 7 (percent)'].append((nlg_counts[6] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 8 (percent)'].append((nlg_counts[7] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['Intent F1-score (Micro)'].append(evaluation_results['intent_f1_micro'])
    results_table['Intent F1-score (Macro)'].append(evaluation_results['intent_f1_macro'])
    results_table['Intent F1-score (Weighted)'].append(evaluation_results['intent_f1_weighted'])
    results_table['Intent Accuracy'].append(evaluation_results['intent_accuracy'])
    results_table['Domain Accuracy'].append(evaluation_results['domain_accuracy'])
    results_table['Domain F1-score (Macro)'].append(evaluation_results['domain_f1_macro'])
    results_table['BLEU Score'].append(evaluation_results['bleu'])
    results_table['ROUGE-1 F1'].append(evaluation_results['rouge1'])
    results_table['ROUGE-2 F1'].append(evaluation_results['rouge2'])
    results_table['ROUGE-L F1'].append(evaluation_results['rougeL'])
    results_table['METEOR Score'].append(evaluation_results['meteor'])
    results_table['Perplexity'].append(evaluation_results['perplexity'])
    results_table['NLU Expert Load Stability (Variance)'].append(np.var(nlu_counts / total_nlu * 100) if total_nlu > 0 else 0)
    results_table['NLG Expert Load Stability (Variance)'].append(np.var(nlg_counts / total_nlg * 100) if total_nlg > 0 else 0)

for key in results_table:
    if results_table[key]:
        results_table[key].append(np.mean(results_table[key]))
    else:
        results_table[key].append(0)

print("\nTraining results saved")
final_df = pd.DataFrame(results_table)
final_df = final_df.T

final_df.columns = [f'Iteration {i+1}' for i in range(num_iterations)] + ['Average']

final_df.to_excel('prediction_deepseekmoe_expert_choice_results.xlsx', index=False)
final_df

2025-07-04 08:09:54.202250: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 08:09:54.501931: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 08:09:54.502031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 08:09:54.526501: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 08:09:54.624110: I tensorflow/core/platform/cpu_feature_guar


Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 08:09:59.133447: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 08:09:59.276388: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 08:09:59.276604: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl
Loaded vocabulary from preprocessor_models/preprocessor_params.json

Loading model from: best_deepseekmoe_expert_choice_model

Model loaded and compiled successfully!
Model: "mo_e_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  784896    
                                                                 
 embedding (Embedding)       multiple                  8576      
                                                                 
 layer_normalization (Layer  multiple                  256       
 Normalization)                                                  
                                                                 
 transformer_decoder_layer   multiple                  264576    
 (TransformerDecoderLayer)     

,Iteration 1,Average
Evaluation Time (seconds),23.802339,23.802339
Prediction Speed (ms/token),2.434583,2.434583
Average CPU Usage (percent),23.217394,23.217394
Average GPU Usage (percent),1.866142,1.866142
Average Memory (GB),5.696944,5.696944
Average GPU Memory (GB),14.504456,14.504456
Average FLOPs Estimate (GFLOPs),2.377871,2.377871
NLU Expert 1 (percent),0.000000,0.000000
NLU Expert 2 (percent),0.000000,0.000000
NLU Expert 3 (percent),18.486042,18.486042


# DeepSeekMoE with Top-P Routing with Adaptive Gating experiment code

In [1]:
def reset_kernel():
    tf.keras.backend.clear_session()
    import gc
    gc.collect()

results_table = {
    'Training Speed (epochs per second)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Training Time (second/epoch)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Total Training Time (second/iteration)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Computational Resource Usage': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average CPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average GPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average GPU Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average FLOPS Estimate (GFLOPS)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Expert NLU Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Expert NLG Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Entity Accuracy': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Intent Accuracy': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Perplexity': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Response Cosine Similarity': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0}
}

best_val_loss = float('inf')
best_model_path = "best_deepseekmoe_topp_adaptive_model"

for iteration in range(5):
    print(f"\nStarting Training Iteration {iteration + 1}")
    reset_kernel()

    def load_tf_datasets_from_disk(load_path):
        print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
        try:
            with open(os.path.join(load_path, "moe_params.json"), "r") as f:
                loaded_moe_params = json.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"moe_params.json not found in {load_path}")

        element_spec = (
            {
                'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
                'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
                'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            },
            {
                'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
                'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
                'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
            }
        )

        try:
            train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
            test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
        except Exception as e:
            raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

        raw_data = {}
        for dataset_name in ['train', 'test']:
            path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
            if os.path.exists(path):
                with open(path, 'rb') as f:
                    raw_data[dataset_name] = pickle.load(f)
                print(f"Loaded raw data for {dataset_name} from {path}")
            else:
                print(f"Warning: Raw data file {path} not found.")
                raw_data[dataset_name] = []

        return {
            "train_dataset": train_tf_dataset,
            "test_dataset": test_tf_dataset,
            "moe_params": loaded_moe_params,
            "raw_data": raw_data
        }

    tf_dataset_save_path = "tf_datasets"
    loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
    train_tf_dataset = loaded_data["train_dataset"]
    moe_params = loaded_data["moe_params"]
    raw_data = loaded_data["raw_data"]

    class TopPAdaptiveRouter:
        def __init__(self, p=0.9, tau=0.1):
            self.p = p
            self.tau = tau

        def __call__(self, gate_logits):
            probs = tf.nn.softmax(gate_logits, axis=-1)
            sorted_probs, sorted_indices = tf.math.top_k(probs, k=tf.shape(probs)[-1])
            top1_prob = sorted_probs[:, 0]
            top2_prob = sorted_probs[:, 1]
            prob_diff = top1_prob - top2_prob
            use_top1 = prob_diff >= self.tau
            max_k = tf.where(use_top1, 1, 2)
            cum_probs = tf.cumsum(sorted_probs, axis=-1)
            p_mask = cum_probs >= self.p
            first_over_p = tf.argmax(tf.cast(p_mask, tf.int32), axis=-1, output_type=tf.int32)
            first_over_p = tf.maximum(first_over_p, 0)
            num_selected_top_p = first_over_p + 1
            num_selected = tf.minimum(num_selected_top_p, max_k)
            max_selected = tf.reduce_max(num_selected)
            batch_size = tf.shape(gate_logits)[0]
            num_experts = tf.shape(gate_logits)[1]
            selection_mask = tf.sequence_mask(num_selected, maxlen=num_experts, dtype=tf.bool)
            selected_indices = tf.where(selection_mask, sorted_indices, num_experts)
            selected_probs = tf.where(selection_mask, sorted_probs, 0.0)
            selected_indices = selected_indices[:, :max_selected]
            selected_probs = selected_probs[:, :max_selected]
            weights_sum = tf.reduce_sum(selected_probs, axis=-1, keepdims=True) + tf.keras.backend.epsilon()
            weights = selected_probs / weights_sum
            valid_mask = tf.sequence_mask(num_selected, maxlen=max_selected, dtype=tf.int32)
            selected_indices = tf.where(valid_mask == 1, selected_indices, -1)
            return selected_indices, weights, num_selected

    class DeepSeekMoELayer(layers.Layer):
        def __init__(self, num_experts, expert_dim, input_dim, m=4, k_s=2, alpha=0.01, top_p=0.9, tau=0.1, name=None):
            super(DeepSeekMoELayer, self).__init__(name=name)
            self.m = m
            self.k_s = k_s
            self.total_experts = num_experts * self.m
            self.routed_experts = self.total_experts - self.k_s
            self.alpha = alpha
            self.top_p = top_p
            self.tau = tau
            self.input_dim = input_dim
            self.seq_length = None
            self.adjusted_expert_dim = expert_dim // self.m
            self.shared_experts = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4), name=f'shared_expert_{i}_dense1'),
                    layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4), name=f'shared_expert_{i}_dense2')
                ], name=f'shared_expert_{i}') for i in range(self.k_s)
            ]
            self.routed_experts_list = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4), name=f'routed_expert_{i}_dense1'),
                    layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4), name=f'routed_expert_{i}_dense2')
                ], name=f'routed_expert_{i}') for i in range(self.k_s, self.total_experts)
            ]
            self.experts = self.shared_experts + self.routed_experts_list
            self.gate = layers.Dense(self.routed_experts, activation=None, name='gate_weights')
            self.router = TopPAdaptiveRouter(p=self.top_p, tau=self.tau)
            self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

        def call(self, inputs, training=False):
            input_rank = inputs.shape.rank
            if input_rank == 3:
                batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
                flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
                is_3d = True
            else:
                batch_size = tf.shape(inputs)[0]
                seq_length = 1
                flat_inputs = inputs
                is_3d = False
            num_tokens = tf.shape(flat_inputs)[0]
            flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
            shared_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
            for i in range(self.k_s):
                shared_output += self.shared_experts[i](flat_inputs)
            gate_logits = self.gate(flat_inputs)
            expert_indices, expert_weights, num_selected = self.router(gate_logits)
            expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
            expert_indices_adjusted = expert_indices + self.k_s
            batch_indices = tf.tile(tf.expand_dims(tf.range(num_tokens), 1), [1, tf.shape(expert_indices)[1]])
            gather_indices = tf.stack([batch_indices, expert_indices_adjusted], axis=-1)
            valid_mask = tf.cast(expert_indices_adjusted >= 0, tf.float32)
            valid_mask = tf.expand_dims(valid_mask, -1)
            selected_outputs = tf.gather_nd(expert_outputs, gather_indices)
            weighted_outputs = selected_outputs * tf.expand_dims(expert_weights, -1) * valid_mask
            routed_output = tf.reduce_sum(weighted_outputs, axis=1)
            output_flat = shared_output + routed_output + flat_inputs
            if is_3d:
                output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
                if self.seq_length is not None:
                    output.set_shape([None, self.seq_length, self.input_dim])
                gate_weights_reshaped = tf.reshape(tf.nn.softmax(gate_logits, axis=-1), [batch_size, seq_length, self.routed_experts])
                if self.seq_length is not None:
                    gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
            else:
                output = output_flat
                gate_weights_reshaped = tf.nn.softmax(gate_logits, axis=-1)
            gate_probs = tf.nn.softmax(gate_logits, axis=-1)
            expert_selection = tf.reduce_sum(tf.one_hot(expert_indices_adjusted, depth=self.total_experts), axis=1)
            expert_selection = expert_selection[:, self.k_s:]
            f_i = tf.reduce_mean(expert_selection, axis=0)
            P_i = tf.reduce_mean(gate_probs, axis=0)
            load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
            self.add_loss(load_balancing_loss)
            self.load_balancing_loss_metric.update_state(load_balancing_loss)
            return output, gate_weights_reshaped

        def get_metrics(self):
            return {f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric}

        def get_config(self):
            config = super().get_config()
            config.update({
                'num_experts': self.total_experts // self.m,
                'expert_dim': self.adjusted_expert_dim * self.m,
                'input_dim': self.input_dim,
                'm': self.m,
                'k_s': self.k_s,
                'alpha': self.alpha,
                'top_p': self.top_p,
                'tau': self.tau,
                'name': self.name
            })
            return config

    class TransformerDecoderLayer(layers.Layer):
        def __init__(self, d_model, num_heads, dff, rate=0.1):
            super(TransformerDecoderLayer, self).__init__()
            self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.ffn = keras.Sequential([
                layers.Dense(dff, activation='relu'),
                layers.Dense(d_model)
            ])
            self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
            self.dropout1 = layers.Dropout(rate)
            self.dropout2 = layers.Dropout(rate)
            self.dropout3 = layers.Dropout(rate)

        def call(self, x, enc_output, training, look_ahead_mask=None):
            if look_ahead_mask is not None:
                look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
                look_ahead_mask = 1.0 - look_ahead_mask
            attn1_output = self.mha1(query=x, value=x, key=x, attention_mask=look_ahead_mask, training=training)
            attn1 = self.dropout1(attn1_output, training=training)
            out1 = self.layernorm1(attn1 + x)
            attn2_output = self.mha2(query=out1, value=enc_output, key=enc_output, training=training)
            attn2 = self.dropout2(attn2_output, training=training)
            out2 = self.layernorm2(attn2 + out1)
            ffn_output = self.ffn(out2)
            ffn_output = self.dropout3(ffn_output, training=training)
            out3 = self.layernorm3(ffn_output + out2)
            return out3

        def get_config(self):
            config = super().get_config()
            mha1_config = self.mha1.get_config()
            config.update({
                'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
                'num_heads': mha1_config['num_heads'],
                'dff': self.ffn.layers[0].units,
                'rate': self.dropout1.rate
            })
            return config

    class MoEModel(models.Model):
        def __init__(self, moe_params):
            super(MoEModel, self).__init__()
            self.embedding_dim = moe_params["embedding_dim"]
            self.max_seq_length = moe_params["max_seq_length"]
            self.num_domains = moe_params["num_domains"]
            self.num_intents = moe_params["num_intents"]
            self.vocab_size = moe_params["vocab_size"]
            self.num_experts = moe_params["num_experts"]
            self.expert_dim = moe_params["expert_dim"]
            self.turn_id_dim = moe_params["turn_id_dim"]
            self.num_heads = 4
            self.dff = self.embedding_dim * 4
            self.dropout_rate = 0.1
            self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                                 self.num_domains + self.turn_id_dim + self.num_intents)
            self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains

            self.embedding = layers.Embedding(
                self.vocab_size,
                self.embedding_dim,
                mask_zero=True,
                embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05),
                name="embedding"
            )
            self.embedding.build((None,))
            with tf.init_scope():
                embedding_weights = self.embedding.get_weights()
                if embedding_weights and len(embedding_weights) > 0:
                    embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                    self.embedding.set_weights(embedding_weights)

            self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
            self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
            self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
            self.decoder_dropout = layers.Dropout(self.dropout_rate)
            self.moe_nlu = DeepSeekMoELayer(
                num_experts=self.num_experts,
                expert_dim=self.expert_dim,
                input_dim=self.nlu_input_dim,
                m=2,
                k_s=2,
                alpha=0.01,
                top_p=0.9,
                tau=0.1,
                name='moe_nlu'
            )
            self.moe_nlg = DeepSeekMoELayer(
                num_experts=self.num_experts,
                expert_dim=self.expert_dim,
                input_dim=self.nlg_input_dim,
                m=2,
                k_s=2,
                alpha=0.01,
                top_p=0.9,
                tau=0.1,
                name='moe_nlg'
            )
            self.intent_output = layers.Dense(self.num_intents, activation='sigmoid', name='intent_output')
            self.domain_output = layers.Dense(self.num_domains, activation='softmax', name='domain_output')
            self.response_output = layers.TimeDistributed(
                layers.Dense(self.vocab_size, activation='softmax'), name='response_output'
            )
            self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)
            self.nlg_projection = layers.TimeDistributed(
                layers.Dense(self.embedding_dim, activation='relu', name='nlg_projection')
            )

        def create_look_ahead_mask(self, size):
            mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
            return mask

        def call(self, inputs, training=False):
            user_utterance_tokens = inputs['user_utterance_tokens']
            prev_system_response_tokens = inputs['prev_system_response_tokens']
            decoder_input_tokens = inputs['decoder_input_tokens']
            domain_onehot_input = inputs['domain_onehot_input']
            turn_id_embedding = inputs['turn_id_embedding']
            ontology_multihot_input = inputs['ontology_multihot_input']

            pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
            user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
            prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
            decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)

            user_enc_out = user_embedded
            prev_system_enc_out = prev_system_embedded

            look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
            dec_output = decoder_embedded
            for i, layer in enumerate(self.decoder_layers):
                dec_output = layer(dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask)
            decoder_output = self.decoder_dropout(dec_output, training=training)

            user_attn_out = self.attn_layer(
                query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), 
                value=user_enc_out, key=user_enc_out, training=training
            )
            prev_system_attn_out = self.attn_layer(
                query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), 
                value=prev_system_enc_out, key=prev_system_enc_out, training=training
            )
            decoder_attn_out = self.attn_layer(
                query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), 
                value=decoder_output, key=decoder_output, training=training
            )

            combined_features = layers.Concatenate()([
                tf.squeeze(user_attn_out, axis=1),
                tf.squeeze(prev_system_attn_out, axis=1),
                tf.squeeze(decoder_attn_out, axis=1),
                domain_onehot_input,
                turn_id_embedding,
                ontology_multihot_input
            ])
            combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
            nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
            nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
            intent_out = self.intent_output(nlu_out)
            domain_out = self.domain_output(nlu_out)

            intent_out_expanded = tf.expand_dims(intent_out, axis=1)
            domain_out_expanded = tf.expand_dims(domain_out, axis=1)
            intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
            domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])

            nlg_input = layers.Concatenate(axis=-1)([
                decoder_output,
                intent_out_tiled,
                domain_out_tiled
            ])
            nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])

            nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
            nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
            response_embeddings = self.nlg_projection(nlg_out)
            response_embeddings = tf.ensure_shape(response_embeddings, [None, self.max_seq_length, self.embedding_dim])
            response_out = self.response_output(nlg_out)

            return {
                'intent_output': intent_out,
                'domain_output': domain_out,
                'response_output': response_out,
                'response_embeddings': response_embeddings,
                'nlu_gate_weights': nlu_gate_weights,
                'nlg_gate_weights': nlg_gate_weights
            }

        def build_graph(self):
            inputs = {
                'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='user_utterance_tokens'),
                'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='prev_system_response_tokens'),
                'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='decoder_input_tokens'),
                'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32, name='domain_onehot_input'),
                'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32, name='turn_id_embedding'),
                'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32, name='ontology_multihot_input'),
            }
            return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

        def get_config(self):
            config = super().get_config()
            config.update({
                'moe_params': {
                    'embedding_dim': self.embedding_dim,
                    'max_seq_length': self.max_seq_length,
                    'num_domains': self.num_domains,
                    'num_intents': self.num_intents,
                    'vocab_size': self.vocab_size,
                    'num_experts': self.num_experts,
                    'expert_dim': self.expert_dim,
                    'turn_id_dim': self.turn_id_dim
                }
            })
            return config
    
    class IntentAccuracy(metrics.Metric):
        def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros', dtype=tf.float32)
            self.total_preds = self.add_weight(name='total_preds', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
            self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
            self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

        def reset_state(self):
            self.correct_preds.assign(0.)
            self.total_preds.assign(0.)

    class IntentPrecision(metrics.Metric):
        def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
            self.predicted_positives = self.add_weight(name='pp', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            predicted_positives = tf.reduce_sum(y_pred)
            self.true_positives.assign_add(true_positives)
            self.predicted_positives.assign_add(predicted_positives)

        def result(self):
            return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.predicted_positives.assign(0.)

    class IntentRecall(metrics.Metric):
        def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
            self.actual_positives = self.add_weight(name='ap', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            actual_positives = tf.reduce_sum(y_true)
            self.true_positives.assign_add(true_positives)
            self.actual_positives.assign_add(actual_positives)

        def result(self):
            return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.actual_positives.assign(0.)

    class IntentF1(metrics.Metric):
        def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.precision = IntentPrecision(threshold=threshold)
            self.recall = IntentRecall(threshold=threshold)

        def update_state(self, y_true, y_pred, sample_weight=None):
            self.precision.update_state(y_true, y_pred, sample_weight)
            self.recall.update_state(y_true, y_pred, sample_weight)

        def result(self):
            p = self.precision.result()
            r = self.recall.result()
            return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

        def reset_state(self):
            self.precision.reset_state()
            self.recall.reset_state()

    class DomainAccuracy(metrics.Metric):
        def __init__(self, name='domain_accuracy', **kwargs):
            super().__init__(name=name, **kwargs)
            self.accuracy = self.add_weight(name='acc', initializer='zeros', dtype=tf.float32)
            self.count = self.add_weight(name='count', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
            pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
            matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
            self.accuracy.assign_add(tf.reduce_sum(matches))
            self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.accuracy / (self.count + tf.keras.backend.epsilon())

        def reset_state(self):
            self.accuracy.assign(0.)
            self.count.assign(0.)

    class Perplexity(metrics.Metric):
        def __init__(self, name='perplexity', **kwargs):
            super().__init__(name=name, **kwargs)
            self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
            self.total_loss = self.add_weight(name='total_loss', initializer='zeros', dtype=tf.float32)
            self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            mask = tf.cast(y_true != 0, dtype=tf.float32)
            loss = self.cross_entropy(y_true, y_pred)
            masked_loss = loss * mask
            sum_loss = tf.reduce_sum(masked_loss)
            num_non_padding_tokens = tf.reduce_sum(mask)
            self.total_loss.assign_add(sum_loss)
            self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

        def result(self):
            avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
            return tf.exp(avg_loss)

        def reset_state(self):
            self.total_loss.assign(0.)
            self.total_non_padding_tokens.assign(0.)

    class ResponseEmbeddingCosineSimilarity(metrics.Metric):
        def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
            super().__init__(name=name)
            self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
            self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            epsilon = tf.keras.backend.epsilon()
            y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
            y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
            y_true_norm = y_true / (y_true_norm_val + epsilon)
            y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
            cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
            non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
            non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
            masked_cosine_sim = cosine_sim_per_token * non_padding_mask
            num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
            avg_cosine_sim_per_sample = tf.where(
                num_non_zero_tokens > 0,
                tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
                tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
            )
            self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
            self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

        def reset_state(self):
            self.total_cosine_sim.assign(0.)
            self.num_samples.assign(0.)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    def calculate_flops(model, batch_size=moe_params["batch_size"]):
        flops = 0
        functional_model = model.build_graph()

        def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
            layer_flops_count = 0

            if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
                return 0

            if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
                for sub_layer in layer_instance.layers:
                    layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
                return layer_flops_count

            if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
                return 0

            if isinstance(layer_instance, layers.Dense):
                input_dim = layer_instance.input_shape[-1]
                output_dim = layer_instance.output_shape[-1]
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                layer_flops_count += 2 * input_dim * output_dim * effective_batch_size

            elif isinstance(layer_instance, layers.LSTM):
                input_dim = layer_instance.input_shape[-1]
                hidden_dim = layer_instance.units
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
                layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size

            elif isinstance(layer_instance, layers.Embedding):
                pass

            elif isinstance(layer_instance, DeepSeekMoELayer):
                moe_input_dim = layer_instance.input_dim
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
                layer_flops_count += gate_flops
                for expert in layer_instance.experts:
                    layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)

            elif isinstance(layer_instance, layers.MultiHeadAttention):
                config = layer_instance.get_config()
                num_heads = config['num_heads']
                key_dim = config['key_dim']
                d_model = num_heads * key_dim
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
                projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
                attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
                context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
                output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
                layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops

            elif isinstance(layer_instance, layers.TimeDistributed):
                inner_layer = layer_instance.layer
                td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
                layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)

            elif isinstance(layer_instance, TransformerDecoderLayer):
                layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
                layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
                ffn_effective_batch_size = current_batch_size * model.max_seq_length
                layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)

            return layer_flops_count

        for layer in functional_model.layers:
            flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)

        return flops / 1e9

    def get_gpu_stats():
        if nvidia_smi is None:
            return 0, 0
        try:
            nvidia_smi.nvmlInit()
            handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
            gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
            mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
            gpu_memory = mem_info.used / (1024 ** 3)
            nvidia_smi.nvmlShutdown()
            return gpu_load, gpu_memory
        except Exception:
            return 0, 0

    class ResourceMonitor(keras.callbacks.Callback):
        def __init__(self, validation_data):
            super().__init__()
            self.resource_stats = []
            self.expert_load_history = []
            self.start_time = None
            self.epoch_start_time = None
            self.flops = None
            self.validation_data = validation_data
            self.nlu_expert_usage_counts = None
            self.nlg_expert_usage_counts = None
            self.nlu_gate_weights_history = []
            self.nlg_gate_weights_history = []

        def on_train_begin(self, logs=None):
            self.start_time = time.time()
            try:
                self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
                self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
                self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
            except Exception:
                self.flops = 0

        def on_epoch_begin(self, epoch, logs=None):
            self.epoch_start_time = time.time()

        def on_epoch_end(self, epoch, logs=None):
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            epoch_time = time.time() - self.epoch_start_time
            self.resource_stats.append({
                'epoch': epoch + 1,
                'epoch_time': epoch_time,
                'cpu_usage': cpu_usage,
                'memory_used': memory.used / (1024 ** 3),
                'memory_total': memory.total / (1024 ** 3),
                'gpu_load': gpu_load,
                'gpu_memory': gpu_memory,
                'loss': logs.get('loss', 0),
                'val_loss': logs.get('val_loss', 0),
                'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
                'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
                'intent_precision': logs.get('intent_output_intent_precision', 0),
                'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
                'intent_recall': logs.get('intent_output_intent_recall', 0),
                'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
                'intent_f1': logs.get('intent_output_intent_f1', 0),
                'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
                'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
                'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
                'perplexity': logs.get('response_output_perplexity', 0),
                'val_perplexity': logs.get('val_response_output_perplexity', 0),
                'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
                'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
                'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
                'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
            })
            if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
                self.expert_load_history.append({
                    'epoch': epoch + 1,
                    'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                    'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
                })

            sample_size = 100
            for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
                inputs, _ = batch
                outputs = self.model(inputs, training=False)
                nlu_gate_weights = outputs['nlu_gate_weights']
                reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
                self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
                nlg_gate_weights = outputs['nlg_gate_weights']
                reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
                self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

        def on_train_end(self, logs=None):
            self.total_time = time.time() - self.start_time

            if len(self.nlu_gate_weights_history) > 0:
                all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
                nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
                for expert_id in nlu_top_indices:
                    if expert_id < self.model.moe_nlu.total_experts:
                        self.nlu_expert_usage_counts[expert_id] += 1

            if len(self.nlg_gate_weights_history) > 0:
                all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
                nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
                for expert_id in nlg_top_indices:
                    if expert_id < self.model.moe_nlg.total_experts:
                        self.nlg_expert_usage_counts[expert_id] += 1

        def visualize_expert_load_distribution(self):
            total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
            total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
            nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
            nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
            nlu_stability = np.var(nlu_percentages)
            nlg_stability = np.var(nlg_percentages)
            return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

    class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
        def __init__(self, d_model, warmup_steps=4000):
            super().__init__()
            self.d_model = tf.cast(d_model, tf.float32)
            self.warmup_steps = warmup_steps

        def __call__(self, step):
            step = tf.cast(step, tf.float32)
            arg1 = tf.math.rsqrt(step)
            arg2 = step * (self.warmup_steps ** -1.5)
            return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

        def get_config(self):
            return {"d_model": self.d_model.numpy(), "warmup_steps": self.warmup_steps}

    model = MoEModel(moe_params)
    embedding_layer = model.embedding

    def create_validation_split(dataset, embedding_layer, validation_split=0.2, batch_size=moe_params["batch_size"]):
        element_spec = dataset.element_spec
        data_list = [(features, targets) for features, targets in dataset.unbatch().as_numpy_iterator()]
        if not data_list:
            raise ValueError("Dataset is empty.")
        if len(data_list) < 2:
            raise ValueError("Dataset too small to split.")
        train_data, val_data = train_test_split(data_list, test_size=validation_split, shuffle=True)

        def create_dataset_from_list(data):
            if not data:
                raise ValueError("Empty data list provided.")
            features = {
                'user_utterance_tokens': np.array([d[0]['user_utterance_tokens'] for d in data], dtype=np.int32),
                'prev_system_response_tokens': np.array([d[0]['prev_system_response_tokens'] for d in data], dtype=np.int32),
                'decoder_input_tokens': np.array([d[0]['decoder_input_tokens'] for d in data], dtype=np.int32),
                'domain_onehot_input': np.array([d[0]['domain_onehot_input'] for d in data], dtype=np.float32),
                'turn_id_embedding': np.array([d[0]['turn_id_embedding'] for d in data], dtype=np.float32),
                'ontology_multihot_input': np.array([d[0]['ontology_multihot_input'] for d in data], dtype=np.float32),
            }
            response_output = np.array([d[1]['response_output'] for d in data], dtype=np.int32)
            response_embeddings = embedding_layer(response_output).numpy()
            targets = {
                'domain_output': np.array([d[1]['domain_output'] for d in data], dtype=np.int32),
                'intent_output': np.array([d[1]['intent_output'] for d in data], dtype=np.float32),
                'response_output': response_output,
                'response_embeddings': response_embeddings
            }
            return tf.data.Dataset.from_tensor_slices((features, targets)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

        train_dataset = create_dataset_from_list(train_data)
        val_dataset = create_dataset_from_list(val_data)
        return train_dataset, val_dataset

    train_tf_dataset, val_tf_dataset = create_validation_split(train_tf_dataset, embedding_layer)

    losses_dict = {
        'intent_output': losses.BinaryCrossentropy(),
        'domain_output': losses.SparseCategoricalCrossentropy(),
        'response_output': losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss
    }

    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }

    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={'intent_output': 0.5, 'domain_output': 0.5, 'response_output': 1.0, 'response_embeddings': 1.0}
    )

    sample_input = next(iter(train_tf_dataset.take(1)))
    model(sample_input[0])

    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=8, mode='min', restore_best_weights=True)
    resource_monitor = ResourceMonitor(val_tf_dataset)
    class TQDMProgressBar(callbacks.Callback):
        def on_epoch_begin(self, epoch, logs=None):
            self.progress_bar = tqdm(total=len(train_tf_dataset), desc=f'Epoch {epoch + 1}')

        def on_batch_end(self, batch, logs=None):
            self.progress_bar.update(1)

        def on_epoch_end(self, epoch, logs=None):
            self.progress_bar.close()

    tqdm_callback = TQDMProgressBar()

    history = model.fit(
        train_tf_dataset,
        epochs=100,
        validation_data=val_tf_dataset,
        callbacks=[early_stopping, resource_monitor, tqdm_callback],
        verbose=0
    )

    expert_stats = resource_monitor.visualize_expert_load_distribution()
    stats = resource_monitor.resource_stats
    avg_epoch_duration = np.mean([s['epoch_time'] for s in stats]) if stats else 0
    total_training_time = resource_monitor.total_time
    avg_cpu_usage = np.mean([s['cpu_usage'] for s in stats]) if stats else 0
    avg_memory = np.mean([s['memory_used'] for s in stats]) if stats else 0
    avg_gpu_load = np.mean([s['gpu_load'] for s in stats]) if stats else 0
    avg_gpu_memory = np.mean([s['gpu_memory'] for s in stats]) if stats else 0
    avg_flops = resource_monitor.flops if resource_monitor.flops else 0
    nlu_counts = resource_monitor.nlu_expert_usage_counts
    nlg_counts = resource_monitor.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) or 1
    total_nlg = np.sum(nlg_counts) or 1
    expert_nlu_percentages = [(nlu_counts[i] / total_nlu) * 100 for i in range(8)]
    expert_nlg_percentages = [(nlg_counts[i] / total_nlg) * 100 for i in range(8)]
    val_intent_accuracy = np.mean([s['val_intent_accuracy'] for s in stats if s['val_intent_accuracy'] > 0]) if stats else 0
    val_entity_accuracy = np.mean([s['val_domain_accuracy'] for s in stats if s['val_domain_accuracy'] > 0]) if stats else 0
    val_perplexity = np.mean([s['val_perplexity'] for s in stats if s['val_perplexity'] > 0]) if stats else 0
    val_cosine_similarity = np.mean([s['val_cosine_similarity'] for s in stats if s['val_cosine_similarity'] > 0]) if stats and any(s['val_cosine_similarity'] > 0 for s in stats) else 0
    
    results_table['Training Speed (epochs per second)'][f'Iteration {iteration + 1}'] = 1 / avg_epoch_duration if avg_epoch_duration > 0 else 0
    results_table['Training Time (second/epoch)'][f'Iteration {iteration + 1}'] = avg_epoch_duration
    results_table['Total Training Time (second/iteration)'][f'Iteration {iteration + 1}'] = total_training_time
    results_table['Computational Resource Usage'][f'Iteration {iteration + 1}'] = avg_cpu_usage + avg_gpu_load
    results_table['Average CPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_cpu_usage
    results_table['Average GPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_gpu_load
    results_table['Average Memory (GB)'][f'Iteration {iteration + 1}'] = avg_memory
    results_table['Average GPU Memory (GB)'][f'Iteration {iteration + 1}'] = avg_gpu_memory
    results_table['Average FLOPS Estimate (GFLOPS)'][f'Iteration {iteration + 1}'] = avg_flops
    results_table['Expert NLU Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlu_stability']
    results_table['Expert NLG Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlg_stability']
    for i in range(8):
        if i < len(expert_stats['nlu_percentages']):
            results_table[f'Average NLU Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlu_percentages'][i]
        if i < len(expert_stats['nlg_percentages']):
            results_table[f'Average NLG Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlg_percentages'][i]
    results_table['Average Validation Entity Accuracy'][f'Iteration {iteration + 1}'] = val_entity_accuracy
    results_table['Average Intent Accuracy'][f'Iteration {iteration + 1}'] = val_intent_accuracy
    results_table['Average Validation Perplexity'][f'Iteration {iteration + 1}'] = val_perplexity
    results_table['Average Validation Response Cosine Similarity'][f'Iteration {iteration + 1}'] = val_cosine_similarity

    val_loss = min([s['val_loss'] for s in stats]) if stats else float('inf')
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        try:
            model.save(best_model_path, save_format='tf', include_optimizer=True)
        except Exception as e:
            print(f"\nFailed to save best model to {best_model_path}: {str(e)}")

for key in results_table:
    results_table[key]['Average'] = np.mean([results_table[key][f'Iteration {i+1}'] for i in range(10)])

final_result = pd.DataFrame(results_table)
final_result = final_result.T
final_result.to_excel('training_deepseekmoe_topp_adaptive_result.xlsx', index=True)
final_result

2025-07-04 08:19:55.073300: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 08:19:55.181673: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 08:19:55.181905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 08:19:55.182999: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 08:19:55.190726: I tensorflow/core/platform/cpu_feature_guar


Starting Training Iteration 1

Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 08:19:58.495729: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 08:19:58.520323: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 08:19:58.520499: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl


Epoch 1:   0%|          | 0/4428 [00:00<?, ?it/s]2025-07-04 08:20:19.857910: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f4f2c1b4740 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-04 08:20:19.857960: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2025-07-04 08:20:19.867321: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-04 08:20:19.890095: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1751617219.988278   18423 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
Epoch 1: 100%|██████████| 4428/4428 [02:50<00:00, 26.04it/s]


INFO:tensorflow:Assets written to: best_deepseekmoe_topp_adaptive_model/assets


INFO:tensorflow:Assets written to: best_deepseekmoe_topp_adaptive_model/assets


,Iteration 1,Iteration 2,Iteration 3,Iteration 4,Iteration 5,Iteration 6,Iteration 7,Iteration 8,Iteration 9,Iteration 10,Average
Training Speed (epochs per second),0.006023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000602
Training Time (second/epoch),166.036098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.603610
Total Training Time (second/iteration),171.864486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.186449
Computational Resource Usage,34.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.460000
Average CPU Usage (percent),25.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.560000
Average GPU Usage (percent),9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.900000
Average Memory (GB),6.590851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.659085
Average GPU Memory (GB),14.531799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.453180
Average FLOPS Estimate (GFLOPS),2.377871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.237787
Expert NLU Load Distribution Stability,239.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.900000


# DeepSeekMoE with Top-P Routing with Adaptive Gating evaluation code

In [1]:
def load_tf_datasets_from_disk(load_path):
    print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
    try:
        with open(os.path.join(load_path, "moe_params.json"), "r") as f:
            loaded_moe_params = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"moe_params.json not found in {load_path}")

    element_spec = (
        {
            'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
            'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
            'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
        },
        {
            'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
            'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
        }
    )

    try:
        train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
        test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
    except Exception as e:
        raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

    raw_data = {}
    for dataset_name in ['train', 'test']:
        path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
        if os.path.exists(path):
            with open(path, 'rb') as f:
                raw_data[dataset_name] = pickle.load(f)
            print(f"Loaded raw data for {dataset_name} from {path}")
        else:
            print(f"Warning: Raw data file {path} not found.")
            raw_data[dataset_name] = []

    return {
        "train_dataset": train_tf_dataset,
        "test_dataset": test_tf_dataset,
        "moe_params": loaded_moe_params,
        "raw_data": raw_data
    }

tf_dataset_save_path = "tf_datasets"
loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
train_tf_dataset = loaded_data["train_dataset"]
test_tf_dataset = loaded_data["test_dataset"]
moe_params = loaded_data["moe_params"]
raw_data = loaded_data["raw_data"]

moe_params["num_experts"] = 4

class TopPAdaptiveRouter:
    def __init__(self, p=0.9, tau=0.1):
        self.p = p
        self.tau = tau

    def __call__(self, gate_logits):
        probs = tf.nn.softmax(gate_logits, axis=-1)
        sorted_probs, sorted_indices = tf.math.top_k(probs, k=tf.shape(probs)[-1])
        top1_prob = sorted_probs[:, 0]
        top2_prob = sorted_probs[:, 1]
        prob_diff = top1_prob - top2_prob
        use_top1 = prob_diff >= self.tau
        max_k = tf.where(use_top1, 1, 2)
        cum_probs = tf.cumsum(sorted_probs, axis=-1)
        p_mask = cum_probs >= self.p
        first_over_p = tf.argmax(tf.cast(p_mask, tf.int32), axis=-1, output_type=tf.int32)
        first_over_p = tf.maximum(first_over_p, 0)
        num_selected_top_p = first_over_p + 1
        num_selected = tf.minimum(num_selected_top_p, max_k)
        max_selected = tf.reduce_max(num_selected)
        batch_size = tf.shape(gate_logits)[0]
        num_experts = tf.shape(gate_logits)[1]
        selection_mask = tf.sequence_mask(num_selected, maxlen=num_experts, dtype=tf.bool)
        selected_indices = tf.where(selection_mask, sorted_indices, num_experts)
        selected_probs = tf.where(selection_mask, sorted_probs, 0.0)
        selected_indices = selected_indices[:, :max_selected]
        selected_probs = selected_probs[:, :max_selected]
        weights_sum = tf.reduce_sum(selected_probs, axis=-1, keepdims=True) + tf.keras.backend.epsilon()
        weights = selected_probs / weights_sum
        valid_mask = tf.sequence_mask(num_selected, maxlen=max_selected, dtype=tf.int32)
        selected_indices = tf.where(valid_mask == 1, selected_indices, -1)
        return selected_indices, weights, num_selected

class DeepSeekMoELayer(layers.Layer):
    def __init__(self, num_experts, expert_dim, input_dim, m=4, k_s=2, alpha=0.01, top_p=0.9, tau=0.1, name=None):
        super(DeepSeekMoELayer, self).__init__(name=name)
        self.m = m
        self.k_s = k_s
        self.total_experts = num_experts * self.m
        self.routed_experts = self.total_experts - self.k_s
        self.alpha = alpha
        self.top_p = top_p
        self.tau = tau
        self.input_dim = input_dim
        self.seq_length = None
        self.adjusted_expert_dim = expert_dim // self.m
        self.shared_experts = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4), name=f'shared_expert_{i}_dense1'),
                layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4), name=f'shared_expert_{i}_dense2')
            ], name=f'shared_expert_{i}') for i in range(self.k_s)
        ]
        self.routed_experts_list = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4), name=f'routed_expert_{i}_dense1'),
                layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4), name=f'routed_expert_{i}_dense2')
            ], name=f'routed_expert_{i}') for i in range(self.k_s, self.total_experts)
        ]
        self.experts = self.shared_experts + self.routed_experts_list
        self.gate = layers.Dense(self.routed_experts, activation=None, name='gate_weights')
        self.router = TopPAdaptiveRouter(p=self.top_p, tau=self.tau)
        self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

    def call(self, inputs, training=False):
        input_rank = inputs.shape.rank
        if input_rank == 3:
            batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
            flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
            is_3d = True
        else:
            batch_size = tf.shape(inputs)[0]
            seq_length = 1
            flat_inputs = inputs
            is_3d = False
        num_tokens = tf.shape(flat_inputs)[0]
        flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
        shared_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
        for i in range(self.k_s):
            shared_output += self.shared_experts[i](flat_inputs)
        gate_logits = self.gate(flat_inputs)
        expert_indices, expert_weights, num_selected = self.router(gate_logits)
        expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
        expert_indices_adjusted = expert_indices + self.k_s
        batch_indices = tf.tile(tf.expand_dims(tf.range(num_tokens), 1), [1, tf.shape(expert_indices)[1]])
        gather_indices = tf.stack([batch_indices, expert_indices_adjusted], axis=-1)
        valid_mask = tf.cast(expert_indices_adjusted >= 0, tf.float32)
        valid_mask = tf.expand_dims(valid_mask, -1)
        selected_outputs = tf.gather_nd(expert_outputs, gather_indices)
        weighted_outputs = selected_outputs * tf.expand_dims(expert_weights, -1) * valid_mask
        routed_output = tf.reduce_sum(weighted_outputs, axis=1)
        output_flat = shared_output + routed_output + flat_inputs
        if is_3d:
            output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
            if self.seq_length is not None:
                output.set_shape([None, self.seq_length, self.input_dim])
            gate_weights_reshaped = tf.reshape(tf.nn.softmax(gate_logits, axis=-1), [batch_size, seq_length, self.routed_experts])
            if self.seq_length is not None:
                gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
        else:
            output = output_flat
            gate_weights_reshaped = tf.nn.softmax(gate_logits, axis=-1)
        gate_probs = tf.nn.softmax(gate_logits, axis=-1)
        expert_selection = tf.reduce_sum(tf.one_hot(expert_indices_adjusted, depth=self.total_experts), axis=1)
        expert_selection = expert_selection[:, self.k_s:]
        f_i = tf.reduce_mean(expert_selection, axis=0)
        P_i = tf.reduce_mean(gate_probs, axis=0)
        load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
        self.add_loss(load_balancing_loss)
        self.load_balancing_loss_metric.update_state(load_balancing_loss)
        return output, gate_weights_reshaped

    def get_metrics(self):
        return {f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric}

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_experts': self.total_experts // self.m,
            'expert_dim': self.adjusted_expert_dim * self.m,
            'input_dim': self.input_dim,
            'm': self.m,
            'k_s': self.k_s,
            'alpha': self.alpha,
            'top_p': self.top_p,
            'tau': self.tau,
            'name': self.name
        })
        return config

class TransformerDecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.ffn = keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask=None):
        if look_ahead_mask is not None:
            look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
            look_ahead_mask = 1.0 - look_ahead_mask
        attn1_output = self.mha1(query=x, value=x, key=x, attention_mask=look_ahead_mask, training=training)
        attn1 = self.dropout1(attn1_output, training=training)
        out1 = self.layernorm1(attn1 + x)
        attn2_output = self.mha2(query=out1, value=enc_output, key=enc_output, training=training)
        attn2 = self.dropout2(attn2_output, training=training)
        out2 = self.layernorm2(attn2 + out1)
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)
        return out3

    def get_config(self):
        config = super().get_config()
        mha1_config = self.mha1.get_config()
        config.update({
            'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
            'num_heads': mha1_config['num_heads'],
            'dff': self.ffn.layers[0].units,
            'rate': self.dropout1.rate
        })
        return config

class MoEModel(models.Model):
    def __init__(self, moe_params):
        super(MoEModel, self).__init__()
        self.embedding_dim = moe_params["embedding_dim"]
        self.max_seq_length = moe_params["max_seq_length"]
        self.num_domains = moe_params["num_domains"]
        self.num_intents = moe_params["num_intents"]
        self.vocab_size = moe_params["vocab_size"]
        self.num_experts = moe_params["num_experts"]
        self.expert_dim = moe_params["expert_dim"]
        self.turn_id_dim = moe_params["turn_id_dim"]
        self.num_heads = 4
        self.dff = self.embedding_dim * 4
        self.dropout_rate = 0.1
        self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                             self.num_domains + self.turn_id_dim + self.num_intents)
        self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains

        self.embedding = layers.Embedding(
            self.vocab_size,
            self.embedding_dim,
            mask_zero=True,
            embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05),
            name="embedding"
        )
        self.embedding.build((None,))
        with tf.init_scope():
            embedding_weights = self.embedding.get_weights()
            if embedding_weights and len(embedding_weights) > 0:
                embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                self.embedding.set_weights(embedding_weights)

        self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
        self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
        self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
        self.decoder_dropout = layers.Dropout(self.dropout_rate)
        self.moe_nlu = DeepSeekMoELayer(
            num_experts=self.num_experts,
            expert_dim=self.expert_dim,
            input_dim=self.nlu_input_dim,
            m=2,
            k_s=2,
            alpha=0.01,
            top_p=0.9,
            tau=0.1,
            name='moe_nlu'
        )
        self.moe_nlg = DeepSeekMoELayer(
            num_experts=self.num_experts,
            expert_dim=self.expert_dim,
            input_dim=self.nlg_input_dim,
            m=2,
            k_s=2,
            alpha=0.01,
            top_p=0.9,
            tau=0.1,
            name='moe_nlg'
        )
        self.intent_output = layers.Dense(self.num_intents, activation='sigmoid', name='intent_output')
        self.domain_output = layers.Dense(self.num_domains, activation='softmax', name='domain_output')
        self.response_output = layers.TimeDistributed(
            layers.Dense(self.vocab_size, activation='softmax'), name='response_output'
        )
        self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)
        self.nlg_projection = layers.TimeDistributed(
            layers.Dense(self.embedding_dim, activation='relu', name='nlg_projection')
        )

    def create_look_ahead_mask(self, size):
        mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
        return mask

    def call(self, inputs, training=False):
        user_utterance_tokens = inputs['user_utterance_tokens']
        prev_system_response_tokens = inputs['prev_system_response_tokens']
        decoder_input_tokens = inputs['decoder_input_tokens']
        domain_onehot_input = inputs['domain_onehot_input']
        turn_id_embedding = inputs['turn_id_embedding']
        ontology_multihot_input = inputs['ontology_multihot_input']

        pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
        user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
        prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
        decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)

        user_enc_out = user_embedded
        prev_system_enc_out = prev_system_embedded

        look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
        dec_output = decoder_embedded
        for i, layer in enumerate(self.decoder_layers):
            dec_output = layer(dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask)
        decoder_output = self.decoder_dropout(dec_output, training=training)

        user_attn_out = self.attn_layer(
            query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), 
            value=user_enc_out, key=user_enc_out, training=training
        )
        prev_system_attn_out = self.attn_layer(
            query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), 
            value=prev_system_enc_out, key=prev_system_enc_out, training=training
        )
        decoder_attn_out = self.attn_layer(
            query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), 
            value=decoder_output, key=decoder_output, training=training
        )

        combined_features = layers.Concatenate()([
            tf.squeeze(user_attn_out, axis=1),
            tf.squeeze(prev_system_attn_out, axis=1),
            tf.squeeze(decoder_attn_out, axis=1),
            domain_onehot_input,
            turn_id_embedding,
            ontology_multihot_input
        ])
        combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
        nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
        nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
        intent_out = self.intent_output(nlu_out)
        domain_out = self.domain_output(nlu_out)

        intent_out_expanded = tf.expand_dims(intent_out, axis=1)
        domain_out_expanded = tf.expand_dims(domain_out, axis=1)
        intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
        domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])

        nlg_input = layers.Concatenate(axis=-1)([
            decoder_output,
            intent_out_tiled,
            domain_out_tiled
        ])
        nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])

        nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
        nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
        response_embeddings = self.nlg_projection(nlg_out)
        response_embeddings = tf.ensure_shape(response_embeddings, [None, self.max_seq_length, self.embedding_dim])
        response_out = self.response_output(nlg_out)

        return {
            'intent_output': intent_out,
            'domain_output': domain_out,
            'response_output': response_out,
            'response_embeddings': response_embeddings,
            'nlu_gate_weights': nlu_gate_weights,
            'nlg_gate_weights': nlg_gate_weights
        }

    def build_graph(self):
        inputs = {
            'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='user_utterance_tokens'),
            'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='prev_system_response_tokens'),
            'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='decoder_input_tokens'),
            'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32, name='domain_onehot_input'),
            'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32, name='turn_id_embedding'),
            'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32, name='ontology_multihot_input'),
        }
        return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

    def get_config(self):
        config = super().get_config()
        config.update({
            'moe_params': {
                'embedding_dim': self.embedding_dim,
                'max_seq_length': self.max_seq_length,
                'num_domains': self.num_domains,
                'num_intents': self.num_intents,
                'vocab_size': self.vocab_size,
                'num_experts': self.num_experts,
                'expert_dim': self.expert_dim,
                'turn_id_dim': self.turn_id_dim
            }
        })
        return config

class IntentAccuracy(metrics.Metric):
    def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros', dtype=tf.float32)
        self.total_preds = self.add_weight(name='total_preds', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
        self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
        self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

    def reset_state(self):
        self.correct_preds.assign(0.)
        self.total_preds.assign(0.)

class IntentPrecision(metrics.Metric):
    def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
        self.predicted_positives = self.add_weight(name='pp', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        predicted_positives = tf.reduce_sum(y_pred)
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)

    def result(self):
        return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.predicted_positives.assign(0.)

class IntentRecall(metrics.Metric):
    def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
        self.actual_positives = self.add_weight(name='ap', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        actual_positives = tf.reduce_sum(y_true)
        self.true_positives.assign_add(true_positives)
        self.actual_positives.assign_add(actual_positives)

    def result(self):
        return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.actual_positives.assign(0.)

class IntentF1(metrics.Metric):
    def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = IntentPrecision(threshold=threshold)
        self.recall = IntentRecall(threshold=threshold)

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

class DomainAccuracy(metrics.Metric):
    def __init__(self, name='domain_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.accuracy = self.add_weight(name='acc', initializer='zeros', dtype=tf.float32)
        self.count = self.add_weight(name='count', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
        pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
        matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
        self.accuracy.assign_add(tf.reduce_sum(matches))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.accuracy / (self.count + tf.keras.backend.epsilon())

    def reset_state(self):
        self.accuracy.assign(0.)
        self.count.assign(0.)

class Perplexity(metrics.Metric):
    def __init__(self, name='perplexity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
        self.total_loss = self.add_weight(name='total_loss', initializer='zeros', dtype=tf.float32)
        self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        mask = tf.cast(y_true != 0, dtype=tf.float32)
        loss = self.cross_entropy(y_true, y_pred)
        masked_loss = loss * mask
        sum_loss = tf.reduce_sum(masked_loss)
        num_non_padding_tokens = tf.reduce_sum(mask)
        self.total_loss.assign_add(sum_loss)
        self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

    def result(self):
        avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
        return tf.exp(avg_loss)

    def reset_state(self):
        self.total_loss.assign(0.)
        self.total_non_padding_tokens.assign(0.)

class ResponseEmbeddingCosineSimilarity(metrics.Metric):
    def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
        super().__init__(name=name)
        self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
        self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true_norm = y_true / (y_true_norm_val + epsilon)
        y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
        cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
        non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
        non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
        masked_cosine_sim = cosine_sim_per_token * non_padding_mask
        num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
        avg_cosine_sim_per_sample = tf.where(
            num_non_zero_tokens > 0,
            tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
            tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
        )
        self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
        self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

    def reset_state(self):
        self.total_cosine_sim.assign(0.)
        self.num_samples.assign(0.)

def contrastive_loss(y_true, y_pred, margin=1.0):
    epsilon = tf.keras.backend.epsilon()
    y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
    y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
    y_true = y_true / (y_true_norm + epsilon)
    y_pred = y_pred / (y_pred_norm + epsilon)
    cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
    positive_loss = 1.0 - cosine_sim
    mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
    masked_loss = positive_loss * mask
    total_loss = tf.reduce_sum(masked_loss)
    num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
    return total_loss / num_tokens

def calculate_flops(model, batch_size=moe_params["batch_size"]):
    flops = 0
    functional_model = model.build_graph()

    def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
        layer_flops_count = 0

        if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
            return 0

        if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
            for sub_layer in layer_instance.layers:
                layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
            return layer_flops_count

        if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
            return 0

        if isinstance(layer_instance, layers.Dense):
            input_dim = layer_instance.input_shape[-1]
            output_dim = layer_instance.output_shape[-1]
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            layer_flops_count += 2 * input_dim * output_dim * effective_batch_size

        elif isinstance(layer_instance, layers.LSTM):
            input_dim = layer_instance.input_shape[-1]
            hidden_dim = layer_instance.units
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
            layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size

        elif isinstance(layer_instance, layers.Embedding):
            pass

        elif isinstance(layer_instance, DeepSeekMoELayer):
            moe_input_dim = layer_instance.input_dim
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
            layer_flops_count += gate_flops
            for expert in layer_instance.experts:
                layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)

        elif isinstance(layer_instance, layers.MultiHeadAttention):
            config = layer_instance.get_config()
            num_heads = config['num_heads']
            key_dim = config['key_dim']
            d_model = num_heads * key_dim
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
            projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
            attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
            context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
            output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
            layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops

        elif isinstance(layer_instance, layers.TimeDistributed):
            inner_layer = layer_instance.layer
            td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
            layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)

        elif isinstance(layer_instance, TransformerDecoderLayer):
            layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
            layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
            ffn_effective_batch_size = current_batch_size * model.max_seq_length
            layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)

        return layer_flops_count

    for layer in functional_model.layers:
        flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)

    return flops / 1e9

def get_gpu_stats():
    if nvidia_smi is None:
        return 0, 0
    try:
        nvidia_smi.nvmlInit()
        handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
        gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
        mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
        gpu_memory = mem_info.used / (1024 ** 3)
        nvidia_smi.nvmlShutdown()
        return gpu_load, gpu_memory
    except Exception:
        return 0, 0

class ResourceMonitor(keras.callbacks.Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.resource_stats = []
        self.expert_load_history = []
        self.start_time = None
        self.epoch_start_time = None
        self.flops = None
        self.validation_data = validation_data
        self.nlu_expert_usage_counts = None
        self.nlg_expert_usage_counts = None
        self.nlu_gate_weights_history = []
        self.nlg_gate_weights_history = []

    def on_train_begin(self, logs=None):
        self.start_time = time.time()
        try:
            self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
            self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
            self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
        except Exception:
            self.flops = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory()
        gpu_load, gpu_memory = get_gpu_stats()
        epoch_time = time.time() - self.epoch_start_time
        self.resource_stats.append({
            'epoch': epoch + 1,
            'epoch_time': epoch_time,
            'cpu_usage': cpu_usage,
            'memory_used': memory.used / (1024 ** 3),
            'memory_total': memory.total / (1024 ** 3),
            'gpu_load': gpu_load,
            'gpu_memory': gpu_memory,
            'loss': logs.get('loss', 0),
            'val_loss': logs.get('val_loss', 0),
            'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
            'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
            'intent_precision': logs.get('intent_output_intent_precision', 0),
            'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
            'intent_recall': logs.get('intent_output_intent_recall', 0),
            'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
            'intent_f1': logs.get('intent_output_intent_f1', 0),
            'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
            'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
            'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
            'perplexity': logs.get('response_output_perplexity', 0),
            'val_perplexity': logs.get('val_response_output_perplexity', 0),
            'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
            'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
            'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
            'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
        })
        if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
            self.expert_load_history.append({
                'epoch': epoch + 1,
                'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
            })

        sample_size = 100
        for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
            inputs, _ = batch
            outputs = self.model(inputs, training=False)
            nlu_gate_weights = outputs['nlu_gate_weights']
            reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
            self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
            nlg_gate_weights = outputs['nlg_gate_weights']
            reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
            self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

    def on_train_end(self, logs=None):
        self.total_time = time.time() - self.start_time

        if len(self.nlu_gate_weights_history) > 0:
            all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
            nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
            for expert_id in nlu_top_indices:
                if expert_id < self.model.moe_nlu.total_experts:
                    self.nlu_expert_usage_counts[expert_id] += 1

        if len(self.nlg_gate_weights_history) > 0:
            all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
            nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
            for expert_id in nlg_top_indices:
                if expert_id < self.model.moe_nlg.total_experts:
                    self.nlg_expert_usage_counts[expert_id] += 1

    def visualize_expert_load_distribution(self):
        total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
        total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
        nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
        nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
        nlu_stability = np.var(nlu_percentages)
        nlg_stability = np.var(nlg_percentages)
        return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {"d_model": self.d_model.numpy(), "warmup_steps": self.warmup_steps}

word_to_id = {}
id_to_word = {}
try:
    possible_paths = [
        os.path.join("preprocessor_models", "preprocessor_params.json"),
        os.path.join("tf_datasets", "preprocessor_params.json"),
        "preprocessor_params.json"
    ]
    
    vocab_loaded = False
    for path in possible_paths:
        if os.path.exists(path):
            with open(path, "r") as f:
                preprocessor_params = json.load(f)
            word_to_id = {k: int(v) for k, v in preprocessor_params['word_to_id'].items()}
            id_to_word = {int(k): v for k, v in preprocessor_params['id_to_word'].items()}
            print(f"Loaded vocabulary from {path}")
            vocab_loaded = True
            break
    
    if not vocab_loaded:
        raise FileNotFoundError("Vocabulary file not found in any location")
except Exception as e:
    print(f"Warning: Could not load vocabulary from preprocessor: {str(e)}")
    word_to_id = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
    id_to_word = {0: '<pad>', 1: '<sos>', 2: '<eos>', 3: '<unk>'}

model_save_path = "best_deepseekmoe_topp_adaptive_model"
print(f"\nLoading model from: {model_save_path}")
custom_objects = {
    "TopPAdaptiveRouter":TopPAdaptiveRouter,
    "DeepSeekMoELayer": DeepSeekMoELayer,
    "CustomLearningRateSchedule": CustomLearningRateSchedule,
    "MoEModel": MoEModel,
    "TransformerDecoderLayer": TransformerDecoderLayer,
    "IntentAccuracy": IntentAccuracy,
    "IntentPrecision": IntentPrecision,
    "IntentRecall": IntentRecall,
    "IntentF1": IntentF1,
    "DomainAccuracy": DomainAccuracy,
    "Perplexity": Perplexity,
    "ResponseEmbeddingCosineSimilarity": ResponseEmbeddingCosineSimilarity
}
try:
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
except Exception as e:
    print(f"Failed to load model from {model_save_path}: {str(e)}")
    raise

def compile_model(model, moe_params):
    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    losses_dict = {
        'intent_output': tf.keras.losses.BinaryCrossentropy(),
        'domain_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss,
    }

    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }

    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={
            'intent_output': 0.5,
            'domain_output': 0.5,
            'response_output': 1.0,
            'response_embeddings': 1.0
        }
    )
    return model

model = compile_model(model, moe_params)
print("\nModel loaded and compiled successfully!")
sample_input = next(iter(train_tf_dataset.take(1)))
model(sample_input[0])
model.summary()

class EvaluationMetrics:
    def __init__(self, model, test_dataset, moe_params, raw_data, word_to_id, id_to_word):
        self.model = model
        self.test_dataset = test_dataset
        self.moe_params = moe_params
        self.raw_data = raw_data
        self.word_to_id = word_to_id
        self.id_to_word = id_to_word
        self.nlu_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.nlg_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.resource_stats = {
            'cpu_usage': [],
            'memory_used': [],
            'gpu_load': [],
            'gpu_memory': [],
            'batch_times': []
        }
        self.special_tokens = {'<pad>', '<sos>', '<eos>', '<unk>'}
        self.nlu_top_k = model.get_layer('moe_nlu').get_config().get('top_k', 2)
        self.nlg_top_k = model.get_layer('moe_nlg').get_config().get('top_k', 2)

    def evaluate(self):
        print("\nEvaluating model on test set...")
        
        def add_response_embeddings(features, targets):
            response_tokens = targets['response_output']
            response_embeddings = self.model.embedding(response_tokens)
            targets['response_embeddings'] = response_embeddings
            return features, targets
        
        test_dataset_with_embeddings = self.test_dataset.map(add_response_embeddings, num_parallel_calls=tf.data.AUTOTUNE)

        start_time = time.time()
        test_results = self.model.evaluate(test_dataset_with_embeddings, verbose=1)
        eval_time = time.time() - start_time
        
        metric_names = self.model.metrics_names
        
        perplexity_value = None
        for name, value in zip(metric_names, test_results):
            if name == 'response_output_perplexity':
                perplexity_value = value
                break
        if perplexity_value is None:
            print("Warning: Could not find perplexity in standard evaluation results.")
            perplexity_value = 0.0
        
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory().used / (1024 ** 3)
        gpu_load, gpu_memory = get_gpu_stats()
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))

        total_tokens = 0
        total_time = 0
        num_batches = 0
        
        test_dataset_small = test_dataset_with_embeddings.unbatch().batch(moe_params["batch_size"])
        
        all_true_intents = []
        all_pred_intents = []
        all_true_domains = []
        all_pred_domains = []
        
        for batch_idx, batch in enumerate(test_dataset_small):
            inputs, targets = batch
            response_tokens = targets['response_output'].numpy()
            non_padding_mask = response_tokens != 0
            total_tokens += np.sum(non_padding_mask)
            
            start_time = time.time()
            outputs = self.model(inputs, training=False)
            batch_time = time.time() - start_time
            
            total_time += batch_time
            num_batches += tf.shape(inputs['user_utterance_tokens'])[0]
            
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            
            self.resource_stats['cpu_usage'].append(cpu_usage)
            self.resource_stats['memory_used'].append(memory.used / (1024 ** 3))
            self.resource_stats['gpu_load'].append(gpu_load)
            self.resource_stats['gpu_memory'].append(gpu_memory)
            self.resource_stats['batch_times'].append(batch_time)
            
            nlu_gate_weights = outputs.get('nlu_gate_weights', tf.zeros([0, self.model.moe_nlu.routed_experts])).numpy()
            nlu_top_indices = np.argsort(-nlu_gate_weights, axis=-1)[:, :self.nlu_top_k] + self.model.moe_nlu.k_s
            for idx in range(self.nlu_top_k):
                expert_ids = nlu_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlu_expert_usage_counts):
                        self.nlu_expert_usage_counts[expert_id] += 1
            
            nlg_gate_weights = outputs.get('nlg_gate_weights', tf.zeros([0, self.model.moe_nlg.routed_experts])).numpy()
            if len(nlg_gate_weights.shape) == 3:
                nlg_gate_weights = nlg_gate_weights.reshape(-1, nlg_gate_weights.shape[-1])
            nlg_top_indices = np.argsort(-nlg_gate_weights, axis=-1)[:, :self.nlg_top_k] + self.model.moe_nlg.k_s
            for idx in range(self.nlg_top_k):
                expert_ids = nlg_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlg_expert_usage_counts):
                        self.nlg_expert_usage_counts[expert_id] += 1
            
            true_intents = targets.get('intent_output', np.zeros((moe_params["batch_size"], self.model.num_intents))).numpy()
            pred_intents = (outputs.get('intent_output', np.zeros_like(true_intents)).numpy() > 0.5).astype(np.int32)
            all_true_intents.append(true_intents)
            all_pred_intents.append(pred_intents)
            
            true_domains = targets.get('domain_output', np.zeros((moe_params["batch_size"], 1))).numpy().flatten()
            pred_domains = np.argmax(outputs.get('domain_output', np.zeros((moe_params["batch_size"], self.model.num_domains))).numpy(), axis=-1)
            all_true_domains.append(true_domains)
            all_pred_domains.append(pred_domains)
            
            if batch_idx % 50 == 0:
                gc.collect()
                tf.keras.backend.clear_session()
        
        avg_time_per_token = (total_time / total_tokens) * 1000 if total_tokens > 0 else 0
        
        avg_cpu = np.mean(self.resource_stats['cpu_usage']) if self.resource_stats['cpu_usage'] else 0
        avg_memory = np.mean(self.resource_stats['memory_used']) if self.resource_stats['memory_used'] else 0
        avg_gpu_load = np.mean(self.resource_stats['gpu_load']) if self.resource_stats['gpu_load'] else 0
        avg_gpu_memory = np.mean(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        peak_gpu_memory = np.max(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))
        
        all_true_intents = np.vstack(all_true_intents)
        all_pred_intents = np.vstack(all_pred_intents)
        all_true_domains = np.concatenate(all_true_domains)
        all_pred_domains = np.concatenate(all_pred_domains)
        
        intent_f1_score = f1_score(all_true_intents, all_pred_intents, average='micro')
        intent_f1_score_macro = f1_score(all_true_intents, all_pred_intents, average='macro')
        intent_f1_score_weighted = f1_score(all_true_intents, all_pred_intents, average='weighted')
        intent_accuracy_score = accuracy_score(all_true_intents, all_pred_intents)
        
        domain_accuracy_score = accuracy_score(all_true_domains, all_pred_domains)
        domain_f1_score_macro = f1_score(all_true_domains, all_pred_domains, average='macro')

        hypotheses = []
        references = []
        meteor_scores = []
        rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        
        domain_metrics = defaultdict(lambda: {
            'bleu': [],
            'rouge1': [],
            'rouge2': [],
            'rougeL': [],
            'meteor': [],
            'count': 0
        })
        
        if not self.raw_data.get('test'):
            print("Warning: raw_data['test'] is empty. Skipping NLG metrics calculation.")
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0
            domain_metrics = {}
        else:
            for i, batch in enumerate(test_dataset_small):
                if i >= len(self.raw_data.get('test', [])):
                    print(f"Warning: raw_data['test'] has fewer items ({len(self.raw_data.get('test', []))}) than test dataset. Stopping NLG metrics calculation.")
                    break
                
                try:
                    inputs, targets = batch
                    with tf.device('/CPU:0'):
                        outputs = self.model(inputs, training=False)
                    
                    domain_id = targets['domain_output'].numpy()[0][0]
                    domain_name = f"domain_{domain_id}"
                    
                    response_probs = outputs['response_output'].numpy()
                    generated_tokens = np.argmax(response_probs, axis=-1)[0]
                    true_tokens = targets['response_output'].numpy()[0]
                    
                    raw_item = self.raw_data['test'][i]
                    ref_text = raw_item.get('raw_next_system_response', '')
                    if not ref_text:
                        continue
                    
                    gen_text = self.tokens_to_text(generated_tokens)
                    ref_tokens = self.tokens_to_text(true_tokens)
                    
                    if not gen_text.strip() or not ref_text.strip():
                        continue
                    
                    hypotheses.append(gen_text)
                    references.append([ref_text])
                    
                    ref_tokens = word_tokenize(ref_text.lower())
                    gen_tokens = word_tokenize(gen_text.lower())
                    
                    try:
                        meteor = meteor_score([ref_tokens], gen_tokens)
                        meteor_scores.append(meteor)
                        domain_metrics[domain_name]['meteor'].append(meteor)
                    except Exception as e:
                        print(f"METEOR calculation error for example {i}: {str(e)}")
                        meteor_scores.append(0)
                        domain_metrics[domain_name]['meteor'].append(0)
                    
                    try:
                        rouge_scores = rouge_scorer_instance.score(ref_text, gen_text)
                        domain_metrics[domain_name]['rouge1'].append(rouge_scores['rouge1'].fmeasure)
                        domain_metrics[domain_name]['rouge2'].append(rouge_scores['rouge2'].fmeasure)
                        domain_metrics[domain_name]['rougeL'].append(rouge_scores['rougeL'].fmeasure)
                    except Exception as e:
                        print(f"ROUGE calculation error for example {i}: {str(e)}")
                    
                    domain_metrics[domain_name]['count'] += 1
                    
                    if i % 50 == 0:
                        gc.collect()
                        tf.keras.backend.clear_session()
                    
                except Exception as e:
                    print(f"Error processing example {i}: {str(e)}")
                    continue
        
        if hypotheses and references:
            hypotheses_tok = []
            references_tok = []
            
            for hyp, ref_list in zip(hypotheses, references):
                if not hyp or not ref_list[0]:
                    continue
                hyp_tokens = word_tokenize(hyp.lower())
                ref_tokens = [word_tokenize(ref.lower()) for ref in ref_list]
                hypotheses_tok.append(hyp_tokens)
                references_tok.append(ref_tokens)
            
            chencherry = SmoothingFunction()
            try:
                bleu_score = corpus_bleu(references_tok, hypotheses_tok, smoothing_function=chencherry.method1)
            except Exception as e:
                print(f"BLEU calculation error: {str(e)}")
                bleu_score = 0.0
            
            rouge_scores = []
            for hyp, ref in zip(hypotheses, references):
                if hyp and ref[0]:
                    try:
                        rouge_scores.append(rouge_scorer_instance.score(ref[0], hyp))
                    except Exception as e:
                        print(f"ROUGE scoring error: {str(e)}")
                        continue
            
            avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_meteor = np.mean(meteor_scores) if meteor_scores else 0.0
        else:
            print("Warning: No valid hypotheses or references for NLG metrics. Setting to 0.0.")
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0

        return {
            'intent_f1_micro': intent_f1_score,
            'intent_f1_macro': intent_f1_score_macro,
            'intent_f1_weighted': intent_f1_score_weighted,
            'intent_accuracy': intent_accuracy_score,
            'domain_accuracy': domain_accuracy_score,
            'domain_f1_macro': domain_f1_score_macro,
            'bleu': bleu_score,
            'rouge1': avg_rouge1,
            'rouge2': avg_rouge2,
            'rougeL': avg_rougeL,
            'meteor': avg_meteor,
            'perplexity': perplexity_value,
            'pred_speed': avg_time_per_token,
            'domain_metrics': domain_metrics,
            'eval_time': eval_time
        }
    
    def tokens_to_text(self, tokens):
        words = []
        for token in tokens:
            if token == 0:
                continue
            word = self.id_to_word.get(token, '<unk>')
            if word not in self.special_tokens:
                words.append(word)
        return " ".join(words).strip()

results_table = {
    'Evaluation Time (seconds)': [],
    'Prediction Speed (ms/token)': [],
    'Average CPU Usage (percent)': [],
    'Average GPU Usage (percent)': [],
    'Average Memory (GB)': [],
    'Average GPU Memory (GB)': [],
    'Average FLOPs Estimate (GFLOPs)': [],
    'NLU Expert 1 (percent)': [],
    'NLU Expert 2 (percent)': [],
    'NLU Expert 3 (percent)': [],
    'NLU Expert 4 (percent)': [],
    'NLU Expert 5 (percent)': [],
    'NLU Expert 6 (percent)': [],
    'NLU Expert 7 (percent)': [],
    'NLU Expert 8 (percent)': [],
    'NLG Expert 1 (percent)': [],
    'NLG Expert 2 (percent)': [],
    'NLG Expert 3 (percent)': [],
    'NLG Expert 4 (percent)': [],
    'NLG Expert 5 (percent)': [],
    'NLG Expert 6 (percent)': [],
    'NLG Expert 7 (percent)': [],
    'NLG Expert 8 (percent)': [],
    'Intent F1-score (Micro)': [],
    'Intent F1-score (Macro)': [],
    'Intent F1-score (Weighted)': [],
    'Intent Accuracy': [],
    'Domain Accuracy': [],
    'Domain F1-score (Macro)': [],
    'BLEU Score': [],
    'ROUGE-1 F1': [],
    'ROUGE-2 F1': [],
    'ROUGE-L F1': [],
    'METEOR Score': [],
    'Perplexity': [],
    'NLU Expert Load Stability (Variance)': [],
    'NLG Expert Load Stability (Variance)': []
}

num_iterations = 5
for iteration in range(1, num_iterations + 1):
    print(f"\nStarting Iteration {iteration}")
    tf.keras.backend.clear_session()
    gc.collect()
    
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
    model = compile_model(model, moe_params)
    
    evaluator = EvaluationMetrics(model, test_tf_dataset, moe_params, raw_data, word_to_id, id_to_word)

    evaluation_results = evaluator.evaluate()
    results_table['Evaluation Time (seconds)'].append(evaluation_results['eval_time'])
    results_table['Prediction Speed (ms/token)'].append(evaluation_results['pred_speed'])
    results_table['Average CPU Usage (percent)'].append(np.mean(evaluator.resource_stats['cpu_usage']) if evaluator.resource_stats['cpu_usage'] else 0)
    results_table['Average GPU Usage (percent)'].append(np.mean(evaluator.resource_stats['gpu_load']) if evaluator.resource_stats['gpu_load'] else 0)
    results_table['Average Memory (GB)'].append(np.mean(evaluator.resource_stats['memory_used']) if evaluator.resource_stats['memory_used'] else 0)
    results_table['Average GPU Memory (GB)'].append(np.mean(evaluator.resource_stats['gpu_memory']) if evaluator.resource_stats['gpu_memory'] else 0)
    results_table['Average FLOPs Estimate (GFLOPs)'].append(calculate_flops(model))
    nlu_counts = evaluator.nlu_expert_usage_counts
    nlg_counts = evaluator.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) if np.sum(nlu_counts) > 0 else 1
    total_nlg = np.sum(nlg_counts) if np.sum(nlg_counts) > 0 else 1
    results_table['NLU Expert 1 (percent)'].append((nlu_counts[0] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 2 (percent)'].append((nlu_counts[1] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 3 (percent)'].append((nlu_counts[2] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 4 (percent)'].append((nlu_counts[3] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 5 (percent)'].append((nlu_counts[4] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 6 (percent)'].append((nlu_counts[5] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 7 (percent)'].append((nlu_counts[6] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 8 (percent)'].append((nlu_counts[7] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLG Expert 1 (percent)'].append((nlg_counts[0] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 2 (percent)'].append((nlg_counts[1] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 3 (percent)'].append((nlg_counts[2] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 4 (percent)'].append((nlg_counts[3] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 5 (percent)'].append((nlg_counts[4] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 6 (percent)'].append((nlg_counts[5] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 7 (percent)'].append((nlg_counts[6] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 8 (percent)'].append((nlg_counts[7] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['Intent F1-score (Micro)'].append(evaluation_results['intent_f1_micro'])
    results_table['Intent F1-score (Macro)'].append(evaluation_results['intent_f1_macro'])
    results_table['Intent F1-score (Weighted)'].append(evaluation_results['intent_f1_weighted'])
    results_table['Intent Accuracy'].append(evaluation_results['intent_accuracy'])
    results_table['Domain Accuracy'].append(evaluation_results['domain_accuracy'])
    results_table['Domain F1-score (Macro)'].append(evaluation_results['domain_f1_macro'])
    results_table['BLEU Score'].append(evaluation_results['bleu'])
    results_table['ROUGE-1 F1'].append(evaluation_results['rouge1'])
    results_table['ROUGE-2 F1'].append(evaluation_results['rouge2'])
    results_table['ROUGE-L F1'].append(evaluation_results['rougeL'])
    results_table['METEOR Score'].append(evaluation_results['meteor'])
    results_table['Perplexity'].append(evaluation_results['perplexity'])
    results_table['NLU Expert Load Stability (Variance)'].append(np.var(nlu_counts / total_nlu * 100) if total_nlu > 0 else 0)
    results_table['NLG Expert Load Stability (Variance)'].append(np.var(nlg_counts / total_nlg * 100) if total_nlg > 0 else 0)

for key in results_table:
    if results_table[key]:
        results_table[key].append(np.mean(results_table[key]))
    else:
        results_table[key].append(0)

print("\nTraining results saved")
final_df = pd.DataFrame(results_table)
final_df = final_df.T
final_df.columns = [f'Iteration {i+1}' for i in range(num_iterations)] + ['Average']
final_df.to_excel('prediction_deepseekmoe_topp_adaptive_results.xlsx', index=False)
final_df

2025-07-04 08:25:30.365036: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 08:25:30.786695: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 08:25:30.786846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 08:25:30.866476: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 08:25:31.029536: I tensorflow/core/platform/cpu_feature_guar


Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 08:25:38.558679: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 08:25:38.830601: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 08:25:38.830824: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl
Loaded vocabulary from preprocessor_models/preprocessor_params.json

Loading model from: best_deepseekmoe_topp_adaptive_model

Model loaded and compiled successfully!
Model: "mo_e_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  784896    
                                                                 
 embedding (Embedding)       multiple                  8576      
                                                                 
 layer_normalization (Layer  multiple                  256       
 Normalization)                                                  
                                                                 
 transformer_decoder_layer   multiple                  264576    
 (TransformerDecoderLayer)     

,Iteration 1,Iteration 2,Average
Evaluation Time (seconds),21.247510,18.251247,19.749379
Prediction Speed (ms/token),2.205226,2.275832,2.240529
Average CPU Usage (percent),22.516535,24.888189,23.702362
Average GPU Usage (percent),1.853973,1.863278,1.858626
Average Memory (GB),5.649051,10.894524,8.271787
Average GPU Memory (GB),14.504456,14.504456,14.504456
Average FLOPs Estimate (GFLOPs),2.377871,2.377871,2.377871
NLU Expert 1 (percent),0.000000,0.000000,0.000000
NLU Expert 2 (percent),0.000000,0.000000,0.000000
NLU Expert 3 (percent),23.443092,6.335004,14.889048


# DeepSeekMoE with Top-P with Expert Choice Balance experiment code

In [1]:
def reset_kernel():
    tf.keras.backend.clear_session()
    import gc
    gc.collect()

results_table = {
    'Training Speed (epochs per second)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Training Time (second/epoch)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Total Training Time (second/iteration)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Computational Resource Usage': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average CPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average GPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average GPU Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average FLOPS Estimate (GFLOPS)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Expert NLU Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Expert NLG Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLU Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average NLG Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Entity Accuracy': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Intent Accuracy': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Perplexity': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0},
    'Average Validation Response Cosine Similarity': {f'Iteration {i+1}': 0 for i in range(10)} | {'Average': 0}
}

best_val_loss = float('inf')
best_model_path = "best_DeepseekTopPExpertChoice_moe_model"

for iteration in range(5):
    print(f"\nStarting Training Iteration {iteration + 1}")
    reset_kernel()

    def load_tf_datasets_from_disk(load_path):
        print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
        try:
            with open(os.path.join(load_path, "moe_params.json"), "r") as f:
                loaded_moe_params = json.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"moe_params.json not found in {load_path}")

        element_spec = (
            {
                'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
                'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
                'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            },
            {
                'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
                'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
                'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
            }
        )

        try:
            train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
            test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
        except Exception as e:
            raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

        raw_data = {}
        for dataset_name in ['train', 'test']:
            path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
            if os.path.exists(path):
                with open(path, 'rb') as f:
                    raw_data[dataset_name] = pickle.load(f)
                print(f"Loaded raw data for {dataset_name} from {path}")
            else:
                print(f"Warning: Raw data file {path} not found.")
                raw_data[dataset_name] = []

        return {
            "train_dataset": train_tf_dataset,
            "test_dataset": test_tf_dataset,
            "moe_params": loaded_moe_params,
            "raw_data": raw_data
        }

    tf_dataset_save_path = "tf_datasets"
    loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
    train_tf_dataset = loaded_data["train_dataset"]
    moe_params = loaded_data["moe_params"]
    raw_data = loaded_data["raw_data"]

    moe_params["num_experts"] = 4
    moe_params["capacity_factor"] = 1.0

    class HybridRouting:
        def __init__(self, p=0.9, capacity_factor=1.0):
            self.p = p
            self.capacity_factor = capacity_factor

        def __call__(self, gate_logits, num_tokens, num_experts, k_s):
            batch_size = tf.shape(gate_logits)[0]
            probs = tf.nn.softmax(gate_logits, axis=-1)
            sorted_probs, sorted_indices = tf.math.top_k(probs, k=tf.shape(probs)[-1])
            cum_probs = tf.cumsum(sorted_probs, axis=-1)
            p_mask = cum_probs >= self.p
            first_over_p = tf.argmax(tf.cast(p_mask, tf.int32), axis=-1, output_type=tf.int32)
            first_over_p = tf.maximum(first_over_p, 0)
            num_selected = first_over_p + 1
            max_selected = tf.reduce_max(num_selected)
            selection_mask = tf.less_equal(tf.range(tf.shape(probs)[-1]), tf.expand_dims(first_over_p, -1))
            top_p_indices = tf.boolean_mask(sorted_indices, selection_mask)
            top_p_probs = tf.boolean_mask(sorted_probs, selection_mask)
            top_p_indices = tf.RaggedTensor.from_row_lengths(top_p_indices, num_selected).to_tensor(shape=[batch_size, max_selected])
            top_p_probs = tf.RaggedTensor.from_row_lengths(top_p_probs, num_selected).to_tensor(shape=[batch_size, max_selected])
            top_p_weights = top_p_probs / tf.reduce_sum(top_p_probs, axis=-1, keepdims=True)
            k = tf.cast(tf.cast(num_tokens, tf.float32) * self.capacity_factor / num_experts, tf.int32)
            k = tf.maximum(k, 1)
            transposed_probs = tf.transpose(probs)
            gating_weights, token_indices = tf.math.top_k(transposed_probs, k=k)
            gating_weights = tf.nn.softmax(gating_weights, axis=-1)
            return top_p_indices, top_p_weights, token_indices, gating_weights

    class MoELayer(layers.Layer):
        def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.01, top_p=0.9, capacity_factor=1.0, name=None):
            super(MoELayer, self).__init__(name=name)
            self.m = m
            self.k_s = k_s
            self.total_experts = num_experts * m
            self.routed_experts = self.total_experts - k_s
            self.alpha = alpha
            self.input_dim = input_dim
            self.top_p = top_p
            self.capacity_factor = capacity_factor
            self.adjusted_expert_dim = expert_dim // m
            self.seq_length = None
            self.shared_experts = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', name=f'shared_expert_{i}_dense1'),
                    layers.Dense(input_dim, activation='relu', name=f'shared_expert_{i}_dense2')
                ], name=f'shared_expert_{i}') for i in range(k_s)
            ]
            self.routed_experts_list = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', name=f'routed_expert_{i}_dense1'),
                    layers.Dense(input_dim, activation='relu', name=f'routed_expert_{i}_dense2')
                ], name=f'routed_expert_{i}') for i in range(k_s, self.total_experts)
            ]
            self.experts = self.shared_experts + self.routed_experts_list
            self.gate = layers.Dense(self.routed_experts, activation='softmax', name='gate_weights')
            self.hybrid_router = HybridRouting(p=self.top_p, capacity_factor=self.capacity_factor)
            self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

        def call(self, inputs, training=False):
            input_rank = inputs.shape.rank
            if input_rank == 3:
                batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
                flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
                is_3d = True
            else:
                batch_size = tf.shape(inputs)[0]
                seq_length = 1
                flat_inputs = inputs
                is_3d = False
            flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
            num_tokens = tf.shape(flat_inputs)[0]
            shared_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
            for i in range(self.k_s):
                shared_output += self.shared_experts[i](flat_inputs)
            gate_probs = self.gate(flat_inputs)
            top_p_indices, top_p_weights, token_indices, gating_weights = self.hybrid_router(
                gate_probs, num_tokens, self.routed_experts, self.k_s
            )
            expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
            top_p_indices_adjusted = top_p_indices + self.k_s
            batch_indices = tf.tile(
                tf.expand_dims(tf.range(num_tokens), 1),
                [1, tf.shape(top_p_indices)[1]]
            )
            gather_indices = tf.stack([batch_indices, top_p_indices_adjusted], axis=-1)
            selected_outputs = tf.gather_nd(expert_outputs, gather_indices)
            top_p_output = tf.reduce_sum(selected_outputs * tf.expand_dims(top_p_weights, -1), axis=1)
            k = tf.shape(token_indices)[1]
            token_indices_expanded = tf.expand_dims(token_indices, axis=2)
            expert_indices = tf.range(self.routed_experts)[:, None, None]
            expert_indices = tf.tile(expert_indices, [1, k, 1])
            indices = tf.concat([token_indices_expanded, expert_indices], axis=2)
            selected_expert_outputs = tf.gather_nd(expert_outputs[:, self.k_s:, :], indices)
            one_hot_tokens = tf.one_hot(token_indices, depth=num_tokens, dtype=tf.float32)
            expert_choice_output = tf.einsum('ek,ekd,ekn->nd', gating_weights, selected_expert_outputs, one_hot_tokens)
            output_flat = shared_output + (0.5 * top_p_output + 0.5 * expert_choice_output) + flat_inputs
            if is_3d:
                output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
                if self.seq_length is not None:
                    output.set_shape([None, self.seq_length, self.input_dim])
                gate_weights_reshaped = tf.reshape(gate_probs, [batch_size, seq_length, self.routed_experts])
                if self.seq_length is not None:
                    gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
            else:
                output = output_flat
                gate_weights_reshaped = gate_probs
            expert_selection = tf.reduce_mean(tf.reduce_max(gate_probs, axis=0), axis=0)
            f_i = tf.reduce_mean(tf.reduce_sum(tf.one_hot(tf.argmax(gate_probs, axis=-1), depth=self.routed_experts), axis=0))
            P_i = tf.reduce_mean(gate_probs, axis=0)
            load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
            self.add_loss(load_balancing_loss)
            self.load_balancing_loss_metric.update_state(load_balancing_loss)
            return output, gate_weights_reshaped

    class TransformerDecoderLayer(layers.Layer):
        def __init__(self, d_model, num_heads, dff, rate=0.1):
            super(TransformerDecoderLayer, self).__init__()
            self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.ffn = keras.Sequential([
                layers.Dense(dff, activation='relu'),
                layers.Dense(d_model)
            ])
            self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
            self.dropout1 = layers.Dropout(rate)
            self.dropout2 = layers.Dropout(rate)
            self.dropout3 = layers.Dropout(rate)

        def call(self, x, enc_output, training, look_ahead_mask=None):
            if look_ahead_mask is not None:
                look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
                look_ahead_mask = 1.0 - look_ahead_mask
            attn1_output = self.mha1(
                query=x, value=x, key=x, attention_mask=look_ahead_mask, training=training, return_attention_scores=True
            )
            attn1, attn_weights1 = attn1_output if isinstance(attn1_output, tuple) else (attn1_output, None)
            attn1 = self.dropout1(attn1, training=training)
            out1 = self.layernorm1(attn1 + x)
            attn2_output = self.mha2(
                query=out1, value=enc_output, key=enc_output, attention_mask=None, training=training, return_attention_scores=True
            )
            attn2, attn_weights2 = attn2_output if isinstance(attn2_output, tuple) else (attn2_output, None)
            attn2 = self.dropout2(attn2, training=training)
            out2 = self.layernorm2(attn2 + out1)
            ffn_output = self.ffn(out2)
            ffn_output = self.dropout3(ffn_output, training=training)
            out3 = self.layernorm3(ffn_output + out2)
            return out3, attn_weights1, attn_weights2

        def get_config(self):
            config = super().get_config()
            mha_config = self.mha1.get_config()
            config.update({
                'd_model': mha_config['key_dim'] * mha_config['num_heads'],
                'num_heads': mha_config['num_heads'],
                'dff': self.ffn.layers[0].units,
                'rate': self.dropout1.rate
            })
            return config

    class MoEModel(models.Model):
        def __init__(self, moe_params):
            super(MoEModel, self).__init__()
            self.embedding_dim = moe_params["embedding_dim"]
            self.max_seq_length = moe_params["max_seq_length"]
            self.num_domains = moe_params["num_domains"]
            self.num_intents = moe_params["num_intents"]
            self.vocab_size = moe_params["vocab_size"]
            self.num_experts = moe_params["num_experts"]
            self.expert_dim = moe_params["expert_dim"]
            self.turn_id_dim = moe_params["turn_id_dim"]
            self.num_heads = 4
            self.dff = self.embedding_dim * 4
            self.dropout_rate = 0.1
            self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                                 self.num_domains + self.turn_id_dim + self.num_intents)
            self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains

            self.embedding = layers.Embedding(
                self.vocab_size,
                self.embedding_dim,
                mask_zero=True,
                embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05),
                name="embedding"
            )
            self.embedding.build((None,))
            with tf.init_scope():
                embedding_weights = self.embedding.get_weights()
                if embedding_weights and len(embedding_weights) > 0:
                    embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                    self.embedding.set_weights(embedding_weights)

            self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
            self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
            self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
            self.decoder_dropout = layers.Dropout(self.dropout_rate)
            self.moe_nlu = MoELayer(
                num_experts=self.num_experts,
                expert_dim=self.expert_dim,
                input_dim=self.nlu_input_dim,
                m=2,
                k_s=2,
                alpha=0.1,
                top_p=0.9,
                capacity_factor=moe_params["capacity_factor"],
                name='moe_nlu'
            )
            self.moe_nlg = MoELayer(
                num_experts=self.num_experts,
                expert_dim=self.expert_dim,
                input_dim=self.nlg_input_dim,
                m=2,
                k_s=2,
                alpha=0.1,
                top_p=0.0,
                capacity_factor=moe_params["capacity_factor"],
                name='moe_nlg'
            )
            self.intent_output = layers.Dense(self.num_intents, activation='sigmoid', name='intent_output')
            self.domain_output = layers.Dense(self.num_domains, activation='softmax', name='domain_output')
            self.response_output = layers.TimeDistributed(
                layers.Dense(self.vocab_size, activation='softmax'), name='response_output'
            )
            self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)
            self.nlg_projection = layers.TimeDistributed(
                layers.Dense(self.embedding_dim, activation='relu', name='nlg_projection')
            )

        def create_look_ahead_mask(self, size):
            mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
            return mask

        def call(self, inputs, training=False):
            user_utterances_tokens = inputs['user_utterance_tokens']
            prev_system_response_tokens = inputs['prev_system_response_tokens']
            decoder_input_tokens = inputs['decoder_input_tokens']
            domain_onehot_input = inputs['domain_onehot_input']
            turn_id_embedding = inputs['turn_id_embedding']
            ontology_multihot_input = inputs['ontology_multihot_input']

            pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
            user_embedded = self.embedding_norm(self.embedding(user_utterances_tokens) + pos_enc)
            prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
            decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)

            user_enc_out = user_embedded
            prev_system_enc_out = prev_system_embedded

            look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
            dec_output = decoder_embedded
            attn_weights = {}
            for i, layer in enumerate(self.decoder_layers):
                dec_output, attn_w1, attn_w2 = layer(
                    dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask
                )
                attn_weights[f'decoder_layer{i+1}_attn1'] = attn_w1
                attn_weights[f'decoder_layer{i+1}_attn2'] = attn_w2

            decoder_output = self.decoder_dropout(dec_output, training=training)

            user_attn_out = self.attn_layer(
                query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), 
                value=user_enc_out, key=user_enc_out, training=training
            )
            prev_system_attn_out = self.attn_layer(
                query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), 
                value=prev_system_enc_out, key=prev_system_enc_out, training=training
            )
            decoder_attn_out = self.attn_layer(
                query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), 
                value=decoder_output, key=decoder_output, training=training
            )

            combined_features = layers.Concatenate()([
                tf.squeeze(user_attn_out, axis=1),
                tf.squeeze(prev_system_attn_out, axis=1),
                tf.squeeze(decoder_attn_out, axis=1),
                domain_onehot_input,
                turn_id_embedding,
                ontology_multihot_input,
            ])
            combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
            nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
            nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
            intent_out = self.intent_output(nlu_out)
            domain_out = self.domain_output(nlu_out)

            intent_out_expanded = tf.expand_dims(intent_out, axis=1)
            domain_out_expanded = tf.expand_dims(domain_out, axis=1)
            intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
            domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])

            nlg_input = layers.Concatenate(axis=-1)([
                decoder_output,
                intent_out_tiled,
                domain_out_tiled
            ])
            nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])

            nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
            nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
            response_embeddings = self.nlg_projection(nlg_out)
            response_embeddings = tf.ensure_shape(response_embeddings, [None, self.max_seq_length, self.embedding_dim])
            response_out = self.response_output(nlg_out)

            return {
                'intent_output': intent_out,
                'domain_output': domain_out,
                'response_output': response_out,
                'response_embeddings': response_embeddings,
                'nlu_gate_weights': nlu_gate_weights,
                'nlg_gate_weights': nlg_gate_weights
            }

        def build_graph(self, input_shape=None):
            inputs = {
                'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='user_utterance_tokens'),
                'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
                'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
                'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32),
                'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32),
                'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32),
            }
            return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

        def get_config(self):
            config = super().get_config()
            config.update({
                'moe_params': {
                    'embedding_dim': self.embedding_dim,
                    'max_seq_length': self.max_seq_length,
                    'num_domains': self.num_domains,
                    'num_intents': self.num_intents,
                    'vocab_size': self.vocab_size,
                    'num_experts': self.num_experts,
                    'expert_dim': self.expert_dim,
                    'turn_id_dim': self.turn_id_dim,
                    'capacity_factor': self.moe_nlg.capacity_factor
                }
            })
            return config

    class IntentAccuracy(metrics.Metric):
        def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros', dtype=tf.float32)
            self.total_preds = self.add_weight(name='total_preds', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
            self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
            self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

        def reset_state(self):
            self.correct_preds.assign(0.)
            self.total_preds.assign(0.)

    class IntentPrecision(metrics.Metric):
        def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
            self.predicted_positives = self.add_weight(name='pp', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            predicted_positives = tf.reduce_sum(y_pred)
            self.true_positives.assign_add(true_positives)
            self.predicted_positives.assign_add(predicted_positives)

        def result(self):
            return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.predicted_positives.assign(0.)

    class IntentRecall(metrics.Metric):
        def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
            self.actual_positives = self.add_weight(name='ap', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            actual_positives = tf.reduce_sum(y_true)
            self.true_positives.assign_add(true_positives)
            self.actual_positives.assign_add(actual_positives)

        def result(self):
            return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.actual_positives.assign(0.)

    class IntentF1(metrics.Metric):
        def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.precision = IntentPrecision(threshold=threshold)
            self.recall = IntentRecall(threshold=threshold)

        def update_state(self, y_true, y_pred, sample_weight=None):
            self.precision.update_state(y_true, y_pred, sample_weight)
            self.recall.update_state(y_true, y_pred, sample_weight)

        def result(self):
            p = self.precision.result()
            r = self.recall.result()
            return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

        def reset_state(self):
            self.precision.reset_state()
            self.recall.reset_state()

    class DomainAccuracy(metrics.Metric):
        def __init__(self, name='domain_accuracy', **kwargs):
            super().__init__(name=name, **kwargs)
            self.accuracy = self.add_weight(name='acc', initializer='zeros', dtype=tf.float32)
            self.count = self.add_weight(name='count', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
            pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
            matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
            self.accuracy.assign_add(tf.reduce_sum(matches))
            self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.accuracy / (self.count + tf.keras.backend.epsilon())

        def reset_state(self):
            self.accuracy.assign(0.)
            self.count.assign(0.)

    class Perplexity(metrics.Metric):
        def __init__(self, name='perplexity', **kwargs):
            super().__init__(name=name, **kwargs)
            self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
            self.total_loss = self.add_weight(name='total_loss', initializer='zeros', dtype=tf.float32)
            self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            mask = tf.cast(y_true != 0, dtype=tf.float32)
            loss = self.cross_entropy(y_true, y_pred)
            masked_loss = loss * mask
            sum_loss = tf.reduce_sum(masked_loss)
            num_non_padding_tokens = tf.reduce_sum(mask)
            self.total_loss.assign_add(sum_loss)
            self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

        def result(self):
            avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
            return tf.exp(avg_loss)

        def reset_state(self):
            self.total_loss.assign(0.)
            self.total_non_padding_tokens.assign(0.)

    class ResponseEmbeddingCosineSimilarity(metrics.Metric):
        def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
            super().__init__(name=name)
            self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
            self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            epsilon = tf.keras.backend.epsilon()
            y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
            y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
            y_true_norm = y_true / (y_true_norm_val + epsilon)
            y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
            cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
            non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
            non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
            masked_cosine_sim = cosine_sim_per_token * non_padding_mask
            num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
            avg_cosine_sim_per_sample = tf.where(
                num_non_zero_tokens > 0,
                tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
                tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
            )
            self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
            self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

        def reset_state(self):
            self.total_cosine_sim.assign(0.)
            self.num_samples.assign(0.)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    def calculate_flops(model, batch_size=moe_params["batch_size"]):
        flops = 0
        functional_model = model.build_graph()

        def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
            layer_flops_count = 0

            if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
                return 0

            if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
                for sub_layer in layer_instance.layers:
                    layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
                return layer_flops_count

            if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
                return 0

            if isinstance(layer_instance, layers.Dense):
                input_dim = layer_instance.input_shape[-1]
                output_dim = layer_instance.output_shape[-1]
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                layer_flops_count += 2 * input_dim * output_dim * effective_batch_size

            elif isinstance(layer_instance, layers.LSTM):
                input_dim = layer_instance.input_shape[-1]
                hidden_dim = layer_instance.units
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
                layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size

            elif isinstance(layer_instance, layers.Embedding):
                pass

            elif isinstance(layer_instance, MoELayer):
                moe_input_dim = layer_instance.input_dim
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
                layer_flops_count += gate_flops
                for expert in layer_instance.experts:
                    layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)

            elif isinstance(layer_instance, layers.MultiHeadAttention):
                config = layer_instance.get_config()
                num_heads = config['num_heads']
                key_dim = config['key_dim']
                d_model = num_heads * key_dim
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
                projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
                attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
                context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
                output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
                layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops

            elif isinstance(layer_instance, layers.TimeDistributed):
                inner_layer = layer_instance.layer
                td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
                layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)

            elif isinstance(layer_instance, TransformerDecoderLayer):
                layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
                layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
                ffn_effective_batch_size = current_batch_size * model.max_seq_length
                layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)

            return layer_flops_count

        for layer in functional_model.layers:
            flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)

        return flops / 1e9

    def get_gpu_stats():
        if nvidia_smi is None:
            return 0, 0
        try:
            nvidia_smi.nvmlInit()
            handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
            gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
            mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
            gpu_memory = mem_info.used / (1024 ** 2)
            nvidia_smi.nvmlShutdown()
            return gpu_load, gpu_memory / 1024
        except Exception:
            return 0, 0

    class ResourceMonitor(keras.callbacks.Callback):
        def __init__(self, validation_data):
            super().__init__()
            self.resource_stats = []
            self.expert_load_history = []
            self.start_time = None
            self.epoch_start_time = None
            self.flops = None
            self.validation_data = validation_data
            self.nlu_expert_usage_counts = None
            self.nlg_expert_usage_counts = None
            self.nlu_gate_weights_history = []
            self.nlg_gate_weights_history = []

        def on_train_begin(self, logs=None):
            self.start_time = time.time()
            try:
                self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
            except Exception:
                self.flops = 0
            try:
                self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
                self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
            except Exception:
                self.nlu_expert_usage_counts = np.zeros(0)
                self.nlg_expert_usage_counts = np.zeros(0)

        def on_epoch_begin(self, epoch, logs=None):
            self.epoch_start_time = time.time()

        def on_epoch_end(self, epoch, logs=None):
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            epoch_time = time.time() - self.epoch_start_time
            self.resource_stats.append({
                'epoch': epoch + 1,
                'epoch_time': epoch_time,
                'cpu_usage': cpu_usage,
                'memory_used': memory.used / (1024 ** 3),
                'memory_total': memory.total / (1024 ** 3),
                'gpu_load': gpu_load,
                'gpu_memory': gpu_memory,
                'loss': logs.get('loss', 0),
                'val_loss': logs.get('val_loss', 0),
                'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
                'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
                'intent_precision': logs.get('intent_output_intent_precision', 0),
                'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
                'intent_recall': logs.get('intent_output_intent_recall', 0),
                'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
                'intent_f1': logs.get('intent_output_intent_f1', 0),
                'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
                'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
                'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
                'perplexity': logs.get('response_output_perplexity', 0),
                'val_perplexity': logs.get('val_response_output_perplexity', 0),
                'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
                'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
                'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
                'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
            })
            if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
                self.expert_load_history.append({
                    'epoch': epoch + 1,
                    'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                    'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
                })

            sample_size = 100
            for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
                inputs, _ = batch
                outputs = self.model(inputs, training=False)
                nlu_gate_weights = outputs['nlu_gate_weights']
                reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
                self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
                nlg_gate_weights = outputs['nlg_gate_weights']
                reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
                self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

        def on_train_end(self, logs=None):
            self.total_time = time.time() - self.start_time

            if len(self.nlu_gate_weights_history) > 0:
                all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
                nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
                for expert_id in nlu_top_indices:
                    if expert_id < self.model.moe_nlu.total_experts:
                        self.nlu_expert_usage_counts[expert_id] += 1

            if len(self.nlg_gate_weights_history) > 0:
                all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
                nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
                for expert_id in nlg_top_indices:
                    if expert_id < self.model.moe_nlg.total_experts:
                        self.nlg_expert_usage_counts[expert_id] += 1

        def visualize_expert_load_distribution(self):
            total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
            total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
            nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
            nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
            nlu_stability = np.var(nlu_percentages)
            nlg_stability = np.var(nlg_percentages)
            return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

    class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
        def __init__(self, d_model, warmup_steps=4000):
            super().__init__()
            self.d_model = tf.cast(d_model, tf.float32)
            self.warmup_steps = warmup_steps

        def __call__(self, step):
            step = tf.cast(step, tf.float32)
            arg1 = tf.math.rsqrt(step)
            arg2 = step * (self.warmup_steps ** -1.5)
            return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

        def get_config(self):
            return {
                "d_model": self.d_model.numpy(),
                "warmup_steps": self.warmup_steps
            }

    model = MoEModel(moe_params)
    embedding_layer = model.embedding

    def create_validation_split(dataset, embedding_layer, validation_split=0.2, batch_size=moe_params["batch_size"]):
        element_spec = dataset.element_spec
        data_list = [(features, targets) for features, targets in dataset.unbatch().as_numpy_iterator()]
        if not data_list:
            raise ValueError("Dataset is empty.")
        if len(data_list) < 2:
            raise ValueError("Dataset too small to split.")
        train_data, val_data = train_test_split(data_list, test_size=validation_split, shuffle=True)

        def create_dataset_from_list(data):
            if not data:
                raise ValueError("Empty data list provided.")
            features = {
                'user_utterance_tokens': np.array([d[0]['user_utterance_tokens'] for d in data], dtype=np.int32),
                'prev_system_response_tokens': np.array([d[0]['prev_system_response_tokens'] for d in data], dtype=np.int32),
                'decoder_input_tokens': np.array([d[0]['decoder_input_tokens'] for d in data], dtype=np.int32),
                'domain_onehot_input': np.array([d[0]['domain_onehot_input'] for d in data], dtype=np.float32),
                'turn_id_embedding': np.array([d[0]['turn_id_embedding'] for d in data], dtype=np.float32),
                'ontology_multihot_input': np.array([d[0]['ontology_multihot_input'] for d in data], dtype=np.float32),
            }
            response_output = np.array([d[1]['response_output'] for d in data], dtype=np.int32)
            response_embeddings = embedding_layer(response_output).numpy()
            targets = {
                'domain_output': np.array([d[1]['domain_output'] for d in data], dtype=np.int32),
                'intent_output': np.array([d[1]['intent_output'] for d in data], dtype=np.float32),
                'response_output': response_output,
                'response_embeddings': response_embeddings
            }
            return tf.data.Dataset.from_tensor_slices((features, targets))

        train_dataset = create_dataset_from_list(train_data).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        val_dataset = create_dataset_from_list(val_data).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return train_dataset, val_dataset

    train_tf_dataset, val_tf_dataset = create_validation_split(train_tf_dataset, embedding_layer)

    losses_dict = {
        'intent_output': losses.BinaryCrossentropy(),
        'domain_output': losses.SparseCategoricalCrossentropy(),
        'response_output': losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss
    }

    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }

    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={'intent_output': 0.5, 'domain_output': 0.5, 'response_output': 1.0, 'response_embeddings': 1.0}
    )

    sample_input = next(iter(train_tf_dataset.take(1)))
    model(sample_input[0])

    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=8, mode='min', restore_best_weights=True)
    resource_monitor = ResourceMonitor(val_tf_dataset)
    class TQDMProgressBar(callbacks.Callback):
        def on_epoch_begin(self, epoch, logs=None):
            self.progress_bar = tqdm(total=len(train_tf_dataset), desc=f'Epoch {epoch + 1}')

        def on_batch_end(self, batch, logs=None):
            self.progress_bar.update(1)

        def on_epoch_end(self, epoch, logs=None):
            self.progress_bar.close()

    tqdm_callback = TQDMProgressBar()

    history = model.fit(
        train_tf_dataset,
        epochs=100,
        validation_data=val_tf_dataset,
        callbacks=[early_stopping, resource_monitor, tqdm_callback],
        verbose=0
    )

    expert_stats = resource_monitor.visualize_expert_load_distribution()
    stats = resource_monitor.resource_stats
    avg_epoch_duration = np.mean([s['epoch_time'] for s in stats]) if stats else 0
    total_training_time = resource_monitor.total_time
    avg_cpu_usage = np.mean([s['cpu_usage'] for s in stats]) if stats else 0
    avg_memory = np.mean([s['memory_used'] for s in stats]) if stats else 0
    avg_gpu_load = np.mean([s['gpu_load'] for s in stats]) if stats else 0
    avg_gpu_memory = np.mean([s['gpu_memory'] for s in stats]) if stats else 0
    avg_flops = resource_monitor.flops if resource_monitor.flops else 0
    nlu_counts = resource_monitor.nlu_expert_usage_counts
    nlg_counts = resource_monitor.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) or 1
    total_nlg = np.sum(nlg_counts) or 1
    expert_nlu_percentages = [(nlu_counts[i] / total_nlu) * 100 for i in range(8)]
    expert_nlg_percentages = [(nlg_counts[i] / total_nlg) * 100 for i in range(8)]
    val_intent_accuracy = np.mean([s['val_intent_accuracy'] for s in stats if s['val_intent_accuracy'] > 0]) if stats else 0
    val_entity_accuracy = np.mean([s['val_domain_accuracy'] for s in stats if s['val_domain_accuracy'] > 0]) if stats else 0
    val_perplexity = np.mean([s['val_perplexity'] for s in stats if s['val_perplexity'] > 0]) if stats else 0
    val_cosine_similarity = np.mean([s['val_cosine_similarity'] for s in stats if s['val_cosine_similarity'] > 0]) if stats else 0

    results_table['Training Speed (epochs per second)'][f'Iteration {iteration + 1}'] = 1 / avg_epoch_duration if avg_epoch_duration > 0 else 0
    results_table['Training Time (second/epoch)'][f'Iteration {iteration + 1}'] = avg_epoch_duration
    results_table['Total Training Time (second/iteration)'][f'Iteration {iteration + 1}'] = total_training_time
    results_table['Computational Resource Usage'][f'Iteration {iteration + 1}'] = avg_cpu_usage + avg_gpu_load
    results_table['Average CPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_cpu_usage
    results_table['Average GPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_gpu_load
    results_table['Average Memory (GB)'][f'Iteration {iteration + 1}'] = avg_memory
    results_table['Average GPU Memory (GB)'][f'Iteration {iteration + 1}'] = avg_gpu_memory
    results_table['Average FLOPS Estimate (GFLOPS)'][f'Iteration {iteration + 1}'] = avg_flops
    results_table['Expert NLU Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlu_stability']
    results_table['Expert NLG Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlg_stability']
    for i in range(8):
        if i < len(expert_stats['nlu_percentages']):
            results_table[f'Average NLU Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlu_percentages'][i]
        if i < len(expert_stats['nlg_percentages']):
            results_table[f'Average NLG Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlg_percentages'][i]
    results_table['Average Validation Entity Accuracy'][f'Iteration {iteration + 1}'] = val_entity_accuracy
    results_table['Average Intent Accuracy'][f'Iteration {iteration + 1}'] = val_intent_accuracy
    results_table['Average Validation Perplexity'][f'Iteration {iteration + 1}'] = val_perplexity
    results_table['Average Validation Response Cosine Similarity'][f'Iteration {iteration + 1}'] = val_cosine_similarity

    val_loss = min([s['val_loss'] for s in stats]) if stats else float('inf')
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        try:
            model.save(best_model_path, save_format='tf', include_optimizer=True)
        except Exception as e:
            print(f"\nFailed to save best model to {best_model_path}: {str(e)}")

for key in results_table:
    results_table[key]['Average'] = np.mean([results_table[key][f'Iteration {i+1}'] for i in range(10)])

final_result = pd.DataFrame(results_table)
final_result = final_result.T
final_result.to_excel('training_DeepseekTopPExpertChoice_result.xlsx', index=True)
final_result

2025-07-04 08:38:21.890266: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 08:38:22.327596: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 08:38:22.327691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 08:38:22.405516: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 08:38:22.559413: I tensorflow/core/platform/cpu_feature_guar


Starting Training Iteration 1

Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 08:38:28.177368: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 08:38:28.459275: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 08:38:28.459561: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl


Epoch 1:   0%|          | 0/4428 [00:00<?, ?it/s]2025-07-04 08:38:51.811388: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f23c029c4b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-04 08:38:51.811479: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2025-07-04 08:38:51.833026: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-04 08:38:51.880942: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1751618331.975172   20995 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
Epoch 1: 100%|██████████| 4428/4428 [03:01<00:00, 24.36it/s]


INFO:tensorflow:Assets written to: best_DeepseekTopPExpertChoice_moe_model/assets


INFO:tensorflow:Assets written to: best_DeepseekTopPExpertChoice_moe_model/assets


,Iteration 1,Iteration 2,Iteration 3,Iteration 4,Iteration 5,Iteration 6,Iteration 7,Iteration 8,Iteration 9,Iteration 10,Average
Training Speed (epochs per second),0.005670,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000567
Training Time (second/epoch),176.382080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.638208
Total Training Time (second/iteration),183.105052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.310505
Computational Resource Usage,49.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.950000
Average CPU Usage (percent),25.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.550000
Average GPU Usage (percent),24.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.400000
Average Memory (GB),6.604259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.660426
Average GPU Memory (GB),14.531799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.453180
Average FLOPS Estimate (GFLOPS),2.377871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.237787
Expert NLU Load Distribution Stability,97.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.750000


# DeepSeekMoE with Top-P with Expert Choice Balance evaluation code

In [1]:
def load_tf_datasets_from_disk(load_path):
    try:
        with open(os.path.join(load_path, "moe_params.json"), "r") as f:
            loaded_moe_params = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"moe_params.json not found in {load_path}")

    element_spec = (
        {
            'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
            'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
            'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
        },
        {
            'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
            'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
        }
    )

    try:
        train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
        test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
    except Exception as e:
        raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

    raw_data = {}
    for dataset_name in ['train', 'test']:
        path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
        if os.path.exists(path):
            with open(path, 'rb') as f:
                raw_data[dataset_name] = pickle.load(f)
        else:
            raw_data[dataset_name] = []

    return {
        "train_dataset": train_tf_dataset,
        "test_dataset": test_tf_dataset,
        "moe_params": loaded_moe_params,
        "raw_data": raw_data
    }

tf_dataset_save_path = "tf_datasets"
loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
train_tf_dataset = loaded_data["train_dataset"]
test_tf_dataset = loaded_data["test_dataset"]
moe_params = loaded_data["moe_params"]
raw_data = loaded_data["raw_data"]

moe_params["num_experts"] = 4

class HybridRouting:
    def __init__(self, p=0.9, capacity_factor=1.0):
        self.p = p
        self.capacity_factor = capacity_factor

    def __call__(self, gate_logits, num_tokens, num_experts, k_s):
        batch_size = tf.shape(gate_logits)[0]
        probs = tf.nn.softmax(gate_logits, axis=-1)
        sorted_probs, sorted_indices = tf.math.top_k(probs, k=tf.shape(probs)[-1])
        cum_probs = tf.cumsum(sorted_probs, axis=-1)
        p_mask = cum_probs >= self.p
        first_over_p = tf.argmax(tf.cast(p_mask, tf.int32), axis=-1, output_type=tf.int32)
        first_over_p = tf.maximum(first_over_p, 0)
        num_selected = first_over_p + 1
        max_selected = tf.reduce_max(num_selected)
        selection_mask = tf.less_equal(tf.range(tf.shape(probs)[-1]), tf.expand_dims(first_over_p, -1))
        top_p_indices = tf.boolean_mask(sorted_indices, selection_mask)
        top_p_probs = tf.boolean_mask(sorted_probs, selection_mask)
        top_p_indices = tf.RaggedTensor.from_row_lengths(top_p_indices, num_selected).to_tensor(shape=[batch_size, max_selected])
        top_p_probs = tf.RaggedTensor.from_row_lengths(top_p_probs, num_selected).to_tensor(shape=[batch_size, max_selected])
        top_p_weights = top_p_probs / tf.reduce_sum(top_p_probs, axis=-1, keepdims=True)
        k = tf.cast(tf.cast(num_tokens, tf.float32) * self.capacity_factor / num_experts, tf.int32)
        k = tf.maximum(k, 1)
        transposed_probs = tf.transpose(probs)
        gating_weights, token_indices = tf.math.top_k(transposed_probs, k=k)
        gating_weights = tf.nn.softmax(gating_weights, axis=-1)
        return top_p_indices, top_p_weights, token_indices, gating_weights

class MoELayer(layers.Layer):
    def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.01, top_p=0.9, capacity_factor=1.0, name=None):
        super(MoELayer, self).__init__(name=name)
        self.m = m
        self.k_s = k_s
        self.total_experts = num_experts * m
        self.routed_experts = self.total_experts - k_s
        self.alpha = alpha
        self.input_dim = input_dim
        self.top_p = top_p
        self.capacity_factor = capacity_factor
        self.adjusted_expert_dim = expert_dim // m
        self.seq_length = None
        self.shared_experts = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', name=f'shared_expert_{i}_dense1'),
                layers.Dense(input_dim, activation='relu', name=f'shared_expert_{i}_dense2')
            ], name=f'shared_expert_{i}') for i in range(k_s)
        ]
        self.routed_experts_list = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', name=f'routed_expert_{i}_dense1'),
                layers.Dense(input_dim, activation='relu', name=f'routed_expert_{i}_dense2')
            ], name=f'routed_expert_{i}') for i in range(k_s, self.total_experts)
        ]
        self.experts = self.shared_experts + self.routed_experts_list
        self.gate = layers.Dense(self.routed_experts, activation='softmax', name='gate_weights')
        self.hybrid_router = HybridRouting(p=self.top_p, capacity_factor=self.capacity_factor)
        self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

    def call(self, inputs, training=False):
        input_rank = inputs.shape.rank
        if input_rank == 3:
            batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
            flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
            is_3d = True
        else:
            batch_size = tf.shape(inputs)[0]
            seq_length = 1
            flat_inputs = inputs
            is_3d = False
        flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
        num_tokens = tf.shape(flat_inputs)[0]
        shared_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
        for i in range(self.k_s):
            shared_output += self.shared_experts[i](flat_inputs)
        gate_probs = self.gate(flat_inputs)
        top_p_indices, top_p_weights, token_indices, gating_weights = self.hybrid_router(
            gate_probs, num_tokens, self.routed_experts, self.k_s
        )
        expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
        top_p_indices_adjusted = top_p_indices + self.k_s
        batch_indices = tf.tile(
            tf.expand_dims(tf.range(num_tokens), 1),
            [1, tf.shape(top_p_indices)[1]]
        )
        gather_indices = tf.stack([batch_indices, top_p_indices_adjusted], axis=-1)
        selected_outputs = tf.gather_nd(expert_outputs, gather_indices)
        top_p_output = tf.reduce_sum(selected_outputs * tf.expand_dims(top_p_weights, -1), axis=1)
        k = tf.shape(token_indices)[1]
        token_indices_expanded = tf.expand_dims(token_indices, axis=2)
        expert_indices = tf.range(self.routed_experts)[:, None, None]
        expert_indices = tf.tile(expert_indices, [1, k, 1])
        indices = tf.concat([token_indices_expanded, expert_indices], axis=2)
        selected_expert_outputs = tf.gather_nd(expert_outputs[:, self.k_s:, :], indices)
        one_hot_tokens = tf.one_hot(token_indices, depth=num_tokens, dtype=tf.float32)
        expert_choice_output = tf.einsum('ek,ekd,ekn->nd', gating_weights, selected_expert_outputs, one_hot_tokens)
        output_flat = shared_output + (0.5 * top_p_output + 0.5 * expert_choice_output) + flat_inputs
        if is_3d:
            output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
            if self.seq_length is not None:
                output.set_shape([None, self.seq_length, self.input_dim])
            gate_weights_reshaped = tf.reshape(gate_probs, [batch_size, seq_length, self.routed_experts])
            if self.seq_length is not None:
                gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
        else:
            output = output_flat
            gate_weights_reshaped = gate_probs
        expert_selection = tf.reduce_mean(tf.reduce_max(gate_probs, axis=0), axis=0)
        f_i = tf.reduce_mean(tf.reduce_sum(tf.one_hot(tf.argmax(gate_probs, axis=-1), depth=self.routed_experts), axis=0))
        P_i = tf.reduce_mean(gate_probs, axis=0)
        load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
        self.add_loss(load_balancing_loss)
        self.load_balancing_loss_metric.update_state(load_balancing_loss)
        return output, gate_weights_reshaped

class TransformerDecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.ffn = keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask=None):
        if look_ahead_mask is not None:
            look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
            look_ahead_mask = 1.0 - look_ahead_mask
        attn1_output = self.mha1(
            query=x, value=x, key=x, attention_mask=look_ahead_mask, training=training, return_attention_scores=True
        )
        attn1, attn_weights1 = attn1_output if isinstance(attn1_output, tuple) else (attn1_output, None)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)
        attn2_output = self.mha2(
            query=out1, value=enc_output, key=enc_output, attention_mask=None, training=training, return_attention_scores=True
        )
        attn2, attn_weights2 = attn2_output if isinstance(attn2_output, tuple) else (attn2_output, None)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)
        return out3, attn_weights1, attn_weights2

    def get_config(self):
        config = super().get_config()
        mha_config = self.mha1.get_config()
        config.update({
            'd_model': mha_config['key_dim'] * mha_config['num_heads'],
            'num_heads': mha_config['num_heads'],
            'dff': self.ffn.layers[0].units,
            'rate': self.dropout1.rate
        })
        return config

class MoEModel(models.Model):
    def __init__(self, moe_params):
        super(MoEModel, self).__init__()
        self.embedding_dim = moe_params["embedding_dim"]
        self.max_seq_length = moe_params["max_seq_length"]
        self.num_domains = moe_params["num_domains"]
        self.num_intents = moe_params["num_intents"]
        self.vocab_size = moe_params["vocab_size"]
        self.num_experts = moe_params["num_experts"]
        self.expert_dim = moe_params["expert_dim"]
        self.turn_id_dim = moe_params["turn_id_dim"]
        self.num_heads = 4
        self.dff = self.embedding_dim * 4
        self.dropout_rate = 0.1
        self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                             self.num_domains + self.turn_id_dim + self.num_intents)
        self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains

        self.embedding = layers.Embedding(
            self.vocab_size,
            self.embedding_dim,
            mask_zero=True,
            embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05),
            name="embedding"
        )
        self.embedding.build((None,))
        with tf.init_scope():
            embedding_weights = self.embedding.get_weights()
            if embedding_weights and len(embedding_weights) > 0:
                embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                self.embedding.set_weights(embedding_weights)

        self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
        self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
        self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
        self.decoder_dropout = layers.Dropout(self.dropout_rate)
        self.moe_nlu = MoELayer(
            num_experts=self.num_experts,
            expert_dim=self.expert_dim,
            input_dim=self.nlu_input_dim,
            m=2,
            k_s=2,
            alpha=0.1,
            top_p=0.9,
            capacity_factor=moe_params["capacity_factor"],
            name='moe_nlu'
        )
        self.moe_nlg = MoELayer(
            num_experts=self.num_experts,
            expert_dim=self.expert_dim,
            input_dim=self.nlg_input_dim,
            m=2,
            k_s=2,
            alpha=0.1,
            top_p=0.0,
            capacity_factor=moe_params["capacity_factor"],
            name='moe_nlg'
        )
        self.intent_output = layers.Dense(self.num_intents, activation='sigmoid', name='intent_output')
        self.domain_output = layers.Dense(self.num_domains, activation='softmax', name='domain_output')
        self.response_output = layers.TimeDistributed(
            layers.Dense(self.vocab_size, activation='softmax'), name='response_output'
        )
        self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)
        self.nlg_projection = layers.TimeDistributed(
            layers.Dense(self.embedding_dim, activation='relu', name='nlg_projection')
        )

    def create_look_ahead_mask(self, size):
        mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
        return mask

    def call(self, inputs, training=False):
        user_utterances_tokens = inputs['user_utterance_tokens']
        prev_system_response_tokens = inputs['prev_system_response_tokens']
        decoder_input_tokens = inputs['decoder_input_tokens']
        domain_onehot_input = inputs['domain_onehot_input']
        turn_id_embedding = inputs['turn_id_embedding']
        ontology_multihot_input = inputs['ontology_multihot_input']

        pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
        user_embedded = self.embedding_norm(self.embedding(user_utterances_tokens) + pos_enc)
        prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
        decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)

        user_enc_out = user_embedded
        prev_system_enc_out = prev_system_embedded

        look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
        dec_output = decoder_embedded
        attn_weights = {}
        for i, layer in enumerate(self.decoder_layers):
            dec_output, attn_w1, attn_w2 = layer(
                dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask
            )
            attn_weights[f'decoder_layer{i+1}_attn1'] = attn_w1
            attn_weights[f'decoder_layer{i+1}_attn2'] = attn_w2

        decoder_output = self.decoder_dropout(dec_output, training=training)

        user_attn_out = self.attn_layer(
            query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), 
            value=user_enc_out, key=user_enc_out, training=training
        )
        prev_system_attn_out = self.attn_layer(
            query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), 
            value=prev_system_enc_out, key=prev_system_enc_out, training=training
        )
        decoder_attn_out = self.attn_layer(
            query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), 
            value=decoder_output, key=decoder_output, training=training
        )

        combined_features = layers.Concatenate()([
            tf.squeeze(user_attn_out, axis=1),
            tf.squeeze(prev_system_attn_out, axis=1),
            tf.squeeze(decoder_attn_out, axis=1),
            domain_onehot_input,
            turn_id_embedding,
            ontology_multihot_input,
        ])
        combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
        nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
        nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
        intent_out = self.intent_output(nlu_out)
        domain_out = self.domain_output(nlu_out)

        intent_out_expanded = tf.expand_dims(intent_out, axis=1)
        domain_out_expanded = tf.expand_dims(domain_out, axis=1)
        intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
        domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])

        nlg_input = layers.Concatenate(axis=-1)([
            decoder_output,
            intent_out_tiled,
            domain_out_tiled
        ])
        nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])

        nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
        nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
        response_embeddings = self.nlg_projection(nlg_out)
        response_embeddings = tf.ensure_shape(response_embeddings, [None, self.max_seq_length, self.embedding_dim])
        response_out = self.response_output(nlg_out)

        return {
            'intent_output': intent_out,
            'domain_output': domain_out,
            'response_output': response_out,
            'response_embeddings': response_embeddings,
            'nlu_gate_weights': nlu_gate_weights,
            'nlg_gate_weights': nlg_gate_weights
        }

    def build_graph(self, input_shape=None):
        inputs = {
            'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='user_utterance_tokens'),
            'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
            'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32),
            'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32),
            'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32),
            'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32),
        }
        return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

    def get_config(self):
        config = super().get_config()
        config.update({
            'moe_params': {
                'embedding_dim': self.embedding_dim,
                'max_seq_length': self.max_seq_length,
                'num_domains': self.num_domains,
                'num_intents': self.num_intents,
                'vocab_size': self.vocab_size,
                'num_experts': self.num_experts,
                'expert_dim': self.expert_dim,
                'turn_id_dim': self.turn_id_dim,
                'capacity_factor': self.moe_nlg.capacity_factor
            }
        })
        return config

class IntentAccuracy(metrics.Metric):
    def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros', dtype=tf.float32)
        self.total_preds = self.add_weight(name='total_preds', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
        self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
        self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

    def reset_state(self):
        self.correct_preds.assign(0.)
        self.total_preds.assign(0.)

class IntentPrecision(metrics.Metric):
    def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
        self.predicted_positives = self.add_weight(name='pp', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        predicted_positives = tf.reduce_sum(y_pred)
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)

    def result(self):
        return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.predicted_positives.assign(0.)

class IntentRecall(metrics.Metric):
    def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros', dtype=tf.float32)
        self.actual_positives = self.add_weight(name='ap', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        actual_positives = tf.reduce_sum(y_true)
        self.true_positives.assign_add(true_positives)
        self.actual_positives.assign_add(actual_positives)

    def result(self):
        return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.actual_positives.assign(0.)

class IntentF1(metrics.Metric):
    def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = IntentPrecision(threshold=threshold)
        self.recall = IntentRecall(threshold=threshold)

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

class DomainAccuracy(metrics.Metric):
    def __init__(self, name='domain_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.accuracy = self.add_weight(name='acc', initializer='zeros', dtype=tf.float32)
        self.count = self.add_weight(name='count', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
        pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
        matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
        self.accuracy.assign_add(tf.reduce_sum(matches))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.accuracy / (self.count + tf.keras.backend.epsilon())

    def reset_state(self):
        self.accuracy.assign(0.)
        self.count.assign(0.)

class Perplexity(metrics.Metric):
    def __init__(self, name='perplexity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
        self.total_loss = self.add_weight(name='total_loss', initializer='zeros', dtype=tf.float32)
        self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        mask = tf.cast(y_true != 0, dtype=tf.float32)
        loss = self.cross_entropy(y_true, y_pred)
        masked_loss = loss * mask
        sum_loss = tf.reduce_sum(masked_loss)
        num_non_padding_tokens = tf.reduce_sum(mask)
        self.total_loss.assign_add(sum_loss)
        self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

    def result(self):
        avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
        return tf.exp(avg_loss)

    def reset_state(self):
        self.total_loss.assign(0.)
        self.total_non_padding_tokens.assign(0.)

class ResponseEmbeddingCosineSimilarity(metrics.Metric):
    def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
        super().__init__(name=name)
        self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
        self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true_norm = y_true / (y_true_norm_val + epsilon)
        y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
        cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
        non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
        non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
        masked_cosine_sim = cosine_sim_per_token * non_padding_mask
        num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
        avg_cosine_sim_per_sample = tf.where(
            num_non_zero_tokens > 0,
            tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
            tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
        )
        self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
        self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

    def reset_state(self):
        self.total_cosine_sim.assign(0.)
        self.num_samples.assign(0.)

def contrastive_loss(y_true, y_pred, margin=1.0):
    epsilon = tf.keras.backend.epsilon()
    y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
    y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
    y_true = y_true / (y_true_norm + epsilon)
    y_pred = y_pred / (y_pred_norm + epsilon)
    cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
    positive_loss = 1.0 - cosine_sim
    mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
    masked_loss = positive_loss * mask
    total_loss = tf.reduce_sum(masked_loss)
    num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
    return total_loss / num_tokens

def calculate_flops(model, batch_size=moe_params["batch_size"]):
    flops = 0
    functional_model = model.build_graph()

    def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
        layer_flops_count = 0

        if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
            return 0

        if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
            for sub_layer in layer_instance.layers:
                layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
            return layer_flops_count

        if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
            return 0

        if isinstance(layer_instance, layers.Dense):
            input_dim = layer_instance.input_shape[-1]
            output_dim = layer_instance.output_shape[-1]
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            layer_flops_count += 2 * input_dim * output_dim * effective_batch_size

        elif isinstance(layer_instance, layers.LSTM):
            input_dim = layer_instance.input_shape[-1]
            hidden_dim = layer_instance.units
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
            layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size

        elif isinstance(layer_instance, layers.Embedding):
            pass

        elif isinstance(layer_instance, MoELayer):
            moe_input_dim = layer_instance.input_dim
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
            layer_flops_count += gate_flops
            for expert in layer_instance.experts:
                layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)

        elif isinstance(layer_instance, layers.MultiHeadAttention):
            config = layer_instance.get_config()
            num_heads = config['num_heads']
            key_dim = config['key_dim']
            d_model = num_heads * key_dim
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
            projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
            attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
            context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
            output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
            layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops

        elif isinstance(layer_instance, layers.TimeDistributed):
            inner_layer = layer_instance.layer
            td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
            layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)

        elif isinstance(layer_instance, TransformerDecoderLayer):
            layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
            layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
            ffn_effective_batch_size = current_batch_size * model.max_seq_length
            layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)

        return layer_flops_count

    for layer in functional_model.layers:
        flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)

    return flops / 1e9

def get_gpu_stats():
    if nvidia_smi is None:
        return 0, 0
    try:
        nvidia_smi.nvmlInit()
        handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
        gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
        mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
        gpu_memory = mem_info.used / (1024 ** 2)
        nvidia_smi.nvmlShutdown()
        return gpu_load, gpu_memory / 1024
    except Exception:
        return 0, 0

class ResourceMonitor(keras.callbacks.Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.resource_stats = []
        self.expert_load_history = []
        self.start_time = None
        self.epoch_start_time = None
        self.flops = None
        self.validation_data = validation_data
        self.nlu_expert_usage_counts = None
        self.nlg_expert_usage_counts = None
        self.nlu_gate_weights_history = []
        self.nlg_gate_weights_history = []

    def on_train_begin(self, logs=None):
        self.start_time = time.time()
        try:
            self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
        except Exception:
            self.flops = 0
        try:
            self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
            self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
        except Exception:
            self.nlu_expert_usage_counts = np.zeros(0)
            self.nlg_expert_usage_counts = np.zeros(0)

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory()
        gpu_load, gpu_memory = get_gpu_stats()
        epoch_time = time.time() - self.epoch_start_time
        self.resource_stats.append({
            'epoch': epoch + 1,
            'epoch_time': epoch_time,
            'cpu_usage': cpu_usage,
            'memory_used': memory.used / (1024 ** 3),
            'memory_total': memory.total / (1024 ** 3),
            'gpu_load': gpu_load,
            'gpu_memory': gpu_memory,
            'loss': logs.get('loss', 0),
            'val_loss': logs.get('val_loss', 0),
            'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
            'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
            'intent_precision': logs.get('intent_output_intent_precision', 0),
            'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
            'intent_recall': logs.get('intent_output_intent_recall', 0),
            'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
            'intent_f1': logs.get('intent_output_intent_f1', 0),
            'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
            'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
            'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
            'perplexity': logs.get('response_output_perplexity', 0),
            'val_perplexity': logs.get('val_response_output_perplexity', 0),
            'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
            'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
            'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
            'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
        })
        if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
            self.expert_load_history.append({
                'epoch': epoch + 1,
                'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
            })

        sample_size = 100
        for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
            inputs, _ = batch
            outputs = self.model(inputs, training=False)
            nlu_gate_weights = outputs['nlu_gate_weights']
            reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
            self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
            nlg_gate_weights = outputs['nlg_gate_weights']
            reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
            self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

    def on_train_end(self, logs=None):
        self.total_time = time.time() - self.start_time

        if len(self.nlu_gate_weights_history) > 0:
            all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
            nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
            for expert_id in nlu_top_indices:
                if expert_id < self.model.moe_nlu.total_experts:
                    self.nlu_expert_usage_counts[expert_id] += 1

        if len(self.nlg_gate_weights_history) > 0:
            all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
            nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
            for expert_id in nlg_top_indices:
                if expert_id < self.model.moe_nlg.total_experts:
                    self.nlg_expert_usage_counts[expert_id] += 1

    def visualize_expert_load_distribution(self):
        total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
        total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
        nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
        nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
        nlu_stability = np.var(nlu_percentages)
        nlg_stability = np.var(nlg_percentages)
        return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {
            "d_model": self.d_model.numpy(),
            "warmup_steps": self.warmup_steps
        }

word_to_id = {}
id_to_word = {}
try:
    possible_paths = [
        os.path.join("preprocessor_models", "preprocessor_params.json"),
        os.path.join("tf_datasets", "preprocessor_params.json"),
        "preprocessor_params.json"
    ]
    
    vocab_loaded = False
    for path in possible_paths:
        if os.path.exists(path):
            with open(path, "r") as f:
                preprocessor_params = json.load(f)
            word_to_id = {k: int(v) for k, v in preprocessor_params['word_to_id'].items()}
            id_to_word = {int(k): v for k, v in preprocessor_params['id_to_word'].items()}
            vocab_loaded = True
            break
    
    if not vocab_loaded:
        raise FileNotFoundError("Vocabulary file not found in any location")
except Exception as e:
    word_to_id = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
    id_to_word = {0: '<pad>', 1: '<sos>', 2: '<eos>', 3: '<unk>'}

model_save_path = "best_DeepseekTopPExpertChoice_moe_model"
custom_objects = {
    "HybridRouting":HybridRouting,
    "MoELayer": MoELayer,
    "CustomLearningRateSchedule": CustomLearningRateSchedule,
    "MoEModel": MoEModel,
    "TransformerDecoderLayer": TransformerDecoderLayer,
    "IntentAccuracy": IntentAccuracy,
    "IntentPrecision": IntentPrecision,
    "IntentRecall": IntentRecall,
    "IntentF1": IntentF1,
    "DomainAccuracy": DomainAccuracy,
    "Perplexity": Perplexity,
    "ResponseEmbeddingCosineSimilarity": ResponseEmbeddingCosineSimilarity
}
try:
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
except Exception as e:
    raise

def compile_model(model, moe_params):
    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    losses_dict = {
        'intent_output': losses.BinaryCrossentropy(),
        'domain_output': losses.SparseCategoricalCrossentropy(),
        'response_output': losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss
    }

    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }

    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={
            'intent_output': 0.5,
            'domain_output': 0.5,
            'response_output': 1.0,
            'response_embeddings': 1.0
        }
    )
    return model

model = compile_model(model, moe_params)
sample_input = next(iter(train_tf_dataset.take(1)))
model(sample_input[0])
model.summary()

class EvaluationMetrics:
    def __init__(self, model, test_dataset, moe_params, raw_data, word_to_id, id_to_word):
        self.model = model
        self.test_dataset = test_dataset
        self.moe_params = moe_params
        self.raw_data = raw_data
        self.word_to_id = word_to_id
        self.id_to_word = id_to_word
        self.nlu_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.nlg_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.resource_stats = {
            'cpu_usage': [],
            'memory_used': [],
            'gpu_load': [],
            'gpu_memory': [],
            'batch_times': []
        }
        self.special_tokens = {'<pad>', '<sos>', '<eos>', '<unk>'}
        self.nlu_top_k = min(model.get_layer('moe_nlu').routed_experts, 2)
        self.nlg_top_k = min(model.get_layer('moe_nlg').routed_experts, 2)

    def evaluate(self):
        def add_response_embeddings(features, targets):
            response_tokens = targets['response_output']
            response_embeddings = self.model.embedding(response_tokens)
            targets['response_embeddings'] = response_embeddings
            return features, targets
        
        test_dataset_with_embeddings = self.test_dataset.map(add_response_embeddings, num_parallel_calls=tf.data.AUTOTUNE)

        start_time = time.time()
        test_results = self.model.evaluate(test_dataset_with_embeddings, verbose=1)
        eval_time = time.time() - start_time
        
        metric_names = self.model.metrics_names
        
        perplexity_value = None
        for name, value in zip(metric_names, test_results):
            if name == 'response_output_perplexity':
                perplexity_value = value
                break
        if perplexity_value is None:
            perplexity_value = 0.0
        
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory().used / (1024 ** 3)
        gpu_load, gpu_memory = get_gpu_stats()
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))

        total_tokens = 0
        total_time = 0
        num_batches = 0
        
        test_dataset_small = test_dataset_with_embeddings.unbatch().batch(moe_params["batch_size"])
        
        all_true_intents = []
        all_pred_intents = []
        all_true_domains = []
        all_pred_domains = []
        
        for batch_idx, batch in enumerate(test_dataset_small):
            inputs, targets = batch
            response_tokens = targets['response_output'].numpy()
            non_padding_mask = response_tokens != 0
            total_tokens += np.sum(non_padding_mask)
            
            start_time = time.time()
            outputs = self.model(inputs, training=False)
            batch_time = time.time() - start_time
            
            total_time += batch_time
            num_batches += tf.shape(inputs['user_utterance_tokens'])[0]
            
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            
            self.resource_stats['cpu_usage'].append(cpu_usage)
            self.resource_stats['memory_used'].append(memory.used / (1024 ** 3))
            self.resource_stats['gpu_load'].append(gpu_load)
            self.resource_stats['gpu_memory'].append(gpu_memory)
            self.resource_stats['batch_times'].append(batch_time)
            
            nlu_gate_weights = outputs.get('nlu_gate_weights', tf.zeros([0, self.model.moe_nlu.routed_experts])).numpy()
            nlu_top_indices = np.argsort(-nlu_gate_weights, axis=-1)[:, :self.nlu_top_k] + self.model.moe_nlu.k_s
            for idx in range(self.nlu_top_k):
                expert_ids = nlu_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlu_expert_usage_counts):
                        self.nlu_expert_usage_counts[expert_id] += 1
            
            nlg_gate_weights = outputs.get('nlg_gate_weights', tf.zeros([0, self.model.moe_nlg.routed_experts])).numpy()
            if len(nlg_gate_weights.shape) == 3:
                nlg_gate_weights = nlg_gate_weights.reshape(-1, nlg_gate_weights.shape[-1])
            nlg_top_indices = np.argsort(-nlg_gate_weights, axis=-1)[:, :self.nlg_top_k] + self.model.moe_nlg.k_s
            for idx in range(self.nlg_top_k):
                expert_ids = nlg_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlg_expert_usage_counts):
                        self.nlg_expert_usage_counts[expert_id] += 1
            
            true_intents = targets.get('intent_output', np.zeros((moe_params["batch_size"], self.model.num_intents))).numpy()
            pred_intents = (outputs.get('intent_output', np.zeros_like(true_intents)).numpy() > 0.5).astype(np.int32)
            all_true_intents.append(true_intents)
            all_pred_intents.append(pred_intents)
            
            true_domains = targets.get('domain_output', np.zeros((moe_params["batch_size"], 1))).numpy().flatten()
            pred_domains = np.argmax(outputs.get('domain_output', np.zeros((moe_params["batch_size"], self.model.num_domains))).numpy(), axis=-1)
            all_true_domains.append(true_domains)
            all_pred_domains.append(pred_domains)
            
            if batch_idx % 50 == 0:
                gc.collect()
                tf.keras.backend.clear_session()
        
        avg_time_per_token = (total_time / total_tokens) * 1000 if total_tokens > 0 else 0
        
        avg_cpu = np.mean(self.resource_stats['cpu_usage']) if self.resource_stats['cpu_usage'] else 0
        avg_memory = np.mean(self.resource_stats['memory_used']) if self.resource_stats['memory_used'] else 0
        avg_gpu_load = np.mean(self.resource_stats['gpu_load']) if self.resource_stats['gpu_load'] else 0
        avg_gpu_memory = np.mean(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        peak_gpu_memory = np.max(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))
        
        all_true_intents = np.vstack(all_true_intents)
        all_pred_intents = np.vstack(all_pred_intents)
        all_true_domains = np.concatenate(all_true_domains)
        all_pred_domains = np.concatenate(all_pred_domains)
        
        intent_f1_score = f1_score(all_true_intents, all_pred_intents, average='micro')
        intent_f1_score_macro = f1_score(all_true_intents, all_pred_intents, average='macro')
        intent_f1_score_weighted = f1_score(all_true_intents, all_pred_intents, average='weighted')
        intent_accuracy_score = accuracy_score(all_true_intents, all_pred_intents)
        
        domain_accuracy_score = accuracy_score(all_true_domains, all_pred_domains)
        domain_f1_score_macro = f1_score(all_true_domains, all_pred_domains, average='macro')

        hypotheses = []
        references = []
        meteor_scores = []
        rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        
        domain_metrics = defaultdict(lambda: {
            'bleu': [],
            'rouge1': [],
            'rouge2': [],
            'rougeL': [],
            'meteor': [],
            'count': 0
        })
        
        if not self.raw_data.get('test'):
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0
            domain_metrics = {}
        else:
            for i, batch in enumerate(test_dataset_small):
                if i >= len(self.raw_data.get('test', [])):
                    break
                
                try:
                    inputs, targets = batch
                    with tf.device('/CPU:0'):
                        outputs = self.model(inputs, training=False)
                    
                    domain_id = targets['domain_output'].numpy()[0][0]
                    domain_name = f"domain_{domain_id}"
                    
                    response_probs = outputs['response_output'].numpy()
                    generated_tokens = np.argmax(response_probs, axis=-1)[0]
                    true_tokens = targets['response_output'].numpy()[0]
                    
                    raw_item = self.raw_data['test'][i]
                    ref_text = raw_item.get('raw_next_system_response', '')
                    if not ref_text:
                        continue
                    
                    gen_text = self.tokens_to_text(generated_tokens)
                    ref_tokens = self.tokens_to_text(true_tokens)
                    
                    if not gen_text.strip() or not ref_text.strip():
                        continue
                    
                    hypotheses.append(gen_text)
                    references.append([ref_text])
                    
                    ref_tokens = word_tokenize(ref_text.lower())
                    gen_tokens = word_tokenize(gen_text.lower())
                    
                    try:
                        meteor = meteor_score([ref_tokens], gen_tokens)
                        meteor_scores.append(meteor)
                        domain_metrics[domain_name]['meteor'].append(meteor)
                    except Exception as e:
                        meteor_scores.append(0)
                        domain_metrics[domain_name]['meteor'].append(0)
                    
                    try:
                        rouge_scores = rouge_scorer_instance.score(ref_text, gen_text)
                        domain_metrics[domain_name]['rouge1'].append(rouge_scores['rouge1'].fmeasure)
                        domain_metrics[domain_name]['rouge2'].append(rouge_scores['rouge2'].fmeasure)
                        domain_metrics[domain_name]['rougeL'].append(rouge_scores['rougeL'].fmeasure)
                    except Exception as e:
                        pass
                    
                    domain_metrics[domain_name]['count'] += 1
                    
                    if i % 50 == 0:
                        gc.collect()
                        tf.keras.backend.clear_session()
                    
                except Exception as e:
                    continue
        
        if hypotheses and references:
            hypotheses_tok = []
            references_tok = []
            
            for hyp, ref_list in zip(hypotheses, references):
                if not hyp or not ref_list[0]:
                    continue
                hyp_tokens = word_tokenize(hyp.lower())
                ref_tokens = [word_tokenize(ref.lower()) for ref in ref_list]
                hypotheses_tok.append(hyp_tokens)
                references_tok.append(ref_tokens)
            
            chencherry = SmoothingFunction()
            try:
                bleu_score = corpus_bleu(references_tok, hypotheses_tok, smoothing_function=chencherry.method1)
            except Exception as e:
                bleu_score = 0.0
            
            rouge_scores = []
            for hyp, ref in zip(hypotheses, references):
                if hyp and ref[0]:
                    try:
                        rouge_scores.append(rouge_scorer_instance.score(ref[0], hyp))
                    except Exception as e:
                        continue
            
            avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_meteor = np.mean(meteor_scores) if meteor_scores else 0.0
        else:
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0

        return {
            'intent_f1_micro': intent_f1_score,
            'intent_f1_macro': intent_f1_score_macro,
            'intent_f1_weighted': intent_f1_score_weighted,
            'intent_accuracy': intent_accuracy_score,
            'domain_accuracy': domain_accuracy_score,
            'domain_f1_macro': domain_f1_score_macro,
            'bleu': bleu_score,
            'rouge1': avg_rouge1,
            'rouge2': avg_rouge2,
            'rougeL': avg_rougeL,
            'meteor': avg_meteor,
            'perplexity': perplexity_value,
            'pred_speed': avg_time_per_token,
            'domain_metrics': domain_metrics,
            'eval_time': eval_time
        }
    
    def tokens_to_text(self, tokens):
        words = []
        for token in tokens:
            if token == 0:
                continue
            word = self.id_to_word.get(token, '<unk>')
            if word not in self.special_tokens:
                words.append(word)
        return " ".join(words).strip()

results_table = {
    'Evaluation Time (seconds)': [],
    'Prediction Speed (ms/token)': [],
    'Average CPU Usage (percent)': [],
    'Average GPU Usage (percent)': [],
    'Average Memory (GB)': [],
    'Average GPU Memory (GB)': [],
    'Average FLOPs Estimate (GFLOPs)': [],
    'NLU Expert 1 (percent)': [],
    'NLU Expert 2 (percent)': [],
    'NLU Expert 3 (percent)': [],
    'NLU Expert 4 (percent)': [],
    'NLU Expert 5 (percent)': [],
    'NLU Expert 6 (percent)': [],
    'NLU Expert 7 (percent)': [],
    'NLU Expert 8 (percent)': [],
    'NLG Expert 1 (percent)': [],
    'NLG Expert 2 (percent)': [],
    'NLG Expert 3 (percent)': [],
    'NLG Expert 4 (percent)': [],
    'NLG Expert 5 (percent)': [],
    'NLG Expert 6 (percent)': [],
    'NLG Expert 7 (percent)': [],
    'NLG Expert 8 (percent)': [],
    'Intent F1-score (Micro)': [],
    'Intent F1-score (Macro)': [],
    'Intent F1-score (Weighted)': [],
    'Intent Accuracy': [],
    'Domain Accuracy': [],
    'Domain F1-score (Macro)': [],
    'BLEU Score': [],
    'ROUGE-1 F1': [],
    'ROUGE-2 F1': [],
    'ROUGE-L F1': [],
    'METEOR Score': [],
    'Perplexity': [],
    'NLU Expert Load Stability (Variance)': [],
    'NLG Expert Load Stability (Variance)': []
}

num_iterations = 5
for iteration in range(1, num_iterations + 1):
    tf.keras.backend.clear_session()
    gc.collect()
    
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
    model = compile_model(model, moe_params)
    
    evaluator = EvaluationMetrics(model, test_tf_dataset, moe_params, raw_data, word_to_id, id_to_word)

    evaluation_results = evaluator.evaluate()
    results_table['Evaluation Time (seconds)'].append(evaluation_results['eval_time'])
    results_table['Prediction Speed (ms/token)'].append(evaluation_results['pred_speed'])
    results_table['Average CPU Usage (percent)'].append(np.mean(evaluator.resource_stats['cpu_usage']) if evaluator.resource_stats['cpu_usage'] else 0)
    results_table['Average GPU Usage (percent)'].append(np.mean(evaluator.resource_stats['gpu_load']) if evaluator.resource_stats['gpu_load'] else 0)
    results_table['Average Memory (GB)'].append(np.mean(evaluator.resource_stats['memory_used']) if evaluator.resource_stats['memory_used'] else 0)
    results_table['Average GPU Memory (GB)'].append(np.mean(evaluator.resource_stats['gpu_memory']) if evaluator.resource_stats['gpu_memory'] else 0)
    results_table['Average FLOPs Estimate (GFLOPs)'].append(calculate_flops(model))
    nlu_counts = evaluator.nlu_expert_usage_counts
    nlg_counts = evaluator.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) if np.sum(nlu_counts) > 0 else 1
    total_nlg = np.sum(nlg_counts) if np.sum(nlg_counts) > 0 else 1
    results_table['NLU Expert 1 (percent)'].append((nlu_counts[0] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 2 (percent)'].append((nlu_counts[1] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 3 (percent)'].append((nlu_counts[2] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 4 (percent)'].append((nlu_counts[3] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 5 (percent)'].append((nlu_counts[4] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 6 (percent)'].append((nlu_counts[5] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 7 (percent)'].append((nlu_counts[6] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 8 (percent)'].append((nlu_counts[7] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLG Expert 1 (percent)'].append((nlg_counts[0] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 2 (percent)'].append((nlg_counts[1] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 3 (percent)'].append((nlg_counts[2] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 4 (percent)'].append((nlg_counts[3] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 5 (percent)'].append((nlg_counts[4] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 6 (percent)'].append((nlg_counts[5] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 7 (percent)'].append((nlg_counts[6] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 8 (percent)'].append((nlg_counts[7] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['Intent F1-score (Micro)'].append(evaluation_results['intent_f1_micro'])
    results_table['Intent F1-score (Macro)'].append(evaluation_results['intent_f1_macro'])
    results_table['Intent F1-score (Weighted)'].append(evaluation_results['intent_f1_weighted'])
    results_table['Intent Accuracy'].append(evaluation_results['intent_accuracy'])
    results_table['Domain Accuracy'].append(evaluation_results['domain_accuracy'])
    results_table['Domain F1-score (Macro)'].append(evaluation_results['domain_f1_macro'])
    results_table['BLEU Score'].append(evaluation_results['bleu'])
    results_table['ROUGE-1 F1'].append(evaluation_results['rouge1'])
    results_table['ROUGE-2 F1'].append(evaluation_results['rouge2'])
    results_table['ROUGE-L F1'].append(evaluation_results['rougeL'])
    results_table['METEOR Score'].append(evaluation_results['meteor'])
    results_table['Perplexity'].append(evaluation_results['perplexity'])
    results_table['NLU Expert Load Stability (Variance)'].append(np.var(nlu_counts / total_nlu * 100) if total_nlu > 0 else 0)
    results_table['NLG Expert Load Stability (Variance)'].append(np.var(nlg_counts / total_nlg * 100) if total_nlg > 0 else 0)

for key in results_table:
    if results_table[key]:
        results_table[key].append(np.mean(results_table[key]))
    else:
        results_table[key].append(0)

final_df = pd.DataFrame(results_table)
final_df = final_df.T
final_df.columns = [f'Iteration {i+1}' for i in range(num_iterations)] + ['Average']
final_df.to_excel('prediction_DeepseekTopPExpertChoic_results.xlsx', index=False)
final_df

2025-07-04 08:44:07.990557: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 08:44:08.305465: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 08:44:08.305620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 08:44:08.352349: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 08:44:08.471319: I tensorflow/core/platform/cpu_feature_guar

Model: "mo_e_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  784896    
                                                                 
 embedding (Embedding)       multiple                  8576      
                                                                 
 layer_normalization (Layer  multiple                  256       
 Normalization)                                                  
                                                                 
 transformer_decoder_layer   multiple                  264576    
 (TransformerDecoderLayer)                                       
                                                                 
 dropout_3 (Dropout)         multiple                  0         
                                                                 
 moe_nlu (MoELayer)          multiple                  2

,Iteration 1,Average
Evaluation Time (seconds),23.373688,23.373688
Prediction Speed (ms/token),2.799684,2.799684
Average CPU Usage (percent),24.294703,24.294703
Average GPU Usage (percent),1.838225,1.838225
Average Memory (GB),5.701781,5.701781
Average GPU Memory (GB),14.504456,14.504456
Average FLOPs Estimate (GFLOPs),2.377871,2.377871
NLU Expert 1 (percent),0.000000,0.000000
NLU Expert 2 (percent),0.000000,0.000000
NLU Expert 3 (percent),23.443092,23.443092


# DeepSeekMoE with Adaptive Gating with Expert Choice Balance experiment code

In [1]:
def reset_kernel():
    tf.keras.backend.clear_session()
    import gc
    gc.collect()

results_table = {
    'Training Speed (epochs per second)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Training Time (second/epoch)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Total Training Time (second/iteration)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Computational Resource Usage': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average CPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average GPU Usage (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average GPU Memory (GB)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average FLOPS Estimate (GFLOPS)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Expert NLU Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)},
    'Expert NLG Load Distribution Stability': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLU Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 1 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 2 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 3 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 4 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 5 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 6 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 7 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average NLG Expert 8 (percent)': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average Validation Entity Accuracy': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average Intent Accuracy': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average Validation Perplexity': {f'Iteration {i+1}': 0 for i in range(10)},
    'Average Validation Response Cosine Similarity': {f'Iteration {i+1}': 0 for i in range(10)}
}
for key in results_table:
    results_table[key]['Average'] = 0

best_val_loss = float('inf')
best_model_path = "best_deepseekmoe_adaptive_gating_expert_choice_model"

for iteration in range(5):
    print(f"\nStarting Training Iteration {iteration + 1}")
    reset_kernel()

    def load_tf_datasets_from_disk(load_path):
        print(f"\nLoading TensorFlow Datasets and MoE parameters from {load_path}...")
        try:
            with open(os.path.join(load_path, "moe_params.json"), "r") as f:
                loaded_moe_params = json.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"moe_params.json not found in {load_path}")

        element_spec = (
            {
                'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
                'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
                'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
                'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            },
            {
                'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
                'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
                'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
            }
        )

        try:
            train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
            test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
        except Exception as e:
            raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

        raw_data = {}
        for dataset_name in ['train', 'test']:
            path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
            if os.path.exists(path):
                with open(path, 'rb') as f:
                    raw_data[dataset_name] = pickle.load(f)
                print(f"Loaded raw data for {dataset_name} from {path}")
            else:
                print(f"Warning: Raw data file {path} not found.")
                raw_data[dataset_name] = []

        return {
            "train_dataset": train_tf_dataset,
            "test_dataset": test_tf_dataset,
            "moe_params": loaded_moe_params,
            "raw_data": raw_data
        }

    tf_dataset_save_path = "tf_datasets"
    loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
    train_tf_dataset = loaded_data["train_dataset"]
    moe_params = loaded_data["moe_params"]
    raw_data = loaded_data["raw_data"]

    class DeepSeekMoELayer(layers.Layer):
        def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.01, tau=0.1, name=None):
            super(DeepSeekMoELayer, self).__init__(name=name)
            self.m = m
            self.k_s = k_s
            self.total_experts = num_experts * self.m
            self.routed_experts = self.total_experts - self.k_s
            self.alpha = alpha
            self.tau = tau
            self.input_dim = input_dim
            self.seq_length = None
            self.adjusted_expert_dim = expert_dim // self.m

            self.shared_experts = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                    layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
                ], name=f'shared_expert_{i}') for i in range(self.k_s)
            ]
            self.routed_experts_list = [
                keras.Sequential([
                    layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                    layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
                ], name=f'routed_expert_{i}') for i in range(self.routed_experts)
            ]
            self.experts = self.shared_experts + self.routed_experts_list
            self.gate = layers.Dense(self.routed_experts, activation=None, name='gate_weights')
            self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

        def call(self, inputs, training=False):
            input_rank = inputs.shape.rank
            if input_rank == 3:
                batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
                flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
                is_3d = True
            else:
                batch_size = tf.shape(inputs)[0]
                seq_length = 1
                flat_inputs = inputs
                is_3d = False

            flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
            num_tokens = tf.shape(flat_inputs)[0]

            shared_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
            for i in range(self.k_s):
                shared_output += self.shared_experts[i](flat_inputs)

            gate_logits = self.gate(flat_inputs)
            gate_weights = tf.nn.softmax(gate_logits, axis=-1)
            sorted_weights = tf.sort(gate_weights, axis=-1, direction='DESCENDING')
            p1 = sorted_weights[:, 0]
            p2 = sorted_weights[:, 1]
            use_top_1 = tf.cast(p1 - p2 >= self.tau, tf.float32)
            k_per_token = tf.where(use_top_1 > 0, 1, 2)

            max_k = 2
            top_k_values, top_k_indices = tf.nn.top_k(gate_weights, k=max_k, sorted=True)
            top_k_indices = top_k_indices + self.k_s
            top_k_weights = top_k_values / (tf.reduce_sum(top_k_values, axis=-1, keepdims=True) + tf.keras.backend.epsilon())

            k_mask = tf.range(max_k, dtype=tf.int32) < tf.expand_dims(k_per_token, -1)
            k_mask = tf.cast(k_mask, tf.float32)
            top_k_weights = top_k_weights * k_mask
            top_k_weights = top_k_weights / (tf.reduce_sum(top_k_weights, axis=-1, keepdims=True) + tf.keras.backend.epsilon())

            expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
            routed_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
            for k in range(max_k):
                kth_weights = top_k_weights[:, k]
                kth_indices = top_k_indices[:, k]
                mask = tf.one_hot(kth_indices, depth=self.total_experts, dtype=tf.float32)
                kth_expert_output = tf.reduce_sum(expert_outputs * tf.expand_dims(mask, -1), axis=1)
                weighted_kth_output = kth_expert_output * tf.expand_dims(kth_weights, -1)
                routed_output += weighted_kth_output

            output_flat = shared_output + routed_output + flat_inputs

            if is_3d:
                output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
                if self.seq_length is not None:
                    output.set_shape([None, self.seq_length, self.input_dim])
                gate_weights_reshaped = tf.reshape(gate_weights, [batch_size, seq_length, self.routed_experts])
                if self.seq_length is not None:
                    gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
            else:
                output = output_flat
                gate_weights_reshaped = gate_weights

            f_i = tf.reduce_mean(tf.reduce_sum(tf.one_hot(tf.argmax(gate_weights, axis=-1), depth=self.routed_experts), axis=0))
            P_i = tf.reduce_mean(gate_weights, axis=0)
            load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
            self.add_loss(load_balancing_loss)
            self.load_balancing_loss_metric.update_state(load_balancing_loss)

            return output, gate_weights_reshaped

        def get_metrics(self):
            return {f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric}

        def get_config(self):
            config = super().get_config()
            config.update({
                'num_experts': self.total_experts // self.m,
                'expert_dim': self.adjusted_expert_dim * self.m,
                'input_dim': self.input_dim,
                'm': self.m,
                'k_s': self.k_s,
                'alpha': self.alpha,
                'tau': self.tau,
                'name': self.name
            })
            return config

    class TransformerDecoderLayer(layers.Layer):
        def __init__(self, d_model, num_heads, dff, rate=0.1):
            super(TransformerDecoderLayer, self).__init__()
            self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
            self.ffn = keras.Sequential([
                layers.Dense(dff, activation='relu'),
                layers.Dense(d_model)
            ])
            self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
            self.dropout1 = layers.Dropout(rate)
            self.dropout2 = layers.Dropout(rate)
            self.dropout3 = layers.Dropout(rate)

        def call(self, x, enc_output, training, look_ahead_mask=None):
            if look_ahead_mask is not None:
                look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
                look_ahead_mask = 1.0 - look_ahead_mask
            attn1_output = self.mha1(query=x, value=x, key=x, attention_mask=look_ahead_mask, training=training)
            attn1 = self.dropout1(attn1_output, training=training)
            out1 = self.layernorm1(attn1 + x)
            attn2_output = self.mha2(query=out1, value=enc_output, key=enc_output, training=training)
            attn2 = self.dropout2(attn2_output, training=training)
            out2 = self.layernorm2(attn2 + out1)
            ffn_output = self.ffn(out2)
            ffn_output = self.dropout3(ffn_output, training=training)
            out3 = self.layernorm3(ffn_output + out2)
            return out3

        def get_config(self):
            config = super().get_config()
            mha1_config = self.mha1.get_config()
            config.update({
                'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
                'num_heads': mha1_config['num_heads'],
                'dff': self.ffn.layers[0].units,
                'rate': self.dropout1.rate
            })
            return config

    class MoEModel(models.Model):
        def __init__(self, moe_params):
            super(MoEModel, self).__init__()
            self.embedding_dim = moe_params["embedding_dim"]
            self.max_seq_length = moe_params["max_seq_length"]
            self.num_domains = moe_params["num_domains"]
            self.num_intents = moe_params["num_intents"]
            self.vocab_size = moe_params["vocab_size"]
            self.num_experts = moe_params["num_experts"]
            self.expert_dim = moe_params["expert_dim"]
            self.turn_id_dim = moe_params["turn_id_dim"]
            self.num_heads = 4
            self.dff = self.embedding_dim * 4
            self.dropout_rate = 0.1
            self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                                 self.num_domains + self.turn_id_dim + self.num_intents)
            self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains
            self.embedding = layers.Embedding(
                self.vocab_size,
                self.embedding_dim,
                mask_zero=True,
                embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05),
                name="embedding"
            )
            self.embedding.build((None,))
            with tf.init_scope():
                embedding_weights = self.embedding.get_weights()
                if embedding_weights and len(embedding_weights) > 0:
                    embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                    self.embedding.set_weights(embedding_weights)
            self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
            self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
            self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
            self.decoder_dropout = layers.Dropout(self.dropout_rate)
            self.moe_nlu = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlu_input_dim, m=2, k_s=2, alpha=0.01, tau=0.1, name='moe_nlu')
            self.moe_nlg = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlg_input_dim, m=2, k_s=2, alpha=0.01, tau=0.1, name='moe_nlg')
            self.intent_output = layers.Dense(self.num_intents, activation='sigmoid', name='intent_output')
            self.domain_output = layers.Dense(self.num_domains, activation='softmax', name='domain_output')
            self.response_output = layers.TimeDistributed(layers.Dense(self.vocab_size, activation='softmax'), name='response_output')
            self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)

        def create_look_ahead_mask(self, size):
            mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
            return mask

        def call(self, inputs, training=False):
            user_utterance_tokens = inputs['user_utterance_tokens']
            prev_system_response_tokens = inputs['prev_system_response_tokens']
            decoder_input_tokens = inputs['decoder_input_tokens']
            domain_onehot_input = inputs['domain_onehot_input']
            turn_id_embedding = inputs['turn_id_embedding']
            ontology_multihot_input = inputs['ontology_multihot_input']
            pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
            user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
            prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
            decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)
            user_enc_out = user_embedded
            prev_system_enc_out = prev_system_embedded
            look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
            dec_output = decoder_embedded
            for i, layer in enumerate(self.decoder_layers):
                dec_output = layer(dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask)
            decoder_output = self.decoder_dropout(dec_output, training=training)
            user_attn_out = self.attn_layer(query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), value=user_enc_out, key=user_enc_out, training=training)
            prev_system_attn_out = self.attn_layer(query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), value=prev_system_enc_out, key=prev_system_enc_out, training=training)
            decoder_attn_out = self.attn_layer(query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), value=decoder_output, key=decoder_output, training=training)
            combined_features = layers.Concatenate()([
                tf.squeeze(user_attn_out, 1),
                tf.squeeze(prev_system_attn_out, 1),
                tf.squeeze(decoder_attn_out, 1),
                domain_onehot_input,
                turn_id_embedding,
                ontology_multihot_input
            ])
            combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
            nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
            nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
            intent_out = self.intent_output(nlu_out)
            domain_out = self.domain_output(nlu_out)
            intent_out_expanded = tf.expand_dims(intent_out, axis=1)
            domain_out_expanded = tf.expand_dims(domain_out, axis=1)
            intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
            domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])
            nlg_input = layers.Concatenate(axis=-1)([
                decoder_output,
                intent_out_tiled,
                domain_out_tiled
            ])
            nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])
            nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
            nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
            response_out = self.response_output(nlg_out)
            return {
                'intent_output': intent_out,
                'domain_output': domain_out,
                'response_output': response_out,
                'response_embeddings': decoder_output,
                'nlu_gate_weights': nlu_gate_weights,
                'nlg_gate_weights': nlg_gate_weights
            }

        def build_graph(self):
            inputs = {
                'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='user_utterance_tokens'),
                'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='prev_system_response_tokens'),
                'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='decoder_input_tokens'),
                'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32, name='domain_onehot_input'),
                'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32, name='turn_id_embedding'),
                'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32, name='ontology_multihot_input'),
            }
            return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

        def get_config(self):
            config = super().get_config()
            config.update({
                'moe_params': {
                    'embedding_dim': self.embedding_dim,
                    'max_seq_length': self.max_seq_length,
                    'num_domains': self.num_domains,
                    'num_intents': self.num_intents,
                    'vocab_size': self.vocab_size,
                    'num_experts': self.num_experts,
                    'expert_dim': self.expert_dim,
                    'turn_id_dim': self.turn_id_dim
                }
            })
            return config

    class IntentAccuracy(metrics.Metric):
        def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros')
            self.total_preds = self.add_weight(name='total_preds', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
            self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
            self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

        def reset_state(self):
            self.correct_preds.assign(0.)
            self.total_preds.assign(0.)

    class IntentPrecision(metrics.Metric):
        def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros')
            self.predicted_positives = self.add_weight(name='pp', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            predicted_positives = tf.reduce_sum(y_pred)
            self.true_positives.assign_add(true_positives)
            self.predicted_positives.assign_add(predicted_positives)

        def result(self):
            return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.predicted_positives.assign(0.)

    class IntentRecall(metrics.Metric):
        def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.threshold = threshold
            self.true_positives = self.add_weight(name='tp', initializer='zeros')
            self.actual_positives = self.add_weight(name='ap', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred > self.threshold, tf.float32)
            true_positives = tf.reduce_sum(y_true * y_pred)
            actual_positives = tf.reduce_sum(y_true)
            self.true_positives.assign_add(true_positives)
            self.actual_positives.assign_add(actual_positives)

        def result(self):
            return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

        def reset_state(self):
            self.true_positives.assign(0.)
            self.actual_positives.assign(0.)

    class IntentF1(metrics.Metric):
        def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
            super().__init__(name=name, **kwargs)
            self.precision = IntentPrecision(threshold=threshold)
            self.recall = IntentRecall(threshold=threshold)

        def update_state(self, y_true, y_pred, sample_weight=None):
            self.precision.update_state(y_true, y_pred, sample_weight)
            self.recall.update_state(y_true, y_pred, sample_weight)

        def result(self):
            p = self.precision.result()
            r = self.recall.result()
            return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

        def reset_state(self):
            self.precision.reset_state()
            self.recall.reset_state()

    class DomainAccuracy(metrics.Metric):
        def __init__(self, name='domain_accuracy', **kwargs):
            super().__init__(name=name, **kwargs)
            self.accuracy = self.add_weight(name='acc', initializer='zeros')
            self.count = self.add_weight(name='count', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
            pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
            matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
            self.accuracy.assign_add(tf.reduce_sum(matches))
            self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.accuracy / (self.count + tf.keras.backend.epsilon())

        def reset_state(self):
            self.accuracy.assign(0.)
            self.count.assign(0.)

    class Perplexity(metrics.Metric):
        def __init__(self, name='perplexity', **kwargs):
            super().__init__(name=name, **kwargs)
            self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
            self.total_loss = self.add_weight(name='total_loss', initializer='zeros')
            self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros')

        def update_state(self, y_true, y_pred, sample_weight=None):
            mask = tf.cast(y_true != 0, dtype=tf.float32)
            loss = self.cross_entropy(y_true, y_pred)
            masked_loss = loss * mask
            sum_loss = tf.reduce_sum(masked_loss)
            num_non_padding_tokens = tf.reduce_sum(mask)
            self.total_loss.assign_add(sum_loss)
            self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

        def result(self):
            avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
            return tf.exp(avg_loss)

        def reset_state(self):
            self.total_loss.assign(0.)
            self.total_non_padding_tokens.assign(0.)

    class ResponseEmbeddingCosineSimilarity(metrics.Metric):
        def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
            super().__init__(name=name, **kwargs)
            self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
            self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

        def update_state(self, y_true, y_pred, sample_weight=None):
            epsilon = tf.keras.backend.epsilon()
            y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
            y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
            y_true_norm = y_true / (y_true_norm_val + epsilon)
            y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
            cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
            non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
            non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
            masked_cosine_sim = cosine_sim_per_token * non_padding_mask
            num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
            avg_cosine_sim_per_sample = tf.where(
                num_non_zero_tokens > 0,
                tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
                tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
            )
            self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
            self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        def result(self):
            return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

        def reset_state(self):
            self.total_cosine_sim.assign(0.)
            self.num_samples.assign(0.)

    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens

    def calculate_flops(model, batch_size=moe_params["batch_size"]):
        flops = 0
        functional_model = model.build_graph()

        def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
            layer_flops_count = 0

            if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
                return 0

            if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
                for sub_layer in layer_instance.layers:
                    layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
                return layer_flops_count

            if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
                return 0

            if isinstance(layer_instance, layers.Dense):
                input_dim = layer_instance.input_shape[-1]
                output_dim = layer_instance.output_shape[-1]
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                layer_flops_count += 2 * input_dim * output_dim * effective_batch_size

            elif isinstance(layer_instance, layers.LSTM):
                input_dim = layer_instance.input_shape[-1]
                hidden_dim = layer_instance.units
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
                layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size

            elif isinstance(layer_instance, layers.Embedding):
                pass

            elif isinstance(layer_instance, DeepSeekMoELayer):
                moe_input_dim = layer_instance.input_dim
                effective_batch_size = current_batch_size
                if len(layer_instance.input_shape) == 3:
                    effective_batch_size *= layer_instance.input_shape[1]
                gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
                layer_flops_count += gate_flops
                for expert in layer_instance.experts:
                    layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)

            elif isinstance(layer_instance, layers.MultiHeadAttention):
                config = layer_instance.get_config()
                num_heads = config['num_heads']
                key_dim = config['key_dim']
                d_model = num_heads * key_dim
                seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
                projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
                attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
                context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
                output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
                layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops

            elif isinstance(layer_instance, layers.TimeDistributed):
                inner_layer = layer_instance.layer
                td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
                layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)

            elif isinstance(layer_instance, TransformerDecoderLayer):
                layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
                layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
                ffn_effective_batch_size = current_batch_size * model.max_seq_length
                layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)

            return layer_flops_count

        for layer in functional_model.layers:
            flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)

        return flops / 1e9

    def get_gpu_stats():
        if nvidia_smi is None:
            return 0, 0
        try:
            nvidia_smi.nvmlInit()
            handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
            gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
            mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
            gpu_memory = mem_info.used / (1024 ** 3)
            nvidia_smi.nvmlShutdown()
            return gpu_load, gpu_memory
        except Exception:
            return 0, 0

    class ResourceMonitor(keras.callbacks.Callback):
        def __init__(self, validation_data):
            super().__init__()
            self.resource_stats = []
            self.expert_load_history = []
            self.start_time = None
            self.epoch_start_time = None
            self.flops = None
            self.validation_data = validation_data
            self.nlu_expert_usage_counts = None
            self.nlg_expert_usage_counts = None
            self.nlu_gate_weights_history = []
            self.nlg_gate_weights_history = []

        def on_train_begin(self, logs=None):
            self.start_time = time.time()
            try:
                self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
                self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
                self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
            except Exception:
                self.flops = 0

        def on_epoch_begin(self, epoch, logs=None):
            self.epoch_start_time = time.time()

        def on_epoch_end(self, epoch, logs=None):
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            epoch_time = time.time() - self.epoch_start_time
            self.resource_stats.append({
                'epoch': epoch + 1,
                'epoch_time': epoch_time,
                'cpu_usage': cpu_usage,
                'memory_used': memory.used / (1024 ** 3),
                'memory_total': memory.total / (1024 ** 3),
                'gpu_load': gpu_load,
                'gpu_memory': gpu_memory,
                'loss': logs.get('loss', 0),
                'val_loss': logs.get('val_loss', 0),
                'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
                'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
                'intent_precision': logs.get('intent_output_intent_precision', 0),
                'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
                'intent_recall': logs.get('intent_output_intent_recall', 0),
                'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
                'intent_f1': logs.get('intent_output_intent_f1', 0),
                'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
                'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
                'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
                'perplexity': logs.get('response_output_perplexity', 0),
                'val_perplexity': logs.get('val_response_output_perplexity', 0),
                'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
                'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
                'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
                'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
            })
            if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
                self.expert_load_history.append({
                    'epoch': epoch + 1,
                    'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                    'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
                })

            sample_size = 100
            for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
                inputs, _ = batch
                outputs = self.model(inputs, training=False)
                nlu_gate_weights = outputs['nlu_gate_weights']
                reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
                self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
                nlg_gate_weights = outputs['nlg_gate_weights']
                reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
                self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

        def on_train_end(self, logs=None):
            self.total_time = time.time() - self.start_time

            if len(self.nlu_gate_weights_history) > 0:
                all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
                nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
                for expert_id in nlu_top_indices:
                    if expert_id < self.model.moe_nlu.total_experts:
                        self.nlu_expert_usage_counts[expert_id] += 1

            if len(self.nlg_gate_weights_history) > 0:
                all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
                nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
                for expert_id in nlg_top_indices:
                    if expert_id < self.model.moe_nlg.total_experts:
                        self.nlg_expert_usage_counts[expert_id] += 1

        def visualize_expert_load_distribution(self):
            total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
            total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
            nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
            nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
            nlu_stability = np.var(nlu_percentages)
            nlg_stability = np.var(nlg_percentages)
            return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

    class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
        def __init__(self, d_model, warmup_steps=4000):
            super().__init__()
            self.d_model = tf.cast(d_model, tf.float32)
            self.warmup_steps = warmup_steps

        def __call__(self, step):
            step = tf.cast(step, tf.float32)
            arg1 = tf.math.rsqrt(step)
            arg2 = step * (self.warmup_steps ** -1.5)
            return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

        def get_config(self):
            return {"d_model": self.d_model.numpy(), "warmup_steps": self.warmup_steps}

    model = MoEModel(moe_params)
    embedding_layer = model.embedding

    def create_validation_split(dataset, embedding_layer, validation_split=0.2, batch_size=moe_params["batch_size"]):
        element_spec = dataset.element_spec
        data_list = [(features, targets) for features, targets in dataset.unbatch().as_numpy_iterator()]
        if not data_list:
            raise ValueError("Dataset is empty.")
        if len(data_list) < 2:
            raise ValueError("Dataset too small to split.")
        train_data, val_data = train_test_split(data_list, test_size=validation_split, shuffle=True)

        def create_dataset_from_list(data):
            if not data:
                raise ValueError("Empty data list provided.")
            features = {
                'user_utterance_tokens': np.array([d[0]['user_utterance_tokens'] for d in data], dtype=np.int32),
                'prev_system_response_tokens': np.array([d[0]['prev_system_response_tokens'] for d in data], dtype=np.int32),
                'decoder_input_tokens': np.array([d[0]['decoder_input_tokens'] for d in data], dtype=np.int32),
                'domain_onehot_input': np.array([d[0]['domain_onehot_input'] for d in data], dtype=np.float32),
                'turn_id_embedding': np.array([d[0]['turn_id_embedding'] for d in data], dtype=np.float32),
                'ontology_multihot_input': np.array([d[0]['ontology_multihot_input'] for d in data], dtype=np.float32),
            }
            response_output = np.array([d[1]['response_output'] for d in data], dtype=np.int32)
            response_embeddings = embedding_layer(response_output).numpy()
            targets = {
                'domain_output': np.array([d[1]['domain_output'] for d in data], dtype=np.int32),
                'intent_output': np.array([d[1]['intent_output'] for d in data], dtype=np.float32),
                'response_output': response_output,
                'response_embeddings': response_embeddings
            }
            return tf.data.Dataset.from_tensor_slices((features, targets)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

        train_dataset = create_dataset_from_list(train_data)
        val_dataset = create_dataset_from_list(val_data)
        return train_dataset, val_dataset

    train_tf_dataset, val_tf_dataset = create_validation_split(train_tf_dataset, embedding_layer)

    losses_dict = {
        'intent_output': losses.BinaryCrossentropy(),
        'domain_output': losses.SparseCategoricalCrossentropy(),
        'response_output': losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss
    }

    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }

    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={'intent_output': 0.5, 'domain_output': 0.5, 'response_output': 1.0, 'response_embeddings': 1.0}
    )

    sample_input = next(iter(train_tf_dataset.take(1)))
    model(sample_input[0])

    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=8, mode='min', restore_best_weights=True)
    resource_monitor = ResourceMonitor(val_tf_dataset)

    class TQDMProgressBar(callbacks.Callback):
        def on_epoch_begin(self, epoch, logs=None):
            self.progress_bar = tqdm(total=len(train_tf_dataset), desc=f'Epoch {epoch + 1}')

        def on_batch_end(self, batch, logs=None):
            self.progress_bar.update(1)

        def on_epoch_end(self, epoch, logs=None):
            self.progress_bar.close()

    tqdm_callback = TQDMProgressBar()

    history = model.fit(
        train_tf_dataset,
        epochs=100,
        validation_data=val_tf_dataset,
        callbacks=[early_stopping, resource_monitor, tqdm_callback],
        verbose=0
    )

    expert_stats = resource_monitor.visualize_expert_load_distribution()
    stats = resource_monitor.resource_stats
    avg_epoch_duration = np.mean([s['epoch_time'] for s in stats]) if stats else 0
    total_training_time = resource_monitor.total_time
    avg_cpu_usage = np.mean([s['cpu_usage'] for s in stats]) if stats else 0
    avg_memory = np.mean([s['memory_used'] for s in stats]) if stats else 0
    avg_gpu_load = np.mean([s['gpu_load'] for s in stats]) if stats else 0
    avg_gpu_memory = np.mean([s['gpu_memory'] for s in stats]) if stats else 0
    avg_flops = resource_monitor.flops if resource_monitor.flops else 0
    nlu_counts = resource_monitor.nlu_expert_usage_counts
    nlg_counts = resource_monitor.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) or 1
    total_nlg = np.sum(nlg_counts) or 1
    expert_nlu_percentages = [(nlu_counts[i] / total_nlu) * 100 for i in range(8)]
    expert_nlg_percentages = [(nlg_counts[i] / total_nlg) * 100 for i in range(8)]
    val_intent_accuracy = np.mean([s['val_intent_accuracy'] for s in stats if s['val_intent_accuracy'] > 0]) if stats else 0
    val_entity_accuracy = np.mean([s['val_domain_accuracy'] for s in stats if s['val_domain_accuracy'] > 0]) if stats else 0
    val_perplexity = np.mean([s['val_perplexity'] for s in stats if s['val_perplexity'] > 0]) if stats else 0
    val_cosine_similarity = np.mean([s['val_cosine_similarity'] for s in stats if s['val_cosine_similarity'] > 0]) if stats and any(s['val_cosine_similarity'] > 0 for s in stats) else 0
    
    results_table['Training Speed (epochs per second)'][f'Iteration {iteration + 1}'] = 1 / avg_epoch_duration if avg_epoch_duration > 0 else 0
    results_table['Training Time (second/epoch)'][f'Iteration {iteration + 1}'] = avg_epoch_duration
    results_table['Total Training Time (second/iteration)'][f'Iteration {iteration + 1}'] = total_training_time
    results_table['Computational Resource Usage'][f'Iteration {iteration + 1}'] = avg_cpu_usage + avg_gpu_load
    results_table['Average CPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_cpu_usage
    results_table['Average GPU Usage (percent)'][f'Iteration {iteration + 1}'] = avg_gpu_load
    results_table['Average Memory (GB)'][f'Iteration {iteration + 1}'] = avg_memory
    results_table['Average GPU Memory (GB)'][f'Iteration {iteration + 1}'] = avg_gpu_memory
    results_table['Average FLOPS Estimate (GFLOPS)'][f'Iteration {iteration + 1}'] = avg_flops
    results_table['Expert NLU Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlu_stability']
    results_table['Expert NLG Load Distribution Stability'][f'Iteration {iteration + 1}'] = expert_stats['nlg_stability']
    for i in range(8):
        if i < len(expert_stats['nlu_percentages']):
            results_table[f'Average NLU Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlu_percentages'][i]
        if i < len(expert_stats['nlg_percentages']):
            results_table[f'Average NLG Expert {i+1} (percent)'][f'Iteration {iteration + 1}'] = expert_stats['nlg_percentages'][i]
    results_table['Average Validation Entity Accuracy'][f'Iteration {iteration + 1}'] = val_entity_accuracy
    results_table['Average Intent Accuracy'][f'Iteration {iteration + 1}'] = val_intent_accuracy
    results_table['Average Validation Perplexity'][f'Iteration {iteration + 1}'] = val_perplexity
    results_table['Average Validation Response Cosine Similarity'][f'Iteration {iteration + 1}'] = val_cosine_similarity

    val_loss = min([s['val_loss'] for s in stats]) if stats else float('inf')
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        try:
            model.save(best_model_path, save_format='tf', include_optimizer=True)
        except Exception as e:
            print(f"\nFailed to save best model to {best_model_path}: {str(e)}")
            
for key in results_table:
    results_table[key]['Average'] = np.mean([results_table[key][f'Iteration {i+1}'] for i in range(10)])

final_result = pd.DataFrame(results_table)
final_result = final_result.T
final_result.to_excel('training_deepseekmoe_adaptive_gating_expert_choice_result.xlsx', index=True)
final_result

2025-07-04 08:55:20.283266: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 08:55:20.656588: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 08:55:20.656709: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 08:55:20.724104: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 08:55:20.848866: I tensorflow/core/platform/cpu_feature_guar


Starting Training Iteration 1

Loading TensorFlow Datasets and MoE parameters from tf_datasets...


2025-07-04 08:55:25.345460: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 08:55:25.549050: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-04 08:55:25.549286: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded raw data for train from tf_datasets/train_raw_data.pkl
Loaded raw data for test from tf_datasets/test_raw_data.pkl


Epoch 1:   0%|          | 0/4428 [00:00<?, ?it/s]2025-07-04 08:55:50.030964: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fb32418b870 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-04 08:55:50.031294: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2025-07-04 08:55:50.060632: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-04 08:55:50.114444: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1751619350.245696   23276 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
Epoch 1: 100%|██████████| 4428/4428 [02:48<00:00, 26.34it/s]


INFO:tensorflow:Assets written to: best_deepseekmoe_adaptive_gating_expert_choice_model/assets


INFO:tensorflow:Assets written to: best_deepseekmoe_adaptive_gating_expert_choice_model/assets


,Iteration 1,Iteration 2,Iteration 3,Iteration 4,Iteration 5,Iteration 6,Iteration 7,Iteration 8,Iteration 9,Iteration 10,Average
Training Speed (epochs per second),0.006088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000609
Training Time (second/epoch),164.254727,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.425473
Total Training Time (second/iteration),169.155386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.915539
Computational Resource Usage,49.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.950000
Average CPU Usage (percent),26.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.650000
Average GPU Usage (percent),23.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.300000
Average Memory (GB),6.567524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.656752
Average GPU Memory (GB),14.529846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.452985
Average FLOPS Estimate (GFLOPS),2.377871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.237787
Expert NLU Load Distribution Stability,480.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.075000


# DeepSeekMoE with Adaptive Gating with Expert Choice Balance evaluation code

In [1]:
def load_tf_datasets_from_disk(load_path):
    try:
        with open(os.path.join(load_path, "moe_params.json"), "r") as f:
            loaded_moe_params = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"moe_params.json not found in {load_path}")

    element_spec = (
        {
            'user_utterance_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'prev_system_response_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'decoder_input_tokens': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32),
            'domain_onehot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_domains"]), dtype=tf.float32),
            'turn_id_embedding': tf.TensorSpec(shape=(None, loaded_moe_params["turn_id_dim"]), dtype=tf.float32),
            'ontology_multihot_input': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
        },
        {
            'domain_output': tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
            'intent_output': tf.TensorSpec(shape=(None, loaded_moe_params["num_intents"]), dtype=tf.float32),
            'response_output': tf.TensorSpec(shape=(None, loaded_moe_params["max_seq_length"]), dtype=tf.int32)
        }
    )

    try:
        train_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "train"), element_spec=element_spec)
        test_tf_dataset = tf.data.Dataset.load(os.path.join(load_path, "test"), element_spec=element_spec)
    except Exception as e:
        raise ValueError(f"Failed to load datasets from {load_path}: {str(e)}")

    raw_data = {}
    for dataset_name in ['train', 'test']:
        path = os.path.join(load_path, f'{dataset_name}_raw_data.pkl')
        if os.path.exists(path):
            with open(path, 'rb') as f:
                raw_data[dataset_name] = pickle.load(f)
        else:
            raw_data[dataset_name] = []

    return {
        "train_dataset": train_tf_dataset,
        "test_dataset": test_tf_dataset,
        "moe_params": loaded_moe_params,
        "raw_data": raw_data
    }

tf_dataset_save_path = "tf_datasets"
loaded_data = load_tf_datasets_from_disk(tf_dataset_save_path)
train_tf_dataset = loaded_data["train_dataset"]
test_tf_dataset = loaded_data["test_dataset"]
moe_params = loaded_data["moe_params"]
raw_data = loaded_data["raw_data"]

moe_params["num_experts"] = 4

class DeepSeekMoELayer(layers.Layer):
    def __init__(self, num_experts, expert_dim, input_dim, m=2, k_s=1, alpha=0.01, tau=0.1, name=None):
        super(DeepSeekMoELayer, self).__init__(name=name)
        self.m = m
        self.k_s = k_s
        self.total_experts = num_experts * self.m
        self.routed_experts = self.total_experts - self.k_s
        self.alpha = alpha
        self.tau = tau
        self.input_dim = input_dim
        self.seq_length = None
        self.adjusted_expert_dim = expert_dim // self.m
        self.shared_experts = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
            ], name=f'shared_expert_{i}') for i in range(self.k_s)
        ]
        self.routed_experts_list = [
            keras.Sequential([
                layers.Dense(self.adjusted_expert_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
                layers.Dense(input_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))
            ], name=f'routed_expert_{i}') for i in range(self.routed_experts)
        ]
        self.experts = self.shared_experts + self.routed_experts_list
        self.gate = layers.Dense(self.routed_experts, activation=None, name='gate_weights')
        self.load_balancing_loss_metric = tf.keras.metrics.Mean(name=f'{name}_load_balancing_loss')

    def call(self, inputs, training=False):
        input_rank = inputs.shape.rank
        if input_rank == 3:
            batch_size, seq_length = tf.shape(inputs)[0], tf.shape(inputs)[1]
            flat_inputs = tf.reshape(inputs, [-1, self.input_dim])
            is_3d = True
        else:
            batch_size = tf.shape(inputs)[0]
            seq_length = 1
            flat_inputs = inputs
            is_3d = False
        flat_inputs = tf.ensure_shape(flat_inputs, [None, self.input_dim])
        num_tokens = tf.shape(flat_inputs)[0]
        shared_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
        for i in range(self.k_s):
            shared_output += self.shared_experts[i](flat_inputs)
        gate_logits = self.gate(flat_inputs)
        gate_weights = tf.nn.softmax(gate_logits, axis=-1)
        sorted_weights = tf.sort(gate_weights, axis=-1, direction='DESCENDING')
        p1 = sorted_weights[:, 0]
        p2 = sorted_weights[:, 1]
        use_top_1 = tf.cast(p1 - p2 >= self.tau, tf.float32)
        k_per_token = tf.where(use_top_1 > 0, 1, 2)
        max_k = 2
        top_k_values, top_k_indices = tf.nn.top_k(gate_weights, k=max_k, sorted=True)
        top_k_indices = top_k_indices + self.k_s
        top_k_weights = top_k_values / (tf.reduce_sum(top_k_values, axis=-1, keepdims=True) + tf.keras.backend.epsilon())
        k_mask = tf.range(max_k, dtype=tf.int32) < tf.expand_dims(k_per_token, -1)
        k_mask = tf.cast(k_mask, tf.float32)
        top_k_weights = top_k_weights * k_mask
        top_k_weights = top_k_weights / (tf.reduce_sum(top_k_weights, axis=-1, keepdims=True) + tf.keras.backend.epsilon())
        expert_outputs = tf.stack([expert(flat_inputs) for expert in self.experts], axis=1)
        routed_output = tf.zeros([num_tokens, self.input_dim], dtype=tf.float32)
        for k in range(max_k):
            kth_weights = top_k_weights[:, k]
            kth_indices = top_k_indices[:, k]
            mask = tf.one_hot(kth_indices, depth=self.total_experts, dtype=tf.float32)
            kth_expert_output = tf.reduce_sum(expert_outputs * tf.expand_dims(mask, -1), axis=1)
            weighted_kth_output = kth_expert_output * tf.expand_dims(kth_weights, -1)
            routed_output += weighted_kth_output
        output_flat = shared_output + routed_output + flat_inputs
        if is_3d:
            output = tf.reshape(output_flat, [batch_size, seq_length, self.input_dim])
            if self.seq_length is not None:
                output.set_shape([None, self.seq_length, self.input_dim])
            gate_weights_reshaped = tf.reshape(gate_weights, [batch_size, seq_length, self.routed_experts])
            if self.seq_length is not None:
                gate_weights_reshaped.set_shape([None, self.seq_length, self.routed_experts])
        else:
            output = output_flat
            gate_weights_reshaped = gate_weights
        f_i = tf.reduce_mean(tf.reduce_sum(tf.one_hot(tf.argmax(gate_weights, axis=-1), depth=self.routed_experts), axis=0))
        P_i = tf.reduce_mean(gate_weights, axis=0)
        load_balancing_loss = self.alpha * tf.reduce_sum(f_i * P_i)
        self.add_loss(load_balancing_loss)
        self.load_balancing_loss_metric.update_state(load_balancing_loss)
        return output, gate_weights_reshaped

    def get_metrics(self):
        return {f'{self.name}_load_balancing_loss': self.load_balancing_loss_metric}

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_experts': self.total_experts // self.m,
            'expert_dim': self.adjusted_expert_dim * self.m,
            'input_dim': self.input_dim,
            'm': self.m,
            'k_s': self.k_s,
            'alpha': self.alpha,
            'tau': self.tau,
            'name': self.name
        })
        return config

class TransformerDecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.ffn = keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask=None):
        if look_ahead_mask is not None:
            look_ahead_mask = tf.cast(look_ahead_mask, tf.float32)
            look_ahead_mask = 1.0 - look_ahead_mask
        attn1_output = self.mha1(query=x, value=x, key=x, attention_mask=look_ahead_mask, training=training)
        attn1 = self.dropout1(attn1_output, training=training)
        out1 = self.layernorm1(attn1 + x)
        attn2_output = self.mha2(query=out1, value=enc_output, key=enc_output, training=training)
        attn2 = self.dropout2(attn2_output, training=training)
        out2 = self.layernorm2(attn2 + out1)
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)
        return out3

    def get_config(self):
        config = super().get_config()
        mha1_config = self.mha1.get_config()
        config.update({
            'd_model': mha1_config['key_dim'] * mha1_config['num_heads'],
            'num_heads': mha1_config['num_heads'],
            'dff': self.ffn.layers[0].units,
            'rate': self.dropout1.rate
        })
        return config

class MoEModel(models.Model):
    def __init__(self, moe_params):
        super(MoEModel, self).__init__()
        self.embedding_dim = moe_params["embedding_dim"]
        self.max_seq_length = moe_params["max_seq_length"]
        self.num_domains = moe_params["num_domains"]
        self.num_intents = moe_params["num_intents"]
        self.vocab_size = moe_params["vocab_size"]
        self.num_experts = moe_params["num_experts"]
        self.expert_dim = moe_params["expert_dim"]
        self.turn_id_dim = moe_params["turn_id_dim"]
        self.num_heads = 4
        self.dff = self.embedding_dim * 4
        self.dropout_rate = 0.1
        self.nlu_input_dim = (self.embedding_dim + self.embedding_dim + self.embedding_dim + 
                             self.num_domains + self.turn_id_dim + self.num_intents)
        self.nlg_input_dim = self.embedding_dim + self.num_intents + self.num_domains
        self.embedding = layers.Embedding(
            self.vocab_size,
            self.embedding_dim,
            mask_zero=True,
            embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05),
            name="embedding"
        )
        self.embedding.build((None,))
        with tf.init_scope():
            embedding_weights = self.embedding.get_weights()
            if embedding_weights and len(embedding_weights) > 0:
                embedding_weights[0][0] = tf.zeros((self.embedding_dim,))
                self.embedding.set_weights(embedding_weights)
        self.pos_encoding = layers.Embedding(self.max_seq_length, self.embedding_dim)
        self.embedding_norm = layers.LayerNormalization(epsilon=1e-6)
        self.decoder_layers = [TransformerDecoderLayer(self.embedding_dim, self.num_heads, self.dff, self.dropout_rate) for _ in range(1)]
        self.decoder_dropout = layers.Dropout(self.dropout_rate)
        self.moe_nlu = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlu_input_dim, m=2, k_s=2, alpha=0.01, tau=0.1, name='moe_nlu')
        self.moe_nlg = DeepSeekMoELayer(num_experts=self.num_experts, expert_dim=self.expert_dim, input_dim=self.nlg_input_dim, m=2, k_s=2, alpha=0.01, tau=0.1, name='moe_nlg')
        self.intent_output = layers.Dense(self.num_intents, activation='sigmoid', name='intent_output')
        self.domain_output = layers.Dense(self.num_domains, activation='softmax', name='domain_output')
        self.response_output = layers.TimeDistributed(layers.Dense(self.vocab_size, activation='softmax'), name='response_output')
        self.attn_layer = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embedding_dim // self.num_heads)

    def create_look_ahead_mask(self, size):
        mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
        return mask

    def call(self, inputs, training=False):
        user_utterance_tokens = inputs['user_utterance_tokens']
        prev_system_response_tokens = inputs['prev_system_response_tokens']
        decoder_input_tokens = inputs['decoder_input_tokens']
        domain_onehot_input = inputs['domain_onehot_input']
        turn_id_embedding = inputs['turn_id_embedding']
        ontology_multihot_input = inputs['ontology_multihot_input']
        pos_enc = self.pos_encoding(tf.range(self.max_seq_length))
        user_embedded = self.embedding_norm(self.embedding(user_utterance_tokens) + pos_enc)
        prev_system_embedded = self.embedding_norm(self.embedding(prev_system_response_tokens) + pos_enc)
        decoder_embedded = self.embedding_norm(self.embedding(decoder_input_tokens) + pos_enc)
        user_enc_out = user_embedded
        prev_system_enc_out = prev_system_embedded
        look_ahead_mask = self.create_look_ahead_mask(self.max_seq_length)
        dec_output = decoder_embedded
        for i, layer in enumerate(self.decoder_layers):
            dec_output = layer(dec_output, prev_system_enc_out, training=training, look_ahead_mask=look_ahead_mask)
        decoder_output = self.decoder_dropout(dec_output, training=training)
        user_attn_out = self.attn_layer(query=tf.reduce_mean(user_enc_out, axis=1, keepdims=True), value=user_enc_out, key=user_enc_out, training=training)
        prev_system_attn_out = self.attn_layer(query=tf.reduce_mean(prev_system_enc_out, axis=1, keepdims=True), value=prev_system_enc_out, key=prev_system_enc_out, training=training)
        decoder_attn_out = self.attn_layer(query=tf.reduce_mean(decoder_output, axis=1, keepdims=True), value=decoder_output, key=decoder_output, training=training)
        combined_features = layers.Concatenate()([
            tf.squeeze(user_attn_out, 1),
            tf.squeeze(prev_system_attn_out, 1),
            tf.squeeze(decoder_attn_out, 1),
            domain_onehot_input,
            turn_id_embedding,
            ontology_multihot_input
        ])
        combined_features = tf.ensure_shape(combined_features, [None, self.nlu_input_dim])
        nlu_out, nlu_gate_weights = self.moe_nlu(combined_features)
        nlu_out = tf.ensure_shape(nlu_out, [None, self.nlu_input_dim])
        intent_out = self.intent_output(nlu_out)
        domain_out = self.domain_output(nlu_out)
        intent_out_expanded = tf.expand_dims(intent_out, axis=1)
        domain_out_expanded = tf.expand_dims(domain_out, axis=1)
        intent_out_tiled = tf.tile(intent_out_expanded, [1, self.max_seq_length, 1])
        domain_out_tiled = tf.tile(domain_out_expanded, [1, self.max_seq_length, 1])
        nlg_input = layers.Concatenate(axis=-1)([
            decoder_output,
            intent_out_tiled,
            domain_out_tiled
        ])
        nlg_input = tf.ensure_shape(nlg_input, [None, self.max_seq_length, self.nlg_input_dim])
        nlg_out, nlg_gate_weights = self.moe_nlg(nlg_input)
        nlg_out = tf.ensure_shape(nlg_out, [None, self.max_seq_length, self.nlg_input_dim])
        response_out = self.response_output(nlg_out)
        return {
            'intent_output': intent_out,
            'domain_output': domain_out,
            'response_output': response_out,
            'response_embeddings': decoder_output,
            'nlu_gate_weights': nlu_gate_weights,
            'nlg_gate_weights': nlg_gate_weights
        }

    def build_graph(self):
        inputs = {
            'user_utterance_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='user_utterance_tokens'),
            'prev_system_response_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='prev_system_response_tokens'),
            'decoder_input_tokens': tf.keras.Input(shape=(self.max_seq_length,), dtype=tf.int32, name='decoder_input_tokens'),
            'domain_onehot_input': tf.keras.Input(shape=(self.num_domains,), dtype=tf.float32, name='domain_onehot_input'),
            'turn_id_embedding': tf.keras.Input(shape=(self.turn_id_dim,), dtype=tf.float32, name='turn_id_embedding'),
            'ontology_multihot_input': tf.keras.Input(shape=(self.num_intents,), dtype=tf.float32, name='ontology_multihot_input'),
        }
        return tf.keras.Model(inputs=inputs, outputs=self.call(inputs))

    def get_config(self):
        config = super().get_config()
        config.update({
            'moe_params': {
                'embedding_dim': self.embedding_dim,
                'max_seq_length': self.max_seq_length,
                'num_domains': self.num_domains,
                'num_intents': self.num_intents,
                'vocab_size': self.vocab_size,
                'num_experts': self.num_experts,
                'expert_dim': self.expert_dim,
                'turn_id_dim': self.turn_id_dim
            }
        })
        return config

class IntentAccuracy(metrics.Metric):
    def __init__(self, name='intent_accuracy', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.correct_preds = self.add_weight(name='correct_preds', initializer='zeros')
        self.total_preds = self.add_weight(name='total_preds', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        correct_batch = tf.reduce_all(tf.equal(y_true, y_pred), axis=-1)
        self.correct_preds.assign_add(tf.reduce_sum(tf.cast(correct_batch, tf.float32)))
        self.total_preds.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.correct_preds / (self.total_preds + tf.keras.backend.epsilon())

    def reset_state(self):
        self.correct_preds.assign(0.)
        self.total_preds.assign(0.)

class IntentPrecision(metrics.Metric):
    def __init__(self, name='intent_precision', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.predicted_positives = self.add_weight(name='pp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        predicted_positives = tf.reduce_sum(y_pred)
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)

    def result(self):
        return self.true_positives / (self.predicted_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.predicted_positives.assign(0.)

class IntentRecall(metrics.Metric):
    def __init__(self, name='intent_recall', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.actual_positives = self.add_weight(name='ap', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        true_positives = tf.reduce_sum(y_true * y_pred)
        actual_positives = tf.reduce_sum(y_true)
        self.true_positives.assign_add(true_positives)
        self.actual_positives.assign_add(actual_positives)

    def result(self):
        return self.true_positives / (self.actual_positives + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0.)
        self.actual_positives.assign(0.)

class IntentF1(metrics.Metric):
    def __init__(self, name='intent_f1', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = IntentPrecision(threshold=threshold)
        self.recall = IntentRecall(threshold=threshold)

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

class DomainAccuracy(metrics.Metric):
    def __init__(self, name='domain_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.accuracy = self.add_weight(name='acc', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(tf.squeeze(y_true, axis=-1), tf.int64)
        pred_labels = tf.argmax(y_pred, axis=1, output_type=tf.int64)
        matches = tf.cast(tf.equal(y_true, pred_labels), tf.float32)
        self.accuracy.assign_add(tf.reduce_sum(matches))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.accuracy / (self.count + tf.keras.backend.epsilon())

    def reset_state(self):
        self.accuracy.assign(0.)
        self.count.assign(0.)

class Perplexity(metrics.Metric):
    def __init__(self, name='perplexity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.cross_entropy = losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
        self.total_loss = self.add_weight(name='total_loss', initializer='zeros')
        self.total_non_padding_tokens = self.add_weight(name='total_non_padding_tokens', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        mask = tf.cast(y_true != 0, dtype=tf.float32)
        loss = self.cross_entropy(y_true, y_pred)
        masked_loss = loss * mask
        sum_loss = tf.reduce_sum(masked_loss)
        num_non_padding_tokens = tf.reduce_sum(mask)
        self.total_loss.assign_add(sum_loss)
        self.total_non_padding_tokens.assign_add(num_non_padding_tokens)

    def result(self):
        avg_loss = self.total_loss / (self.total_non_padding_tokens + tf.keras.backend.epsilon())
        return tf.exp(avg_loss)

    def reset_state(self):
        self.total_loss.assign(0.)
        self.total_non_padding_tokens.assign(0.)

class ResponseEmbeddingCosineSimilarity(metrics.Metric):
    def __init__(self, name='response_embedding_cosine_similarity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.total_cosine_sim = self.add_weight(name='total_cosine_sim', initializer='zeros', dtype=tf.float32)
        self.num_samples = self.add_weight(name='num_samples', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm_val = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm_val = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true_norm = y_true / (y_true_norm_val + epsilon)
        y_pred_norm = y_pred / (y_pred_norm_val + epsilon)
        cosine_sim_per_token = tf.reduce_sum(y_true_norm * y_pred_norm, axis=-1)
        non_padding_mask = tf.cast(y_true_norm_val > epsilon, tf.float32) * tf.cast(y_pred_norm_val > epsilon, tf.float32)
        non_padding_mask = tf.squeeze(non_padding_mask, axis=-1)
        masked_cosine_sim = cosine_sim_per_token * non_padding_mask
        num_non_zero_tokens = tf.reduce_sum(non_padding_mask, axis=-1)
        avg_cosine_sim_per_sample = tf.where(
            num_non_zero_tokens > 0,
            tf.reduce_sum(masked_cosine_sim, axis=-1) / num_non_zero_tokens,
            tf.zeros_like(num_non_zero_tokens, dtype=tf.float32)
        )
        self.total_cosine_sim.assign_add(tf.reduce_sum(avg_cosine_sim_per_sample))
        self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.total_cosine_sim / (self.num_samples + tf.keras.backend.epsilon())

    def reset_state(self):
        self.total_cosine_sim.assign(0.)
        self.num_samples.assign(0.)

def contrastive_loss(y_true, y_pred, margin=1.0):
    epsilon = tf.keras.backend.epsilon()
    y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
    y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
    y_true = y_true / (y_true_norm + epsilon)
    y_pred = y_pred / (y_pred_norm + epsilon)
    cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
    positive_loss = 1.0 - cosine_sim
    mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
    masked_loss = positive_loss * mask
    total_loss = tf.reduce_sum(masked_loss)
    num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
    return total_loss / num_tokens

def calculate_flops(model, batch_size=moe_params["batch_size"]):
    flops = 0
    functional_model = model.build_graph()
    def _calculate_layer_flops(layer_instance, current_batch_size, current_seq_length):
        layer_flops_count = 0
        if isinstance(layer_instance, (tf.keras.layers.InputLayer, type(tf.keras.Input(shape=())))):
            return 0
        if hasattr(layer_instance, 'layers') and isinstance(layer_instance.layers, (list, tuple)):
            for sub_layer in layer_instance.layers:
                layer_flops_count += _calculate_layer_flops(sub_layer, current_batch_size, current_seq_length)
            return layer_flops_count
        if not hasattr(layer_instance, 'input_shape') or layer_instance.input_shape is None:
            return 0
        if isinstance(layer_instance, layers.Dense):
            input_dim = layer_instance.input_shape[-1]
            output_dim = layer_instance.output_shape[-1]
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            layer_flops_count += 2 * input_dim * output_dim * effective_batch_size
        elif isinstance(layer_instance, layers.LSTM):
            input_dim = layer_instance.input_shape[-1]
            hidden_dim = layer_instance.units
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else 1
            layer_flops_count += 8 * (input_dim + hidden_dim) * hidden_dim * seq_length * current_batch_size
        elif isinstance(layer_instance, layers.Embedding):
            pass
        elif isinstance(layer_instance, DeepSeekMoELayer):
            moe_input_dim = layer_instance.input_dim
            effective_batch_size = current_batch_size
            if len(layer_instance.input_shape) == 3:
                effective_batch_size *= layer_instance.input_shape[1]
            gate_flops = 2 * moe_input_dim * layer_instance.routed_experts * effective_batch_size
            layer_flops_count += gate_flops
            for expert in layer_instance.experts:
                layer_flops_count += _calculate_layer_flops(expert, effective_batch_size, 1)
        elif isinstance(layer_instance, layers.MultiHeadAttention):
            config = layer_instance.get_config()
            num_heads = config['num_heads']
            key_dim = config['key_dim']
            d_model = num_heads * key_dim
            seq_length = layer_instance.input_shape[1] if len(layer_instance.input_shape) > 1 else current_seq_length
            projection_flops = 3 * (2 * d_model * d_model) * seq_length * current_batch_size
            attn_score_flops = num_heads * (2 * seq_length * key_dim * seq_length) * current_batch_size
            context_flops = num_heads * (2 * seq_length * seq_length * key_dim) * current_batch_size
            output_projection_flops = 2 * d_model * d_model * seq_length * current_batch_size
            layer_flops_count += projection_flops + attn_score_flops + context_flops + output_projection_flops
        elif isinstance(layer_instance, layers.TimeDistributed):
            inner_layer = layer_instance.layer
            td_effective_batch_size = current_batch_size * layer_instance.input_shape[1] if len(layer_instance.input_shape) == 3 else current_batch_size
            layer_flops_count += _calculate_layer_flops(inner_layer, td_effective_batch_size, 1)
        elif isinstance(layer_instance, TransformerDecoderLayer):
            layer_flops_count += _calculate_layer_flops(layer_instance.mha1, current_batch_size, model.max_seq_length)
            layer_flops_count += _calculate_layer_flops(layer_instance.mha2, current_batch_size, model.max_seq_length)
            ffn_effective_batch_size = current_batch_size * model.max_seq_length
            layer_flops_count += _calculate_layer_flops(layer_instance.ffn, ffn_effective_batch_size, 1)
        return layer_flops_count
    for layer in functional_model.layers:
        flops += _calculate_layer_flops(layer, batch_size, model.max_seq_length)
    return flops / 1e9

def get_gpu_stats():
    if nvidia_smi is None:
        return 0, 0
    try:
        nvidia_smi.nvmlInit()
        handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
        gpu_load = nvidia_smi.nvmlDeviceGetUtilizationRates(handle).gpu
        mem_info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
        gpu_memory = mem_info.used / (1024 ** 3)
        nvidia_smi.nvmlShutdown()
        return gpu_load, gpu_memory
    except Exception:
        return 0, 0

class ResourceMonitor(keras.callbacks.Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.resource_stats = []
        self.expert_load_history = []
        self.start_time = None
        self.epoch_start_time = None
        self.flops = None
        self.validation_data = validation_data
        self.nlu_expert_usage_counts = None
        self.nlg_expert_usage_counts = None
        self.nlu_gate_weights_history = []
        self.nlg_gate_weights_history = []

    def on_train_begin(self, logs=None):
        self.start_time = time.time()
        try:
            self.flops = calculate_flops(self.model, batch_size=moe_params["batch_size"])
            self.nlu_expert_usage_counts = np.zeros(self.model.moe_nlu.total_experts)
            self.nlg_expert_usage_counts = np.zeros(self.model.moe_nlg.total_experts)
        except Exception:
            self.flops = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory()
        gpu_load, gpu_memory = get_gpu_stats()
        epoch_time = time.time() - self.epoch_start_time
        self.resource_stats.append({
            'epoch': epoch + 1,
            'epoch_time': epoch_time,
            'cpu_usage': cpu_usage,
            'memory_used': memory.used / (1024 ** 3),
            'memory_total': memory.total / (1024 ** 3),
            'gpu_load': gpu_load,
            'gpu_memory': gpu_memory,
            'loss': logs.get('loss', 0),
            'val_loss': logs.get('val_loss', 0),
            'intent_accuracy': logs.get('intent_output_intent_accuracy', 0),
            'val_intent_accuracy': logs.get('val_intent_output_intent_accuracy', 0),
            'intent_precision': logs.get('intent_output_intent_precision', 0),
            'val_intent_precision': logs.get('val_intent_output_intent_precision', 0),
            'intent_recall': logs.get('intent_output_intent_recall', 0),
            'val_intent_recall': logs.get('val_intent_output_intent_recall', 0),
            'intent_f1': logs.get('intent_output_intent_f1', 0),
            'val_intent_f1': logs.get('val_intent_output_intent_f1', 0),
            'domain_accuracy': logs.get('domain_output_domain_accuracy', 0),
            'val_domain_accuracy': logs.get('val_domain_output_domain_accuracy', 0),
            'perplexity': logs.get('response_output_perplexity', 0),
            'val_perplexity': logs.get('val_response_output_perplexity', 0),
            'cosine_similarity': logs.get('response_embeddings_response_embedding_cosine_similarity', 0),
            'val_cosine_similarity': logs.get('val_response_embeddings_response_embedding_cosine_similarity', 0),
            'nlu_load_balancing_loss': logs.get('moe_nlu_load_balancing_loss', 0),
            'nlg_load_balancing_loss': logs.get('moe_nlg_load_balancing_loss', 0),
        })
        if 'moe_nlu_load_balancing_loss' in logs or 'moe_nlg_load_balancing_loss' in logs:
            self.expert_load_history.append({
                'epoch': epoch + 1,
                'nlu_expert_load_balance_loss': float(logs.get('moe_nlu_load_balancing_loss', 0)),
                'nlg_expert_load_balance_loss': float(logs.get('moe_nlg_load_balancing_loss', 0))
            })
        sample_size = 100
        for batch in self.validation_data.take(sample_size // moe_params["batch_size"]):
            inputs, _ = batch
            outputs = self.model(inputs, training=False)
            nlu_gate_weights = outputs['nlu_gate_weights']
            reshaped_nlu_weights = tf.reshape(nlu_gate_weights, [-1, self.model.moe_nlu.routed_experts])
            self.nlu_gate_weights_history.append(reshaped_nlu_weights.numpy())
            nlg_gate_weights = outputs['nlg_gate_weights']
            reshaped_nlg_weights = tf.reshape(nlg_gate_weights, [-1, self.model.moe_nlg.routed_experts])
            self.nlg_gate_weights_history.append(reshaped_nlg_weights.numpy())

    def on_train_end(self, logs=None):
        self.total_time = time.time() - self.start_time
        if len(self.nlu_gate_weights_history) > 0:
            all_nlu_weights = np.concatenate(self.nlu_gate_weights_history, axis=0)
            nlu_top_indices = np.argmax(all_nlu_weights, axis=1) + self.model.moe_nlu.k_s
            for expert_id in nlu_top_indices:
                if expert_id < self.model.moe_nlu.total_experts:
                    self.nlu_expert_usage_counts[expert_id] += 1
        if len(self.nlg_gate_weights_history) > 0:
            all_nlg_weights = np.concatenate(self.nlg_gate_weights_history, axis=0)
            nlg_top_indices = np.argmax(all_nlg_weights, axis=1) + self.model.moe_nlg.k_s
            for expert_id in nlg_top_indices:
                if expert_id < self.model.moe_nlg.total_experts:
                    self.nlg_expert_usage_counts[expert_id] += 1

    def visualize_expert_load_distribution(self):
        total_nlu = np.sum(self.nlu_expert_usage_counts) or 1
        total_nlg = np.sum(self.nlg_expert_usage_counts) or 1
        nlu_percentages = (self.nlu_expert_usage_counts / total_nlu) * 100
        nlg_percentages = (self.nlg_expert_usage_counts / total_nlg) * 100
        nlu_stability = np.var(nlu_percentages)
        nlg_stability = np.var(nlg_percentages)
        return {'nlu_percentages': nlu_percentages, 'nlg_percentages': nlg_percentages, 'nlu_stability': nlu_stability, 'nlg_stability': nlg_stability}

class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {"d_model": self.d_model.numpy(), "warmup_steps": self.warmup_steps}

word_to_id = {}
id_to_word = {}
try:
    possible_paths = [
        os.path.join("preprocessor_models", "preprocessor_params.json"),
        os.path.join("tf_datasets", "preprocessor_params.json"),
        "preprocessor_params.json"
    ]
    vocab_loaded = False
    for path in possible_paths:
        if os.path.exists(path):
            with open(path, "r") as f:
                preprocessor_params = json.load(f)
            word_to_id = {k: int(v) for k, v in preprocessor_params['word_to_id'].items()}
            id_to_word = {int(k): v for k, v in preprocessor_params['id_to_word'].items()}
            vocab_loaded = True
            break
    if not vocab_loaded:
        raise FileNotFoundError("Vocabulary file not found in any location")
except Exception as e:
    word_to_id = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
    id_to_word = {0: '<pad>', 1: '<sos>', 2: '<eos>', 3: '<unk>'}

model_save_path = "best_deepseekmoe_adaptive_gating_expert_choice_model"
custom_objects = {
    "DeepSeekMoELayer": DeepSeekMoELayer,
    "CustomLearningRateSchedule": CustomLearningRateSchedule,
    "MoEModel": MoEModel,
    "TransformerDecoderLayer": TransformerDecoderLayer,
    "IntentAccuracy": IntentAccuracy,
    "IntentPrecision": IntentPrecision,
    "IntentRecall": IntentRecall,
    "IntentF1": IntentF1,
    "DomainAccuracy": DomainAccuracy,
    "Perplexity": Perplexity,
    "ResponseEmbeddingCosineSimilarity": ResponseEmbeddingCosineSimilarity
}
try:
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
except Exception as e:
    raise

def compile_model(model, moe_params):
    learning_rate = CustomLearningRateSchedule(moe_params["embedding_dim"])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
    def contrastive_loss(y_true, y_pred, margin=1.0):
        epsilon = tf.keras.backend.epsilon()
        y_true_norm = tf.norm(y_true, axis=-1, keepdims=True)
        y_pred_norm = tf.norm(y_pred, axis=-1, keepdims=True)
        y_true = y_true / (y_true_norm + epsilon)
        y_pred = y_pred / (y_pred_norm + epsilon)
        cosine_sim = tf.reduce_sum(y_true * y_pred, axis=-1)
        positive_loss = 1.0 - cosine_sim
        mask = tf.cast(tf.reduce_sum(tf.abs(y_true), axis=-1) > 0, dtype=tf.float32)
        masked_loss = positive_loss * mask
        total_loss = tf.reduce_sum(masked_loss)
        num_tokens = tf.reduce_sum(mask) + tf.keras.backend.epsilon()
        return total_loss / num_tokens
    losses_dict = {
        'intent_output': tf.keras.losses.BinaryCrossentropy(),
        'domain_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_output': tf.keras.losses.SparseCategoricalCrossentropy(),
        'response_embeddings': contrastive_loss,
    }
    metrics_dict = {
        'intent_output': [IntentAccuracy(), IntentPrecision(), IntentRecall(), IntentF1()],
        'domain_output': [DomainAccuracy()],
        'response_output': [Perplexity()],
        'response_embeddings': [ResponseEmbeddingCosineSimilarity()]
    }
    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        metrics=metrics_dict,
        loss_weights={
            'intent_output': 0.5,
            'domain_output': 0.5,
            'response_output': 1.0,
            'response_embeddings': 1.0
        }
    )
    return model

model = compile_model(model, moe_params)
sample_input = next(iter(train_tf_dataset.take(1)))
model(sample_input[0])
model.summary()

class EvaluationMetrics:
    def __init__(self, model, test_dataset, moe_params, raw_data, word_to_id, id_to_word):
        self.model = model
        self.test_dataset = test_dataset
        self.moe_params = moe_params
        self.raw_data = raw_data
        self.word_to_id = word_to_id
        self.id_to_word = id_to_word
        self.nlu_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.nlg_expert_usage_counts = np.zeros(moe_params["num_experts"] * moe_params.get("m", 2)) if "num_experts" in moe_params else np.zeros(1)
        self.resource_stats = {
            'cpu_usage': [],
            'memory_used': [],
            'gpu_load': [],
            'gpu_memory': [],
            'batch_times': []
        }
        self.special_tokens = {'<pad>', '<sos>', '<eos>', '<unk>'}
        self.nlu_top_k = model.get_layer('moe_nlu').get_config().get('top_k', 2)
        self.nlg_top_k = model.get_layer('moe_nlg').get_config().get('top_k', 2)

    def evaluate(self):
        def add_response_embeddings(features, targets):
            response_tokens = targets['response_output']
            response_embeddings = self.model.embedding(response_tokens)
            targets['response_embeddings'] = response_embeddings
            return features, targets
        test_dataset_with_embeddings = self.test_dataset.map(add_response_embeddings, num_parallel_calls=tf.data.AUTOTUNE)
        start_time = time.time()
        test_results = self.model.evaluate(test_dataset_with_embeddings, verbose=1)
        eval_time = time.time() - start_time
        metric_names = self.model.metrics_names
        perplexity_value = None
        for name, value in zip(metric_names, test_results):
            if name == 'response_output_perplexity':
                perplexity_value = value
                break
        if perplexity_value is None:
            perplexity_value = 0.0
        cpu_usage = psutil.cpu_percent()
        memory = psutil.virtual_memory().used / (1024 ** 3)
        gpu_load, gpu_memory = get_gpu_stats()
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))
        total_tokens = 0
        total_time = 0
        num_batches = 0
        test_dataset_small = test_dataset_with_embeddings.unbatch().batch(moe_params["batch_size"])
        all_true_intents = []
        all_pred_intents = []
        all_true_domains = []
        all_pred_domains = []
        for batch_idx, batch in enumerate(test_dataset_small):
            inputs, targets = batch
            response_tokens = targets['response_output'].numpy()
            non_padding_mask = response_tokens != 0
            total_tokens += np.sum(non_padding_mask)
            start_time = time.time()
            outputs = self.model(inputs, training=False)
            batch_time = time.time() - start_time
            total_time += batch_time
            num_batches += tf.shape(inputs['user_utterance_tokens'])[0]
            cpu_usage = psutil.cpu_percent()
            memory = psutil.virtual_memory()
            gpu_load, gpu_memory = get_gpu_stats()
            self.resource_stats['cpu_usage'].append(cpu_usage)
            self.resource_stats['memory_used'].append(memory.used / (1024 ** 3))
            self.resource_stats['gpu_load'].append(gpu_load)
            self.resource_stats['gpu_memory'].append(gpu_memory)
            self.resource_stats['batch_times'].append(batch_time)
            nlu_gate_weights = outputs.get('nlu_gate_weights', tf.zeros([0, self.model.moe_nlu.routed_experts])).numpy()
            nlu_top_indices = np.argsort(-nlu_gate_weights, axis=-1)[:, :self.nlu_top_k] + self.model.moe_nlu.k_s
            for idx in range(self.nlu_top_k):
                expert_ids = nlu_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlu_expert_usage_counts):
                        self.nlu_expert_usage_counts[expert_id] += 1
            nlg_gate_weights = outputs.get('nlg_gate_weights', tf.zeros([0, self.model.moe_nlg.routed_experts])).numpy()
            if len(nlg_gate_weights.shape) == 3:
                nlg_gate_weights = nlg_gate_weights.reshape(-1, nlg_gate_weights.shape[-1])
            nlg_top_indices = np.argsort(-nlg_gate_weights, axis=-1)[:, :self.nlg_top_k] + self.model.moe_nlg.k_s
            for idx in range(self.nlg_top_k):
                expert_ids = nlg_top_indices[:, idx]
                for expert_id in expert_ids:
                    if expert_id < len(self.nlg_expert_usage_counts):
                        self.nlg_expert_usage_counts[expert_id] += 1
            true_intents = targets.get('intent_output', np.zeros((moe_params["batch_size"], self.model.num_intents))).numpy()
            pred_intents = (outputs.get('intent_output', np.zeros_like(true_intents)).numpy() > 0.5).astype(np.int32)
            all_true_intents.append(true_intents)
            all_pred_intents.append(pred_intents)
            true_domains = targets.get('domain_output', np.zeros((moe_params["batch_size"], 1))).numpy().flatten()
            pred_domains = np.argmax(outputs.get('domain_output', np.zeros((moe_params["batch_size"], self.model.num_domains))).numpy(), axis=-1)
            all_true_domains.append(true_domains)
            all_pred_domains.append(pred_domains)
            if batch_idx % 50 == 0:
                gc.collect()
                tf.keras.backend.clear_session()
        avg_time_per_token = (total_time / total_tokens) * 1000 if total_tokens > 0 else 0
        avg_cpu = np.mean(self.resource_stats['cpu_usage']) if self.resource_stats['cpu_usage'] else 0
        avg_memory = np.mean(self.resource_stats['memory_used']) if self.resource_stats['memory_used'] else 0
        avg_gpu_load = np.mean(self.resource_stats['gpu_load']) if self.resource_stats['gpu_load'] else 0
        avg_gpu_memory = np.mean(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        peak_gpu_memory = np.max(self.resource_stats['gpu_memory']) if self.resource_stats['gpu_memory'] else 0
        flops = calculate_flops(self.model, batch_size=self.moe_params.get("batch_size", moe_params["batch_size"]))
        all_true_intents = np.vstack(all_true_intents)
        all_pred_intents = np.vstack(all_pred_intents)
        all_true_domains = np.concatenate(all_true_domains)
        all_pred_domains = np.concatenate(all_pred_domains)
        intent_f1_score = f1_score(all_true_intents, all_pred_intents, average='micro')
        intent_f1_score_macro = f1_score(all_true_intents, all_pred_intents, average='macro')
        intent_f1_score_weighted = f1_score(all_true_intents, all_pred_intents, average='weighted')
        intent_accuracy_score = accuracy_score(all_true_intents, all_pred_intents)
        domain_accuracy_score = accuracy_score(all_true_domains, all_pred_domains)
        domain_f1_score_macro = f1_score(all_true_domains, all_pred_domains, average='macro')
        hypotheses = []
        references = []
        meteor_scores = []
        rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        domain_metrics = defaultdict(lambda: {
            'bleu': [],
            'rouge1': [],
            'rouge2': [],
            'rougeL': [],
            'meteor': [],
            'count': 0
        })
        if not self.raw_data.get('test'):
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0
            domain_metrics = {}
        else:
            for i, batch in enumerate(test_dataset_small):
                if i >= len(self.raw_data.get('test', [])):
                    break
                try:
                    inputs, targets = batch
                    with tf.device('/CPU:0'):
                        outputs = self.model(inputs, training=False)
                    domain_id = targets['domain_output'].numpy()[0][0]
                    domain_name = f"domain_{domain_id}"
                    response_probs = outputs['response_output'].numpy()
                    generated_tokens = np.argmax(response_probs, axis=-1)[0]
                    true_tokens = targets['response_output'].numpy()[0]
                    raw_item = self.raw_data['test'][i]
                    ref_text = raw_item.get('raw_next_system_response', '')
                    if not ref_text:
                        continue
                    gen_text = self.tokens_to_text(generated_tokens)
                    ref_tokens = self.tokens_to_text(true_tokens)
                    if not gen_text.strip() or not ref_text.strip():
                        continue
                    hypotheses.append(gen_text)
                    references.append([ref_text])
                    ref_tokens = word_tokenize(ref_text.lower())
                    gen_tokens = word_tokenize(gen_text.lower())
                    try:
                        meteor = meteor_score([ref_tokens], gen_tokens)
                        meteor_scores.append(meteor)
                        domain_metrics[domain_name]['meteor'].append(meteor)
                    except Exception as e:
                        meteor_scores.append(0)
                        domain_metrics[domain_name]['meteor'].append(0)
                    try:
                        rouge_scores = rouge_scorer_instance.score(ref_text, gen_text)
                        domain_metrics[domain_name]['rouge1'].append(rouge_scores['rouge1'].fmeasure)
                        domain_metrics[domain_name]['rouge2'].append(rouge_scores['rouge2'].fmeasure)
                        domain_metrics[domain_name]['rougeL'].append(rouge_scores['rougeL'].fmeasure)
                    except Exception as e:
                        pass
                    domain_metrics[domain_name]['count'] += 1
                    if i % 50 == 0:
                        gc.collect()
                        tf.keras.backend.clear_session()
                except Exception as e:
                    continue
        if hypotheses and references:
            hypotheses_tok = []
            references_tok = []
            for hyp, ref_list in zip(hypotheses, references):
                if not hyp or not ref_list[0]:
                    continue
                hyp_tokens = word_tokenize(hyp.lower())
                ref_tokens = [word_tokenize(ref.lower()) for ref in ref_list]
                hypotheses_tok.append(hyp_tokens)
                references_tok.append(ref_tokens)
            chencherry = SmoothingFunction()
            try:
                bleu_score = corpus_bleu(references_tok, hypotheses_tok, smoothing_function=chencherry.method1)
            except Exception as e:
                bleu_score = 0.0
            rouge_scores = []
            for hyp, ref in zip(hypotheses, references):
                if hyp and ref[0]:
                    try:
                        rouge_scores.append(rouge_scorer_instance.score(ref[0], hyp))
                    except Exception as e:
                        continue
            avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores]) if rouge_scores else 0.0
            avg_meteor = np.mean(meteor_scores) if meteor_scores else 0.0
        else:
            bleu_score = avg_rouge1 = avg_rouge2 = avg_rougeL = avg_meteor = 0.0
        return {
            'intent_f1_micro': intent_f1_score,
            'intent_f1_macro': intent_f1_score_macro,
            'intent_f1_weighted': intent_f1_score_weighted,
            'intent_accuracy': intent_accuracy_score,
            'domain_accuracy': domain_accuracy_score,
            'domain_f1_macro': domain_f1_score_macro,
            'bleu': bleu_score,
            'rouge1': avg_rouge1,
            'rouge2': avg_rouge2,
            'rougeL': avg_rougeL,
            'meteor': avg_meteor,
            'perplexity': perplexity_value,
            'pred_speed': avg_time_per_token,
            'domain_metrics': domain_metrics,
            'eval_time': eval_time
        }
    
    def tokens_to_text(self, tokens):
        words = []
        for token in tokens:
            if token == 0:
                continue
            word = self.id_to_word.get(token, '<unk>')
            if word not in self.special_tokens:
                words.append(word)
        return " ".join(words).strip()

results_table = {
    'Evaluation Time (seconds)': [],
    'Prediction Speed (ms/token)': [],
    'Average CPU Usage (percent)': [],
    'Average GPU Usage (percent)': [],
    'Average Memory (GB)': [],
    'Average GPU Memory (GB)': [],
    'Average FLOPs Estimate (GFLOPs)': [],
    'NLU Expert 1 (percent)': [],
    'NLU Expert 2 (percent)': [],
    'NLU Expert 3 (percent)': [],
    'NLU Expert 4 (percent)': [],
    'NLU Expert 5 (percent)': [],
    'NLU Expert 6 (percent)': [],
    'NLU Expert 7 (percent)': [],
    'NLU Expert 8 (percent)': [],
    'NLG Expert 1 (percent)': [],
    'NLG Expert 2 (percent)': [],
    'NLG Expert 3 (percent)': [],
    'NLG Expert 4 (percent)': [],
    'NLG Expert 5 (percent)': [],
    'NLG Expert 6 (percent)': [],
    'NLG Expert 7 (percent)': [],
    'NLG Expert 8 (percent)': [],
    'Intent F1-score (Micro)': [],
    'Intent F1-score (Macro)': [],
    'Intent F1-score (Weighted)': [],
    'Intent Accuracy': [],
    'Domain Accuracy': [],
    'Domain F1-score (Macro)': [],
    'BLEU Score': [],
    'ROUGE-1 F1': [],
    'ROUGE-2 F1': [],
    'ROUGE-L F1': [],
    'METEOR Score': [],
    'Perplexity': [],
    'NLU Expert Load Stability (Variance)': [],
    'NLG Expert Load Stability (Variance)': []
}

num_iterations = 5
for iteration in range(1, num_iterations + 1):
    tf.keras.backend.clear_session()
    gc.collect()
    model = tf.keras.models.load_model(model_save_path, custom_objects=custom_objects, compile=False)
    model = compile_model(model, moe_params)
    evaluator = EvaluationMetrics(model, test_tf_dataset, moe_params, raw_data, word_to_id, id_to_word)
    evaluation_results = evaluator.evaluate()
    results_table['Evaluation Time (seconds)'].append(evaluation_results['eval_time'])
    results_table['Prediction Speed (ms/token)'].append(evaluation_results['pred_speed'])
    results_table['Average CPU Usage (percent)'].append(np.mean(evaluator.resource_stats['cpu_usage']) if evaluator.resource_stats['cpu_usage'] else 0)
    results_table['Average GPU Usage (percent)'].append(np.mean(evaluator.resource_stats['gpu_load']) if evaluator.resource_stats['gpu_load'] else 0)
    results_table['Average Memory (GB)'].append(np.mean(evaluator.resource_stats['memory_used']) if evaluator.resource_stats['memory_used'] else 0)
    results_table['Average GPU Memory (GB)'].append(np.mean(evaluator.resource_stats['gpu_memory']) if evaluator.resource_stats['gpu_memory'] else 0)
    results_table['Average FLOPs Estimate (GFLOPs)'].append(calculate_flops(model))
    nlu_counts = evaluator.nlu_expert_usage_counts
    nlg_counts = evaluator.nlg_expert_usage_counts
    total_nlu = np.sum(nlu_counts) if np.sum(nlu_counts) > 0 else 1
    total_nlg = np.sum(nlg_counts) if np.sum(nlg_counts) > 0 else 1
    results_table['NLU Expert 1 (percent)'].append((nlu_counts[0] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 2 (percent)'].append((nlu_counts[1] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 3 (percent)'].append((nlu_counts[2] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 4 (percent)'].append((nlu_counts[3] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 5 (percent)'].append((nlu_counts[4] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 6 (percent)'].append((nlu_counts[5] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 7 (percent)'].append((nlu_counts[6] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLU Expert 8 (percent)'].append((nlu_counts[7] / total_nlu * 100 if total_nlu > 0 else 0))
    results_table['NLG Expert 1 (percent)'].append((nlg_counts[0] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 2 (percent)'].append((nlg_counts[1] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 3 (percent)'].append((nlg_counts[2] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 4 (percent)'].append((nlg_counts[3] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 5 (percent)'].append((nlg_counts[4] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 6 (percent)'].append((nlg_counts[5] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 7 (percent)'].append((nlg_counts[6] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['NLG Expert 8 (percent)'].append((nlg_counts[7] / total_nlg * 100 if total_nlg > 0 else 0))
    results_table['Intent F1-score (Micro)'].append(evaluation_results['intent_f1_micro'])
    results_table['Intent F1-score (Macro)'].append(evaluation_results['intent_f1_macro'])
    results_table['Intent F1-score (Weighted)'].append(evaluation_results['intent_f1_weighted'])
    results_table['Intent Accuracy'].append(evaluation_results['intent_accuracy'])
    results_table['Domain Accuracy'].append(evaluation_results['domain_accuracy'])
    results_table['Domain F1-score (Macro)'].append(evaluation_results['domain_f1_macro'])
    results_table['BLEU Score'].append(evaluation_results['bleu'])
    results_table['ROUGE-1 F1'].append(evaluation_results['rouge1'])
    results_table['ROUGE-2 F1'].append(evaluation_results['rouge2'])
    results_table['ROUGE-L F1'].append(evaluation_results['rougeL'])
    results_table['METEOR Score'].append(evaluation_results['meteor'])
    results_table['Perplexity'].append(evaluation_results['perplexity'])
    results_table['NLU Expert Load Stability (Variance)'].append(np.var(nlu_counts / total_nlu * 100) if total_nlu > 0 else 0)
    results_table['NLG Expert Load Stability (Variance)'].append(np.var(nlg_counts / total_nlg * 100) if total_nlg > 0 else 0)

for key in results_table:
    if results_table[key]:
        results_table[key].append(np.mean(results_table[key]))
    else:
        results_table[key].append(0)

final_df = pd.DataFrame(results_table)
final_df = final_df.T
final_df.columns = [f'Iteration {i+1}' for i in range(num_iterations)] + ['Average']
final_df.to_excel('prediction_deepseekmoe_adaptive_gating_expert_choice_results.xlsx', index=False)
final_df

2025-07-04 08:59:34.141854: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 08:59:34.180462: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 08:59:34.180506: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 08:59:34.182729: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-04 08:59:34.192030: I tensorflow/core/platform/cpu_feature_guar

Model: "mo_e_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  784896    
                                                                 
 embedding (Embedding)       multiple                  8576      
                                                                 
 layer_normalization (Layer  multiple                  256       
 Normalization)                                                  
                                                                 
 transformer_decoder_layer   multiple                  264576    
 (TransformerDecoderLayer)                                       
                                                                 
 dropout_3 (Dropout)         multiple                  0         
                                                                 
 moe_nlu (DeepSeekMoELayer)  multiple                  2

,Iteration 1,Average
Evaluation Time (seconds),19.436955,19.436955
Prediction Speed (ms/token),1.996718,1.996718
Average CPU Usage (percent),22.000072,22.000072
Average GPU Usage (percent),1.992842,1.992842
Average Memory (GB),5.648967,5.648967
Average GPU Memory (GB),14.504456,14.504456
Average FLOPs Estimate (GFLOPs),2.377871,2.377871
NLU Expert 1 (percent),0.000000,0.000000
NLU Expert 2 (percent),0.000000,0.000000
NLU Expert 3 (percent),10.325698,10.325698
